# Collect all the data that has been tracked and associated


In [1]:
import numpy as np
import os
import json
import sys
import glob
import time
import datetime
from pympler.tracker import SummaryTracker
from scipy import stats, signal
from scipy.signal import argrelextrema
from sklearn import linear_model
import cv2
import h5py
from itertools import groupby
from math import ceil
import pickle
import math
from scipy.linalg import norm
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA


# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from matplotlib.patches import Rectangle, Circle
from matplotlib.patches import Polygon, Ellipse
from matplotlib import patches
from matplotlib.collections import PatchCollection
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D

# import statsmodels.api as sm

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/AntTrackCode')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker

# import multiprocessing
# import threading
import subprocess
from subprocess import call
import pandas as pd
import seaborn as sns
import copy


# stop telling me i have a nan in an array with a logical comparison
with np.errstate(invalid='ignore'):
    np.less([np.nan, 0], 1)

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


In [2]:
%qtconsole

## Build file list 

In [3]:
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
file_list = []

# searches for files
file_list = glob.glob(os.path.join(vid_locations, '**/**/*0000.mp4'))
# file_list = glob.glob(os.path.join(vid_locations, 'Tunnel_20180329-30/**/*0000.mp4'))
file_list = sorted(file_list)
print('Total Number of Videos: ',len(file_list))
                
file_list = file_list
len(file_list)

Total Number of Videos:  8266


8266

### Calibrations 

In [4]:
# TUNNEL EXPERIMENTS
pix2mm = 959.7563/30 # Measured 3cm in 4 cameras
fps = 239.16

v_multiplier=  fps / pix2mm
print('%0.3f pixels per mm'%pix2mm)


# ARENA EXPERIMENT
# pix2cm = np.mean([ 3/np.mean([869.047, 873.866, 870.576]),  2/583.876])
# print('Pixels per cm:', 1/(3/np.mean([869.047, 873.866, 870.576])),' and ', 1/(2/583.876))

# BRIDGE EXPERIMENT
# pix2cm = np.mean([ 2/np.mean([926.069, 930.107]),  2.4/1110.773, 2.5/1158.836])

31.992 pixels per mm


# OPTIONAL: BUILD DATA SET AND SAVE AS PICKLE


In [ ]:
# FOR EACH COLONY, BUILD DATA AND SAVE AS PICKLE

vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
colonies = glob.glob(os.path.join(vid_locations, 'Tunnel**/'))
# %run LoadInTrackedData.ipynb # load in function of interest
from ipynb.fs.defs.LoadInTrackedData import load_and_analyze_videos_to_df

for col in colonies:
    print('\n', col)
    f_list = sorted(glob.glob(os.path.join(col, '**/*0000.mp4')))
    load_and_analyze_videos_to_df(f_list)
    

# # LOAD IN DATA
# %run LoadInTrackedData.ipynb
# df, trial_info = load_and_analyze_videos_to_df(file_list)
# # df, trial_info = load_and_analyze_videos_to_df(file_list)


# # if just building trial list
# # trial_info = create_trial_info(file_list)

del f_list, col, colonies

#### Find long trials and bootstrap

In [ ]:
# FIND LONG TRIALS 
# import random


temp = df.copy()
# look only at trials longer than 50fr
idcs = [index for index, row in temp.iterrows() if len(row.v)>50]
longtracks = df.loc[idcs,]

# # MEDIAN V
# colony_R = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
# date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
# day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
# subs_string = longtracks['substrate'].values.tolist()
# substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
# v_med_R = np.array(longtracks['median_v'])
# df_med_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
#                        "v_med" : v_med_R } )

# del idcs, colony_R, day_R, substrate_R, v_med_R, date_days, subs_string, temp



# # BOOTSTRAP
# n_boot = 50;

# # make a numpy array of each random sample for one entry in dataframe
# sample = []
# for boot in range(0,100):
# #     sample.append( np.array([random.choice(tr) for tr in list(temp.v)]) )
#     sample.append( np.array([random.choice(row.v) if len(row.v)>50 else np.nan for index, row in longtracks.iterrows() ]))
    
# # convert sample list into np array
# allsamples = np.transpose(np.vstack(sample))
# samplesubset = allsamples[:,0:n_boot]

# # make new dataframe to be output to R as feather
# shortcolnames = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
# colony_R = np.repeat(np.array(shortcolnames), n_boot)
# date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
# col_days = [col.split('-').index(day) for day, col in zip(date_days, shortcolnames)]
# day_R = np.repeat(np.array(col_days), n_boot)
# subs_string = longtracks['substrate'].values.tolist()
# subs_int = np.array([int(s.split('mm')[0]) for s in subs_string])
# substrate_R = np.repeat(subs_int, n_boot)
# indiv_R = np.repeat(np.array(longtracks.index), n_boot)
# df_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
#                        "indiv" : indiv_R, "v" : np.concatenate(samplesubset)})

# del allsamples, samplesubset, shortcolnames, colony_R, day_R, substrate_R, date_days, subs_string, subs_int, indiv_R, sample
del temp
print('done')

## Manually save things as pickle or feather

In [18]:
# SAVE DATAFRAME AS FEATHER
# import time
# import feather

# # save master dataframe as pickle
# df_preference = (df>0).astype(np.int8)
# start_time = time.time()
# df.to_pickle(vid_locations + 'AllTracks')#, compression='infer', protocol=4) 
# print('Saved all trackways dataframe as pickle - duration: ', time.time()-start_time)

# save master dataframe with TDs for each colony
# colonies = glob.glob(os.path.join(vid_locations, 'Tunnel**/'))
# for col in colonies:
#     col_OI = col.split('/')[-2][-5:]
#     df.loc[(df['colony']!='Tunnel_201803'+col_OI)].to_pickle(vid_locations + '/AllTracks_TDs_%s'%col_OI, protocol = 4)#, compression='infer', protocol=4) 
#     print(col_OI)
# print('    Done saving pickle')
    
# save master dataframe as feather
# start_time = time.time()
# # newdf = df[['ID','colony','datetime','date','median_v','substrate','x_raw']].copy()
# # feather.write_dataframe(newdf, vid_locations + 'AllTracks.feather')
# print('Saved all trackways dataframe as feather - duration: ', time.time()-start_time)

# save long tracks dataframe as pickle
# start_time = time.time()
# longtracks.to_pickle(vid_locations + 'LongTracks')
# print('Saved long trackways dataframe as pickle - duration: ', time.time()-start_time)

# save long tracks dataframe as feather
# start_time = time.time()
# # feather.write_dataframe(longtracks, vid_locations + 'LongTracks.feather')
# print('Saved long trackways dataframe as feathers - duration: ', time.time()-start_time)

# print('Saved median and bootstrap files as feathers')
# feather.write_dataframe(df_med_R, vid_locations + 'Median.feather')
# # feather.write_dataframe(df_R, vid_locations + 'Bootstrap.feather')
# del df_med_R, df_R

13-14
15-16
20-21
22-23
24-25
27-28
29-30
08-09
    Done saving pickle


# LOAD IN DATA FROM PICKLE

In [5]:
# LOAD IN DATA FROM PICKLE OR FEATHER (SKIP ALL NEXT DATAFRAME SECTIONS) - SPECIFY WHETHER TO INCLUDE TD DATA
print('loading in all trackways pickle')
start_time = time.time()
colonies = glob.glob(os.path.join(vid_locations, 'Tunnel**/'))
for col in colonies[0:]:
    col_OI = col.split('/')[-2][-5:]
    print('   colony: %s'%col_OI)
    tmp = pd.read_pickle(vid_locations + 'AllTracks_' + col_OI)
#     tmp = pd.read_pickle(vid_locations + 'AllTracks_TDs_' + col_OI)
    if col == colonies[0]:
        df = tmp.copy()
    else:
        df = pd.concat([df,tmp], ignore_index=True)
    del tmp, col_OI
print('read in all trackways pickle -- duration: ', time.time()-start_time)


# load in master dataframe
# print('loading in all trackways pickle')
# start_time = time.time()
# df = pd.read_pickle(vid_locations + 'AllTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)

# make list of trials
# %run LoadInTrackedData.ipynb # loads all functions
# from ipynb.fs.defs.LoadInTrackedData import create_trial_info # loads only one function
# trial_info = create_trial_info(file_list)

# load in dataframe of long trackways
# start_time = time.time()
# longtracks = pd.read_pickle(vid_locations + 'LongTracks')
# print('read in all trackways pickle -- duration: ', time.time()-start_time)

loading in all trackways pickle
   colony: 13-14
   colony: 15-16
   colony: 20-21
   colony: 22-23
   colony: 24-25
   colony: 27-28
   colony: 29-30
   colony: 08-09
read in all trackways pickle -- duration:  53.11151313781738


In [6]:
# how to delete a bunch of columns from a dataframe
columns_to_drop = ['LA', 'RA', 'area', 'error', 'sinuosity',
                  'xy_cov_matrix', 'median_vx', 'median_vy', 'measurements', 'xfilt','yfilt', 'x_final', 'y_final', 'vx_final', 'vy_final', 'v_final']
for colmn in columns_to_drop:
    if colmn in df: # remove columns if already exist
        df = df.drop(colmn, axis = 1)

# for joint_num in range(0,6):
#     columns_to_drop = ['TD_dists_all', 'TD_dist_x', 'TD_dist_y', 'TD_dist_idcs',
#                       'joint%i_TD_dists'%joint_num, 'joint%i_TD_dist_x'%joint_num, 'joint%i_TD_dist_y'%joint_num, 'joint%i_TD_dist_idcs'%joint_num]

# PLOT THINGS

## Plot out all walking trajectories

For each colony, plots out trajectories on each substrate. Good way of visualizing if there are pheromone paths etc.

In [ ]:
# FIND PHEROMONE TRAILS BASED ON Y LOCATION

allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
allcols = [tr['colony'] for tr in trial_info]
coltypes = sorted(list(set(allcols)))
wid = 1000
hei = 550

for coltype in coltypes:
    plt.figure(figsize = (20,4))
    plt.clf()
    col_data = df.loc[df['colony'] == coltype]
    
    for kk, subtype in enumerate(subtypes):
        dataOI = col_data.loc[col_data['substrate'] == subtype]
        
        # define subplot
        spwid=0.23
#         ax=plt.axes([0.05+spwid*int(kk), 0, spwid-.01-.07, 1])
        
        gs = gridspec.GridSpec(1,4)
        gs.update(left = 0.05+spwid*int(kk), right = 0.05+spwid*(int(kk)+1), wspace = 0)
        
        # plot all traces
        for index, row in dataOI.iterrows():
            ax1 = plt.subplot(gs[0,:-1])
            ax1.plot(row['x_raw'], row['y_raw'], '-k', linewidth = 1, alpha = 0.1)
            ax1.set_xlim((0, wid))
            ax1.set_ylim((0, hei))
        ax1.set_aspect(1)
        ax1.invert_yaxis()
        if kk > 0:
            ax1.get_yaxis().set_visible(False)
#         n_long = len(longtracks.loc[longtracks['colony'] == coltype].loc[ longtracks['substrate'] == subtype ])
#         plt.title(('%s -- n: %s -- n > 50 fr: %s' % (subtype, len(dataOI), n_long)), horizontalalignment = 'left', x = 0)
        plt.title(('%s -- n: %s' % (subtype, len(dataOI))), horizontalalignment = 'left', x = 0)
        
#         # plot hist of all y values
#         ax2 = plt.subplot(gs[0,-1])
        
#         # plotting through pandas
# #         all_ys = pd.DataFrame( {"y_raw" : np.concatenate(dataOI['y_raw'].values)})
# #         ax2=plt.axes([0.05+spwid*int(kk)+(spwid-0.07), .32, .07, .36])
# #         all_ys['y_raw'].hist(orientation = 'horizontal', alpha = 0.5)

#         # ploting through matplotlib
#         all_ys = np.concatenate(dataOI['y_raw'].values)
#         hist, bins = np.histogram(all_ys, bins = 20)
#         barwidth = bins[1]-bins[0]
#         centers = (bins[:-1]+bins[1:])/2                          
#         ax2.barh(centers, hist, align = 'center', height = barwidth, alpha = 0.5)
#         ax2.set_ylim(0,hei)
#         ax2.set_ylim(ax2.get_ylim()[::-1])
#         ax2.set_aspect(ax2.get_xbound()[1]/wid*3)
#         ax2.get_yaxis().set_visible(False)
#         ax2.get_xaxis().set_visible(False)
#         ax2.set_frame_on(False)
   
#         # identify pheromone trails using mode
#         ax1.axhline(centers[np.argmax(hist)],color='r', linestyle = '--', alpha = 0.5)
#         ax2.axhline(centers[np.argmax(hist)],color='r', linestyle = '--', alpha = 0.5, xmax = 0.95)
#         ax1.axhspan(centers[np.argmax(hist)]-2*barwidth, centers[np.argmax(hist)]+2*barwidth, color = 'r', alpha = 0.2)
        
#         # identify pheromone trails using median
#         ax1.axhline(np.mean(all_ys), alpha = 0.5)
#         ax1.axhspan(np.mean(all_ys)-np.std(all_ys), np.mean(all_ys)+np.std(all_ys), alpha = 0.2)
        
        
        
        
    plt.suptitle('Colony: %s' % coltype, x=0.02, y = 0.93, fontsize = 16, horizontalalignment = 'left')
    
    plt.savefig(vid_locations + 'Figures/' + 'PheromoneTrail_%s.png' % coltype)

In [ ]:
# PLOT AND SAVE TRAJECTORIES FOR EACH HOUR OF RECORDING
import matplotlib.animation as animation


alldates = [tr['date'] for tr in trial_info]
alltimes = [tr['time'] for tr in trial_info]
dates = sorted(list(set(alldates)))

dpi = 300
nfig = 0

for date in dates:
    print('Date: ', date)
    whichones = [i for i,x in enumerate(alldates) if x == date]
    todaystimes = [alltimes[x] for x in whichones]
    todayshours = [int(x[0:2]) for x in todaystimes]
    
    
    for hour  in list(range(0,24)):
        whichoftoday = [whichones[i] for i,x in enumerate(todayshours) if 
                        (x == hour)]
        tracksubset = [tracks[i] for i in whichoftoday]
        trialnames = [file_list[i] for i in whichoftoday]
        
        fig = plt.figure()
        plt.clf()
        
        num_flights = 0
        cnt = 0
        for cnt, track in enumerate(tracksubset):
            if track != []:
                for k, obj in track.items():

                    if len(obj['x']) < 10:
                        continue

                    # plot with each trial as separate color IN PIXELS
                    plt.plot(obj['xfilt'], obj['yfilt'], '-')

                    # vary color based on velocity
        #             plt.set_cmap('copper')
        #             temp=obj['vfilt']/np.amax(obj['vfilt'])
        #             plt.scatter(pix2cm*obj['xfilt'][1::], pix2cm*obj['yfilt'][1::], #'-' ,
        #                      c=temp, edgecolor='none')#.tolist()).tolist()

                    num_flights += 1

#         print('Trials in hour ', hour, ': ', cnt) #whichoftoday)
        plt.xlabel('x (cm)')
        plt.ylabel('y (cm)')
        plt.title('%s -- %d:00-%d:00.\nn = %d' %(date, hour, hour+1,cnt) ,loc = 'left')
        plt.gca().set_aspect(1)
        plt.gca().invert_yaxis()
        plt.xlim((0, 1280))
        plt.ylim((0, 700))
        plt.gcf()
        
        # save figure
        pname = os.path.join(vid_locations, 'Trackway%d.png'%(nfig))
        plt.savefig(pname)
        plt.close('all')
        
        nfig = nfig +1
            
# save images as movie
os.system("ffmpeg -r 4 -i '" + vid_locations  + "Trackway%01d.png' -vcodec mpeg4 -y '" 
          + vid_locations + "Trackways.mp4'")

# delete all trackway vids
pics2delete = glob.glob(os.path.join(vid_locations, 'Trackway*.png'))
for pic in pics2delete:
    os.remove(pic)

## VIOLIN PLOTS

In [ ]:
# violin plot of median speed vs. substrate FOR ALL TRIALS

plt.figure()
pltcolors = ['#464F56', '#BA4246', '#087E8B', '#701C6F']
my_pal = {"0mm": '#464F56', "1mm": '#BA4246', "3mm": '#087E8B', "5mm": '#701C6F'}
ax = sns.violinplot(x = 'substrate', y = 'median_v',  
                    data = df[(df['v'].map(lambda x: np.sum(np.isfinite(x))>50).values) &
                             (df['colony'] != 'Tunnel_20180508-09')], cut = 0 , palette=my_pal) #hue = 'substrate',
ax.set_ylabel('median v [pix/s]')
# sns.stripplot(x = 'treatment', y = 'mean_vx', hue = 'headwind', data = df, split = True, jitter = True)
plt.gcf()
# ax = sns.violinplot(x = 'substrate', y = 'median_v',  data = df, cut = 0 ) #hue = 'substrate',
# add how many trials in each plot to figure
# nobs = df['substrate'].value_counts().values
nobs = df[df['v'].map(lambda x: np.sum(np.isfinite(x))>50).values]['substrate'].value_counts().values
nobs = nobs[::-1]
pos = range(len(nobs))

# if want in mm/s
plt.yticks(np.arange(0,1500, step = pix2mm*5), np.arange(0,int(1500/pix2mm), step = 5))
ax.set_ylabel('median v [mm/s]')

for tick,label in zip(pos, ax.get_xticklabels()):
    ax.text(pos[tick], -20, ('n_tracks = ' + str(nobs[tick])), 
            horizontalalignment = 'center', size = 'x-small', weight = 'semibold')


subtypes = sorted(list(set(df['substrate'])))
for sub in subtypes:
    print(
        np.median(df[(df['v'].map(lambda x: np.sum(np.isfinite(x))>50).values) & (df['colony'] != 'Tunnel_20180508-09') & (df['substrate'] == sub)]['median_v'].values)/pix2mm)

In [ ]:
# make bootstrap histograms of speeds on each substrate

# subsample all bootstraps
lens = [50] #range(1,51,20)
nfig = 0
for n_boot in lens:
    print(n_boot)
#     print(n_boot)
    boot_df = []
    samplesubset = []
    samplesubset = allsamples[:,0:n_boot]
    boot_df = pd.DataFrame( {"substrate" : np.repeat(longtracks['substrate'].values, n_boot),
                           "v" : np.concatenate(samplesubset), "colony" : np.repeat(longtracks['colony'].values, n_boot)})
    
    # FIGURE FOR ALL COLONIES
    plt.figure()
    ax = sns.violinplot(x = 'substrate', y = 'v',  data = boot_df, cut = 0 )
    ax.set_ylabel('v [pix/s]')
    plt.text(-.45,2400, 'All Colonies -- # iterations: %i' % n_boot)
    plt.gca().set_ylim([0, 2500])
    
    # save figure
    pname = os.path.join(vid_locations, 'Figures','Bootstrap_All.png')
    plt.savefig(pname)
    plt.close('all')
    
    
    
    # FIGURE FOR EACH COLONY
    # *** CAN I MAKE A LIST OF COLTYPES JUST FROM DF?
    allcols = [tr['colony'] for tr in trial_info]
    coltypes = list(set(allcols))

    for coltype in sorted(coltypes):
        plt.figure()
        ax = sns.violinplot(x = 'substrate', y = 'v',  data = boot_df.loc[boot_df['colony'] == coltype], cut = 0 )
        ax.set_ylabel('v [pix/s]')
        ntr = len(boot_df.loc[boot_df['colony'] == coltype])/n_boot
        plt.text(-.45,2400, '%s -- # trials: %i --- # iterations: %i' % (coltype, ntr, n_boot))
        plt.gca().set_ylim([0, 2500])
        
        
        # SAVE THINGS?
#         pname = os.path.join(vid_locations, 'Figures', 'Bootstrap_%s.png'%(coltype))
#         plt.savefig(pname)
   # plt.close('all')
    
    
    
    
    
#     nfig = nfig +1
            
# save images as movie
# os.system("ffmpeg -r 4 -i '" + vid_locations  + "Bootstrap%02d.png' -vcodec mpeg4 -y '" 
#           + vid_locations + "Bootstrap.mp4'")

# delete all trackway vids
# pics2delete = glob.glob(os.path.join(vid_locations, 'Bootstrap*.png'))
# for pic in pics2delete:
#     os.remove(pic)

### CINNAMON COMPARISON

In [ ]:
# ** DISTRIBUTION PLOT FOR PERCENT OF TOTAL DIST TRAVELED AT EACH SPEED FOR JUST CINNAMON TRIALS - FOR RIGHT AFTER BURST VS **
def get_dist_traveled_df(x, var_OI, bins):
    glenna=x[var_OI]/pix2mm
    ordering = np.argsort(np.digitize(glenna,bins))
    ordered = glenna[ordering]
    split = np.split(ordered,np.where(np.diff(np.digitize(glenna,bins)[ordering])>0)[0]+1)
    split_sum = np.array([np.sum(x) for x in split])
    bin_idcs = np.unique(np.digitize(glenna,bins))
    bin_sums = np.zeros(bins.shape)
#     print(bin_idcs, [x[0] for x in split])
    bin_sums[bin_idcs.astype(int)] = split_sum/fps
    
    return bin_sums

longtracks = df[df['v'].map(lambda x: np.sum(np.isfinite(x))>50).values]

var_OI = 'movave_v'
sp_max = 60
precision = 10 # how many bins per 1 unit
bins = np.linspace(0,sp_max, sp_max*precision+1)
longtracks['hist_dist'] =df.apply( get_dist_traveled_df, args = (var_OI, bins), axis=1)
subtypes = sorted(list(set(df['substrate'])))

plt.close('all')
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
time_starts = [80000, 90000]
time_stops = [90000, 120000]
percent_cutoff = 0.02

plt.figure(figsize = (14,4))
for coltype in ['Tunnel_20180508-09']:#coltypes[1:2]:
    print( coltype)

    for ss,subtype in enumerate(subtypes[0:4]):
        ax = plt.subplot(1,4,ss+1)
        print(' -- ', subtype)
        sub_df = longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype)]
        
        # plot no cinnamon time
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: int(x)<90000)).values]['hist_dist']
        total_dist_traveled = np.sum(vals_OI.sum())
        hist_OI = vals_OI.sum()/total_dist_traveled
#         plt.bar(bins, hist_OI , width =1/precision, color = 'k', alpha = 0.2, align = 'edge')
        print(' -- -- total dist: %0.2f mm'%total_dist_traveled)
        kde_data = np.repeat(bins+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins)/precision
        if ss == 0:
            ref_speed = bins[np.argmax(kde_fit)]+1/(2*precision)
            plt.xlabel('speed (mm/s)')
            plt.ylabel('fraction of dist traveled')
        plt.axvline(x=ref_speed, ymin = 0, ymax = 1, color = 'k', linestyle = ':', alpha = 0.4)
        top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
        plt.axvline(x=top_percent, ymin = 0, ymax = 0.6, color = 'k', linestyle = '-', alpha = 0.4)
        plt.plot(bins[hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = 'k', alpha = 0.4)
        plt.text(30, 0.11/precision, '%i'%len(vals_OI), color = 'k', alpha = 0.4)
        plt.text(40, 0.11/precision, '%0.2f cm'%(total_dist_traveled/10), color = 'k', alpha = 0.4)
        
        # plot cinammon time far from blast
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1>=2))).values]['hist_dist']
        total_dist_traveled = np.sum(vals_OI.sum())
        hist_OI = vals_OI.sum()/total_dist_traveled
#         plt.bar(bins[:-1], hist_OI , width =1/precision, color = pltcolors[ss], alpha = 0.05, align = 'edge')
        kde_data = np.repeat(bins+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins)/precision
        plt.plot(bins[hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '--', color = pltcolors[ss], alpha = 0.5)
        top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
        plt.axvline(x=top_percent, ymin = 0, ymax = 0.6, color = pltcolors[ss], linestyle = '--', alpha = 0.5)
        plt.text(30, 0.10/precision, '%i'%len(vals_OI), color = pltcolors[ss], alpha = 0.5)
        plt.text(40, 0.10/precision, '%0.2f'%(total_dist_traveled/10), color = pltcolors[ss], alpha = 0.5)
        
        # plot cinammon time close to blast
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1<2))).values]['hist_dist']
        total_dist_traveled = np.sum(vals_OI.sum())
        hist_OI = vals_OI.sum()/total_dist_traveled
#         plt.bar(bins[:-1], hist_OI , width =1/precision, color = pltcolors[ss], alpha = 0.3, align = 'edge')
        kde_data = np.repeat(bins+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins)/precision
        plt.plot(bins[hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = pltcolors[ss])
        top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
        plt.axvline(x=top_percent, ymin = 0, ymax = 0.6, color = pltcolors[ss], linestyle = '-')
        plt.text(30, 0.09/precision, '%i'%len(vals_OI), color = pltcolors[ss], alpha = 1)
        plt.text(40, 0.09/precision, '%0.2f'%(total_dist_traveled/10), color = pltcolors[ss], alpha = 1)
        
        plt.ylim([0, 0.12/precision])
    #         plt.yscale('log')
    #         plt.ylim([.0001, 1])
        plt.xlim([0,sp_max])
        if ss>0:
            plt.gca().get_yaxis().set_visible(False)

plt.savefig('cinnamon_distance.svg', transparency = True)



# glenna=longtracks[var_OI].iloc[67]/pix2mm
# ordering = np.argsort(np.digitize(glenna,bins))
# ordered = glenna[ordering]
# split = np.split(ordered,np.where(np.diff(np.digitize(glenna,bins)[ordering])==1)[0]+1)
# split_sum = np.array([np.sum(x) for x in split])
# bin_idcs = np.unique(np.digitize(glenna,bins))
# bin_sums = np.zeros(bins.shape)
# bin_sums[bin_idcs.astype(int)] = split_sum/fps


# BAR PLOT OF CINNAMON STUFF:
plt.figure(figsize=(3,8))
cin_vals = np.full((3,4,2), np.nan)
percent_cutoffs = [0.5,0.05,0.02]
for pp, percent_cutoff in enumerate(percent_cutoffs):
    plt.subplot(3,1,pp+1)
    for ss,subtype in enumerate(subtypes[0:4]):
        sub_df = longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype)]
        
        # no cinnamon:
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: int(x)<90000)).values]['hist_dist']
        total_dist_traveled = np.sum(vals_OI.sum())
        hist_OI = vals_OI.sum()/total_dist_traveled
        top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
        cin_vals[pp, ss, 0] = top_percent
        
        # close to blast:
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1<2))).values]['hist_dist']
        total_dist_traveled = np.sum(vals_OI.sum())
        hist_OI = vals_OI.sum()/total_dist_traveled
        top_percent = np.flip(bins,0)[np.cumsum(np.flip(hist_OI,0))>percent_cutoff][0]
        cin_vals[pp, ss, 1] = top_percent
        
        plt.plot(cin_vals[pp,ss,:], (4-ss)*np.ones(2), '.', color= pltcolors[ss])
        plt.plot(cin_vals[pp,ss,:], (4-ss)*np.ones(2), '-', color= pltcolors[ss])      
        plt.xlim((0,50))
        plt.ylabel('cutoff: %0.2f'%percent_cutoff)

In [ ]:
#  ** DISTRIBUTION PLOT FOR PROPORTION OF TRIAL AT EACH SPEED FOR JUST CINNAMON TRIALS - FOR RIGHT AFTER BURST VS **
plt.close('all')
longtracks = df[df['v'].map(lambda x: np.sum(np.isfinite(x))>50).values]
subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())

var_OI = 'movave_v'
sp_max = 55
precision = 1 # how many bins per 1 unit
bins = np.linspace(0,sp_max, sp_max*precision+1)


longtracks['hist']=longtracks[var_OI].apply(lambda x: np.histogram(x/pix2mm, bins = bins)[0]/np.sum(np.isfinite(x)))

pltcolors = ['#464F56', '#BA4246', '#087E8B', '#701C6F']

time_starts = [80000, 90000]
time_stops = [90000, 120000]


plt.figure(figsize = (14,4))
for coltype in ['Tunnel_20180508-09']:#coltypes[1:2]:
    print( coltype)

    for ss,subtype in enumerate(subtypes[0:4]):
        ax = plt.subplot(1,4,ss+1)
        print(subtype)
        sub_df = longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype)]
        
        # plot no cinnamon time
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: int(x)<90000)).values]['hist']
        hist_OI = vals_OI.mean()
#         plt.bar(bins[:-1], hist_OI , width =1/precision, color = 'k', alpha = 0.2, align = 'edge')
        kde_data = np.repeat(bins[:-1]+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins[:-1])/precision
        if ss == 0:
            ref_speed = bins[np.argmax(kde_fit)]+1/(2*precision)
        plt.axvline(x=ref_speed, ymin = 0, ymax = 1, color = 'k', linestyle = ':', alpha = 0.4)
        plt.plot(bins[:-1][hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = 'k', alpha = 0.4)
        plt.text(40, 0.09, 'n: %i'%len(vals_OI), color = 'k', alpha = 0.4)
        
        
        # plot cinammon time far from blast
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1>=2))).values]['hist']
        hist_OI = vals_OI.mean()
#         plt.bar(bins[:-1], hist_OI , width =1/precision, color = pltcolors[ss], alpha = 0.05, align = 'edge')
        kde_data = np.repeat(bins[:-1]+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins[:-1])/precision
        plt.plot(bins[:-1][hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '--', color = pltcolors[ss], alpha = 0.5)
        plt.text(40, 0.08, 'n: %i'%len(vals_OI), color = pltcolors[ss], alpha = 0.5)
        
        # plot cinammon time close to blast
        vals_OI = sub_df.loc[(sub_df['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1<2))).values]['hist']
        hist_OI = vals_OI.mean()
#         plt.bar(bins[:-1], hist_OI , width =1/precision, color = pltcolors[ss], alpha = 0.3, align = 'edge')
        kde_data = np.repeat(bins[:-1]+1/(2*precision),np.round((hist_OI*10000)).astype(int))
        kde = stats.gaussian_kde(kde_data)
        kde_fit = kde.evaluate(bins[:-1])/precision
        plt.plot(bins[:-1][hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = pltcolors[ss])
        plt.text(40, 0.07, 'n: %i'%len(vals_OI), color = pltcolors[ss], alpha = 1)
        
        
        plt.ylim([0,0.12/precision])
#         plt.yscale('log')
#         plt.ylim([.0001, 1])
        plt.xlim([0,55])

        if ss==0:
            plt.xlabel('windowed moving ave vel. (mm/s)')
            plt.ylabel('fraction of recorded time')
            
del longtracks, sub_df, kde, kde_fit, ref_speed, kde_data, vals_OI, hist_OI

In [ ]:
# FIND PROPORTION OF FRAMES AT EACH SPEED FOR COLONY BEFORE AND AFTER PERTURBATION
plt.close('all')
longtracks = df[df['v_final'].map(lambda x: np.sum(np.isfinite(x))>50).values]
subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())


sp_max = 2000
ac_max = 150000
n_bins = int(1800/20)
data_for_hist = []
n_precision = 10000 # 100: one data point per 1% of time at that speed, 1000: one data point per 0.1% of time at that speed
n_col = 8

time_starts = [80000, 90000]
time_stops = [90000, 120000]

filmed_time = {}
for timerange in range(0,2):

    for coltype in ['Tunnel_20180508-09']:#coltypes[1:2]:
        print( coltype)

        for ss,subtype in enumerate(subtypes[0:4]):

            counts_all = []
            tmp = dict()
            
            # compile proportion of time in each velcoity bin for 
            number_of_frames = longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype) & 
                           (pd.to_numeric(longtracks['time'])> time_starts[timerange])& 
                           (pd.to_numeric(longtracks['time'])< time_stops[timerange])]['movave_v'].apply(lambda x: len(x)).sum()
            filmed_time[timerange, ss] = number_of_frames/fps
            print('timerange %i - %s - %i frames - %0.1f s'%(timerange, subtype, number_of_frames, filmed_time[trange, ss]))
            for k,track in longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype) 
                                         & (pd.to_numeric(longtracks['time'])> time_starts[timerange])
                                         & (pd.to_numeric(longtracks['time'])< time_stops[timerange])].iterrows():
                counts, _ = np.histogram(track['movave_v'], bins=n_bins, range=(0,sp_max)) # average velocity over moving window
#                 counts, _ = np.histogram(track['v_final'], bins=n_bins, range=(0,sp_max)) # instantaneous velocity
#                 counts, _ = np.histogram(np.diff(track['v_final'])*240/np.diff(track['frames_final']), bins=n_bins, range=(0,ac_max)) # instantaneous accel
                counts_all.append(counts/counts.sum())

            counts_ave = np.mean(counts_all, axis=0)


            # how much precision would you like (hundredths = *100)
            vp_vs = np.repeat(np.linspace((sp_max/n_bins)/2,sp_max-(sp_max/n_bins)/2,n_bins), np.round(counts_ave*n_precision).astype(int)) # for velocities
#             vp_vs = np.repeat(np.linspace((ac_max/n_bins)/2,ac_max-(ac_max/n_bins)/2,n_bins), np.round(counts_ave*n_precision).astype(int)) # for accelerations
            tmp['substrate'] = subtype
            tmp['colony'] = coltype
            tmp['time'] = timerange
            tmp['velocity'] = vp_vs
            tmp['minute'] = 
    #         print(len(vp_vs))

            data_for_hist.append(tmp)


    tmp_df = pd.DataFrame(data_for_hist)
    vp_df = pd.DataFrame( {"substrate" : np.repeat(tmp_df['substrate'].values, [len(a) for a in tmp_df['velocity']]),
                            "velocity" : np.hstack(tmp_df['velocity']), 
                           "colony" : np.repeat(tmp_df['colony'].values, [len(a) for a in tmp_df['velocity']]),
                          "timerange" : np.repeat(tmp_df['time'].values, [len(a) for a in tmp_df['velocity']]),
                          })
    

# pre- and during perturbations for colonies 9+
plt.figure(figsize = (5,7))
ymax = 0.004
pltcolors = ['r','g','c','b']

for tt, trange in enumerate([1,0]):
    for coltype in ['Tunnel_20180508-09']:
        ax=plt.axes([0.15, 0.1+0.45*tt, 0.8, 0.35])
        
        comparison_curve,_ = np.histogram(vp_df.loc[
            (vp_df['substrate']=='0mm') & (vp_df['colony']==coltype) & (vp_df['timerange']==0)]['velocity'].values,
                                        bins = np.linspace(0,sp_max,n_bins+1))
        bins = np.linspace(0,sp_max,n_bins+1)
        flat_cutoff = np.argmax(comparison_curve)
        plt.axvline(x=bins[flat_cutoff], ymin =0, ymax =10, c='r', ls=':')
        plt.text(1300, 0.0022, '% above cutoff:', color = 'k')
        plt.text(1300, 0.0036, 'max vel (mm/s):', color = 'k')

        for ss,subtype in enumerate(subtypes[0:4]):
            # FOR VELOCITY
            data_OI = vp_df.loc[(vp_df['substrate']==subtype) & (vp_df['colony']==coltype)
                                  & (vp_df['timerange']==trange)]['velocity']
            sns.distplot(data_OI, bins = bins, label = '%s'%subtype, color = pltcolors[ss], kde_kws={'cut': 0})
            curve,_ = np.histogram(data_OI.values, bins = bins)
            curve_max = np.max(data_OI.values)
            percent_above_cutoff = np.sum(curve[flat_cutoff:])/np.sum(curve)*100
            plt.axvline(x=curve_max, ymin = 0, ymax = 0.1, c=pltcolors[ss], ls='-')
            plt.text( 1300, 0.002-0.0002*ss, '%s - %0.1f = %0.1f s'%(subtype, percent_above_cutoff, 
                                                                   percent_above_cutoff/100*filmed_time[trange,ss]), color = pltcolors[ss] )
            plt.text( 1300, 0.0034-0.0002*ss, '%s - %0.1f'%(subtype, curve_max/pix2mm), color = pltcolors[ss] )
            
            # FOR ACCEL
#             sns.distplot(vp_df.loc[(vp_df['substrate']==subtype) & (vp_df['colony']==coltype)
#                                   & (vp_df['timerange']==trange)]['velocity'], 
#                          bins = np.linspace(0,ac_max,n_bins+1), label = '%s'%subtype)
        plt.title('%i to %i'%(time_starts[trange]/100, time_stops[trange]/100), loc = 'left')
        
#         plt.ylim((0,ymax))
#         plt.xlim((0,sp_max))
#         plt.gca().set_xticks(np.arange(0,1600, step = 500))
#         plt.gca().set_xticklabels(np.arange(0,1600, step = 500))
#         plt.gca().set_yticklabels(np.arange(0,ymax + 0.001, step = 0.001)*100)
#         plt.gca().set_yticks(np.arange(0,ymax + 0.001, step = 0.001))
        
        # in mm/s
        plt.ylim((0,ymax))
        plt.xlim((0,sp_max))
        plt.gca().set_xticks(np.arange(0,sp_max, step = 10*pix2mm))
        plt.gca().set_xticklabels(np.arange(0,sp_max/pix2mm, step =  10))
        plt.gca().set_yticklabels(np.arange(0,ymax + 0.001, step = 0.001)*100)
        plt.gca().set_yticks(np.arange(0,ymax + 0.001, step = 0.001))
        if tt ==0:
            plt.xlabel('moving window ave. velocity (mm/s)')
        else:
            plt.xlabel('')
        
        
        # accel in mm/s2
#         pix2mm = 1000/32
#         plt.ylim((0,0.00007))
#         plt.xlim((0,ac_max))
#         plt.gca().set_xticks(np.arange(0,ac_max, step = 1000*pix2mm))
#         plt.gca().set_xticklabels(np.arange(0,ac_max/pix2mm/1000, step =  1))
#         plt.xlabel('ac (m/s2)')
        
    
        
        
        plt.ylabel('Percent of time')
#         plt.gca().set_yscale('log')
#         plt.ylim((1e-7,1e-2))

# del longtracks

## Plot velocity profile for given trial

takes the longest track from a given trial
finds points that are within ROI
plot points on first frame of video, outside of ROI = blue, inside ROI = colored by total speed
plots velocity profile and histogram of inside ROI points

part 2 plots ave dist or speed within a window and uses that to identify "moving points"


In [ ]:
# PLOT VELOCITY PROFILE FROM GIVEN TRACK

tr_num = 405# 593#105 #91 #105
videofile = df.video[tr_num]
cap = cv2.VideoCapture(videofile)
cap.set(1,int(df.frames_final[tr_num][0])+1)
ret, frame = cap.read()
wid = frame.shape[1]
hei = frame.shape[0]
buffer = 60

if not ret:
    print('what happened?')
    
# draw ROI on image
cv2.rectangle(frame, (0+buffer-1,0+buffer-1), (frame.shape[1]-buffer-1,frame.shape[0]-buffer-1),
              (255,255,255), thickness=2, lineType=8, shift=0)

# set figure features/colors
plt.figure()
pix2mm = 1000/32
# plt.set_cmap('cool')
# temp=track[longest_trial]['frames']/np.amax(track[longest_trial]['frames'])
temp=np.array(df.v[tr_num])/pix2mm  #/np.amax(track[longest_trial]['vfilt'])
# temp = np.append(temp, temp[len(temp)-1])
atemp = np.diff(df.v[tr_num])/(np.diff(df.frames_final[tr_num]))*240/pix2mm # acceleration
norm2 = colors.Normalize(vmin=0, vmax = 50)

# plot trajectory on top of image of first frame
ax1=plt.axes([0.1, 0.53, 0.9, 0.4])
plt.title(videofile[51::])
plt.imshow(frame)
sc=plt.scatter(df.x[tr_num], df.y[tr_num], cmap=cm.cool, #'-' ,
         c=temp, edgecolor='none', norm=norm2)
plt.scatter(df.x[tr_num][1], df.y[tr_num][1], s=15, c="r",
        edgecolor='none')
plt.xlim((0,wid-1));
plt.ylim((0,hei-1));
plt.gca().invert_yaxis()
plt.gca().set_aspect('equal')
plt.colorbar(sc)

# plot speed vs. frame
ax2=plt.axes([0.12, 0.1, 0.6, 0.35])
# plot moving points rectangles
# pc = PatchCollection(rects, facecolor = 'r', alpha = 0.3, edgecolor = 'r')
# ax2.add_collection(pc)
plt.plot(df.frames_final[tr_num], temp, '-k');
plt.scatter(df.frames_final[tr_num], temp, #'-' ,
         cmap=cm.cool, c=temp, edgecolor='none', norm=norm2)
# plt.plot(df.frames_final[tr_num], df.loc[tr_num].dist_90fr*(240/90),'.r', markersize = 1)
plt.plot(df.frames_final[tr_num], np.array(df.loc[tr_num].v_movave)/pix2mm,'-r', markersize = 1)
# plt.scatter(track[longest_trial]['frames'][1], track[longest_trial]['vfilt'][1], s=15, c="r",
#         edgecolor='none')
plt.ylabel('speed (mm/s)')
# plt.ylim((-20,35));
plt.show()


# # acceleration plot
# ax3 = ax2.twinx()
# plt.plot(df.frames_final[tr_num][:-1], atemp, '-g');
# plt.ylim((-2000,3500));

In [ ]:
# PLOT VELOCITY PROFILE OF LONGEST TRACK IN GIVEN VIDEO
# Tests out methods of finding ROI points and moving points
plt.close('all')
import cv2

#which video to look at?
# 85 = good steady walking on 0mm
# 550 = shows stopping and cut-off
# 723 = 3 mm walking no stopping
# 293 = steady 1 mm walking
vid_num = 550
# track = tracks[vid_num]
videofile = file_list[vid_num]
# videofile = np.unique(df['video'].values)[vid_num]
track = df[df['video']==videofile]
print(videofile)

# how long are the trials?
# lengths = [len(obj['x']) for k, obj in track.items()] # using variables generated (takes a long time to read in data vs. pickle of dataframe)
lengths = [len(obj['x']) for index, obj in track.iterrows()] # using dataframe
print('trial lengths: ', lengths)
longest_trial = np.argmax(np.array(lengths))
print('longest trial: ', longest_trial)
# longest_trial = 1
# trackOI = track[longest_trial] # using variable
trackOI = track.loc[track.index[0]] # using dataframe
print('len of track: ', len(trackOI['frames']))

# load video for first frame
cap = cv2.VideoCapture(videofile)
cap.set(1,trackOI['frames'][0]+1)
ret, frame = cap.read()
wid = frame.shape[1]
hei = frame.shape[0]
if not ret:
    print('what happened?')
    
# find which frames are close to edge
buffer = 60 
edgeidcs = ((np.array(trackOI['x_raw']) < buffer) | (np.array(trackOI['x_raw']) > wid-(buffer+20)) | 
            (np.array(trackOI['y_raw']) < buffer) | (np.array(trackOI['y_raw']) > hei-buffer))
ROIidcs = ~edgeidcs

# draw ROI on image
cv2.rectangle(frame, (0+buffer-1,0+buffer-1), (frame.shape[1]-buffer-20-1,frame.shape[0]-buffer-1),
              (255,255,255), thickness=2, lineType=8, shift=0)
    
    
# FIND MOVING POINTS
ROIpts = trackOI['vfilt'][ROIidcs[0:-1]]
ROIfrs = trackOI['frames'][0:-1][ROIidcs[0:-1]]
Mov_val = []
Mov_allfr = []


# DISTANCE METHOD

# variables
b_wid = 10
b_sep = 40
dist_thres = 50 # pix/window - ~90 pix is 2 mm

for fr in ROIfrs:
    fr = int(fr)
    
    if fr - b_sep < 0:
        continue
        
    # what frames do we want to measure position in
    box1frs = np.array(range(int(fr) - b_wid - b_sep, int(fr) - b_sep))
    box2frs = np.array(range(int(fr) + b_sep, int(fr) + b_sep + b_wid ))
    
    # if box frames aren't in track, do not keep point
    box1idcs = np.isin(trackOI['frames'], box1frs) #np.searchsorted(trackOI['frames'], box1frs)
    box2idcs = np.isin(trackOI['frames'], box2frs) 
    if ~np.any(box1idcs) | ~np.any(box2idcs):
        # print('one of box frames is not in track - removing fr ', fr)
        continue

    # find ave location of ant in time box
    box1x = np.mean(np.array(trackOI['x_raw'])[box1idcs])
    box1y = np.mean(np.array(trackOI['y_raw'])[box1idcs])
    box2x = np.mean(np.array(trackOI['x_raw'])[box2idcs])
    box2y = np.mean(np.array(trackOI['y_raw'])[box2idcs])
    dist_traveled = np.sqrt(np.power(box1x-box2x,2) + np.power(box1y-box2y,2))
    
    Mov_allfr = np.append(Mov_allfr, fr)
    Mov_val = np.append(Mov_val, dist_traveled)

Mov_fr = Mov_allfr[Mov_val>dist_thres] # frs when measure is above threshold
Mov_val_keep = Mov_val[Mov_val>dist_thres] # vals when measure is above threshold
Mov_idc = np.array([i for i, x in enumerate(trackOI['frames']) if x in Mov_fr]) # where in original data are moving points
Mov_v = np.array([trackOI['vfilt'][i] for i in Mov_idc])

    
# MEDIAN SPEED METHOD
wMov_val = []
wMov_fr = []

# variables
win_wid = 12 # actually half the width
nfr_cutoff = 20
speed_cutoff = 2 # pix/win

for fr in ROIfrs:
    fr = int(fr)
    winfrs = np.array(range(fr - win_wid, fr + win_wid))
    winidcs = np.isin(trackOI['frames'], winfrs)
    
    if np.sum(winidcs) < nfr_cutoff:
        continue   
    win_medspeed = np.median(trackOI['vfilt'][winidcs[0:-1]])
    win_meanspeed = np.mean(trackOI['vfilt'][winidcs[0:-1]])
#     print(fr, ' - ', win_medspeed)
    
    wMov_fr = np.append(wMov_fr, fr)
#     wMov_val = np.append(wMov_val, win_medspeed)
    wMov_val = np.append(wMov_val, win_meanspeed)
wMov_val_keep = wMov_val[[wfr in Mov_fr for wfr in wMov_fr]] # vals when measure is above threshold
wMov_fr_keep = wMov_fr[[wfr in Mov_fr for wfr in wMov_fr]] # frs when measure is above threshold

    
# Find rectangles of when measurement is above threshold
rects = []
if len(Mov_fr)>2:
    temp = np.diff(Mov_fr) > 1
    find_stops = np.append(temp, True)
    find_starts = np.append(np.array([True]),temp)
    rect_starts = Mov_fr[find_starts]
    rect_stops = Mov_fr[find_stops]
else:
    rect_starts = []
    rect_stops = []

for kk,rst in enumerate(rect_starts):
#         print(kk)
#         print(rst)
#         print(rect_stops[kk]-rst)
    rect = Rectangle([rst, 0], (rect_stops[kk]-rst), 40, angle=0)
    rects.append(rect)
    

    
    
    
# PLOT THINGS

# set figure features/colors
plt.figure()
# plt.set_cmap('cool')
# temp=track[longest_trial]['frames']/np.amax(track[longest_trial]['frames'])
temp=trackOI['vfilt']#/np.amax(track[longest_trial]['vfilt'])
temp = np.append(temp, temp[len(temp)-1])
norm2 = colors.Normalize(vmin=0, vmax = 40)

# PANEL A
# plot trajectory on top of image of first frame
ax1=plt.axes([0.1, 0.53, 0.8, 0.4])
plt.title(videofile[62::])
plt.imshow(frame)
# plt.plot(track[longest_trial]['xfilt'][edgeidcs], track[longest_trial]['yfilt'][edgeidcs], '-')
plt.scatter(np.array(trackOI['x_raw'])[edgeidcs], np.array(trackOI['y_raw'])[edgeidcs], s=2, c=(0.1,0.1,0.6),
        edgecolor='none')
sc=plt.scatter(np.array(trackOI['x_raw'])[ROIidcs], np.array(trackOI['y_raw'])[ROIidcs], cmap=cm.cool, s=6, #'-' ,
         c=temp[ROIidcs]/pix2mm, edgecolor='none', norm=norm2)
plt.scatter(trackOI['x_raw'][1], trackOI['y_raw'][1], s=15, c="y",
        edgecolor='none')
plt.xlim((0,wid-1));
plt.ylim((0,hei-1));
plt.gca().invert_yaxis()
plt.gca().set_aspect('equal')
plt.colorbar(sc)


# PANEL B
# plot speed vs. frame
ax2=plt.axes([0.08, 0.1, 0.6, 0.35])
# plot moving points rectangles
pc = PatchCollection(rects, facecolor = 'r', alpha = 0.1, edgecolor = [])
ax2.add_collection(pc)
# plot all data within ROI
# plt.scatter(track[longest_trial]['frames'][0:-1][ROIidcs[0:-1]], track[longest_trial]['vfilt'][ROIidcs[0:-1]], #'-' ,
#          cmap=cm.cool, c=temp[0:-1][ROIidcs[0:-1]], edgecolor='none', norm=norm2)
# plot only moving points
plt.scatter(Mov_fr, Mov_v/pix2mm, s=20, #'-' ,
         cmap=cm.cool, c=Mov_v/pix2mm, edgecolor='none', norm=norm2)
plt.plot(trackOI['frames'][0:-1], trackOI['vfilt']/pix2mm, '-k', alpha = 0.5);
plt.scatter(trackOI['frames'][0], (trackOI['vfilt']/pix2mm)[0], s=15, c="y",
        edgecolor='none')
plt.ylabel('speed (mm/s)')
plt.ylim((0,40));
plt.show()

# Plot moving point measures - dist traveled/as speed
ax3 = ax2.twinx()
ax3.set_ylim((0,40/(240/90)));
ax3.spines['right'].set_color('red')
ax3.tick_params(axis= 'y', colors = 'red')
ax3.yaxis.label.set_color('red')
# plot as dist_traveled
plt.scatter(Mov_allfr, Mov_val/pix2mm, s=3, c='r', edgecolor = 'None',alpha = 0.3)
plt.scatter(Mov_fr, Mov_val_keep/pix2mm, s=3, c='r', edgecolor = 'None')
plt.ylabel('net mm traveled/0.375s')

# plot as speed
# plt.scatter(Mov_allfr, Mov_val*(240/90), s=3, c='r', edgecolor = 'None')
# plt.ylabel('net dist traveled per 1s [pix]')
# plt.ylim((0,1500));

# plot moving point measure - median/mean speed 
plt.sca(ax2)
plt.plot(wMov_fr, wMov_val/pix2mm, 'g', alpha = 0.3)
plt.scatter(wMov_fr_keep, wMov_val_keep/pix2mm, s=3, c='g', edgecolor = 'None')

if len(Mov_v)>1:
    # plot violine plot
    # plt.subplots_adjust(bottom = 0.1, right = 0.8, top = 0.9)
    hax =plt.axes([0.75, 0.1, 0.1, 0.35])
    
    # track[longest_trial]['vfilt'][ROIidcs[0:-1]]
    hax.violinplot(Mov_v/pix2mm, showmeans = True, showmedians = True)
    hax.get_xaxis().set_visible(False)
    hax.get_yaxis().set_visible(False)
    plt.ylim((0,40));
#     hax.spines['right'].set_visible(False)
#     hax.spines['top'].set_visible(False)
#     hax.spines['bottom'].set_visible(False)
#     hax.set_yticklabels([])
#     hax.tick_params(direction = 'in')
    hax.set_frame_on(False)
    
    
    # average/median speed violin plot
    haxME =plt.axes([0.8, 0.1, 0.1, 0.35])
    parts = haxME.violinplot(wMov_val_keep/pix2mm, showmeans = True, showmedians = True)
    for pc in parts['bodies']:
        pc.set_facecolor('green')
    for partname in ('cbars','cmins','cmaxes','cmeans','cmedians'):
        pc = parts[partname]
        pc.set_edgecolor('green')
    haxME.get_xaxis().set_visible(False)
    haxME.get_yaxis().set_visible(False)
    haxME.set_frame_on(False)
    plt.ylim((0,40));
    haxME.patch.set_alpha(0)
    
    # dist traveled violin plot
    haxMA =plt.axes([0.85, 0.1, 0.1, 0.35])
    parts = haxMA.violinplot(Mov_val_keep/pix2mm, showmeans = True, showmedians = True)
    for pc in parts['bodies']:
        pc.set_facecolor('red')
    for partname in ('cbars','cmins','cmaxes','cmeans','cmedians'):
        pc = parts[partname]
        pc.set_edgecolor('red')
    haxMA.get_xaxis().set_visible(False)
#     haxMA.get_yaxis().set_visible(False)
    haxMA.spines['left'].set_visible(False)
    haxMA.spines['top'].set_visible(False)
    haxMA.spines['bottom'].set_visible(False)
    haxMA.tick_params(direction = 'in')
    haxMA.get_yaxis().tick_right()
    haxMA.set_yticklabels([])
#     haxMA.set_frame_on(False)
    plt.ylim((0,40/(240/90)));
    haxMA.patch.set_alpha(0)

In [ ]:
# plot median speed vs. time
# import matplotlib.dates as mdates
# plt.figure(figsize=(16,6))
# plt.plot(df.datetime, df.median_v, '.k')
# alldates = [tr.date() for tr in df.datetime];
# dates = sorted(list(set(alldates)))
# lightson = [datetime.datetime.combine(d, datetime.time(6,59)) for d in dates]
# lightsoff = [datetime.datetime.combine(d, datetime.time(19,9)) for d in dates]
# lightsoff = [datetime.datetime.combine(dates[0], datetime.time(0,0))] + lightsoff
# lightson = lightson + [datetime.datetime.combine(dates[-1], datetime.time(23,59)) ]

# for loff,lon in zip(lightsoff,lightson):
# #     print(loff, lon)
#     plt.axvspan(loff,lon,facecolor='k',alpha = 0.2)

# plt.tight_layout()
# plt.ylabel('median v (pix/s)')
# plt.text(datetime.datetime(2018,2,20,19,9), 1150,'webcam wont\ntrigger\nin dark', horizontalalignment='left')
# plt.text(datetime.datetime(2018,2,21,9,30), 1150,'cant record\nwhile\nworking', horizontalalignment='left')
# plt.text(datetime.datetime(2018,2,22,9,30), 1150,'short recording\nsessions\nwhile working', horizontalalignment='left')
# plt.text(datetime.datetime(2018,2,22,19,9), 950,'fixed\nnighttime\ntriggering', horizontalalignment='left')
# plt.text(datetime.datetime(2018,2,22,23,0), 1150,'computer\nfroze\novernight', horizontalalignment='left')
# plt.text(datetime.datetime(2018,2,23,13,0), 1150,'wont trigger\nuntil after \n3min pause', horizontalalignment='left')

# JOINT TRAJECTORIES

#### Plot video of tracking on raw footage <br>

In [9]:
# plot images with tracked data on it
tr_num = 105#105 #91 #105
videofile = df.video[tr_num]
print(videofile)
cap = cv2.VideoCapture(videofile)
plt.close('all')

def WRTant_to_WRTframe(val_x, val_y, frame_center_x, frame_center_y, ant_ang_deg):
    ant_ang = ant_ang_deg *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    rotated_vals = np.dot(R,np.array([val_x-100,val_y-100]))
    translated_vals = rotated_vals*np.array([1,1]) + np.array([frame_center_x, frame_center_y])  
    return translated_vals[0], translated_vals[1];

def plot_ant_pt(ant_part, ant_part_num, df, tr_num, fr_num, ant_x, ant_y, ant_ang_deg, buffer):
    x = df['%s%i_x'%(ant_part,ant_part_num)][tr_num][fr_num]
    y = df['%s%i_y'%(ant_part,ant_part_num)][tr_num][fr_num]
    conf = df['%s%i_conf'%(ant_part,ant_part_num)][tr_num][fr_num]
    (newx, newy) = WRTant_to_WRTframe(x, y, ant_x, ant_y, ant_ang_deg)
#     print('old vals: %i, %i  TO %0.1f, %0.1f'%(x,y,newx, newy))

    # define colormap to show confidence
    norm2 = colors.Normalize(vmin=0, vmax=1)
    plt.scatter(newx+buffer, newy+buffer, c = conf, s = 10, cmap = cm.bwr,
               edgecolor = 'none', norm=norm2)# '.g')
    return;

def save_image(vlocation, nfig, name_base):
    pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
    plt.savefig(pname)
    nfig = nfig + 1
    plt.pause(0.2)
#     plt.close('all')
    return nfig

def save_video(vlocation, name_base):
    # save images as movie
    if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
        os.remove(vlocation + "/%s.mp4"%name_base)
        print('** Deleted %s.mp4 file'%name_base)
    print('saving %s.mp4 file'%name_base)
    command_p1 = "ffmpeg -r 20 -i '%s/%s"%(vlocation, name_base)
    command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
    command = command_p1 + "%01d.png'" + command_p2
#     print(command)
    os.system(command)
    plt.pause(10)

    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
    for pic in pics2delete:
        os.remove(pic)
    return


fig = plt.figure(figsize=(10,6))
for im_n, fr_OI in enumerate( df.frames_final[tr_num]):
    plt.clf()
    # load frame
    ff=np.where(df.frames[tr_num]==fr_OI)[0][0]
    cap.set(1,int(fr_OI))
    ret, frame = cap.read()
    x_dim = frame.shape[1]
    y_dim = frame.shape[0]
    
    # load ant x, y and angle
    x = df.x_raw[tr_num][ff]
    y = df.y_raw[tr_num][ff]
    ang = df.angle_improved[tr_num][ff]
    (thorax_x, thorax_y) = WRTant_to_WRTframe(df.thorax_x[tr_num][ff], df.thorax_y[tr_num][ff], x, y, ang)
#     print(x,y,ang)
    
    
    
    
    # PLOT THINGS
    ax1=fig.add_axes([0.03,0.1, 0.5, 0.3])
    ax1.set_position([0.03,0.1, 0.5, 0.3])
    plt.imshow(frame)
    plt.plot(x, y, '.w')
    plt.axis('off')
    
    ax2=fig.add_axes([0.35,0.1, 0.7, 0.7]) #plt.axes()
#     ax2.set_position([0.35,0.1, 0.7, 0.7], which = 'both')
    # zoom into around ant
    buffer = 150
    blank_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.uint8)* 1# 1.001# np.max(temp) # gray background  1.0001#
    bframe = blank_frame.copy()
    bframe[buffer:-buffer, buffer:-buffer,:] = frame
    if not np.isnan(ang):
        xrange = range(int(round(thorax_x)), int(round(thorax_x+2*buffer)))
        yrange = range(int(round(thorax_y)), int(round(thorax_y+2*buffer)))
    else:
        xrange = range(int(round(x)), int(round(x+2*buffer)))
        yrange = range(int(round(y)), int(round(y+2*buffer)))
    xrange_actual = np.array(sorted(list( set(xrange) & set(range(0, x_dim+2*buffer) ) )))[[0,-1]]
    yrange_actual = np.array(sorted(list( set(yrange) & set(range(0, y_dim+2*buffer) ) )))[[0,-1]]
    frame_zoom = bframe[np.ix_(yrange_actual, xrange_actual)]
    plt.xlim(xrange_actual)
    plt.ylim(yrange_actual)
    plt.text(xrange_actual[0]+20, yrange_actual[0]+20, 'Fr: %i'%fr_OI, color='w')
    plt.imshow(bframe)
    cmap = cm.bwr
    plt.scatter(x+buffer, y+buffer, s=20, c=np.array(0.5), 
            norm = colors.Normalize(vmin=0, vmax=1), marker= 'o')
    
    
    if not np.isnan(ang):
        plt.scatter(thorax_x+buffer, thorax_y+buffer, c = df.thorax_conf[tr_num][ff], s = 10, 
                cmap = cmap, norm = colors.Normalize(vmin=0, vmax=1))
        for jj in range(0,6):
            plot_ant_pt('joint',jj, df, tr_num, ff, x, y, ang, buffer)
        for aa in range(0,2):
            plot_ant_pt('antenna',aa, df, tr_num, ff, x, y, ang, buffer)
            
            
    
    
    
    plt.gca().invert_yaxis()
    plt.axis('off')
    cax = plt.axes([0.93,0.1,0.02,0.7])
    plt.colorbar(cax=cax)
    plt.clim(0,1)
    plt.set_cmap(cm.bwr)
    
    
    vlocation = '/media/gravishlab/SeagateExpansionDrive/AntTrack'
    save_image(vlocation, im_n, 'Model_Predictions')
save_video(vlocation, 'Model_Predictions')
    #     plt.pause(0.1)
    
    
    
    

    
    
plt.close()

/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_101027_16276736-0000.mp4
** Deleted Model_Predictions.mp4 file
saving Model_Predictions.mp4 file


### Define functions to analyze tracked data, removing low confidence points and lowpass filter -- apply to individual trial/limb

In [441]:
# INTERPOLATING AND LOWPASS FILTERING FUNCTIONS

def remove_lowconf_pts(arr, conf, conf_cutoff, jump_limit):
    arr_highconf = arr.copy()
    # get rid of low confidence pts
    arr_highconf[conf<conf_cutoff]=np.nan
    return arr_highconf

def remove_jumps(arr, jump_limit):
    yy = np.isnan(arr)
    xx = range(len(yy))
    arr_nojump = np.empty(yy.shape)*np.nan
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == False: # if is not a group of nan
            g = np.array(list(g))
#             print(len(g))
            if len(g)>3:
                # get rid of drastic changes
                if jump_limit != None:
                    arr_OI = arr[g]
                    d_arr = np.abs(np.diff(arr_OI))
                    d_jump = np.abs(d_arr) > jump_limit
                    d_jump_cumsum = np.cumsum(np.insert(d_jump,0,0))
                    d_jump_opp = (d_jump_cumsum%2).astype(bool)
                    if np.sum(d_jump_opp==True)>np.sum(d_jump_opp == False):
                        d_jump_opp = np.logical_not(d_jump_opp)
                    arr_OI[d_jump_opp]=np.nan
                arr_nojump[g]=arr_OI
                    
    return arr_nojump
    


def middle_half(alist, wanted_parts=4):
    alist= alist[np.logical_not(np.isnan(alist))]
    length = len(alist)
    sections = np.array([ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ])
    middle_half = np.concatenate(sections[1:3])
    if not len(middle_half)>0:
        return  np.nan, np.nan
    else:
        full_range = (np.max(alist)-np.min(alist))
        middle_range = (np.max(middle_half)-np.min(middle_half))
        med = np.mean(middle_half)
#         print(middle_range/full_range)
        if middle_range/full_range < 0.93: # theoretically for normal distribution mid_range/full_range = 0.16625
#             print('not  normal dist')
            sigma = middle_range/3
#             print('removed outliers')
        else:
            sigma = full_range/4
#         print(med, sigma)
    return med, sigma

def remove_outliers(arr):
    med, sigma = middle_half(arr)
    where_far_away = (np.abs(arr-med)> 2*sigma)
    arr[where_far_away]=np.nan
    return arr

def remove_outliers2d(arr_x, arr_y):
    med_x, sigma_x = middle_half(arr_x)
    med_y, sigma_y = middle_half(arr_y)
#     if sigma_x < 15:
#         sigma_x = 15
    where_far_away = np.logical_or(np.abs(arr_x-med_x)> 2*sigma_x, np.abs(arr_y-med_y)> 2*sigma_y)
#     print('removing %i outliers'%np.sum(where_far_away))
    arr_x[where_far_away]=np.nan
    arr_y[where_far_away]=np.nan
    return arr_x, arr_y

def remove_outliers(arr_x, arr_y):
    med_x, sigma_x = middle_half(arr_x)
    med_y, sigma_y = middle_half(arr_y)
    
    where_far_away = np.logical_or(np.abs(arr_x-med_x)> 2*sigma_x, np.abs(arr_y-med_y)> 2*sigma_y)
    arr_x[where_far_away]=np.nan
    arr_y[where_far_away]=np.nan
    
    return arr_x, arr_y

def find_nan_gaps(arr, limit):  
    yy = np.isnan(arr)
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
#                 print('remove: ', len(g), g)
                continue
                
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
#                 print('interp: ', len(g), g)
            else:
                where_othergaps[np.array(g)]=True
#                 print('remove: ', len(g), g)
    return where_gapOI, where_othergaps

def find_interp_idcs(where_interpolate):
    interp_idcs = []
    for val in [-1,0,1]:
        interp_idcs = np.concatenate([interp_idcs,np.where(where_interpolate)[0]+val])
    interp_idcs = np.sort(np.array(list(set(interp_idcs)))) # get of repeat elements
    interp_idcs = interp_idcs[np.logical_and(interp_idcs>-1, interp_idcs < len(where_interpolate))].astype(np.uint32) # only elements in range
    return interp_idcs

def interp_vals(arr, interp_idcs): # array includes nan values
    interp = arr.copy()
    if len(interp_idcs)>0:
        temp = arr[interp_idcs]
        interpolated_vals = np.interp(
            interp_idcs, 
            interp_idcs[np.logical_not(np.isnan(temp))], temp[np.logical_not(np.isnan(temp))] )
        interp[interp_idcs] = interpolated_vals
    return interp
    
def lowpass_filt_sections(arr):
    yy = np.isnan(arr)
    xx = range(len(yy))
    full_filtered = np.empty(yy.shape)*np.nan
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == False: # if is a group of nan
            g = list(g)
#             print('section to lowpass fitler: ', len(g))
            if len(g)>9:
                b, a = signal.butter(2,0.3,btype='low')
                filtered = signal.filtfilt(b, a, arr[np.array(g)])
                full_filtered[np.array(g)]=filtered
    return full_filtered



def interpolate_filter_tracking(df, tr_num, tracked_pt, conf_cutoff, jump_limit, nan_gap_limit, plots = False):
    
    all_frames = df['frames'][tr_num]
    frames_OI = df['frames_final'][tr_num]
    frames_OI_idcs = np.isin(all_frames, frames_OI)
    
    # initialize variables
    joint_x, joint_y, x_offset, y_offset, conf, \
    joint_x_highconf, joint_y_highconf, joint_x_interp, joint_y_interp, joint_x_filt, joint_y_filt = \
    (np.empty(all_frames.shape)*np.nan for i in range(11))
    
    # set up variables
    joint_x = df['%s_x'%tracked_pt][tr_num]
    joint_y = df['%s_y'%tracked_pt][tr_num]
    if tracked_pt != 'thorax': # account for inaccurate initial guess of body center, make relative to LEAP tracked thorax
        if 'thorax_x_filt' in df:
            x_offset = df['thorax_x_filt'][tr_num]
            y_offset = df['thorax_y_filt'][tr_num]
        else:
            x_offset = df['thorax_x'][tr_num]
            y_offset = df['thorax_y'][tr_num]
        joint_x = joint_x - x_offset
        joint_y = joint_y - y_offset
    conf = df['%s_conf'%tracked_pt][tr_num]
    joint_x_highconf = remove_lowconf_pts(joint_x, conf, conf_cutoff, jump_limit)
    joint_y_highconf = remove_lowconf_pts(joint_y, conf, conf_cutoff, jump_limit)
    
    # remove big jumps
    joint_x_highconf = remove_jumps(joint_x_highconf, jump_limit)
    joint_y_highconf = remove_jumps(joint_y_highconf, jump_limit)
    
    # remove outliers
    print('before removing outliers: ', np.sum(np.isnan(joint_x_highconf)), ' nan of ', len(joint_x_highconf) )
    joint_x_highconf, joint_y_highconf = remove_outliers2d(joint_x_highconf, joint_y_highconf)
    print('after removing outliers: ', np.sum(np.isnan(joint_x_highconf)), ' nan of ', len(joint_x_highconf) )
    
    # interpolate 
    where_interpolate, where_remove = find_nan_gaps(joint_x_highconf, nan_gap_limit)
    joint_x_interp = interp_vals(joint_x_highconf, find_interp_idcs(where_interpolate))
    joint_y_interp = interp_vals(joint_y_highconf, find_interp_idcs(where_interpolate))

    # lowpass filter
    joint_x_filt = lowpass_filt_sections(joint_x_interp)
    joint_y_filt = lowpass_filt_sections(joint_y_interp)
#     print(joint_x_filt.shape)
    
    if plots:
        # PLOT THINGS
        fig = plt.figure(figsize=(15,5))
        ax1=plt.subplot(2,1,1)
        ax_limits=[]
        ax_limits.append([np.nanmin(joint_x)-10, np.nanmax(joint_x)+10])
        ax2 = plt.subplot(2,1,2)
        ax_limits.append([np.nanmin(joint_y)-10, np.nanmax(joint_y)+10])

        for xx,ax in enumerate([ax1, ax2]):
            for kk, inter in enumerate(all_frames[where_interpolate]):
                if kk == 0:
                    rect = Rectangle((inter-0.5, ax_limits[xx][0]),
                                     1, np.diff(ax_limits[xx]), alpha = 0.2, fc = 'm', ec = None, label = 'interpolated')
                else:
                    rect = Rectangle((inter-0.5, ax_limits[xx][0]),
                                     1, np.diff(ax_limits[xx]), alpha = 0.2, fc = 'm', ec = None)
                ax.add_patch(rect)
            for kk,remov in enumerate(all_frames[where_remove]):
                if kk == 0:
                    rect = Rectangle((remov-0.5, ax_limits[xx][0]),
                                     1, np.diff(ax_limits[xx]), alpha = 0.05, fc = 'k', ec = None, label = 'removed')
                else:
                    rect = Rectangle((remov-0.5, ax_limits[xx][0]),
                                     1, np.diff(ax_limits[xx]), alpha = 0.05, fc = 'k', ec = None)
                ax.add_patch(rect)

        plt.sca(ax1)
        cmap = cm.bwr
        plt.scatter(all_frames, joint_x, c = conf, s = 10, 
                cmap = cmap, norm = colors.Normalize(vmin=0, vmax=1), label = 'raw tracking')
# #         plt.plot(frames_OI, joint_x_highconf, '-k', alpha = 0.2)
        plt.plot(all_frames, joint_x_interp, '.k', alpha = 0.5, MarkerSize = 2)#, label = 'interpolated')
        plt.plot(all_frames, joint_x_filt, '-g', alpha = 0.5, label = 'filtered', )
        plt.ylabel('x (pix)')
        plt.legend(loc = 'upper right', frameon=False, fontsize = 7)
        titleparts = videofile.split('/')
        plt.title('%s -- %s -- %s\n%s -- conf. cutoff: %0.1f -- jump limit: %i -- max nan gap for interp: %i'
                  %(titleparts[-2], titleparts[-1].split('_')[0], titleparts[-1].split('_')[1],
                    tracked_pt, conf_cutoff, jump_limit, nan_gap_limit),
                 loc = 'left')
#         plt.axhline(y=-58.82, xmin = 0, xmax = .8) # for file 589, trackway 833 -- show outlier range
#         plt.axhline(y=-58.82+2*15.99, xmin = 0, xmax = .8)
#         plt.axhline(y=-58.82-2*15.99, xmin = 0, xmax = .8)
        


        plt.sca(ax2)
#         plt.plot(frames_OI, joint_y_highconf, '-k', alpha = 0.5)
        plt.plot(all_frames, joint_y_interp, '.k', alpha = 0.5, label = 'interpolated', MarkerSize = 2)
        plt.plot(all_frames, joint_y_filt, '-g', alpha = 0.5, label = 'filtered', )
#         plt.plot(frames_OI, joint_y_filt, '.g', alpha = 0.5, label = 'filtered', MarkerSize = 3)
        plt.scatter(all_frames, joint_y, c = conf, s = 10, 
                cmap = cmap, norm = colors.Normalize(vmin=0, vmax=1))
        plt.ylabel('y (pix)')
        plt.gca().invert_yaxis()
#         plt.axhline(y=30.01, xmin = 0, xmax = .9) # for file 589, trackway 833 -- show outlier range
#         plt.axhline(y=30.01+2*7.4, xmin = 0, xmax = .9)
#         plt.axhline(y=30.01-2*7.4, xmin = 0, xmax = .9)
        
        
        

        cax = plt.axes([0.93,0.1,0.02,0.8])
        plt.colorbar(cax=cax, label='confidence')
        plt.clim(0,1)
    
    if tracked_pt != 'thorax':
        joint_x_filt = joint_x_filt + x_offset
        joint_y_filt = joint_y_filt + y_offset
    print('final filter len: ', np.sum(np.isnan(joint_x_filt)), ' nan of ', len(joint_x_filt) )
    del rect, 
    return;






# RUN ON SPECIFIC TRIAL & PLOT
plt.close('all')
for tr_num in [9]:#range(833,834):#105 #91 #105
    videofile = df.video[tr_num]
    print(videofile)
    for joint_num in range(0,1):
        tracked_pt = 'joint%i'%joint_num
        interpolate_filter_tracking(df, tr_num, tracked_pt, 
                                conf_cutoff = 0.6, jump_limit = 10, nan_gap_limit = 5, plots = True)
del tracked_pt, videofile
#     tracked_pt = 'thorax'
#     interpolate_filter_tracking(df, tr_num, tracked_pt, 
#                                 conf_cutoff = 0.6, jump_limit = 10, nan_gap_limit = 5, plots = True)
    
    
    
        



def WRTant_to_WRTframe(val_x, val_y, frame_center_x, frame_center_y, ant_ang_deg):
    ant_ang = ant_ang_deg *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    if R.ndim == 3:
        rotated_vals = np.einsum('ijk, ki->kj', R, (np.array([val_x,val_y])-100).T).T
#         print(R.shape, rotated_vals.shape, rotated_vals[:,0])
        translated_vals = rotated_vals + np.array([frame_center_x, frame_center_y])  
        return translated_vals[0,:], translated_vals[1,:];
    elif R.ndim == 2:
        rotated_vals = np.einsum('ij, i->j', R, (np.array([val_x,val_y])-100).T).T
#         print(R.shape, rotated_vals.shape, rotated_vals[:])
        translated_vals = rotated_vals + np.array([frame_center_x, frame_center_y])  
        return translated_vals[0], translated_vals[1];
    else:
        print('something went wrong with R dimensions')
        return np.nan, np.nan

/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080521_16276736-0000.mp4
before removing outliers:  46  nan of  118
after removing outliers:  54  nan of  118
final filter len:  44  nan of  118


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in greater


## Analyze tracked data in WHOLE DATAFRAME, removing low confidence points and lowpass filter -- rotate wrt full frame, rotate wrt tracked neck and thorax
-- apply to whole dataframe: thorax, neck, and limbs<br>

In [411]:
# DATAFRAME FUNCTIONS

def find_nan_gaps(arr, limit):  
    yy = np.isnan(arr)
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
#                 print('remove: ', len(g), g)
                continue
                
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
#                 print('interp: ', len(g), g)
            else:
                where_othergaps[np.array(g)]=True
#                 print('remove: ', len(g), g)
    return where_gapOI, where_othergaps


def interpolate_filter_tracking_df(x, ant_part, conf_cutoff, jump_limit, nan_gap_limit):
#     print('TRIAL: ', x.name, ' -- substrate: ', x.substrate)
    all_frames = x['frames']
    frames_OI = x['frames_final']
    frames_OI_idcs = np.isin(all_frames, frames_OI)
    
    joint={'x': x[ant_part+'_x'] , 'y': x[ant_part+'_y']}
    conf = x[ant_part + '_conf']
    if 'thorax_x_filt' in df:
        thorax = {'x': x['thorax_x_filt'], 'y': x['thorax_y_filt']} 
    else:
        thorax = {'x': x['thorax_x'], 'y': x['thorax_y']} 

    #initialize new dicts
    joint_highconf = {}
    joint_no_outliers = {}
    joint_interp = {}
    joint_filt = {}

    for coord in ['x','y']:
        if 'thorax' not in ant_part: # account for inaccurate initial guess of body center, make relative to LEAP tracked thorax
            joint[coord] = joint[coord] - thorax[coord]

        joint_highconf[coord] = remove_lowconf_pts(joint[coord], conf, conf_cutoff, jump_limit)
        joint_highconf[coord] = remove_jumps(joint_highconf[coord], jump_limit)
    joint_no_outliers['x'], joint_no_outliers['y'] = remove_outliers2d(joint_highconf['x'], joint_highconf['y'])   
    for coord in ['x','y']:
        where_interpolate, where_remove = find_nan_gaps(joint_no_outliers[coord], nan_gap_limit)
        if np.sum(where_interpolate==True)>0: # only interpolate if needed
            joint_interp[coord] = interp_vals(joint_no_outliers[coord], find_interp_idcs(where_interpolate))
        else:
            joint_interp[coord] = joint_no_outliers[coord]
        joint_filt[coord] = lowpass_filt_sections(joint_interp[coord])
    
        if 'thorax' not in ant_part: # account for inaccurate initial guess of body center, make relative to LEAP tracked thorax
            joint_filt[coord] = joint_filt[coord] + thorax[coord]

    if np.sum(np.logical_not(np.isnan(joint_filt['x']))) < 50: # if fewer than 50 non-nan points in trial, remove
        joint_filt['x'][np.logical_not(np.isnan(joint_filt['x']))]=np.nan
        joint_filt['y'][np.logical_not(np.isnan(joint_filt['y']))]=np.nan
        
    if np.nanmax(np.linalg.norm([np.diff(joint_filt['x']), np.diff(joint_filt['y'])],axis=0))> 30 : # if unreasonably high velocities
        joint_filt['x'][np.logical_not(np.isnan(joint_filt['x']))]=np.nan
        joint_filt['y'][np.logical_not(np.isnan(joint_filt['y']))]=np.nan
        
     
    return joint_filt['x'], joint_filt['y'] #joint_x_filt, joint_y_filt;


def WRTant_to_WRTframe_df(df, ant_part):
    frame_idcs = np.isin(df['frames'], df['frames_final'])
    ant_x = np.array(df['x_raw'])#[frame_idcs]
    ant_y = np.array(df['y_raw'])#[frame_idcs]
    ang = np.array(df['angle_improved'])*-1#[frame_idcs]*-1
    val_x = df['%s_x_filt'%ant_part]
    val_y = df['%s_y_filt'%ant_part]

    ant_ang = ang *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    
    if R.ndim == 3:
        rotated_vals = np.einsum('ijk, ki->kj', R, (np.array([val_x,val_y])-100).T).T
        translated_vals = rotated_vals + np.array([ant_x, ant_y])
        xs = translated_vals[0,:]
        ys = translated_vals[1,:]
    elif R.ndim == 2:
        rotated_vals = np.einsum('ij, i->j', R, (np.array([val_x,val_y])-100).T).T
        translated_vals = rotated_vals + np.array([ant_x, ant_y]) 
        xs = translated_vals[0]
        ys = translated_vals[1]    
    else:
        print('something went wrong with R dimensions')
        return np.nan, np.nan
    
    # remove any crazy rotated points that result from slight diff in rotation angle +180/-180
    fast_velocity_pts = np.linalg.norm([np.diff(xs), np.diff(ys)],axis=0)> 30
    xs[1:][fast_velocity_pts] = np.nan
    ys[1:][fast_velocity_pts] = np.nan
    
    # remove any points close to edge of frame
    buffer = 60
    wid = 1000
    hei = 500
    edgeidcs = ((xs < buffer) | (xs > wid-(buffer)) | (ys < buffer) | (ys > hei-buffer))
    xs[edgeidcs]=np.nan
    ys[edgeidcs]=np.nan

    return xs,ys
    

def WRTant_to_WRTneck_df(df, ant_part):
    x = df['%s_x_filt'%ant_part]
    y = df['%s_y_filt'%ant_part]
    thorax_x = df['thorax_x_filt']
    thorax_y = df['thorax_y_filt']
    neck_x = df['neck_x_filt']
    neck_y = df['neck_y_filt']
    
    val_coord = np.array([x,y])-np.array([thorax_x,thorax_y])
    neck_coord = np.array([neck_x-thorax_x,neck_y-thorax_y])
    ang = np.arctan( (neck_y-thorax_y)/(neck_x-thorax_x))
    c, s = np.cos(ang), np.sin(ang)
    Rx = np.array([c,s])
    Ry = np.array([-s,c])
    newx = np.einsum('mn,mn->n', val_coord, Rx)
    newy = np.einsum('mn,mn->n', val_coord, Ry)
    return newx, newy

In [412]:
# APPLY INTERPOLATION AND LOWPASS FILTERING TO DATAFRAME
# removes points with low confidence or large jumps in tracking (likely errors), replaces with nan
# interpolates nan gaps less than specified size
# low pass filters (butterworth) and saves

print('For whole dataframe: filter x,y wrt ant and calc filtered x,y wrt lab')
conf_cutoff = 0.6
jump_cutoff = 10
nan_gap_limit = 5

# # thorax
df['thorax_x_filt'], df['thorax_y_filt'] = zip(*df.apply(
    interpolate_filter_tracking_df, args = ('thorax', conf_cutoff, jump_cutoff, nan_gap_limit), axis=1))
df['thorax_x_filt_fullfr'], df['thorax_y_filt_fullfr'] = zip(*df.apply(
        WRTant_to_WRTframe_df, args = ('thorax',), axis=1))
print('thorax analyzed')

# neck
df['neck_x_filt'], df['neck_y_filt'] = zip(*df.apply(
    interpolate_filter_tracking_df, args = ('neck', conf_cutoff, jump_cutoff, nan_gap_limit), axis=1))
df['neck_x_filt_fullfr'], df['neck_y_filt_fullfr'] = zip(*df.apply(
        WRTant_to_WRTframe_df, args = ('neck',), axis=1))
print('neck analyzed')

# antennae
for joint_num in range(0,2):
    df['antenna%i_x_filt'%joint_num], df['antenna%i_y_filt'%joint_num] = zip(*df.apply(
        interpolate_filter_tracking_df, args = ('antenna%i'%joint_num, conf_cutoff, jump_cutoff, nan_gap_limit), axis=1))
    df['antenna%i_x_filt_fullfr'%joint_num], df['antenna%i_y_filt_fullfr'%joint_num] = zip(*df.apply(
        WRTant_to_WRTframe_df, args = ('antenna%i'%joint_num,), axis=1))
print('antennae analyzed')

# legs
for joint_num in range(0,6):
    print('joint %i'%joint_num)
    df['joint%i_x_filt'%joint_num], df['joint%i_y_filt'%joint_num] = zip(*df.apply(
        interpolate_filter_tracking_df, args = ('joint%i'%joint_num, conf_cutoff, jump_cutoff, nan_gap_limit), axis=1))
    df['joint%i_x_filt_fullfr'%joint_num], df['joint%i_y_filt_fullfr'%joint_num] = zip(*df.apply(
        WRTant_to_WRTframe_df, args = ('joint%i'%joint_num,), axis=1))
    
print('legs analyzed')


# rotate relative to thorax and neck if exist
print('\nRotate leg and antennae point WRT tracked neck direction:')
if ('thorax_x_filt' in df) and ('neck_x_filt' in df):
    for joint_num in range(0,2):
        df['antenna%i_x_filt_WRTneck'%joint_num], df['antenna%i_y_filt_WRTneck'%joint_num] = zip(*df.apply(
            WRTant_to_WRTneck_df, args = ('antenna%i'%joint_num,), axis=1))
    print('antennae analyzed')
    for joint_num in range(0,6):
        df['joint%i_x_filt_WRTneck'%joint_num], df['joint%i_y_filt_WRTneck'%joint_num] = zip(*df.apply(
            WRTant_to_WRTneck_df, args = ('joint%i'%joint_num,), axis=1))
    print('legs analyzed')
    
del conf_cutoff, jump_cutoff, nan_gap_limit


# something there's a weird rotation resulting in a really high velocity 

print('\nall donezo')

For whole dataframe: filter x,y wrt ant and calc filtered x,y wrt lab


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in greater
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: All-NaN slice encountered
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:101: RuntimeWarning: invalid value encountered in greater
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in greater


thorax analyzed
neck analyzed
antennae analyzed
joint 0
joint 1
joint 2
joint 3
joint 4
joint 5
legs analyzed

Rotate leg and antennae point WRT tracked neck direction:
antennae analyzed
legs analyzed

all donezo


## Test out removing data when ant is close to wall or stationary

In [419]:
# for single trial

tr_num = 0
for n_tr in np.arange(0,10):
#     tr_num = 2279#
    tr_num = random.randint(0,11000)
    x = df['thorax_x_filt_fullfr'][tr_num]
    y = df['thorax_y_filt_fullfr'][tr_num]
    v = np.linalg.norm(np.array([np.diff(x), np.diff(y)]), axis =0)

    # remove points close to edge
    buffer = 60
    wid = 1000
    hei = 500
    edgeidcs = ((x < buffer) | (x > wid-(buffer)) | (y < buffer) | (y > hei-buffer))
    ROIidcs = ~edgeidcs

    # find moving points based on distance traveled
    b_wid = 10
    b_sep = 40
    dist_thres = 40 # pix/window - ~90 pix is 2 mm
    
    temp = np.add.outer(np.arange(len(x)), -np.arange(len(x)))
    before_wid = np.logical_and(temp >= -1*b_sep - (b_wid/2),temp <= -1*b_sep + (b_wid/2)).astype(np.float)
    before_wid[before_wid==0] = np.nan
    after_wid = np.logical_and(temp >= b_sep - (b_wid/2),temp <= b_sep + (b_wid/2)).astype(np.float)
    after_wid[after_wid==0] = np.nan
    before_x = np.nanmean(before_wid*x[:,np.newaxis], axis=0)
    after_x = np.nanmean(after_wid*x[:,np.newaxis], axis=0)
    before_y = np.nanmean(before_wid*y[:,np.newaxis], axis=0)
    after_y = np.nanmean(after_wid*y[:,np.newaxis], axis=0)
    dist_traveled = np.linalg.norm([after_x-before_x, after_y-before_y], axis = 0)
    slow_idcs = dist_traveled < dist_thres
    

    plt.close('all')
    plt.figure(figsize=(6,8))
    plt.subplot(2,1,1)
    plt.plot(x,y,'-k')
    plt.plot(x[edgeidcs],y[edgeidcs], '.b')
    plt.plot(x[slow_idcs],y[slow_idcs], '.r')
    rect = patches.Rectangle([buffer, buffer], wid-2*buffer, hei-2*buffer, ec='k', fc = 'none', alpha = 0.2)
    plt.gca().add_patch(rect)
    plt.xlim(0,wid)
    plt.ylim(0,hei)
    plt.gca().invert_yaxis()
    plt.title('Tr Num: %i, Sub: %s'%(tr_num, df['substrate'][tr_num]))
    
    plt.subplot(2,1,2)
    plt.plot(v,'-k')
    plt.plot(dist_traveled/90, '--r')
    plt.axhline(y=dist_thres/90, c='k', alpha = 0.3)

#     #plt.axis('equal')
    plt.pause(3)

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: Mean of empty slice
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in les

In [ ]:
# apply to dataframe

## FIND TOUCHDOWN AND STRIDES

In [450]:
# FOR A GIVEN TRIAL, FIND TOUCHDOWNS, STRIDE LENGTHS, AND STRIDE DURATIONS

def find_start_of_stance(vel, vel_cutoff, win_size, n_slow_in_window):
    where_slow = vel< vel_cutoff
    first_slows = np.diff(np.insert(where_slow, len(where_slow), np.nan).astype(np.int8)) == 1
    n = n_slow_in_window # how many nans must there be in window of w width after each first_slow
    w = win_size
    where_slow_cumsum = np.cumsum(np.insert(where_slow,len(where_slow),np.zeros(w)))
    n_subsequent_slow = where_slow_cumsum[w:]-where_slow_cumsum[:-(w)]
    starts = np.logical_and(first_slows, n_subsequent_slow>=n)
    prior_starts = np.cumsum(np.insert(starts,0,np.zeros(10)))[10:] - np.cumsum(np.insert(starts,0,np.zeros(10)))[0:-10]
    n_prior_slow = np.insert(where_slow_cumsum,0,0)-np.insert(where_slow_cumsum, 0, np.zeros(w+1))[:-(w)]
    TDs = np.where(np.logical_and(np.logical_and(starts, n_prior_slow[:-(w+1)]<=n), prior_starts ==1))[0]
#     TDs = np.where(np.logical_and(starts, prior_starts == 1))[0]
    return TDs

def find_start_of_stance2(vel, vel_cutoff, gaps_size_to_close, win_size, n_slow_in_window):
    where_slow = vel < vel_cutoff
    # interpolate gaps
    where_slow[find_nan_gaps(np.logical_not(where_slow), gaps_size_to_close)] = True
    
    first_slows = np.diff(np.insert(where_slow, len(where_slow), False).astype(np.int8)) == 1
    TDs = np.where(first_slows)[0]
    
    n = n_slow_in_window # how many nans must there be in window of w width after each first_slow
    w = win_size
    where_slow_cumsum = np.cumsum(np.insert(where_slow,len(where_slow),np.zeros(w)))
    n_subsequent_slow = where_slow_cumsum[w:]-where_slow_cumsum[:-(w)]
    starts = np.logical_and(first_slows, n_subsequent_slow>=n)
    prior_starts = np.cumsum(np.insert(starts,0,np.zeros(w)))[w:] - np.cumsum(np.insert(starts,0,np.zeros(w)))[0:-w]
#     n_prior_slow = np.insert(where_slow_cumsum,0,0)-np.insert(where_slow_cumsum, 0, np.zeros(w+1))[:-(w)]
#     TDs = np.where(np.logical_and(np.logical_and(starts, n_prior_slow[:-(w+1)]<=n), prior_starts ==1))[0]
    TDs = np.where(np.logical_and(starts, prior_starts == 1))[0] + 2
    return TDs

def find_nan_gaps(arr, limit):  
    from itertools import groupby
    yy = arr
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
                continue       
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
    return where_gapOI


def find_TD_position(df, joint, TDs, x_or_y, tr_num, window_size):
    idcs = (np.tile(TDs,(window_size,1))+np.arange(0,window_size)[:,np.newaxis])
    #idcs = np.clip(idcs, 0, len(df['%s_%s_filt_fullfr'%(joint, x_or_y)][tr_num])-1)
    positions = np.mean(df['%s_%s_filt_fullfr'%(joint, x_or_y)][tr_num][idcs],axis=0)
    return positions


def remove_close_TDs(TDs, TDs_x, TDs_y, cutoff):
    for k in TDs.keys():
        d_TDs = np.linalg.norm(np.diff(np.array([TDs_x[k], TDs_y[k]])), axis=0)
        d_TDs = np.insert(d_TDs, 0, cutoff+2) # delete second TD, not first
        TDs[k]= np.delete(TDs[k], np.where(d_TDs<cutoff))
        TDs_x[k]= np.delete(TDs_x[k], np.where(d_TDs<cutoff))
        TDs_y[k]= np.delete(TDs_y[k], np.where(d_TDs<cutoff))
    return TDs, TDs_x, TDs_y

    

def find_TD_frames(df, tr_num, TDs_x, TDs_y, n_nearby_frs, pix_cutoff):
    if n_nearby_frs%2 == 1: # make sure is even
        n_nearby_frs = n_nearby_frs + 1
    TDs_fr={}
    TDs_fr_idcs = {}
    for k in TDs_x.keys():
        idcs = (np.tile(TDs[k],(n_nearby_frs+1,1))+np.arange(-n_nearby_frs/2,n_nearby_frs/2+1)[:,np.newaxis]).astype(np.int64)
        idcs= np.clip(idcs, 0, len(df['%s_x_filt_fullfr'%(k)][tr_num])-1)
        d_TDs_x = TDs_x[k][np.newaxis,:] - df['%s_x_filt_fullfr'%(k)][tr_num][idcs]
        d_TDs_y = TDs_y[k][np.newaxis,:] - df['%s_y_filt_fullfr'%(k)][tr_num][idcs]
        d_TDs_total = np.linalg.norm(np.array([d_TDs_x, d_TDs_y]), axis=0)
        TDs_fr_idcs[k] = idcs[np.argmax(d_TDs_total<pix_cutoff, axis = 0), range(len(TDs_x[k]))]
        TDs_fr[k] = df['frames'][tr_num][TDs_fr_idcs[k]]
    return TDs_fr_idcs, TDs_fr


def remove_uncertain_TD_frames(TDs_fr_idcs, vel, where_slow, min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff): # remove any TD_frs that don't have at least __ non-nan frames before
    good_TDs = {}
    good_strides = {}
    for k in vel.keys():
        v_chunks = np.split(vel[k], TDs_fr_idcs[k])
        ws_chunks = np.split(where_slow[k], TDs_fr_idcs[k])
        wf_chunks = np.split(np.logical_not(where_slow[k]), TDs_fr_idcs[k])
        n_finite_fr_before = np.array([0]+ [np.sum(np.isfinite(c[-min_n_fr_before_TD:])) for c in v_chunks][:-1])
        fraction_tracked = np.array([np.sum(np.isfinite(c))/len(c) for c in v_chunks])
        n_slow_in_center = np.array([np.sum(np.logical_not(np.trim_zeros(np.logical_not(np.trim_zeros(c,'f'))))) for c in ws_chunks ])
        good_TDs[k] = (n_finite_fr_before == min_n_fr_before_TD)[1:]
        tmp = np.logical_and(good_TDs[k][:-1], good_TDs[k][1:]) # good strides must be between two good TDs
        tmp2 = np.logical_and.reduce((n_finite_fr_before==min_n_fr_before_TD, n_slow_in_center<=max_slow_during_swing, fraction_tracked > fraction_track_cutoff))[1:-1]
        good_strides[k] = np.logical_and(tmp, tmp2)
    return good_TDs, good_strides


def find_stride_dur(TDs_fr_idcs, good_strides):
    stride_durations = {}
    stride_dur_sta = {}
    stride_dur_sto = {}
    for jj in range(0,6):
        k = 'joint%i'%jj
        stride_durations[k] = np.diff(TDs_fr_idcs[k])[good_strides[k]]
        stride_dur_sta[k] = TDs_fr_idcs[k][0:-1][good_strides[k]]
        stride_dur_sto[k] = TDs_fr_idcs[k][1:][good_strides[k]]-1
    return stride_durations, stride_dur_sta, stride_dur_sto


def find_stride_dist(df, tr_num, TDs_fr_idcs, good_strides):
    stride_dists = {}
    stride_dists_sta = {}
    stride_dists_sto = {}
    for jj in range(0,6):
        k = 'joint%i'%jj
        d_x = np.diff(df['%s_x_filt_fullfr'%k][tr_num][TDs_fr_idcs[k]])[good_strides[k]]
        d_y = np.diff(df['%s_y_filt_fullfr'%k][tr_num][TDs_fr_idcs[k]])[good_strides[k]]
        stride_dists[k] = np.linalg.norm(np.array([d_x, d_y]), axis = 0)
        stride_dists_sta[k] = TDs_fr_idcs[k][0:-1][good_strides[k]]
        stride_dists_sto[k] = TDs_fr_idcs[k][1:][good_strides[k]]-1
    return stride_dists, stride_dists_sta, stride_dists_sto
    

def plot_TD_circles(TDs_x, TDs_y, rad):
    for k in TDs_x.keys():
        for ii in range(len(TDs_x[k])):
            circ =Circle((TDs_x[k][ii] , TDs_y[k][ii]), radius=rad, ec = 'k' , fc= 'k', alpha = 0.4)
            ax1.add_patch(circ)
    return


########################################################
# user set parameters
for tr_num in [1244]: # range(75,76):
    print('trial: %s, time %s'%(tr_num, df['time'][tr_num]))
#     tr_num = 410#310#18

    # from df version below
    vel_cutoff= 3 # what velocity cutoff for identifying stances
    gap_size_to_close = 3 # interpolate where slow data
    n_stance_fr = 5 # how many frames of "stance" for it to be a legit "stance"
    min_TD_separation = 10 # how close can subsequent TDs be to each other
    n_nearby_frs = 12 # how many frames around TD to look for TD frame (first fr when foot within __ pix of TD location)
    pix_cutoff = 3 # how large of a radius around TD location to look for first TD frame
    min_n_fr_before_TD = 3 # how many non-nan frames before TD to make sure you're getting the correct TD timing
    max_slow_during_swing = 1 # how many slow points allowed outside of initial slow section
    fraction_track_cutoff = 0.9 # percent of stride that needs to be tracked (not missing extra TD)

    #########################################################



    TDs = {}
    vel = {}
    where_slow = {}
    for jj in range(0,6):
        k = 'joint%i'%jj
        vel[k] = np.linalg.norm(np.diff(np.array([df['joint%i_x_filt_fullfr'%jj][tr_num], df['joint%i_y_filt_fullfr'%jj][tr_num]])), axis=0)
        # how much noise is there?
        if np.sum(vel[k]<1)/np.sum(np.isfinite(vel[k]))>0.6:
            vel_cutoff = 1
            pix_cutoff = 2
            print('%0.2f of velocity less than 1 -- vel_cutoff = %0.1f'%(np.sum(vel[k]<1)/np.sum(np.isfinite(vel[k])), vel_cutoff))
        elif np.sum(vel[k]<1.5)/np.sum(np.isfinite(vel[k]))>0.6:
            vel_cutoff = 1.5
            pix_cutoff = 2
            print('%0.2f of velocity less than 1.5 -- vel_cutoff = %0.1f'%(np.sum(vel[k]<1.5)/np.sum(np.isfinite(vel[k])), vel_cutoff))
        elif np.sum(vel[k]<2)/np.sum(np.isfinite(vel[k]))>0.5:
            vel_cutoff = 2
            print('%0.2f of velocity less than 2 -- vel_cutoff = %0.1f'%(np.sum(vel[k]<2)/np.sum(np.isfinite(vel[k])), vel_cutoff))
        else:
            vel_cutoff = 3
            print('%0.2f of velocity less than 2 -- vel_cutoff = %0.1f'%(np.sum(vel[k]<1.5)/np.sum(np.isfinite(vel[k])), vel_cutoff))

        if not np.any(np.isfinite(vel[k])):
            TDs[k] = np.nan
            continue
        where_slow[k] = vel[k]<vel_cutoff
        where_slow[k][find_nan_gaps(np.logical_not(where_slow[k]), gap_size_to_close)] = True
        TDs[k] = find_start_of_stance2(vel[k], vel_cutoff, gap_size_to_close, n_stance_fr, n_stance_fr)
    TDs_raw = TDs.copy()
    TDs_x = {k: find_TD_position(df, k, v, 'x', tr_num, n_stance_fr) for k, v in TDs.items()} # general guess of where stance is
    TDs_y = {k: find_TD_position(df, k, v, 'y', tr_num, n_stance_fr) for k, v in TDs.items()}
    TDs, TDs_x, TDs_y = remove_close_TDs(TDs, TDs_x, TDs_y, cutoff=min_TD_separation) # remove TDs that are close to eachother
    TDs_fr_idcs, TDs_fr = find_TD_frames(df, tr_num, TDs_x, TDs_y, n_nearby_frs, pix_cutoff) # find frames of touchdown
    good_TDs, good_strides = remove_uncertain_TD_frames(TDs_fr_idcs, vel, where_slow, min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff)
    stride_dur, stride_dur_sta, stride_dur_sto = find_stride_dur(TDs_fr_idcs, good_strides)
    stride_dist, stride_dist_sta, stride_dist_sto = find_stride_dist(df, tr_num, TDs_fr_idcs, good_strides)


    ##### PLOT THINGS
    # plt.close('all')
    cs = ['r','m', 'y', 'g','c','b']
    ls = ['--', ':', '-','--',':','-']
    fig=plt.figure(figsize=(22,12))
    ax1=fig.add_axes([0.1,0.6, 0.8, 0.3])
    plt.plot(df['thorax_x_filt_fullfr'][tr_num], df['thorax_y_filt_fullfr'][tr_num], 'k-', alpha = 0.2)
    for jj in range(0,6,1):
        plt.plot(df['joint%i_x_filt_fullfr'%jj][tr_num], df['joint%i_y_filt_fullfr'%jj][tr_num], '.', MarkerSize = 2, color = cs[jj])
        plt.plot(df['joint%i_x_filt_fullfr'%jj][tr_num], df['joint%i_y_filt_fullfr'%jj][tr_num], '-', alpha = 0.1, color = cs[jj])
        plt.text(600, 100+15*jj, 'joint%i'%jj, color = cs[jj])
    plt.plot(df['joint0_x_filt_fullfr'][tr_num][TDs_raw['joint0'][0]] , df['joint0_y_filt_fullfr'][tr_num][TDs_raw['joint0'][0]], 
             'o', color = 'k' , fillstyle = 'none')
    plt.plot(df['joint3_x_filt_fullfr'][tr_num][TDs_raw['joint3'][0]] , df['joint3_y_filt_fullfr'][tr_num][TDs_raw['joint3'][0]], 
             'o', color = 'k' , fillstyle = 'none')
    plot_TD_circles(TDs_x, TDs_y, pix_cutoff)
    plt.ylabel('y')
    plt.xlabel('x')
    plt.gca().axis('equal')
    plt.gca().invert_yaxis()
    plt.title((' - ').join(df.video[tr_num].split('/')[-2:]))


    ax2=fig.add_axes([0.1,0.35, 0.8, 0.2])
    plt.axhline(y = vel_cutoff, color = 'k', linestyle = ':', alpha=.3)
    plt.ylabel('total vel (pix/fr)')
    ax3=fig.add_axes([0.1,0.1, 0.8, 0.2])
    plt.axhline(y = vel_cutoff, color = 'k', linestyle = ':', alpha=.3)
    plt.ylabel('total vel (pix/fr)')

    for jj in range(0,6):
        k = 'joint%i'%jj

        plt.sca(ax1)
        # plot final TD locations
        temp = TDs_fr_idcs[k][good_TDs[k]][np.logical_not(np.isnan(TDs_fr_idcs[k][good_TDs[k]]))].astype(np.int64)
        plt.plot( df['%s_x_filt_fullfr'%(k)][tr_num][temp], df['%s_y_filt_fullfr'%(k)][tr_num][temp], '*', color = 'k', alpha = 0.8, MarkerSize = 8)
        plt.plot( df['%s_x_filt_fullfr'%(k)][tr_num][temp], df['%s_y_filt_fullfr'%(k)][tr_num][temp], '.', color = cs[jj], alpha = 0.8, MarkerSize = 4)
        # plot stride distances measured
        temp = np.array([stride_dur_sta[k], stride_dur_sto[k]+1])
        plt.plot(df['%s_x_filt_fullfr'%(k)][tr_num][temp.T].T, df['%s_y_filt_fullfr'%(k)][tr_num][temp.T].T, color = cs[jj], alpha = 0.4)

        if jj%2 == 0:
            plt.sca(ax2)
        else:
            plt.sca(ax3)

        # plot velocity trace
        plt.plot(df['frames'][tr_num][:-1], vel[k], '-', color = cs[jj])
        # plot where velocity is slow (a potential "stance")
        for sw in np.array(np.where(where_slow[k]))[0]:
            rect = Rectangle((df['frames'][tr_num][sw]-0.5, -1*(jj+2)/4), 1, 1/3, alpha = 0.4, fc = cs[jj], ec = None)
            plt.gca().add_patch(rect)
        # plot initial guess of TD frame
        plt.plot(df['frames'][tr_num][TDs_raw[k]] , np.ones(TDs_raw[k].shape)*(-1*(jj+2)/4)+(1/6), '.', color = cs[jj] , alpha = 1, MarkerSize = 3)
        plt.plot(df['frames'][tr_num][TDs[k]] , np.ones(TDs[k].shape)*(-1*(jj+2)/4)+(1/6), '.', color = cs[jj] )
        # plot final TD frames
        for TD_fr in TDs_fr[k]:
            plt.axvline(x = TD_fr, color = cs[jj], linestyle = ls[jj], alpha=.1)
        for TD_fr in TDs_fr[k][good_TDs[k]]:
            plt.axvline(x = TD_fr, color = cs[jj], linestyle = ls[jj], alpha=.4)
        # plot final TD dist/durations
        temp = np.array([stride_dur_sta[k]+1, stride_dur_sto[k]])
        plt.plot(df['frames'][tr_num][temp], (20+jj)*np.ones(temp.shape), '-', color = cs[jj])


    del TDs, TDs_x, TDs_y, TDs_raw, good_TDs, good_strides, stride_dur, stride_dur_sta, stride_dur_sto, stride_dist, stride_dist_sta, stride_dist_sto
    del TDs_fr_idcs, TDs_fr, vel, k

trial: 1244, time 101212
0.44 of velocity less than 2 -- vel_cutoff = 3.0
0.62 of velocity less than 1.5 -- vel_cutoff = 1.5
0.66 of velocity less than 2 -- vel_cutoff = 2.0
0.29 of velocity less than 2 -- vel_cutoff = 3.0
nan of velocity less than 2 -- vel_cutoff = 3.0
0.08 of velocity less than 2 -- vel_cutoff = 3.0


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:166: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:174: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:179: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:184: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:173: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.vir

IndexError: arrays used as indices must be of integer (or boolean) type

In [420]:
# WHOLE DATAFRAME -- FIND TOUCHDOWNS, STRIDE LENGTHS, STRIDE DURATIONS, TRAVEL DIR., FACING ROTATIONS
    
def find_start_of_stance_df(vel, vel_cutoff, gaps_size_to_close, n_slow):
    stance_cutoff = vel_cutoff
    
#     if not np.any(np.isfinite(vel)):
#         return np.nan
    where_slow = vel< stance_cutoff
    
    # interpolate gaps
    where_slow[find_nan_gaps(np.logical_not(where_slow), gaps_size_to_close)] = True
    
    first_slows = np.diff(np.insert(where_slow, len(where_slow), False).astype(np.int8)) == 1
    TDs = np.where(first_slows)[0]
    
    n = n_slow # how many nans must there be in window of w width after each first_slow
    where_slow_cumsum = np.cumsum(np.insert(where_slow,len(where_slow),np.zeros(n)))
    n_subsequent_slow = where_slow_cumsum[n:]-where_slow_cumsum[:-(n)]
    starts = np.logical_and(first_slows, n_subsequent_slow>=n)
    prior_starts = np.cumsum(np.insert(starts,0,np.zeros(n)))[n:] - np.cumsum(np.insert(starts,0,np.zeros(n)))[0:-n]
    TDs = np.where(np.logical_and(starts, prior_starts == 1))[0] +1
    return TDs

def find_nan_gaps(arr, limit):  
    from itertools import groupby
    yy = arr
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
                continue       
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
    return where_gapOI


def find_TD_position_df(df, joint_num, TDs, x_or_y, window_size):
    idcs = (np.tile(TDs,(window_size,1))+np.arange(0,window_size)[:,np.newaxis])
    positions = np.mean(df['joint%i_%s_filt_fullfr'%(joint_num, x_or_y)][idcs],axis=0)
    return positions

def remove_close_TDs_df(TDs, cutoff, df, jj, window_size):
    TDs_x = find_TD_position_df(df, jj, TDs, 'x', window_size)
    TDs_y = find_TD_position_df(df, jj, TDs, 'y', window_size)
    d_TDs = np.linalg.norm(np.diff(np.array([TDs_x, TDs_y])), axis=0)
    d_TDs = np.insert(d_TDs, 0, cutoff+2) # delete second TD, not first
    TDs= np.delete(TDs, np.where(d_TDs<cutoff))
    TDs_x = np.delete(TDs_x, np.where(d_TDs<cutoff))
    TDs_y = np.delete(TDs_y, np.where(d_TDs<cutoff))
    return TDs, TDs_x, TDs_y
    

def find_TD_frames_df(df, jj, TDs, TDs_x, TDs_y, n_nearby_frs, pix_cutoff):
    if n_nearby_frs%2 == 1: # make sure is even
        n_nearby_frs = n_nearby_frs + 1
    idcs = (np.tile(TDs,(n_nearby_frs+1,1))+np.arange(-n_nearby_frs/2,n_nearby_frs/2+1)[:,np.newaxis]).astype(np.int64)
#     idcs[idcs>= len(df['joint%i_x_filt_fullfr'%jj])] = len(df['joint%i_x_filt_fullfr'%jj])-1
    idcs= np.clip(idcs, 0, len(df['joint%i_x_filt_fullfr'%(jj)])-1)
    d_TDs_x = TDs_x[np.newaxis,:] - df['joint%i_x_filt_fullfr'%jj][idcs]
    d_TDs_y = TDs_y[np.newaxis,:] - df['joint%i_y_filt_fullfr'%jj][idcs]
    d_TDs_total = np.linalg.norm(np.array([d_TDs_x, d_TDs_y]), axis=0)
    TDs_fr_idcs = idcs[np.argmax(d_TDs_total<pix_cutoff, axis = 0), range(len(TDs_x))]+1
    TDs_fr = df['frames'][TDs_fr_idcs]
    return TDs_fr_idcs, TDs_fr


def remove_uncertain_TD_frames_df(TDs_fr_idcs, vel, where_slow, min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff): # remove any TD_frs that don't have at least __ non-nan frames before
    v_chunks = np.split(vel, TDs_fr_idcs-1)
    ws_chunks = np.split(where_slow, TDs_fr_idcs-1)
    wf_chunks = np.split(np.logical_not(where_slow), TDs_fr_idcs-1)
    n_finite_fr_before = np.array([0]+ [np.sum(np.isfinite(c[-min_n_fr_before_TD:])) for c in v_chunks][:-1])
    fraction_tracked = np.array([np.sum(np.isfinite(c))/len(c) for c in v_chunks])
    n_slow_in_center = np.array([np.sum(np.logical_not(np.trim_zeros(np.logical_not(np.trim_zeros(c,'f'))))) for c in ws_chunks ])
    good_TDs = (n_finite_fr_before == min_n_fr_before_TD)[1:]
    tmp = np.logical_and(good_TDs[:-1], good_TDs[1:]) # good strides must be between two good TDs
    tmp2 = np.logical_and.reduce((n_finite_fr_before==min_n_fr_before_TD, n_slow_in_center<=max_slow_during_swing, fraction_tracked > fraction_track_cutoff))[1:-1]
    good_strides = np.logical_and(tmp, tmp2)
    return good_TDs, good_strides


def find_stride_dur_df(TDs_fr_idcs, TDs_fr, good_strides):
    stride_durations = np.diff(TDs_fr_idcs)[good_strides]
    stride_TDstart = TDs_fr[:-1][good_strides].astype(int)
    stride_TDstop = TDs_fr[1:][good_strides].astype(int)
    return stride_durations, stride_TDstart, stride_TDstop


def find_stride_dist_df(df, jj, TDs_fr_idcs, good_strides):
    k = 'joint%i'%jj
    # stride length
    d_x = np.diff(df['%s_x_filt_fullfr'%k][TDs_fr_idcs])
    d_y = np.diff(df['%s_y_filt_fullfr'%k][TDs_fr_idcs])
    stride_dists = np.linalg.norm(np.array([d_x, d_y]), axis = 0)[good_strides]
    # stride thorax total distance
    d_tx = np.diff(df['thorax_x_filt_fullfr'][TDs_fr_idcs])
    d_ty = np.diff(df['thorax_y_filt_fullfr'][TDs_fr_idcs])
    thorax_dists_total = np.linalg.norm(np.array([d_tx, d_ty]), axis = 0)[good_strides]
    # stride thorax straight distance
    d_nx = np.diff([df['thorax_x_filt_fullfr'][TDs_fr_idcs], df['neck_x_filt_fullfr'][TDs_fr_idcs]], axis = 0)[0]
    d_ny = np.diff([df['thorax_y_filt_fullfr'][TDs_fr_idcs], df['neck_y_filt_fullfr'][TDs_fr_idcs]], axis = 0)[0]
    a=[d_nx[:-1], d_ny[:-1]]
    b=[d_tx, d_ty]
    thorax_dists_straight = np.einsum('ik,ik->k', b, a/np.linalg.norm(a, axis =0))[good_strides]
    # stride travel direction rotation
    a=[d_nx[:-1], d_ny[:-1]]
    b=[d_tx, d_ty]
    travel_dir = np.rad2deg(np.arctan2(np.cross(a,b,axis=0),np.einsum('ik,ik->k',a,b)))[good_strides]
    # stride facing rotation
    a=[d_nx[:-1],d_ny[:-1]]
    b=[d_nx[1:],d_ny[1:]]
    rotations = np.rad2deg(np.arctan2(np.cross(a,b,axis=0),np.einsum('ik,ik->k',a,b)))[good_strides]
    return stride_dists, thorax_dists_total, thorax_dists_straight, travel_dir, rotations
    
    
def find_touchdowns_df(df, jj, vel_cutoff, gap_size_to_close, n_stance_fr, min_TD_separation, n_nearby_frs, pix_cutoff,
                       min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff): 
    vel = np.linalg.norm(np.diff(np.array([df['joint%i_x_filt_fullfr'%jj], df['joint%i_y_filt_fullfr'%jj]])), axis=0)
    # how much noise is there?
    if np.sum(vel<1)/np.sum(np.isfinite(vel))>0.6:
        vel_cutoff = 1
        pix_cutoff = 2
    elif np.sum(vel<1.5)/np.sum(np.isfinite(vel))>0.6:
        vel_cutoff = 1.5
        pix_cutoff = 2
    elif np.sum(vel<2)/np.sum(np.isfinite(vel))>0.5:
        vel_cutoff = 2
    else:
        vel_cutoff = 3
    where_slow = vel<vel_cutoff
    where_slow[find_nan_gaps(np.logical_not(where_slow), gap_size_to_close)] = True
    TDs = find_start_of_stance_df(vel, vel_cutoff, gap_size_to_close, n_stance_fr)
    TDs, TDs_x, TDs_y = remove_close_TDs_df(TDs, min_TD_separation, df, jj, n_stance_fr) # remove TDs that are close to eachother
    TDs_fr_idcs, TDs_fr = find_TD_frames_df(df, jj, TDs, TDs_x, TDs_y, n_nearby_frs, pix_cutoff) # find frames of touchdown
    good_TDs, good_strides = remove_uncertain_TD_frames_df(TDs_fr_idcs, vel, where_slow, min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff)
    stride_dur, stride_TDstart, stride_TDstop = find_stride_dur_df(TDs_fr_idcs, TDs_fr, good_strides)
    stride_dist, thorax_dists_total, thorax_dists_straight, travel_dir, rotations = find_stride_dist_df(df, jj, TDs_fr_idcs, good_strides)
#     print(TDs.shape, TDs_f, r_raw.shape, stride_dur_idcs.shape, stride_dist_idcs.shape)
    return vel, TDs_fr_idcs, stride_TDstart, stride_TDstop, stride_dur, stride_dist, thorax_dists_total, \
            thorax_dists_straight, travel_dir, rotations, good_TDs, good_strides


# user set parameters
vel_cutoff= 3 # what velocity cutoff for identifying stances
gap_size_to_close = 3 # interpolate where slow data
n_stance_fr = 5 # how many frames of "stance" for it to be a legit "stance"
min_TD_separation = 10 # how close can subsequent TDs be to each other
n_nearby_frs = 12 # how many frames around TD to look for TD frame (first fr when foot within __ pix of TD location)
pix_cutoff = 3 # how large of a radius around TD location to look for first TD frame
min_n_fr_before_TD = 3 # how many non-nan frames before TD to make sure you're getting the correct TD timing
max_slow_during_swing = 1 # how many slow points allowed outside of initial slow section
fraction_track_cutoff = 0.8 # percent of stride that needs to be tracked (not missing extra TD)


for joint_num in range(0,6):
    print(joint_num)
#     shapeOI = df['frames'].shape
    
    columns_to_drop = ['joint%i_vel'%joint_num, 'joint%i_TD_idcs'%joint_num, 'joint%i_St_start'%joint_num, 'joint%i_St_stop'%joint_num,
                       'joint%i_St_Len'%joint_num, 'joint%i_St_Dur'%joint_num,
                      'joint%i_St_tdist_total'%joint_num, 'joint%i_St_tdist_straight'%joint_num, 'joint%i_St_rotation'%joint_num,
                      'joint%i_good_TDs'%joint_num, 'joint%i_good_strides'%joint_num, 'joint%i_St_travel_dir'%joint_num,
                      'St_Len_all', 'St_Dur_all', 'St_tdist_total', 'St_tdist_straight', 'St_rotation', 'St_travel_dir',
                      'joint%i_TD_x'%joint_num, 'joint%i_TD_y'%joint_num]
    for colmn in columns_to_drop:
        if colmn in df: # remove columns if already exist
            df = df.drop(colmn, axis = 1)

    df = df.reindex( columns = df.columns.tolist() + ['joint%i_vel'%joint_num, 'joint%i_TD_idcs'%joint_num, 'joint%i_St_Len'%joint_num,  
        'joint%i_St_tdist_total'%joint_num, 'joint%i_St_start'%joint_num, 'joint%i_St_stop'%joint_num,
        'joint%i_St_tdist_straight'%joint_num, 'joint%i_St_rotation'%joint_num, 'joint%i_St_travel_dir'%joint_num,
        'joint%i_St_Dur'%joint_num, 'joint%i_good_TDs'%joint_num, 'joint%i_good_strides'%joint_num] )
    df['joint%i_vel'%joint_num], df['joint%i_TD_idcs'%joint_num],  df['joint%i_St_start'%joint_num],  df['joint%i_St_stop'%joint_num], \
        df['joint%i_St_Dur'%joint_num], df['joint%i_St_Len'%joint_num], \
        df['joint%i_St_tdist_total'%joint_num], df['joint%i_St_tdist_straight'%joint_num], df['joint%i_St_travel_dir'%joint_num], \
        df['joint%i_St_rotation'%joint_num], df['joint%i_good_TDs'%joint_num], df['joint%i_good_strides'%joint_num] = zip(*df.apply(
        find_touchdowns_df, args = (joint_num, vel_cutoff, gap_size_to_close, n_stance_fr, min_TD_separation, n_nearby_frs, pix_cutoff, 
                                    min_n_fr_before_TD, max_slow_during_swing, fraction_track_cutoff), axis=1))
    
del colmn, columns_to_drop

print('all done')

0


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:126: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:129: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in long_scalars
/home/gravis

1
2
3
4
5
all done


## Make images and video of raw tracked & analyzed data on background subtract and raw footage
-- for user given trackway number

In [107]:
# plot images with tracked data and lowpass filtered data
tr_num = 1007 #105 #91 #105 # 1662


def load_video(raw_video_path, frame_range, verbose):
    """
    Independent of the frame range loaded, background has to be computed over total video or else can run into
    tracking problems
    """
    vid = cv2.VideoCapture(raw_video_path)
    Height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    NumFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    if not (NumFrames > 0):
        raise IOError('Codec issue: cannot read number of frames.')

    # restrict to desired range of frames
    if frame_range is None:
        frame_range = (0, int(NumFrames))
    else:
        # check doesn't exceed number of frames
        if frame_range[0] + frame_range[1] > NumFrames:
            frame_range = (int(frame_range[0]), int(NumFrames - frame_range[0]))

    # initialize blank frames
    frames = np.zeros((frame_range[1], Height, Width), np.uint8)

    # set the first frame to read in
    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for kk in range(frame_range[0]):
        tru, ret = self.vid.read(1)
    # vid.set(cv.CAP_PROP_POS_FRAMES, frame) # this way of setting the frame doesn't work on all cv versions

    # read in all frames
    for kk in range(frame_range[1]):
        tru, ret = vid.read(1)

        # check if video frames are being loaded
        if not tru:
            raise IOError('Codec issue: cannot load frames.')
        frames[kk, :, :] = ret[:, :, 0]  # assumes loading color
        if ((kk % 100) == 0) and verbose:
            print(kk)
    return frames, NumFrames, frame_range, vid


def remove_background(raw_video_path, frame_range, bkg_method, bkg_sep, verbose):

    # load in video, get video features
    frames, NumFrames, frame_range, vid = load_video(raw_video_path, frame_range, verbose)

    # if all frames loaded, do as normal
    if frame_range[1] == NumFrames:
        background = np.float32(np.median(frames[0::bkg_sep,:,:], axis = 0))
        if verbose:
            print('all_loaded!')
    else: # still use full video for background, not just desired output range
        background = []
        for kk in range(0, NumFrames, bkg_sep):
            vid.set(cv2.CAP_PROP_POS_FRAMES, kk)
            tru, ret = vid.read(1)

            # check if video frames are being loaded
            if not tru:
                raise IOError('Codec issue: cannot load frames.')

            background.append(ret[:,:,0])  # assumes loading color
        background = np.array(background, dtype='float32')
        background = np.float32(np.median(background, axis=0))

    # add a small number to background to not have divide by zeros for division
    background = background + np.float32(1E-6)
    if verbose:
        print('Background calculated')
    if bkg_method == 'div':
        norm_bkg = np.mean(background[:])  # normalize for mean intensity of image
        # norm_frm = np.mean(frames, axis=(1,2)) # normalize for mean intensity of current frame. For flicker
        frames_normed = (frames / norm_bkg) / (background / norm_bkg)  # broadcasting
    elif bkg_method == 'sub':
        raise IOError('Code does not currently support background subtraction, only division')
    else:
        raise IOError('Background divsion/subtraction method not recognized. Use div.')
    if verbose:
        print('Background removed')
    return frames_normed

def augment_contrast(frames, invert, cutoff, verbose):

    # center around 0 (neg = darker than background, pos = lighter than background)
    frames = frames-1

    # if there is a light backgroud, invert images
    if invert:
        frames = -1*frames
        if verbose:
            print('Inverted frames')

    # find max pixel value for each frame
    max_pixel_vals = frames.max(1).max(1)

    # divide by max so that all darker than background pixels are from -1 to 0 and lighter pixels are from 0 to 1
    frames_maxone = frames / max_pixel_vals[:,None,None]
    if frames_maxone.max(1).max(1).max(0) > 1:
        raise IOError('Error in normalized image. Did you background divide?')

    # shift pixel values to determine how dark the background should be
    frames_contrast = (255-cutoff)*frames_maxone + cutoff
    frames_contrast[frames_contrast<0]=0

    return frames_contrast


def WRTant_to_WRTframe(val_x, val_y, frame_center_x, frame_center_y, ant_ang_deg):
    ant_ang = ant_ang_deg *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    rotated_vals = np.dot(R,np.array([val_x-100,val_y-100]))
    translated_vals = rotated_vals*np.array([1,1]) + np.array([frame_center_x, frame_center_y])  
    return translated_vals[0], translated_vals[1];

def plot_ant_pt(ant_part, ant_part_num, filt, df, tr_num, idx, ant_x, ant_y, ant_ang_deg, buffer): #filt = '' if want raw data
    x = df['%s%s_x%s'%(ant_part,str(ant_part_num),filt)][tr_num][idx]
    y = df['%s%s_y%s'%(ant_part,str(ant_part_num),filt)][tr_num][idx]
    conf = df['%s%s_conf'%(ant_part,str(ant_part_num))][tr_num][idx]
#     (newx, newy)= (x,y)
    (newx, newy) = WRTant_to_WRTframe(x, y, ant_x, ant_y, ant_ang_deg)
#     print('old vals: %i, %i  TO %0.1f, %0.1f'%(x,y,newx, newy))

    if ('joint' in ant_part) or ('antenna' in ant_part):
        if '_filt' in filt:
            
            # plot strides and touchdowns
            if 'joint%i_TD_idcs'%ant_part_num in df:
                if idx in df['joint%i_TD_idcs'%ant_part_num][tr_num][df['joint%i_good_TDs'%ant_part_num][tr_num]]:
#                     print('Joint %i -- Fr %i'%(ant_part_num, idx))
                    sca = plt.scatter(newx+buffer, newy+buffer, s = 10, facecolor = 'none', edgecolor = 'w')
                dur_starts = df['joint%i_TD_idcs'%ant_part_num][tr_num][:-1][df['joint%i_good_strides'%ant_part_num][tr_num]]
                dur_stops = df['joint%i_TD_idcs'%ant_part_num][tr_num][1:][df['joint%i_good_strides'%ant_part_num][tr_num]]
                if np.any(np.logical_and( ff>dur_starts, ff<dur_stops)):
                    str_OI = dur_starts[ np.logical_and( ff>dur_starts, ff<dur_stops)]
                    plt.plot([df['joint%i_x_filt_fullfr'%ant_part_num][tr_num][str_OI]+buffer, newx+buffer],
                             [df['joint%i_y_filt_fullfr'%ant_part_num][tr_num][str_OI]+buffer, newy+buffer], '-w', alpha = 0.2)
                
            # plot actual feet points
            if ant_part_num < 3:
                sca = plt.scatter(newx+buffer, newy+buffer, c = 'c', s = 10, edgecolor = 'none')# '.g')
            else:
                sca = plt.scatter(newx+buffer, newy+buffer, c = 'm', s = 10, edgecolor = 'none')
#             if ('joint' in ant_part):
#                 if df['frames_final'][tr_num][fr_num] in df['%s%i_TD_frs'%(ant_part, ant_part_num)][tr_num]:
#                     sca.set_edgecolor('w')
        else:
            # define colormap to show confidence
            norm2 = colors.Normalize(vmin=0, vmax=1)
            plt.scatter(newx+buffer, newy+buffer, c = conf, s = 10, cmap = cm.bwr,
                       edgecolor = 'none', norm=norm2)# '.g')
    else:
        plt.scatter(newx+buffer, newy+buffer, c = 'w', s = 10, edgecolor = 'none')
        
    return;


def crop_to_view(variable_to_use, tr_num, fr, x_dim, y_dim, buffer, axisOI):
    x = df[variable_to_use%'x'][tr_num][ff]
    y = df[variable_to_use%'y'][tr_num][ff]
    xrange = range(int(round(x)), int(round(x+2*buffer)))
    yrange = range(int(round(y)), int(round(y+2*buffer)))
    # account for if range goes outside of video frame
    xrange_actual = np.array(sorted(list( set(xrange) & set(range(0, x_dim+2*buffer) ) )))[[0,-1]]
    yrange_actual = np.array(sorted(list( set(yrange) & set(range(0, y_dim+2*buffer) ) )))[[0,-1]]
    plt.sca(axisOI)
    plt.xlim(xrange_actual)
    plt.ylim(yrange_actual)
    
    return xrange_actual, yrange_actual


def save_image(vlocation, nfig, name_base):
    pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
    plt.savefig(pname)
    nfig = nfig + 1
    plt.pause(0.2)
#     plt.close('all')
    return nfig


def save_video(vlocation, name_base):
    # save images as movie
    if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
        os.remove(vlocation + "/%s.mp4"%name_base)
        print('** Deleted %s.mp4 file'%name_base)
    print('saving %s.mp4 file'%name_base)
    command_p1 = "ffmpeg -r 10 -i '%s/%s"%(vlocation, name_base)
    command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
    command = command_p1 + "%01d.png'" + command_p2
#     print(command)
    os.system(command)
    plt.pause(10)

    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
    for pic in pics2delete:
        os.remove(pic)
    return




# DO THE THINGS
videofile = df.video[tr_num]
print(videofile)
cap = cv2.VideoCapture(videofile)
plt.close('all')
# calc bkgd sub
frames_normed = remove_background(videofile, None, 'div', 50, verbose = False)

fig = plt.figure(figsize=(19.8,10.8))

plt.gcf().text(0.05, 0.7, 'Rough substrates constrain walking speed in ants but not due to large limb perturbations', fontsize = 24)
plt.gcf().text(0.05, 0.62, 'G. T. Clifton, D. Holway, N. Gravish', fontsize = 18)
plt.gcf().text(0.05, 0.54, 'University of California, San Diego', fontsize = 18)
plt.gcf().text(0.05, 0.2, '2019', fontsize = 18)
for title_fr in range(0,10*2):
    save_image(vlocation, title_fr, 'LEAPtracking_filter_PAPER')


limbs = ['LH','LM','LF', 'RH', 'RM', 'RF']
for im_n, fr_OI in enumerate( range(310,510)):# df.frames[tr_num][:]):
#     fr_id = np.where(df.frames_final[tr_num] == fr_OI)[0][0]
    plt.clf()
    # load frame
    ff=np.where(df.frames[tr_num]==fr_OI)[0][0]
    cap.set(1,int(fr_OI))
    ret, frame = cap.read()
    x_dim = frame.shape[1]
    y_dim = frame.shape[0]
    bkgdframe = np.stack((frames_normed[int(fr_OI),:,:],)*3,-1)
    
    # load ant x, y and angle
    x = df.x_raw[tr_num][ff]
    y = df.y_raw[tr_num][ff]
    ang = df.angle_improved[tr_num][ff]
    (thorax_x, thorax_y) = WRTant_to_WRTframe(df.thorax_x[tr_num][ff], df.thorax_y[tr_num][ff], x, y, ang)
    (neck_x, neck_y) = WRTant_to_WRTframe(df.neck_x[tr_num][ff], df.neck_y[tr_num][ff], x, y, ang)
#     print(x,y,ang)

    # PLOT THINGS
    
    # BCKGD SUB IMAGE WITH TRACKED DATA & CONFIDENCE
    ax2=fig.add_axes([0.02,0.1, 0.35, 0.7]) #plt.axes()
    # zoom into around ant
    buffer = 150
    white_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.float32)
    wframe = white_frame.copy()
    wframe[buffer:-buffer, buffer:-buffer,:] = bkgdframe
    wframe = wframe/np.max(wframe)
    plt.imshow(wframe)
    
    xrange_actual, yrange_actual = crop_to_view('%s_raw', tr_num, ff, x_dim, y_dim, buffer, ax2)
#     plt.text(xrange_actual[0]+20, yrange_actual[0]+20, 'Fr: %i'%fr_OI, color='k')
    plt.imshow(wframe)
    cmap = cm.bwr
    plt.scatter(x+buffer, y+buffer, s=20, c=np.array(0.5), norm = colors.Normalize(vmin=0, vmax=1), marker= 'o')
    plt.scatter(x+10, y+2*buffer-10, s=20, c=np.array(0.5), norm = colors.Normalize(vmin=0, vmax=1), marker= 'o')
    plt.text(x+15, y+2*buffer-10, 'estimated ant location from full body tracking', color = 'k', verticalalignment = "center")
    
    if not np.isnan(ang):
        plt.scatter(thorax_x+buffer, thorax_y+buffer, c = df.thorax_conf[tr_num][ff], s = 10, 
                cmap = cmap, norm = colors.Normalize(vmin=0, vmax=1))
        plt.scatter(neck_x+buffer, neck_y+buffer, c = df.neck_conf[tr_num][ff], s = 10, 
                cmap = cmap, norm = colors.Normalize(vmin=0, vmax=1))
        for jj in range(0,6):
            plot_ant_pt('joint',jj, '', df, tr_num, ff, x, y, ang, buffer)
            #plot_ant_pt('joint',jj,'_filt', df, tr_num, ff, x, y, ang, buffer)
        for aa in range(0,2):
            plot_ant_pt('antenna',aa, '', df, tr_num, ff, x, y, ang, buffer)
#     plt.plot(df['joint0_x_filt'][tr_num][ff],df['joint0_y_filt'][tr_num][ff], '.g')
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.gcf().text(0.02, 0.82, 'Raw LEAP output on background-subtracted, cropped view', color='k', FontSize = 12)
    plt.gcf().text(0.02, 0.79, 'frame: %i'%fr_OI, color='k', FontSize = 12)
    
    # COLORBAR
    cax = plt.axes([0.02,0.08,0.35, 0.03])
    plt.colorbar(cax=cax, label = 'confidence', orientation = 'horizontal')
    plt.clim(0,1)
    plt.set_cmap(cm.bwr)
    
    # RAW IMAGE WITH FILTERED DATA
    ax3=fig.add_axes([0.40,0.1, 0.35, 0.7]) #plt.axes()
    black_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.uint8)* 1# 1.001# np.max(temp) # gray background  1.0001#
    bframe = black_frame.copy()
    bframe[buffer:-buffer, buffer:-buffer,:] = frame
    plt.imshow(bframe)
    plt.text(xrange_actual[0]+10, yrange_actual[1]-30, 'Trusted Strides: ', color= 'w')
    for jj in range(0,6):
            plot_ant_pt('joint',jj,'_filt', df, tr_num, ff, x, y, ang, buffer)
            if 'joint%i_TD_idcs'%jj in df:
    
                # plot duration info
                dur_starts = df['joint%i_TD_idcs'%jj][tr_num][:-1][df['joint%i_good_strides'%jj][tr_num]]
                dur_stops = df['joint%i_TD_idcs'%jj][tr_num][1:][df['joint%i_good_strides'%jj][tr_num]]
                if ff in dur_starts:
                    plt.text( xrange_actual[0]+65+jj*15, yrange_actual[1]-30, limbs[jj], color = [1,1,1])
                elif np.any(np.logical_and( ff>dur_starts, ff<dur_stops)):
                    plt.text( xrange_actual[0]+65+jj*15, yrange_actual[1]-30, limbs[jj], color=  ['c','m'][int(jj/3)])
        
    plot_ant_pt('thorax', '', '_filt', df, tr_num, ff, x, y, ang, buffer)
    plot_ant_pt('neck', '', '_filt', df, tr_num, ff, x, y, ang, buffer)
    plt.xlim(xrange_actual)
    plt.ylim(yrange_actual)
    plt.gca().invert_yaxis()
    plt.axis('off')
    
    plt.scatter(xrange_actual[0]+10, yrange_actual[1]-20, s = 10, facecolor = 'none', edgecolor = 'w')
    plt.text( xrange_actual[0]+17, yrange_actual[1]-20, 'touchdown', color = [1,1,1], verticalalignment = "center")
    plt.plot([xrange_actual[0]+10, xrange_actual[0]+14],[yrange_actual[1]-10, yrange_actual[1]-10], '-w', alpha = 0.2)
    plt.text( xrange_actual[0]+17, yrange_actual[1]-10, 'swing trajectory', color = [1,1,1], verticalalignment = "center")
    plt.gcf().text(0.4, 0.82, 'Post-processed body and limb tracking', color='k', FontSize = 12)
    plt.gcf().text(0.4, 0.79, 'low confidence removed, lowpass filtered', color='k', FontSize = 12)
    
    # FULL FRAME
    ax1=fig.add_axes([0.78,0.77, 0.2, 0.23])
    ax1.set_position([0.78,0.77, 0.2, 0.23])
    plt.imshow(frame)
    plt.plot(x, y, '.w')
    plt.axis('off')
    
    # VELOCITY TRACES OF LEGS
    ax4=fig.add_axes([0.78,0.14, 0.2, 0.60])#plt.axes()
    for jj in range(0,6):
#         plt.plot(range(1,ff),df['joint%i_x_filt'%jj][tr_num][1:ff]-df.thorax_x_filt[tr_num][1:ff], '-', color= col)
        plt.plot(range(1,ff),df['joint%i_vel'%jj][tr_num][1:ff]+jj%3*20, '-', color = ['c','m'][int(jj/3)], alpha = 0.3)
        if 'joint%i_TD_idcs'%jj in df:
            TDs = df['joint%i_TD_idcs'%jj][tr_num][df['joint%i_good_TDs'%jj][tr_num]]
            idcs_OI = TDs[TDs<=ff]
#                 plt.plot(idcs_OI,df['joint%i_x_filt'%jj][tr_num][idcs_OI]-df.thorax_x_filt[tr_num][idcs_OI], '.', color = col)
            plt.plot(idcs_OI,df['joint%i_vel'%jj][tr_num][idcs_OI]+jj%3*20, '.', color = ['c','m'][int(jj/3)])              
    plt.xlim((-10,len(df['frames'][tr_num])))
    plt.ylim((0,60))
    plt.text(-10,51,'forelimbs')
    plt.text(-10,31,'midlimbs')
    plt.text(-10,11,'hindlimbs')
    plt.text(-10,57,'limbs:')
    plt.text(70,57,'left', color = 'c')
    plt.text(120,57,'right', color = 'm')
    plt.text(-10,55,'identified touchdowns:')
    plt.plot(250,55.4,'.', color = 'c')
    plt.plot(272,55.4,'.', color = 'm')
    plt.plot([500,500],[50,50-10/v_multiplier],'-k')
    plt.text(490,50-5/v_multiplier,'10 mm/s', color ='k', horizontalalignment='right', verticalalignment='center')
    plt.gcf().text(0.78, 0.74, 'limb velocities', color='k', FontSize = 12)
    plt.axis('off')
    
    titleparts = videofile.split('/')
    plt.suptitle( '%s -- %s -- %s'
                  %(titleparts[-2], titleparts[-1].split('_')[0], titleparts[-1].split('_')[1]),x=0.02, y=.95, horizontalalignment = 'left')
    plt.pause(0.1)
    
    vlocation = '/media/gravishlab/SeagateExpansionDrive/AntTrack'
    save_image(vlocation, im_n+10*2, 'LEAPtracking_filter_PAPER')
save_video(vlocation, 'LEAPtracking_filter_PAPER')
    
# plt.close()

/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/3mm/20180314_094259_16276718-0000.mp4
** Deleted LEAPtracking_filter_PAPER.mp4 file
saving LEAPtracking_filter_PAPER.mp4 file


#### Run through all tracks and visualize

In [ ]:
# plot images with tracked data and lowpass filtered data
print('running data visualization')

def load_video(raw_video_path, frame_range, verbose):
    """
    Independent of the frame range loaded, background has to be computed over total video or else can run into
    tracking problems
    """
    vid = cv2.VideoCapture(raw_video_path)
    Height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    NumFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    if not (NumFrames > 0):
        raise IOError('Codec issue: cannot read number of frames.')

    # restrict to desired range of frames
    if frame_range is None:
        frame_range = (0, int(NumFrames))
    else:
        # check doesn't exceed number of frames
        if frame_range[0] + frame_range[1] > NumFrames:
            frame_range = (int(frame_range[0]), int(NumFrames - frame_range[0]))

    # initialize blank frames
    frames = np.zeros((frame_range[1], Height, Width), np.uint8)

    # set the first frame to read in
    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for kk in range(frame_range[0]):
        tru, ret = self.vid.read(1)
    # vid.set(cv.CAP_PROP_POS_FRAMES, frame) # this way of setting the frame doesn't work on all cv versions

    # read in all frames
    for kk in range(frame_range[1]):
        tru, ret = vid.read(1)

        # check if video frames are being loaded
        if not tru:
            raise IOError('Codec issue: cannot load frames.')
        frames[kk, :, :] = ret[:, :, 0]  # assumes loading color
        if ((kk % 100) == 0) and verbose:
            print(kk)
    return frames, NumFrames, frame_range, vid


def remove_background(frames, bkg_method, bkg_sep, verbose):

    # load in video, get video features
#     frames, NumFrames, frame_range, vid = load_video(raw_video_path, frame_range, verbose)

    # if all frames loaded, do as normal
    background = np.float32(np.median(frames[0::bkg_sep,:,:], axis = 0))
    if verbose:
        print('all_loaded!')
    background = np.array(background, dtype='float32')
    background = np.float32(np.median(background, axis=0))

    # add a small number to background to not have divide by zeros for division
    background = background + np.float32(1E-6)
    if verbose:
        print('Background calculated')
    if bkg_method == 'div':
        norm_bkg = np.mean(background[:])  # normalize for mean intensity of image
        # norm_frm = np.mean(frames, axis=(1,2)) # normalize for mean intensity of current frame. For flicker
        frames_normed = (frames / norm_bkg) / (background / norm_bkg)  # broadcasting
    elif bkg_method == 'sub':
        raise IOError('Code does not currently support background subtraction, only division')
    else:
        raise IOError('Background divsion/subtraction method not recognized. Use div.')
    if verbose:
        print('Background removed')
    return frames_normed

def augment_contrast(frames, invert, cutoff, verbose):

    # center around 0 (neg = darker than background, pos = lighter than background)
    frames = frames-1

    # if there is a light backgroud, invert images
    if invert:
        frames = -1*frames
        if verbose:
            print('Inverted frames')

    # find max pixel value for each frame
    max_pixel_vals = frames.max(1).max(1)

    # divide by max so that all darker than background pixels are from -1 to 0 and lighter pixels are from 0 to 1
    frames_maxone = frames / max_pixel_vals[:,None,None]
    if frames_maxone.max(1).max(1).max(0) > 1:
        raise IOError('Error in normalized image. Did you background divide?')

    # shift pixel values to determine how dark the background should be
    frames_contrast = (255-cutoff)*frames_maxone + cutoff
    frames_contrast[frames_contrast<0]=0

    return frames_contrast


def WRTant_to_WRTframe(val_x, val_y, frame_center_x, frame_center_y, ant_ang_deg):
    ant_ang = ant_ang_deg *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    rotated_vals = np.dot(R,np.array([val_x-100,val_y-100]))
    translated_vals = rotated_vals*np.array([1,1]) + np.array([frame_center_x, frame_center_y])  
    return translated_vals[0], translated_vals[1];

def plot_full_frame(fr_OI, x, y):
    plt.sca(ax1)
    frame_toshow = np.stack((frames[int(fr_OI),:,:],)*3,-1)
#     ff=np.where(df.frames[tr_num]==fr_OI)[0][0]
#     x = df.x_raw[tr_num][ff]
#     y = df.y_raw[tr_num][ff]
    
    ax1.cla()
    plt.imshow(frame_toshow)
    plt.plot(x, y, '.w')
    plt.axis('off')
    return

def plot_raw_tracking(fr_OI, x, y, ff, ang):
    plt.sca(ax2)
    ax2.cla()
    # BCKGD SUB IMAGE WITH TRACKED DATA & CONFIDENCE
    bkgdframe = np.stack((frames_normed[int(fr_OI),:,:],)*3,-1)

    # zoom into around ant
    buffer = 150
    white_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.float32)
    wframe = white_frame.copy()
    wframe[buffer:-buffer, buffer:-buffer,:] = bkgdframe
    wframe = wframe/np.max(wframe)
    
    plt.imshow(wframe)
    xrange_actual, yrange_actual = crop_to_view('%s_raw', tr_num, ff, x_dim, y_dim, buffer, ax2)
    plt.text(xrange_actual[0]+20, yrange_actual[0]+20, 'Fr: %i'%fr_OI, color='k')
    cmap = cm.bwr

    if not np.isnan(ang):
        plot_ant_pt('neck','', '', df, tr_num, ff, x, y, ang, buffer)
        plot_ant_pt('thorax', '', '', df, tr_num, ff, x, y, ang, buffer)
        for jj in range(0,6):
            plot_ant_pt('joint',jj, '', df, tr_num, ff, x, y, ang, buffer)
        for aa in range(0,2):
            plot_ant_pt('antenna',aa, '', df, tr_num, ff, x, y, ang, buffer)
    
    plt.gca().invert_yaxis()
    plt.axis('off')
    return

def plot_final_tracking(fr_OI, x, y, ff, ang):
    plt.sca(ax3)
    ax3.cla()
    buffer = 150
    frame = np.stack((frames[int(fr_OI),:,:],)*3,-1)
    black_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.uint8)* 1# 1.001# np.max(temp) # gray background  1.0001#
    bframe = black_frame.copy()
    bframe[buffer:-buffer, buffer:-buffer,:] = frame
    plt.imshow(bframe)
    if fr_OI in df.frames_final[tr_num]:
        fr_id = np.where(df.frames_final[tr_num] == fr_OI)[0][0]
        for jj in range(0,6):
                plot_ant_pt('joint',jj,'_filt', df, tr_num, ff, x, y, ang, buffer)
        plot_ant_pt('thorax', '', '_filt', df, tr_num, ff, x, y, ang, buffer)
        plot_ant_pt('neck', '', '_filt', df, tr_num, ff, x, y, ang, buffer)
    xrange_actual, yrange_actual = crop_to_view('%s_raw', tr_num, ff, x_dim, y_dim, buffer, ax3)
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.text(xrange_actual[0]+20, yrange_actual[0]+20, 'Low conf removed, lowpass filtered', color='w')
    return

def plot_ant_pt(ant_part, ant_part_num, filt, df, tr_num, fr_num, ant_x, ant_y, ant_ang_deg, buffer): #filt = '' if want raw data
    x = df['%s%s_x%s'%(ant_part,str(ant_part_num),filt)][tr_num][fr_num]
    y = df['%s%s_y%s'%(ant_part,str(ant_part_num),filt)][tr_num][fr_num]
    conf = df['%s%s_conf'%(ant_part,str(ant_part_num))][tr_num][fr_num]
#     (newx, newy)= (x,y)
    (newx, newy) = WRTant_to_WRTframe(x, y, ant_x, ant_y, ant_ang_deg)
#     print('old vals: %i, %i  TO %0.1f, %0.1f'%(x,y,newx, newy))

    if ('joint' in ant_part) or ('antenna' in ant_part):
        if '_filt' in filt:
            if ant_part_num < 3:
                sca = plt.scatter(newx+buffer, newy+buffer, c='c', s = 10, edgecolor = 'none')
            else:
                sca = plt.scatter(newx+buffer, newy+buffer, c = 'm', s = 10, edgecolor = 'none')
                
#             if ('joint' in ant_part):
#                 if df['frames_final'][tr_num][fr_num] in df['%s%i_TD_frs'%(ant_part, ant_part_num)][tr_num]:
#                     sca.set_edgecolor('w')
        else:
            # define colormap to show confidence
            norm2 = colors.Normalize(vmin=0, vmax=1)
            plt.scatter(newx+buffer, newy+buffer, c = conf, s = 10, cmap = cm.bwr,
                       edgecolor = 'none', norm=norm2)# '.g')
    else:
        plt.scatter(newx+buffer, newy+buffer, c = 'w', s = 10, edgecolor = 'none')
        
    return;


def crop_to_view(variable_to_use, tr_num, ff, x_dim, y_dim, buffer, axisOI):
    x = df[variable_to_use%'x'][tr_num][ff]
    y = df[variable_to_use%'y'][tr_num][ff]
    xrange = range(int(round(x)), int(round(x+2*buffer)))
    yrange = range(int(round(y)), int(round(y+2*buffer)))
    # account for if range goes outside of video frame
    xrange_actual = np.array(sorted(list( set(xrange) & set(range(0, x_dim+2*buffer) ) )))[[0,-1]]
    yrange_actual = np.array(sorted(list( set(yrange) & set(range(0, y_dim+2*buffer) ) )))[[0,-1]]
    plt.sca(axisOI)
    plt.xlim(xrange_actual)
    plt.ylim(yrange_actual)
    return xrange_actual, yrange_actual


def key_event(e):
    global fr_OI
    global close_fig

    if e.key == "right":
        fr_OI = fr_OI + 1
    elif e.key == "left":
        fr_OI = fr_OI - 1
    elif e.key == "w":
        close_fig = True
        return
    else:
        return
    fr_OI = int(
        (fr_OI-df.frames_final[tr_num][0]) % (df.frames_final[tr_num][-1]-df.frames_final[tr_num][0]+1) 
        + df.frames_final[tr_num][0])
    ff=np.where(df.frames[tr_num]==fr_OI)[0][0]
    x = df.x_raw[tr_num][ff]
    y = df.y_raw[tr_num][ff]
    ang = df.angle_improved[tr_num][ff]
    (thorax_x, thorax_y) = WRTant_to_WRTframe(df.thorax_x[tr_num][ff], df.thorax_y[tr_num][ff], x, y, ang)
    (neck_x, neck_y) = WRTant_to_WRTframe(df.neck_x[tr_num][ff], df.neck_y[tr_num][ff], x, y, ang)
    
#     plt.plot(x, y, '.w')
    plt.axis('off')

    plot_full_frame(fr_OI, x, y)
    plot_raw_tracking(fr_OI, x, y, ff, ang)
    plot_final_tracking(fr_OI, x, y, ff, ang)


    



#     plt.title('%s -- %s -- %s'%(folder_bits[-2], ('_').join(folder_bits[-1].split('_')[0:2]), h_path.split('/')[-1][:-3]), loc = 'left')
    fig.canvas.draw()

def close_out_of_program():
    continueon = input('Press any key to exit:  ')
    print('\nClosing down viewer...')
    return #close_up_shop
    
    
    
    
i=threading.Thread(target=close_out_of_program, args = ())
i.start()
print('*** PRESS "w" TO CLOSE A FIGURE AND MOVE TO THE NEXT ANT H5 ***')
for kk, tr_num in enumerate(range(45,49)):
    close_fig = False
    videofile = df.video[tr_num]
    print(videofile)
    cap = cv2.VideoCapture(videofile)
    plt.close('all')
    # calc bkgd sub
    frames, NumFrames, frame_range, vid = load_video(videofile, None, verbose=False)
    frames_normed = remove_background(frames, 'div', 50, verbose = False)
    
    fig = plt.figure(figsize=(16,6))
    fr_OI = int(df.frames_final[tr_num][0])
    ff=np.where(df.frames[tr_num]==fr_OI)[0][0]

    fr_id = np.where(df.frames_final[tr_num] == fr_OI)[0][0]
    frame = frames[fr_OI]
    x_dim = frame.shape[1]
    y_dim = frame.shape[0]
    bkgdframe = np.stack((frames_normed[int(fr_OI),:,:],)*3,-1)

    # load ant x, y and angle
    x = df.x_raw[tr_num][ff]
    y = df.y_raw[tr_num][ff]
    ang = df.angle_improved[tr_num][ff]
    (thorax_x, thorax_y) = WRTant_to_WRTframe(df.thorax_x[tr_num][ff], df.thorax_y[tr_num][ff], x, y, ang)
    (neck_x, neck_y) = WRTant_to_WRTframe(df.neck_x[tr_num][ff], df.neck_y[tr_num][ff], x, y, ang)
    
    
    mngr = plt.get_current_fig_manager()
    mngr.window.setGeometry(1550,300,1000,400)
    
    ax1=fig.add_axes([0.03,0.1, 0.2, 0.3])
    plot_full_frame(fr_OI,x , y)
    
    ax2=fig.add_axes([0.25,0.1, 0.3, 0.8]) 
    plot_raw_tracking(fr_OI, x, y, ff, ang)
    
    # COLORBAR
    cax = plt.axes([0.58,0.1,0.02,0.8])
    plt.colorbar(cax=cax, label = 'confidence')
    plt.clim(0,1)
    plt.set_cmap(cm.bwr)
    
    ax3=fig.add_axes([0.65,0.1, 0.3, 0.8]) 
    plot_final_tracking(fr_OI, x, y, ff, ang)
    
    
    fig.canvas.mpl_connect('key_press_event', key_event)
    plt.show(block=False)

    fig_open = plt.fignum_exists(fig.number)
    while (not close_fig) and fig_open:
        plt.pause(3)
        if not i.isAlive():
#                 print('User input has ended. Closing...')
            break
    plt.close(fig)

    if not i.isAlive():
#             print('User input has ended. Closing...')
        break

    


    
#     # POSITION TRACES OF LEGS
#     ax4=fig.add_axes([0.03,0.35, 0.2, 0.6])#plt.axes()

#     for jj in range(0,6):
#         if jj <3 :
#             plt.plot(range(1,fr_id),df['joint%i_x_filt'%jj][tr_num][1:fr_id]-df.thorax_x_filt[tr_num][1:fr_id], '.c')
#             plt.plot(range(1,fr_id),df['joint%i_x_filt'%jj][tr_num][1:fr_id]-df.thorax_x_filt[tr_num][1:fr_id], '-c')
#         else:
#             plt.plot(range(1,fr_id),df['joint%i_x_filt'%jj][tr_num][1:fr_id]-df.thorax_x_filt[tr_num][1:fr_id], '-m')
#     plt.xlim((-10,len(df['frames_final'][tr_num])))
#     plt.ylim((-100,70))
#     plt.text(-10,30,'fore')
#     plt.text(-10,-20,'mid')
#     plt.text(-10,-70,'hind')
#     plt.axis('off')
    
#     titleparts = videofile.split('/')
#     plt.suptitle( '%s -- %s -- %s'
#                   %(titleparts[-2], titleparts[-1].split('_')[0], titleparts[-1].split('_')[1]),x=0.03, y=.95, horizontalalignment = 'left')
#     plt.pause(0.1)
    
    
# plt.close()

In [ ]:
# PLOT FULLFRAME
tr_num = 46#105 #91 #105
videofile = df.video[tr_num]
print(videofile)
cap = cv2.VideoCapture(videofile)
plt.close('all')
# calc bkgd sub
frames_normed = remove_background(videofile, None, 'div', 50, verbose = False)


# def plot_ant_pt(ant_part, ant_part_num, filt, df, tr_num, fr_num, ant_x, ant_y, ant_ang_deg, buffer): #filt = '' if want raw data
#     x = df['%s%s_x%s'%(ant_part,str(ant_part_num),filt)][tr_num][fr_num]
#     y = df['%s%s_y%s'%(ant_part,str(ant_part_num),filt)][tr_num][fr_num]
#     conf = df['%s%s_conf'%(ant_part,str(ant_part_num))][tr_num][fr_num]
#     (newx, newy)= (x,y)
# #     print('%s%s_x%s'%(ant_part,str(ant_part_num),filt), tr_num, fr_num, newx,newy)
# #     (newx, newy) = WRTant_to_WRTframe(x, y, ant_x, ant_y, ant_ang_deg)
# #     print('old vals: %i, %i  TO %0.1f, %0.1f'%(x,y,newx, newy))

#     if ('joint' in ant_part) or ('antenna' in ant_part):
#         if '_filt' in filt:
#             if ant_part_num < 3:
#                 sca = plt.scatter(newx+buffer, newy+buffer, c = 'c', s = 10, edgecolor = 'none')# '.g')
#             else:
#                 sca = plt.scatter(newx+buffer, newy+buffer, c = 'm', s = 10, edgecolor = 'none')
                
#             if ('joint' in ant_part):
#                 if df['frames_final'][tr_num][fr_num] in df['%s%i_TD_frs'%(ant_part, ant_part_num)][tr_num]:
#                     sca.set_edgecolor('w')
#         else:
#             # define colormap to show confidence
#             norm2 = colors.Normalize(vmin=0, vmax=1)
#             plt.scatter(newx+buffer, newy+buffer, c = conf, s = 10, cmap = cm.bwr,
#                        edgecolor = 'none', norm=norm2)# '.g')
#     else:
#         plt.scatter(newx+buffer, newy+buffer, c = 'w', s = 10, edgecolor = 'none')
        
#     return;


fig = plt.figure(figsize=(16,6))
for im_n, fr_OI in enumerate( df.frames_final[tr_num][20:]):
    fr_id = np.where(np.isin( df.frames_final[tr_num], fr_OI))
    plt.clf()
    # load frame
    ff=np.where(df.frames[tr_num]==fr_OI)[0][0]
    cap.set(1,int(fr_OI))
    ret, frame = cap.read()
    x_dim = frame.shape[1]
    y_dim = frame.shape[0]
    bkgdframe = np.stack((frames_normed[int(fr_OI),:,:],)*3,-1)
    
    # load ant x, y and angle
    x = df.x_raw[tr_num][ff]
    y = df.y_raw[tr_num][ff]
    ang = df.angle_improved[tr_num][ff]
    (thorax_x, thorax_y) = WRTant_to_WRTframe(df.thorax_x[tr_num][ff], df.thorax_y[tr_num][ff], x, y, ang)
    (neck_x, neck_y) = WRTant_to_WRTframe(df.neck_x[tr_num][ff], df.neck_y[tr_num][ff], x, y, ang)
#     print(x,y,ang)
    
    

    # PLOT THINGS
#     ax1=fig.add_axes([0.03,0.1, 0.2, 0.3])
#     ax1.set_position([0.03,0.1, 0.2, 0.3])
    plt.imshow(frame)
#     plt.plot(x, y, '.w')
#     plt.axis('off')
    
    for jj in range(0,6):
            plot_ant_pt('joint',jj,'_filt_fullfr', df, tr_num, fr_id, x, y, ang, buffer=0)
    plot_ant_pt('thorax', '', '_filt_fullfr', df, tr_num, fr_id, x, y, ang, buffer=0)
    plot_ant_pt('neck', '', '_filt_fullfr', df, tr_num, fr_id, x, y, ang, buffer=0)
    plt.text(10,30, 'Fr: %i'%fr_OI, color = 'w')
    titleparts = df.video[tr_num].split('/')
    plt.suptitle( 'TRIAL %i:  %s -- %s -- %s'%(tr_num, titleparts[-2], 
                        titleparts[-1].split('_')[0], titleparts[-1].split('_')[1]),x=0.03, y=.95, horizontalalignment = 'left')
#     plt.pause(0.1)
    vlocation = '/media/gravishlab/SeagateExpansionDrive/AntTrack'
    save_image(vlocation, im_n, 'LEAPtracking_filter_fullframe')
save_video(vlocation, 'LEAPtracking_filter_fullframe')

# Touchdown heights

### Calculate height of feet for whole dataframe

In [13]:
# CALCULATE STEP HEIGHTS FOR WHOLE DATAFRAME


# load in all step height profiles into dataframe based on colony and substrate
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
sh_folder_list = []
sh_folder_list = glob.glob(os.path.join(vid_locations, '**/*[0-9]mm/'))
sh_folder_list = sorted(sh_folder_list)
print('Total Number of Videos: ',len(sh_folder_list))
step_height_data = []
for folder in sh_folder_list:
    file = ('/').join(folder.split('/')[:-1])+'/'+('_').join(folder.split('/')[-3:])+'Step_Height.pkl'
    colony = folder.split('/')[-3]
    substrate = folder.split('/')[-2]
#     print(file)
    temp = {}
    with open(file, 'rb') as f:
        temp['hlines'], temp['vlines'], temp['height_profile'],_ = pickle.load(f)
    f.close()
    temp['colony']=colony
    temp['substrate']=substrate
    step_height_data.append(temp)
    
step_height_df = pd.DataFrame(step_height_data).copy()
# del step_height_data
          
    
# def get_TD_location_df(x, part):
#     all_x = x['%s_x_filt_fullfr'%part]
#     all_y = x['%s_y_filt_fullfr'%part]
#     TD_x = all_x[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
#     TD_y = all_y[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]]
#     TD_x = TD_x
#     TD_y = TD_y
#     return TD_x, TD_y

def get_TD_height_df(x, part, step_height_df):
    all_x = x['%s_x_filt_fullfr'%part]
    all_y = x['%s_y_filt_fullfr'%part]
    TD_x = all_x[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    TD_y = all_y[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]]
    TD_x = TD_x
    TD_y = TD_y
    n_pts = len(TD_x)
    
    col_OI = x['colony']
    sub_OI = x['substrate']
    step_heights = np.ones(TD_x.shape)
    if sub_OI == '0mm': # all steps are on top for flat substrate
        return TD_x, TD_y, step_heights
    
    step_height_OI = step_height_df[
        step_height_df['substrate'].str.contains(sub_OI) & step_height_df['colony'].str.contains(col_OI)]
    hlines = step_height_OI['hlines'].values[0]
    n_hlines = len(hlines)
    vlines = step_height_OI['vlines'].values[0]
    n_vlines = len(vlines)
    height_profile = step_height_OI['height_profile'].values[0]
    
    y_pred = np.repeat(TD_x[np.newaxis,:],n_hlines, axis =0)*np.repeat(hlines[:,1][:,np.newaxis],n_pts,axis=1)+ \
        np.repeat(hlines[:,0][:,np.newaxis],n_pts,axis=1)
    y_idcs = np.sum(y_pred<TD_y,axis =0)-1
    x_pred = np.repeat(TD_y[np.newaxis,:],n_vlines, axis =0)*1/(np.repeat(vlines[:,1][:,np.newaxis],n_pts,axis=1))- \
        np.repeat((vlines[:,0]/vlines[:,1])[:,np.newaxis],n_pts,axis=1)
    x_idcs = np.sum(x_pred<TD_x,axis =0)-1
    
    off_substrate = np.logical_or(np.logical_or(x_idcs<0,x_idcs>=(n_vlines-1)), np.logical_or(y_idcs<0,y_idcs>=(n_hlines-1)))
    on_substrate = np.logical_not(off_substrate)
    step_heights[on_substrate]  = height_profile[y_idcs[on_substrate], x_idcs[on_substrate]]
#     print(col_OI, sub_OI, TD_y, y_idcs)
    return TD_x, TD_y, step_heights


# compile data of TD locations and heights
print('\ncalculating step heights')
for joint_num in range(0,6):
#     df = df.drop(['joint%i_St_Heights'%joint_num], axis = 1)
    df['joint%i_St_TD_x'%joint_num], df['joint%i_St_TD_y'%joint_num], df['joint%i_St_Heights'%joint_num] = zip(*
                                                        df.apply(get_TD_height_df, args = ('joint%i'%joint_num, step_height_df), axis=1))

    #     df = df.drop(['joint%i_TD_x'%joint_num, 'joint%i_TD_y'%joint_num], axis = 1)
#     df['joint%i_St_TD_x'%joint_num], df['joint%i_St_TD_y'%joint_num] = zip(*df.apply(get_TD_location_df, args = ('joint%i'%joint_num,), axis=1))
print('\nDone analyzing step locations and heights!')

Total Number of Videos:  32

calculating step heights


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in less
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in less



Done analyzing step locations and heights!


### Test out step height calculations for given trackway

In [ ]:
# try calculating joint height for a given trial to see what's going wrong
part = 'joint0'
tr = 10763
all_x = df['%s_x_filt_fullfr'%part][tr]
all_y = df['%s_y_filt_fullfr'%part][tr]
TD_x = all_x[df['%s_TD_idcs'%part][tr][:-1][df['%s_good_strides'%part][tr]]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
TD_y = all_y[df['%s_TD_idcs'%part][tr][:-1][df['%s_good_strides'%part][tr]]]
TD_x = TD_x
TD_y = TD_y
n_pts = len(TD_x)
print(TD_x, TD_y)

step_heights = np.ones(TD_x.shape)

col_OI = df['colony'][tr]
sub_OI = df['substrate'][tr]
if sub_OI == '0mm': # all steps are on top for flat substrate
    step_heights = np.ones(TD_x.shape)

print(col_OI, sub_OI, df['datetime'][tr])

step_height_OI = step_height_df[
    step_height_df['substrate'].str.contains(sub_OI) & step_height_df['colony'].str.contains(col_OI)]
hlines = step_height_OI['hlines'].values[0]
n_hlines = len(hlines)
vlines = step_height_OI['vlines'].values[0]
n_vlines = len(vlines)
height_profile = step_height_OI['height_profile'].values[0]

y_pred = np.repeat(TD_x[np.newaxis,:],n_hlines, axis =0)*np.repeat(hlines[:,1][:,np.newaxis],n_pts,axis=1)+ \
    np.repeat(hlines[:,0][:,np.newaxis],n_pts,axis=1)
y_idcs = np.sum(y_pred<TD_y,axis =0)-1

x_pred = np.repeat(TD_y[np.newaxis,:],n_vlines, axis =0)*1/(np.repeat(vlines[:,1][:,np.newaxis],n_pts,axis=1))- \
    np.repeat((vlines[:,0]/vlines[:,1])[:,np.newaxis],n_pts,axis=1)
x_idcs = np.sum(x_pred<TD_x,axis =0)-1
off_substrate = np.logical_or(np.logical_or(x_idcs<0,x_idcs>=(n_vlines-1)), np.logical_or(y_idcs<0,y_idcs>=(n_hlines-1)))
on_substrate = np.logical_not(off_substrate)
print(x_idcs, y_idcs)




# print(col_OI, sub_OI, y_idcs, TD_x)
step_heights[on_substrate]  = height_profile[y_idcs[on_substrate], x_idcs[on_substrate]]

### Calculate % of touchdowns that are on peak vs. valley for each substrate/colony

In [ ]:
# DETERMINE % OF STEPS ON PEAKS VS. VALLEYS

# Calculate stride heights
df['St_Heights'] = df.filter(regex='_St_Heights$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
pix2mm = 959.7563/30
fps = 240
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))
df['St_TD_x'] = df.filter(regex='_St_TD_x$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_TD_y'] = df.filter(regex='_St_TD_y$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
lens = [len(item) for item in df['St_Heights']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values), 
                        "St_Heights" : np.concatenate(df['St_Heights'].values),
                        "St_TD_x" : np.concatenate(df['St_TD_x'].values), "St_TD_y" : np.concatenate(df['St_TD_y'].values)})


# # plot distribution of all analyzed TDS to make sure no weirdness
# allsubs = [tr['substrate'] for tr in trial_info]
# subtypes = sorted(list(set(allsubs)))
# plt.close('all')
# plt.figure(figsize=(17,3))
# for ss,subtype in enumerate(subtypes[0:4]):
#     plt.subplot(1,4,ss+1)
#     xs = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_TD_x'].values
#     ys = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_TD_y'].values
#     plt.plot(xs,ys,'.k', alpha = 0.01, MarkerSize = 2)
#     rect = Rectangle((0,0),1000,550, linewidth =1, edgecolor = 'k', facecolor = 'none')
#     plt.gca().add_patch(rect)
#     plt.title('%s'%subtype, horizontalalignment='left', loc = 'left')
#     plt.axis('equal')
#     plt.axis('off')


# # plot distribution of all analyzed TDs for each colony to make sure no weirdness
# allsubs = [tr['substrate'] for tr in trial_info]
# subtypes = sorted(list(set(allsubs)))
# coltypes = sorted(list(set(df['colony'].values)))
# plt.close('all')
# plt.figure(figsize=(12,12))
# for cc,coltype in enumerate(coltypes[:-1]):
#     for ss,subtype in enumerate(subtypes[0:4]):
#         plt.subplot(7,4,ss+1+(cc*4))
#         xs = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']==coltype)]['St_TD_x'].values
#         ys = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']==coltype)]['St_TD_y'].values
#         plt.plot(xs,ys,'.k', alpha = 0.015, MarkerSize = 2)
#         rect = Rectangle((0,0),1000,550, linewidth =1, edgecolor = 'k', facecolor = 'none')
#         plt.gca().add_patch(rect)
#         plt.axis('equal')
#         plt.axis('off')
#         if cc==0:
#             plt.title('%s'%subtype, horizontalalignment='left', loc = 'left')



# how much of ground is peak vs. valley
allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
percent_of_substrate_peak = np.full(4,np.nan)
percent_of_substrate_peak[0]=100
for kk in range(1,4):
    idx = subtypes.index(step_height_data[kk]['substrate'])
    profile = step_height_data[kk]['height_profile']
    box_size = (idx-1)*2+1
    total_peak_area = np.sum(
        profile*np.repeat(np.concatenate([np.full(profile.shape[0]-1, box_size),[1]])[:,np.newaxis], profile.shape[1],axis=1)*box_size)
    total_area = 16*30
    percent_of_substrate_peak[idx] = total_peak_area/total_area*100


# what percentage of all recorded strides is on a peak for each substrate & colony
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
coltypes = sorted(list(set(df['colony'].values)))
TD_height_data = np.full((4,6,3),np.nan)
for ss, subtype in enumerate(subtypes[0:4]):
    print('\nSUBSTRATE: %s, %0.2f percent of substrate is peaks'%(subtype, percent_of_substrate_peak[ss]))
    for cc,coltype in enumerate(coltypes[:-2]):
        xvals = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']==coltype)]['St_Heights'].values
        TD_height_data[ss,cc,:]=[np.sum(xvals), len(xvals)-np.sum(np.sum(xvals)), len(xvals)]
        print('  %s: n = %i --- %0.2f percent on peaks'%(coltype, len(xvals), np.sum(xvals)/len(xvals)*100))
plt.figure()
percent_TD_peaks = TD_height_data[:,:,0]/TD_height_data[:,:,2]*100
yerrs = np.abs([np.min(percent_TD_peaks,axis=1), np.max(percent_TD_peaks,axis=1)]-np.mean(percent_TD_peaks, axis=1))
plt.bar(np.arange(4), np.mean(percent_TD_peaks, axis =1), width=1, yerr = yerrs, color = pltcolors)
plt.plot(np.vstack([np.arange(0,4)-.5, np.arange(0,4)+0.5]), np.repeat(percent_of_substrate_peak[:,np.newaxis],2, axis =1).T, ':k')
plt.ylim([0,100])
plt.xlim([-0.5,3.5])
plt.xticks(np.arange(4),['0','1','3','5'])
plt.title('% of TDs that are on peaks', loc= 'left')
plt.xlabel('substrate size (mm)')

# is there a preference for a given foot to be on peak vs. valley
print('\n')
for ss, subtype in enumerate(subtypes[0:4]):
    print('\nSUBSTRATE: %s, %0.2f percent of substrate is peaks'%(subtype, percent_of_substrate_peak[ss]))
    xvals = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony'] != coltypes[-1])]['St_Heights'].values
    print('SUB TOTAL n TDs: %i'%len(xvals))
    print('p-value: ', stats.ttest_1samp(percent_TD_peaks[ss,:], percent_of_substrate_peak[ss])[1]) # two-sided t-test for each substrate
     
    for jj in range(0,6):
        xvals = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['Joints_all']!='joint%s'%jj)]['St_Heights'].values
        print('  %s: n = %i --- %0.2f percent on peaks'%('Joint%i'%jj, len(xvals), np.sum(xvals)/len(xvals)*100))

In [ ]:
# LOOK AT PROPORTION OF STRIDES WITH ANT ON UNEVEN GROUND VS. LEVEL FOOTSTEPS


# ######### CALCULATE UNEVENESS OF CONCURRENT STEPS
# df['St_jointnum'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
#     lambda x: np.concatenate([x]), axis = 1).map(
#     lambda x: np.repeat([0,1,2,3,4,5], x))


# def find_concurrent_eveness_df(x, cutoff = 5):
#     TD_idcs = np.concatenate(x.filter(regex='_TD_idcs$').map(lambda x: x[:-1]).tolist())
#     good_strides = np.concatenate(x.filter(regex='_good_strides$').tolist())
    
#     St_idcs_unsorted = TD_idcs[good_strides]
#     sort_idcs = np.argsort(St_idcs_unsorted)
#     St_idcs = St_idcs_unsorted[sort_idcs]
#     St_jointnum = x['St_jointnum'][sort_idcs]
#     St_heights = np.concatenate(x.filter(regex='_St_Heights$').tolist())[sort_idcs]
    
#     where_split = np.where( np.insert(np.diff(St_idcs)>cutoff,0,False))[0]
#     clumped_heights =np.split(St_heights, where_split)
#     n_concurrent_feet = np.array([np.sum(c) for c in clumped_heights])
#     frac_peak = np.array([np.sum(c)/len(c) for c in clumped_heights])
#     n_3even =np.sum(np.logical_and(n_concurrent_feet>2, frac_peak%1==0))
#     n_3uneven = np.sum(np.logical_and(n_concurrent_feet>2, frac_peak%1>0))
#     n_2even = np.sum(np.logical_and(n_concurrent_feet==2, frac_peak%1==0))
#     n_2uneven = np.sum(np.logical_and(n_concurrent_feet==2, frac_peak%1>0))
#     n_strides = len(clumped_heights)
#     return np.array([n_3even, n_3uneven, n_2even, n_2uneven, n_strides])


# concurrent_cutoff = 5 #how many frames to determine if stances are concurrent
# df['St_concurrent_eveness'] = df.apply(find_concurrent_eveness_df, args = (concurrent_cutoff,), axis=1)





####### ANALYZE
plt.close('all')
uneveness_data = np.full((4,7,5),np.nan)
for ss,subtype in enumerate(subtypes[0:4]):
    print('\n%s'%subtype)
    for cc,coltype in enumerate(coltypes[:-1]):
        xs = np.vstack(df.loc[(df['substrate']==subtype) & (df['colony']==coltype)]['St_concurrent_eveness'].values)
        print(np.sum(xs, axis =0))
        uneveness_data[ss,cc,:]=np.sum(xs, axis =0)
    print('sums: ', np.sum(uneveness_data[ss,:,:], axis =1))


plt.figure(figsize=(13,5))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']

# % concurrent strides identified
plt.subplot(1,3,1)
n_concurrent_strides  = np.sum(uneveness_data[:,:,:-1],axis=2)
percent_concurrent_strides = n_concurrent_strides/uneveness_data[:,:,-1]*100
percent_tripod_strides = np.sum(uneveness_data[:,:,0:2],axis=2)/uneveness_data[:,:,-1]*100
percent_duo_strides = np.sum(uneveness_data[:,:,2:4],axis=2)/uneveness_data[:,:,-1]*100

yerrs = np.abs([np.min(percent_concurrent_strides,axis=1), np.max(percent_concurrent_strides,axis=1)]
               -np.mean(percent_concurrent_strides, axis=1))
plt.bar(np.arange(4), np.mean(percent_concurrent_strides, axis =1), width=1, yerr = yerrs, color = pltcolors, alpha = 0.4)
yerrs = np.abs([np.min(percent_tripod_strides,axis=1), np.max(percent_tripod_strides,axis=1)]
               -np.mean(percent_tripod_strides, axis=1))
plt.bar(np.arange(4), np.mean(percent_tripod_strides, axis =1),
        width=1, yerr = yerrs, color = pltcolors)
plt.text(2, 95, 'total n strides:', color = 'k')
for ss in range(0,4):
    plt.text(2, 90-ss*5, '%i'%np.sum(uneveness_data[:,:,-1], axis = 1)[ss], color=pltcolors[ss])
plt.ylim([0,100])
plt.xlim([-0.5,3.5])
plt.xticks(np.arange(4),['0','1','3','5'])
plt.title('% of identified strides concurrent', loc= 'left')

# % of identified tripods that are even
plt.subplot(1,3,2)
percent_even_tripods = uneveness_data[:,:,0]/np.sum(uneveness_data[:,:,0:2],axis=2)*100
yerrs = np.abs([np.min(percent_even_tripods,axis=1), np.max(percent_even_tripods,axis=1)]
               -np.mean(percent_even_tripods, axis=1))
plt.bar(np.arange(4), np.mean(percent_even_tripods, axis =1), width=1, yerr = yerrs, color = pltcolors)
plt.ylim([0,100])
plt.xlim([-0.5,3.5])
plt.xticks(np.arange(4),['0','1','3','5'])
plt.title('% of identified tripods that are even', loc= 'left')
plt.xlabel('substrate size (mm)')

# % of identified duos that are even
plt.subplot(1,3,3)
percent_even_duos = uneveness_data[:,:,2]/np.sum(uneveness_data[:,:,2:4],axis=2)*100
yerrs = np.abs([np.min(percent_even_duos,axis=1), np.max(percent_even_duos,axis=1)]
               -np.mean(percent_even_duos, axis=1))
plt.bar(np.arange(4), np.mean(percent_even_duos, axis =1), width=1, yerr = yerrs, color = pltcolors, alpha = 0.4)
plt.ylim([0,100])
plt.xlim([-0.5,3.5])
plt.xticks(np.arange(4),['0','1','3','5'])
plt.title('% of identified duos that are even', loc= 'left')

### Look at transitions in step height during sequential strides

In [ ]:
def find_sequential_strides_df(x, joint_num, parameter): #'rotation', 'Len', 'Dur', 'travel_dir'
    arr = x['joint%s_St_%s'%(joint_num, parameter)]
    good_st = x['joint%s_good_strides'%joint_num]
    idcs = range(len(good_st))
    st_idcs = np.cumsum(good_st)-1
    st_len_groups  = [arr[st_idcs[np.array(list(g))]] for k,g in groupby(iter(idcs), lambda x: good_st[x]) if k == True]
    arr_0 = [g[:-1] for g in st_len_groups if len(g)>1]
    arr_1 = [g[1:] for g in st_len_groups if len(g)>1]
    if len(arr_0)>0:
        arr_0 = np.concatenate(arr_0)
        arr_1 = np.concatenate(arr_1)
    return arr_0, arr_1

for parameter  in ['Len', 'Dur', 'tdist_total', 'travel_dir', 'rotation', 'Heights']:
    print(parameter)
    if 'SeqSt0_%s'%(parameter) in df.columns:
        df = df.drop(['SeqSt0_%s'%(parameter), 'SeqSt1_%s'%(parameter)], axis =1)
    # for each joint, find the 1st (0) and 2nd (1) parameter values for sequential strides
    for joint_num in range(0,6):
        df['joint%s_SeqSt0_%s'%(joint_num, parameter)], df['joint%s_SeqSt1_%s'%(joint_num, parameter)]= \
            zip(*df.apply(find_sequential_strides_df, args = (joint_num, parameter), axis=1))
    # combine all joints
    df['SeqSt0_%s'%parameter] = df.filter(regex='_SeqSt0_%s$'%parameter, axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
    df['SeqSt1_%s'%parameter] = df.filter(regex='_SeqSt1_%s$'%parameter, axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)

# find joints used for each sequential stride pair
df['SeqSt_Joints'] = df.filter(regex='_SeqSt0_Len$', axis=1).applymap(lambda x: len(x)).apply(lambda x: np.concatenate([x]), axis = 1).map(lambda x: np.repeat([0,1,2,3,4,5], x))
    
# remove colulmns for individual joint sequential stride info
for parameter  in ['Len', 'Dur', 'tdist_total', 'travel_dir', 'rotation', 'Heights']:
    for joint_num in range(0,6):
        df = df.drop(['joint%s_SeqSt0_%s'%(joint_num, parameter), 'joint%s_SeqSt1_%s'%(joint_num, parameter)], axis =1)
    
    
    
# new df with just sequential stride info of interest
lens = [len(item) for item in df['SeqSt0_travel_dir']]
seq_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['SeqSt_Joints'].values),
                        "Len_0" : np.concatenate(df['SeqSt0_Len'].values/pix2mm), "Len_1" : np.concatenate(df['SeqSt1_Len'].values/pix2mm),
                        "Dur_0" : np.concatenate(df['SeqSt0_Dur'].values/fps), "Dur_1" : np.concatenate(df['SeqSt1_Dur'].values/fps),
                        "travel_dist_0" : np.concatenate(df['SeqSt0_tdist_total'].values/pix2mm), 
                        "travel_dist_1" : np.concatenate(df['SeqSt1_tdist_total'].values/pix2mm),
                        "speed_0" : np.concatenate(df['SeqSt0_tdist_total'].values/pix2mm)/np.concatenate(df['SeqSt0_Dur'].values/fps), 
                        "speed_1" : np.concatenate(df['SeqSt1_tdist_total'].values/pix2mm)/np.concatenate(df['SeqSt1_Dur'].values/fps),
                        "travel_dir_0" : np.concatenate(df['SeqSt0_travel_dir'].values), "travel_dir_1" : np.concatenate(df['SeqSt1_travel_dir'].values),
                        "rotation_0" : np.concatenate(df['SeqSt0_rotation'].values), "rotation_1" : np.concatenate(df['SeqSt1_rotation'].values),
                        "height_0" : np.concatenate(df['SeqSt0_Heights'].values), "height_1" : np.concatenate(df['SeqSt1_Heights'].values)    })

print('\ndone analyzing')




# # plot step transition type vased on travel distance of previous and current strides
# # percent of same level sequential strides that are peaks vs. valleys
# # plt.close('all')
# plt.figure(figsize=(15,4))
# clrs = np.array([[0,0,1],[0,0,0],[1,0,0]])
# for ss,subtype in enumerate(subtypes[0:4]):
#     print('\n%s'%subtype)
#     plt.subplot(1,4,ss+1)
#     seq_strides_OI = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']!=coltypes[-1])]
#     xs = seq_strides_OI['travel_dist_0'].values
#     ys = seq_strides_OI['travel_dist_1'].values
    
#     step_type = (seq_strides_OI['height_1'].values-seq_strides_OI['height_0'].values+1).astype(np.uint8)
#     colors = clrs[step_type]
    
#     # plot all points at once
# #     plt.scatter(xs,ys, s=2, c =colors, alpha = 0.01)
        
#     # plot each step type one at a time
#     for st_OI in [1]:#range(0,3):
#         plt.scatter(xs[step_type==st_OI],ys[step_type==st_OI], s=2, c =clrs[st_OI], alpha = 0.01)
    
#     plt.xlim(0,5)
#     plt.ylim(0,5)
    
#     print('-- same level: %i'%np.sum(step_type==1))
#     print('---- valley: %i'%np.sum(np.logical_and(step_type==1, seq_strides_OI['height_0'].values==0)))
#     print('---- peak: %i'%np.sum(np.logical_and(step_type==1, seq_strides_OI['height_0'].values==1)))
#     print('-- step down: %i'%np.sum(step_type==0))
#     print('-- step up: %i'%np.sum(step_type==2))
    
    
# PLOT % OF SAME LEVEL SEQUENTIAL STEPS ON PEAKS VS. VALLEYS
sequential_data = np.full( (4,7,3),np.nan)
for ss,subtype in enumerate(subtypes[0:4]):
    print('\n%s'%subtype)
    for cc,coltype in enumerate(coltypes[:-1]):
        seq_strides_OI = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']==coltype)]
        step_type = (seq_strides_OI['height_1'].values-seq_strides_OI['height_0'].values+1).astype(np.uint8)
        sequential_data[ss,cc,-1] = len(step_type)
        sequential_data[ss,cc,0] = np.sum(np.logical_and(step_type==1,seq_strides_OI['height_0'].values==0) )
        sequential_data[ss,cc,1] = np.sum(np.logical_and(step_type==1,seq_strides_OI['height_0'].values==1) )
        
plt.close('all')
plt.figure(figsize=(7,5))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
percent_samelevel_peaks = sequential_data[:,:,1]/(np.sum(sequential_data[:,:,0:2],axis=2))*100
means = np.mean(percent_samelevel_peaks,axis = 1)
# yerrs = np.std(percent_samelevel_peaks,axis = 1)
yerrs = np.abs([np.min(percent_samelevel_peaks,axis=1), np.max(percent_samelevel_peaks,axis=1)]
               -means)
# plt.bar(np.arange(4), np.mean(percent_concurrent_strides, axis =1), width=1, yerr = yerrs, color = pltcolors, alpha = 0.4)
# yerrs = np.abs([np.min(percent_tripod_strides,axis=1), np.max(percent_tripod_strides,axis=1)]
#                -np.mean(percent_tripod_strides, axis=1))
plt.bar(np.arange(4), means,width=1, yerr = yerrs, color = pltcolors)
# plt.text(2, 95, 'total n strides:', color = 'k')
for ss in range(0,4):
    plt.text(ss, 90, 'n: %i'%np.sum(np.sum(sequential_data[:,:,0:2],axis=2),axis =1)[ss], horizontalalignment='center')
plt.ylim([0,100])
plt.xlim([-0.5,3.5])
plt.xticks(np.arange(4),['0','1','3','5'])
plt.title('% peak to peak vs. valley to valley sequential strides', loc= 'left', FontSize = 10)
plt.ylabel('% of strides that are peak to peak')
plt.xlabel('substrate box size (mm)')

for ss in range(1,4):
    print('p-value: ', stats.ttest_1samp(percent_samelevel_peaks[ss,:], 50)) # two-sided t-test for each substrate

# Point clouds of leg positions WRT ant on diff substrates 

In [10]:
# Point cloud of TDs based on whether it's a straight/turning stride - convex hull + PCA

def get_TD_location_df(x, part):
    all_x = x['%s_x_filt_WRTneck'%part]
    all_y = x['%s_y_filt_WRTneck'%part]
    TD_x = all_x[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    TD_y = all_y[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]]
    TD_x = TD_x
    TD_y = TD_y
    return TD_x, TD_y

def plot_convex_hull(points, pltcolors, joint_num):
    hull = ConvexHull(points)
    cent =np.mean(points, 0)
    pts = []
    for pt in points[hull.simplices]:
        pts.append(pt[0].tolist())
        pts.append(pt[1].tolist())
    pts.sort(key=lambda p: np.arctan2(p[1]-cent[1], p[0] - cent[0]))
    pts = pts[0::2]
    pts.insert(len(pts), pts[0])
    k= 1.0
    poly = Polygon(k*(np.array(pts)-cent) + cent, closed = True, facecolor = pltcolors[joint_num], alpha = 0.05)
    poly.set_capstyle('round')
    plt.gca().add_patch(poly)
    plt.plot(cent[0],cent[1],'.', color = pltcolors[joint_num])
    return 

def plt_pca_ellipse(points, pltcolors, joint_num, trial_type):
    
    cent =np.mean(points, axis = 0)
    pca = PCA(n_components = 2)
    pca.fit_transform(points)
    projected = pca.transform(points)
    a = np.linalg.norm(pca.inverse_transform([np.std(projected, axis =0)[0],0])-cent)
    b = np.linalg.norm(pca.inverse_transform([np.std(projected, axis =0)[1],0])-cent)
    new_unit =pca.inverse_transform([1,0])-cent
    el_angle = np.rad2deg(np.arctan2(new_unit[1], new_unit[0]))
    
    if trial_type == 'straight':
        lstyle = '-'
        xoffset = 0
    else:
        lstyle = '--'
        xoffset = 100
    el = Ellipse(cent, 2*a, 2*b, angle = el_angle, ec = pltcolors[joint_num], fc = 'None', LineStyle = lstyle)
    plt.gca().add_patch(el)
    el = Ellipse(cent, 2*1.96*a, 2*1.96*b, angle = el_angle, ec = pltcolors[joint_num], fc = 'None', LineStyle = lstyle)
    plt.gca().add_patch(el)
    plt.text(-100 + xoffset, -115 + 10*joint_num, '%i'%len(points), color = pltcolors[joint_num])
    SD = np.sqrt(a*b) # radius of circle with same area as ellipse
    plt.text(-60 + xoffset, -115 + 10*joint_num, '%0.1f'%SD, color = pltcolors[joint_num])
    return SD

# combine all values from all joints into one column of dataframe
for joint_num in range(0,6):
    #     df = df.drop(['joint%i_TD_x'%joint_num, 'joint%i_TD_y'%joint_num], axis = 1)
    df['joint%i_St_TD_x_WRTant'%joint_num], df['joint%i_St_TD_y_WRTant'%joint_num] = zip(*df.apply(get_TD_location_df, args = ('joint%i'%joint_num,), axis=1))
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_TD_x_WRTant'] = df.filter(regex='_St_TD_x_WRTant$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_TD_y_WRTant'] = df.filter(regex='_St_TD_y_WRTant$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_Heights'] = df.filter(regex='_St_Heights$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))
    
# make a dataframe where each row is its own stride
lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values), 
                        "St_heights" : np.concatenate(df['St_Heights'].values),
                        "St_TD_x_WRTant" : np.concatenate(df['St_TD_x_WRTant'].values), "St_TD_y_WRTant" : np.concatenate(df['St_TD_y_WRTant'].values) })
        
plt.close('all')
subtypes = sorted(list(set(df['substrate'].values)))
plt.figure(figsize = (18,6))



stddev = np.empty((2,4,6))
flat_XY = np.empty((2,6))
footX = np.empty((2,4,6))
footY = np.empty((2,4,6))
for ss,subtype in enumerate(subtypes[0:4]): 
    sub_TD = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!='Tunnel_20180329-30')]
    
    plt.subplot(1,4,ss+1)
    plt.title('%s'%subtype, loc = 'left')
    plt.plot(0,0,'+k')
    plt.text(-100, -125, 'n:', color = 'k')
    plt.text(-60, -125, 'SD:', color = 'k')
    plt.axis('off')

    if ss == 3:
        plt.plot([-50, -50+2*pix2mm],[0,0], '-k')
        plt.plot([0,0],[-50, -50+2*pix2mm], '-k')
        plt.text(-50+1*pix2mm+5, 5, '2 mm', horizontalalignment = 'center', color = 'k')
    
    # PLOT STRAIGHT STRIDES
    sub_TD_OI = sub_TD.loc[np.abs(sub_TD['St_travel_dir']) < 15]
    pltcolors = ['xkcd:aqua', 'xkcd:azure', 'xkcd:crimson', 'xkcd:fuchsia', 'xkcd:goldenrod', 'xkcd:green']
    
    for joint_num in range(0,6):
        # PLOT CONVEX HULL
        joint_df = sub_TD_OI.loc[sub_TD_OI['Joints_all'] == 'joint%i'%joint_num]
        points = np.array([joint_df['St_TD_x_WRTant'].values, joint_df['St_TD_y_WRTant'].values]).T
        plot_convex_hull(points, pltcolors, joint_num)
        # PLOT PCA ELIPSE
        stddev[0, ss, joint_num] = plt_pca_ellipse(points, pltcolors, joint_num, 'straight')
        # SAVE OR PLOT "NORMAL" POINT ON FLAT GROUND
        if subtype == '0mm':
                flat_XY[:,joint_num] = np.mean(points, axis = 0)
        else:
            plt.plot(flat_XY[0],flat_XY[1],'.k', alpha=1)
            footX[0, ss, joint_num] =np.mean(points, axis = 0)[0]
            footY[0, ss, joint_num] =np.mean(points, axis = 0)[1]
        del joint_df
    del sub_TD_OI
        
        
#     # PLOT UNICYCLE STRIDES
#     sub_TD_OI = sub_TD.loc[(np.abs(sub_TD['St_travel_dir']) > 15) & ( np.abs(sub_TD['St_rotation']-sub_TD['St_travel_dir']) < 15) ]
#     pltcolors = ['c', 'b', 'r', 'm', 'y', 'g']
#     for joint_num in range(0,6):
#         # PLOT CONVEX HULL
#         joint_df = sub_TD_OI.loc[(np.isin(sub_TD_OI['Joints_all'], ['joint%i'%(joint_num%3), 'joint%i'%(joint_num%3 + 3)])) & # it's a hind, mid, or fore limb
#                                 (sub_TD_OI['Joints_all'].map( # same side or opp side of turn
#                                     lambda x: np.sign(int(x[-1])//3*2-1)) == np.sign(sub_TD_OI['St_travel_dir'])*(joint_num//3*-2+1) )] 
#         points = np.array([joint_df['St_TD_x'].values, joint_df['St_TD_y'].values*-1*np.sign(joint_df['St_travel_dir'].values)]).T
#         plot_convex_hull(points, pltcolors, joint_num)
#         # PLOT PCA ELIPSE
#         plt_pca_ellipse(points, pltcolors, joint_num, 'turns')
#         del joint_df
        

    
#     # PLOT PEAK AND VALLEY TDs SEPARATELY
#     pltcolors = ['xkcd:goldenrod','xkcd:crimson', 'xkcd:fuchsia',  'xkcd:green', 'xkcd:azure', 'xkcd:aqua']
#     trial_types = ['valleys','straight']
#     for step_height in [1,0]:
#         sub_TD_OI = sub_TD.loc[(np.abs(sub_TD['St_travel_dir']) < 15) & (sub_TD['St_heights']==step_height)]
#         if len(sub_TD_OI)==0:
#             continue

#         for joint_num in range(0,6):
#             # PLOT CONVEX HULL
#             joint_df = sub_TD_OI.loc[sub_TD_OI['Joints_all'] == 'joint%i'%joint_num]
#             points = np.array([joint_df['St_TD_x_WRTant'].values, joint_df['St_TD_y_WRTant'].values]).T
#             plot_convex_hull(points, pltcolors, joint_num)
#             # PLOT PCA ELIPSE
#             if (step_height == 1) & (ss==0):
#                 flat_XY[:,joint_num] = np.mean(points, axis = 0)
#             else:
#                 plt.plot(flat_XY[0],flat_XY[1],'.k', alpha=0.05)
#             stddev[step_height, ss, joint_num] = plt_pca_ellipse(points, pltcolors, joint_num, trial_types[step_height])
#             footX[step_height, ss, joint_num] =np.mean(points, axis = 0)[0]
#             footY[step_height, ss, joint_num] =np.mean(points, axis = 0)[1]
#             del joint_df
#         del sub_TD_OI
        
    plt.axis('equal')
    plt.xlim((-100,75))
    plt.ylim((-110,100))
    plt.gca().invert_yaxis()

        
    # PLOT SIDESTEP STRIDES
#     sub_TD_OI = sub_TD.loc[(np.abs(sub_TD['St_travel_dir']) > 15) & ( np.abs(sub_TD['St_travel_dir'])-np.abs(sub_TD['St_rotation']) > 15) &
#                          ( np.sign(sub_TD['St_rotation']-15) == np.sign(sub_TD['St_travel_dir']) ) ]


# bar plot of standard deviations
plt.figure()
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
jjs = [0,3,1,4,2,5]
for step_height in [0]:# range(0,2): # if both peak and valley
    for jj in np.arange(0,6):
        joint_num = jjs[jj]
#         print(joint_num)
        xs = (step_height*4) + np.arange(0,4) + jj*0.1
        ys = stddev[step_height, :,joint_num]/pix2mm
        plt.bar(xs,ys, color = pltcolors, width = 0.1, alpha = 0.3, align = 'edge')
    plt.bar((step_height*4)+np.linspace(0,3,4),np.mean(stddev[step_height,:,:],axis = 1)/pix2mm, 
            yerr = np.std(stddev[step_height,:,:]/pix2mm, axis =1 ), edgecolor = pltcolors, align = 'edge',
           width = 0.6, alpha = 0.4, facecolor= 'none')
plt.ylim((0,0.3))

# # find p-values for comparisons amongst substrates
for ss in range(0,3):
    for ss2 in range(ss+1,4):
        print('%i vs. %i p-value: '%(ss,ss2), stats.f_oneway(stddev[0,ss,:],stddev[0,ss2,:])[1])
#     print('p-value: ', stats.ttest_ind(stddev[0,ss,:],stddev[1,ss,:])[1]) # give same result as above
    
# # find p-values for if variance is diff on peaks vs. valleys
# for ss in range(1,4):
#     print('p-value: ', stats.f_oneway(stddev[0,ss,:],stddev[1,ss,:])[1])
# #     print('p-value: ', stats.ttest_ind(stddev[0,ss,:],stddev[1,ss,:])[1]) # give same result as above
    
del sub_TD, all_strides
with open('%sAnt_Straight_Foot_Placement.pkl'%vid_locations, 'wb') as f:
    pickle.dump([footX, footY], f)

0 vs. 1 p-value:  1.922176162363998e-07
0 vs. 2 p-value:  1.129346043889994e-09
0 vs. 3 p-value:  4.972621229642762e-08
1 vs. 2 p-value:  0.033132177957727306
1 vs. 3 p-value:  0.0037536989395684215
2 vs. 3 p-value:  3.646206995063139e-07


### Bar plots of average foot placement shifts

In [66]:
# bar plot of ave foot placement shift
pltcolors = ['#BA4246', '#087E8B', '#701C6F']
jjs = [0,3,1,4,2,5]

# anterior-posterior
plt.figure()
for step_height in [0]:# range(0,2): # if both peak and valley
    for jj in np.arange(0,6):
        joint_num = jjs[jj]
#         print(joint_num)
        xs = (step_height*3) + np.arange(0,3) + jj*0.1
        ys = (footX[0,1:,joint_num] - flat_XY[0,joint_num])/pix2mm 
        plt.bar(xs,ys, color = pltcolors, width = 0.1, alpha = 0.3, align = 'edge')

plt.ylim((-0.15,0.15))
plt.ylabel('posterior <--> anterior')

# medial-lateral
plt.figure()
for step_height in [0]:# range(0,2): # if both peak and valley
    for jj in np.arange(0,6):
        joint_num = jjs[jj]
#         print(joint_num)
        xs = (step_height*3) + np.arange(0,3) + jj*0.1
        ys = (footY[0,1:,joint_num] - flat_XY[1,joint_num])/pix2mm * (joint_num//3*2-1)*-1 # flip so that medial shifts are positive
        plt.bar(xs,ys, color = pltcolors, width = 0.1, alpha = 0.3, align = 'edge')
plt.ylim((-0.15,0.15))
plt.ylabel('lateral <--> medial')

0 [ 0.04184733 -0.02367528 -0.00821908]
3 [ 0.03401038 -0.02273975 -0.031048  ]
1 [0.13012102 0.04032828 0.02972958]
4 [0.13177092 0.04378043 0.02719511]
2 [ 0.01198763 -0.03550424 -0.00862545]
5 [ 0.0118878  -0.02710736 -0.01349742]


Text(0,0.5,'lateral <--> medial')

In [ ]:
# Point cloud of just TDs - imshow + transparent cmap - SHANNON ENTROPY CALC
from matplotlib.colors import LinearSegmentedColormap
def get_TD_location_df(x, part):
    all_x = x['%s_x_filt_WRTneck'%part]
    all_y = x['%s_y_filt_WRTneck'%part]
    TD_x = all_x[x['%s_TD_idcs'%part][x['%s_good_TDs'%part]]]
    TD_y = all_y[x['%s_TD_idcs'%part][x['%s_good_TDs'%part]]]
    TD_x = TD_x[np.isfinite(TD_x)]
    TD_y = TD_y[np.isfinite(TD_y)]
    return TD_x, TD_y
    
allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
allcols = [tr['colony'] for tr in trial_info]
coltypes = sorted(list(set(allcols)))
plt.close('all') 
for joint_num in range(0,6):
    #     df = df.drop(['joint%i_TD_x'%joint_num, 'joint%i_TD_y'%joint_num], axis = 1)
    df['joint%i_TD_x'%joint_num], df['joint%i_TD_y'%joint_num] = zip(*df.apply(get_TD_location_df, args = ('joint%i'%joint_num,), axis=1))
        
plt.close('all')
plt.figure(figsize = (18,5))
pltcolors = ['xkcd:aqua', 'xkcd:azure', 'xkcd:crimson', 'xkcd:fuchsia', 'xkcd:goldenrod', 'xkcd:green']
for ss,subtype in enumerate(subtypes[0:4]): 
    plt.subplot(1,4,ss+1)
    plt.text(0, 10, 'H: ', color = 'k')
    plt.text(0, 195, 'n: ', color = 'k')
    plt.title('%s'%subtype, loc = 'left')
    plt.plot(100,100,'k+')
    plt.axis('off')
    for joint_num in range(0,6):
        lens = [len(item) for item in df.loc[df['substrate'] == subtype]['joint%i_TD_x'%joint_num]]
        sub_TD = pd.DataFrame( {"substrate" : np.repeat(df.loc[df['substrate'] == subtype]['substrate'].values, lens), 
                                "colony" : np.repeat(df.loc[df['substrate'] == subtype]['colony'].values, lens),
                                "TD_x" : np.concatenate(df.loc[df['substrate'] == subtype]['joint%i_TD_x'%joint_num].values), 
                                "TD_y" : np.concatenate(df.loc[df['substrate'] == subtype]['joint%i_TD_y'%joint_num].values)})
        
        TD_hist, _, _ = np.histogram2d(sub_TD['TD_x'].values, sub_TD['TD_y'].values , bins=np.arange(-100,101,1), normed = True)
        c_array = colors.ListedColormap(pltcolors[joint_num])(range(0,100))
        c_array[:, -1]=np.arange(0,1,0.01)
        new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[joint_num], colors = c_array)
        plt.imshow(TD_hist.T, cmap = new_cmap, norm = colors.Normalize(vmin=0, vmax = 0.01))
        
        entropy = -1*np.nansum(np.multiply(TD_hist, np.log2(TD_hist)))
        plt.text(20+30*joint_num, 10, '%0.2f'%entropy, color = pltcolors[joint_num], horizontalalignment = 'left')
        plt.text(20+30*joint_num, 195, '%i'%len(sub_TD), color = pltcolors[joint_num], horizontalalignment = 'left', fontsize = 8)
#         print( 'Sub: %s -- Joint: %i -- H = %0.2f'%(subtype, joint_num, entropy))

In [ ]:
# OLD VERSION - JUST SCATTER PLOT

allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
allcols = [tr['colony'] for tr in trial_info]
coltypes = sorted(list(set(allcols)))
plt.close('all')

def rotate_wrt_neck(x, y, thorax_x, thorax_y, neck_x, neck_y):
    
    val_coord = np.array([x,y])-np.array([thorax_x,thorax_y])
    neck_coord = np.array([neck_x-thorax_x,neck_y-thorax_y])
    ang = np.arctan( (neck_y-thorax_y)/(neck_x-thorax_x))
    c, s = np.cos(ang), np.sin(ang)
    Rx = np.array([c,s])
    Ry = np.array([-s,c])
    newx = np.einsum('mn,mn->n', val_coord, Rx)
    newy = np.einsum('mn,mn->n', val_coord, Ry)
#     R = np.array([[c,s],[-1*s,c]])
#     newxy = np.einsum('mn,mqn->mn', val_coord, R)
#     print('val: ', val_coord[0,:], val_coord[1,:])
#     print('neck: ', neck_coord[0,:], neck_coord[1,:])
#     print('ang: ', ang)
#     print('newxy: ', newxy[0,0:10], newxy[1,0:10])
#     print('Rx,Ry', Rx, Ry)
#     print('newx: ', newx[0:10])
#     print('newy: ', newy)
    
    return newx, newy

def plot_joint_pointcloud(x, ant_part):
#     print('%s_x_filt'%ant_part)
    joint_x = x['%s_x_filt'%ant_part]
    joint_y = x['%s_y_filt'%ant_part]
    thorax_x = x['thorax_x_filt']
    thorax_y = x['thorax_y_filt']
    neck_x = x['neck_x_filt']
    neck_y = x['neck_y_filt']
    frames = x['frames']
    frames_final = x['frames_final']
    vals_OI = np.isin(frames, frames_final)
    cs = plt.cm.get_cmap('viridis',6)
    alpha = 0.1
#     print('joint', len(joint_x), 'thorax', len(thorax_x), 'neck', len(neck_x), 'frames', len(vals_OI))
    
    if 'thorax' in ant_part:
        plt.plot((joint_x-thorax_x), (joint_y-thorax_y), '.k', alpha=alpha, 
                 markeredgecolor = None, markeredgewidth = 0)
    elif 'neck' in ant_part:
        new_x, new_y = rotate_wrt_neck(joint_x, joint_y, thorax_x, thorax_y, neck_x, neck_y)
        plt.plot(new_x, new_y, '.', alpha=alpha, c = 'r', 
                 markeredgecolor = None, markeredgewidth = 0)
    elif 'anten' in ant_part:
        new_x, new_y = rotate_wrt_neck(joint_x, joint_y, thorax_x, thorax_y, neck_x, neck_y)
        plt.plot(new_x, new_y, '.', alpha=alpha, c = 'k', 
                 markeredgecolor = None, markeredgewidth = 0)
    else:
        new_x, new_y = rotate_wrt_neck(joint_x, joint_y, thorax_x, thorax_y, neck_x, neck_y)
        plt.plot(new_x, new_y, '.', alpha=alpha, c = cs(int(ant_part[-1])), 
                 markeredgecolor = None, markeredgewidth = 0)
        
#     # neck not forced to be +x axis
#     if 'neck' in ant_part:
#         plt.plot((neck_x-thorax_x)[vals_OI], (neck_y-thorax_y)[vals_OI], '.r', alpha=alpha, 
#                  markeredgecolor = None, markeredgewidth = 0)
#     elif 'joint' in ant_part:
#         plt.plot((joint_x-thorax_x)[vals_OI], (joint_y-thorax_y)[vals_OI], 
#                  '.', alpha=alpha, c = cs(int(ant_part[-1])), 
#                  markeredgecolor = None, markeredgewidth = 0)

    return #joint_x, joint_y, new_xaxis_unit


for coltype in coltypes:
    
    col_data = df.loc[df['colony'] == coltype]
    plt.figure(figsize = (14,4))
    plt.clf()
    for kk, subtype in enumerate(subtypes):
        dataOI = col_data.loc[col_data['substrate'] == subtype]
        
        plt.subplot(1,len(subtypes),kk+1)
        
        if kk != 0:
            plt.gca().get_yaxis().set_visible(False)
    
        plt.plot(0,0,'.k')
#         dataOI.apply(plot_joint_pointcloud, args = ('thorax',), axis=1)
#         dataOI.apply(plot_joint_pointcloud, args = ('neck',), axis=1)
        
        # antennae
#         for joint_num in range(0,2):
#             dataOI.apply(plot_joint_pointcloud, args = ('antenna%i'%joint_num,), axis=1)
            
        # legs
        for joint_num in range(0,6):
            dataOI.apply(plot_joint_pointcloud, args = ('joint%i'%joint_num,), axis=1)
            
            
        plt.title('Substrate: %s'%subtype, loc='left')
#         plt.ylim((-100,100))
        plt.axis('equal')
        plt.xlim((-100,100))
#         plt.ylim((-100,100))
        plt.gca().invert_yaxis()

In [ ]:
def find_max_df(x, part):
    new_x = x['%s_x_filt'%part]
    new_y = x['%s_y_filt'%part]
#     print(np.nanmax(new_x))
    return '*X* max: %0.0f, min: %0.0f  --  *Y* max: %0.0f, min: %0.0f'%(np.nanmax(new_x), np.nanmin(new_x), np.nanmax(new_y), np.nanmin(new_y))

print('LEGS')
for joint_num in [3]:#range(0,1):
    df.apply(find_max_df, args = ('joint%i'%joint_num,), axis=1)

## COMPARE TD SHIFT TO SIMPLE ANT MODEL

### Calculate shift of average TD location wrt flat

In [ ]:
# plot distance from flat ave step down
# footX[step_height, ss, joint_num]


pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
jjs = [0,3,1,4,2,5]
titles = ['hind','mid','fore']

# bar plot
# plt.figure()
# for step_height in [0,1]:
#     Xdif = (footX[step_height,1:,:]-footX[1,0,:])#*[1,-1,-1,1,-1,-1]
#     Ydif = (footY[step_height,1:,:]-footY[1,0,:])*[1,1,1,-1,-1,-1]

#     for jj in np.arange(0,3):
#         joint_num = [jj,jj+3]
        
#         # medial/lateral shift
#         plt.subplot(2,3,jj+1)
#         xs = np.linspace(0,0.4,3)
#         ys = np.mean(Ydif[:,joint_num]/pix2mm, axis =1)
#         if step_height == 0:
#             plt.bar(xs,ys, color = pltcolors[1:], alpha = 0.3, width = 0.2, align = 'edge', hatch="/")
#         else:
#             plt.bar(xs,ys, color = pltcolors[1:], alpha = 0.7, width = 0.2, align = 'edge')
        
#         plt.gca().get_xaxis().set_visible(False)
#         plt.axhline(y=0, linestyle = ':', color = pltcolors[0])
#         plt.title(titles[jj]+'limb')
#         if jj == 0:
#             plt.ylabel('lateral <---> medial')
#         else:
#             plt.gca().get_yaxis().set_visible(False)
#         plt.axis('equal')
#         plt.ylim([-0.3,0.3])

        
#         # anterior/posterior shift
#         plt.subplot(2,3,jj+4)
#         xs = -1 * np.linspace(0,0.4,3)
#         ys = np.mean(Xdif[:,joint_num]/pix2mm, axis =1)
#         if step_height == 0:
#             plt.barh(xs,ys, color = pltcolors[1:], alpha = 0.3, height = 0.2, align = 'edge', hatch="/")
#         else:
#             plt.barh(xs,ys, color = pltcolors[1:], alpha = 0.7, height = 0.2, align = 'edge')
        
#         plt.gca().get_yaxis().set_visible(False)
#         plt.axvline(x=0, linestyle = ':', color = pltcolors[0])
#         plt.axis('equal')
#         if jj ==1:
#             plt.xlabel('posterior <---> anterior')
#         plt.xlim([-0.3, 0.3])


# plt.close('all')
plt.figure()
import matplotlib as mpl
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color = pltcolors[1:])
c_x = np.array([-1,0,1]) #np.mean(footX[1,0,:].reshape((2,3)),axis=0)/pix2mm
c_y = np.zeros(3) #np.mean(np.abs(footY[1,0,:]).reshape((2,3)),axis=0)/pix2mm
plt.plot(c_x,c_y,'.k')
linestyles=['--','-']
for step_height in [0,1]:
    Xdif = (footX[step_height,1:,:]-footX[1,0,:])#*[1,-1,-1,1,-1,-1]
    Ydif = (footY[step_height,1:,:]-footY[1,0,:])*[1,1,1,-1,-1,-1]
    for jj in np.arange(0,3):
        joint_num = [jj,jj+3]
        ys = np.mean(Ydif[:,joint_num]/pix2mm, axis =1)
        xs = np.mean(Xdif[:,joint_num]/pix2mm, axis =1)
        plt.plot(np.array([np.ones(3)*c_x[jj],c_x[jj]+xs]), np.array([np.ones(3)*c_y[jj],c_y[jj]+ys]),linestyle= linestyles[step_height], alpha = 0.8)

plt.axis('equal')
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)
# del fig

### Define functions for simple ant model 

In [ ]:
# define important functions

def plot_tripods(xs,ys,zs,col):
#     ax.scatter(xs[1]-xs[1],ys[1]-ys[1],zs[1],s=20, c=col)
    ax.scatter(xs[::4]-xs[1],ys[::4]-ys[1],zs[::4],s=4, c=col)
    ax.scatter(xs[2::4]-xs[1],ys[2::4]-ys[1],zs[2::4],s=20, c=col)
    ax.plot3D(xs-xs[1],ys-ys[1],zs, ':', color = col)
    
def rotate(xs, ys, x0, y0, ang): # angle in radians
    new_xs = x0 + math.cos(ang) * (xs - x0) - math.sin(ang) * (ys - y0)
    new_ys = y0 + math.sin(ang) * (xs - x0) + math.cos(ang) * (ys - y0)
    return new_xs, new_ys

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
    
def plot_cylinder(x0, x1, y, z0, z1, cyl_rad, col):

    origin = np.array([0, 0, 0])
    #axis and radius
    p0 = np.array([x0, y, z0])
    p1 = np.array([x1, y, z1])
    R = cyl_rad
    #vector in direction of axis
    v = p1 - p0
    #find magnitude of vector
    mag = norm(v)
    #unit vector in direction of axis
    v = v / mag
    #make some vector not in the same direction as v
    not_v = np.array([1, 0, 0])
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    #make vector perpendicular to v
    n1 = np.cross(v, not_v)
    #normalize n1
    n1 /= norm(n1)
    #make unit vector perpendicular to v and n1
    n2 = np.cross(v, n1)
    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    t = np.linspace(0, mag, 100)
    theta = np.linspace(0, 2 * np.pi, 100)
    #use meshgrid to make 2d arrays
    t, theta = np.meshgrid(t, theta)
    #generate coordinates for surface
    X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta) * n1[i] + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    ax.plot_surface(X, Y, Z, color = col)
    plt.show()
    
# #plot axis
# ax.plot(*zip(p0, p1), color = 'red')
# ax.set_xlim(0, 10)
# ax.set_ylim(0, 10)
# ax.set_zlim(0, 10)

### Rotate ant model around back limb

In [ ]:
# MODEL FOOT PLACEMENT FOR DIFF BODY ORIENTATIONS - PLOT 3D - rotate around back stance foot
plt.close('all')

flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0]
allY = np.hstack([-1*flatY,flatY])
body_height = 3/4#2/3
angle_OI = 1/4

# flat ground option
xs = np.vstack([allX, np.zeros(6)]).flatten(order='F')
ys = np.vstack([allY, np.zeros(6)]).flatten(order='F')
zs = np.vstack([np.zeros(6), body_height*np.ones(6)]).flatten(order='F')

# PITCH UP
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1,2,1, projection = '3d')
plot_tripods(xs,ys,zs,'k')
p_angle = np.tan(angle_OI)
p_xs, p_zs = rotate(xs,zs, xs[0],zs[0], p_angle)
p_ys = ys
plot_tripods(p_xs,p_ys,p_zs,'r')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1)
ax.plot_surface(np.array([[0.5,0.5],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[1,1],[1,1]]), alpha = 0.1)

# annotate
# plt.gcf().text(0.4, 0.85, 'F:')
# plt.gcf().text(0.4, 0.82, 'M:')
# plt.gcf().text(0.4, 0.79, 'H:')

# plt.gcf().text(0.43, 0.88, 'ant')
# plt.gcf().text(0.43, 0.85, '%0.2f'%((p_xs[-2]-p_xs[1])-xs[-2]))
# plt.gcf().text(0.43, 0.82, '%0.2f'%((p_xs[2]-p_xs[1])-xs[2]))
# plt.gcf().text(0.43, 0.79, '%0.2f'%((p_xs[6]-p_xs[1])-xs[6]))

# plt.gcf().text(0.47, 0.88, 'med')
# plt.gcf().text(0.47, 0.85, '%0.2f'%(np.abs(ys[-2])-np.abs(p_ys[-2]-p_ys[1])))
# plt.gcf().text(0.47, 0.82, '%0.2f'%(np.abs(ys[2])-np.abs(p_ys[2]-p_ys[1])))
# plt.gcf().text(0.47, 0.79, '%0.2f'%(np.abs(ys[6])-np.abs(p_ys[6]-p_ys[1])))


ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
plt.title('pitching up')


print(p_xs)
print(xs)


# PITCH DOWN
ax = fig.add_subplot(1,2,2, projection = '3d')
plot_tripods(xs,ys,zs,'k')
p_angle = -1*np.tan(angle_OI)
p_xs, p_zs = rotate(xs,zs, xs[0],zs[0], p_angle)
p_ys = ys
plot_tripods(p_xs,p_ys,p_zs,'r')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1)
ax.plot_surface(np.array([[0.5,0.5],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[-1,-1],[-1,-1]]), alpha = 0.1)

# annotate
plt.gcf().text(0.82, 0.85, 'F:')
plt.gcf().text(0.82, 0.82, 'M:')
plt.gcf().text(0.82, 0.79, 'H:')

plt.gcf().text(0.85, 0.88, 'ant')
plt.gcf().text(0.85, 0.85, '%0.2f'%((p_xs[-2]-p_xs[1])-xs[-2]))
plt.gcf().text(0.85, 0.82, '%0.2f'%((p_xs[2]-p_xs[1])-xs[2]))
plt.gcf().text(0.85, 0.79, '%0.2f'%((p_xs[6]-p_xs[1])-xs[6]))

plt.gcf().text(0.9, 0.88, 'med')
plt.gcf().text(0.9, 0.85, '%0.2f'%(np.abs(ys[-2])-np.abs(p_ys[-2]-p_ys[1])))
plt.gcf().text(0.9, 0.82, '%0.2f'%(np.abs(ys[2])-np.abs(p_ys[2]-p_ys[1])))
plt.gcf().text(0.9, 0.79, '%0.2f'%(np.abs(ys[6])-np.abs(p_ys[6]-p_ys[1])))

ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
plt.title('pitching down')


# # ROLL UP
# fig = plt.figure(figsize = (12,6))
# ax = fig.add_subplot(1,2,1, projection = '3d')
# plot_tripods(xs,ys,zs,'k')
# r_angle = np.tan(1/3)
# p_ys, p_zs = rotate(ys,zs, ys[2],zs[2], r_angle)
# p_xs = xs
# plot_tripods(p_xs,p_ys,p_zs,'r')
# ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1)
# ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[0.5,2],[0.5,2]]), np.array([[1,1],[1,1]]), alpha = 0.1)

# # annotate
# plt.gcf().text(0.4, 0.85, 'F:')
# plt.gcf().text(0.4, 0.82, 'M:')
# plt.gcf().text(0.4, 0.79, 'H:')

# plt.gcf().text(0.43, 0.88, 'ant')
# plt.gcf().text(0.43, 0.85, '%0.2f'%((p_xs[-2]-p_xs[1])-xs[-2]))
# plt.gcf().text(0.43, 0.82, '%0.2f'%((p_xs[2]-p_xs[1])-xs[2]))
# plt.gcf().text(0.43, 0.79, '%0.2f'%((p_xs[6]-p_xs[1])-xs[6]))

# plt.gcf().text(0.47, 0.88, 'med')
# plt.gcf().text(0.47, 0.85, '%0.2f'%(np.abs(ys[-2])-np.abs(p_ys[-2]-p_ys[1])))
# plt.gcf().text(0.47, 0.82, '%0.2f'%(np.abs(ys[2])-np.abs(p_ys[2]-p_ys[1])))
# plt.gcf().text(0.47, 0.79, '%0.2f'%(np.abs(ys[6])-np.abs(p_ys[6]-p_ys[1])))

# ax.set_xlabel('post --> ant')
# ax.set_ylabel('mediolateral')
# set_axes_equal(ax)
# plt.title('roll up')


# # ROLL DOWN
# ax = fig.add_subplot(1,2,2, projection = '3d')
# plot_tripods(xs,ys,zs,'k')
# r_angle = -1*np.tan(1/3)
# p_ys, p_zs = rotate(ys,zs, ys[2],zs[2], r_angle)
# p_xs = xs
# plot_tripods(p_xs,p_ys,p_zs,'r')
# ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1)
# ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[0.5,2],[0.5,2]]), -1*np.array([[1,1],[1,1]]), alpha = 0.1)

# # annotate
# plt.gcf().text(0.82, 0.85, 'F:')
# plt.gcf().text(0.82, 0.82, 'M:')
# plt.gcf().text(0.82, 0.79, 'H:')

# plt.gcf().text(0.85, 0.88, 'ant')
# plt.gcf().text(0.85, 0.85, '%0.2f'%((p_xs[-2]-p_xs[1])-xs[-2]))
# plt.gcf().text(0.85, 0.82, '%0.2f'%((p_xs[2]-p_xs[1])-xs[2]))
# plt.gcf().text(0.85, 0.79, '%0.2f'%((p_xs[6]-p_xs[1])-xs[6]))

# plt.gcf().text(0.9, 0.88, 'med')
# plt.gcf().text(0.9, 0.85, '%0.2f'%(np.abs(ys[-2])-np.abs(p_ys[-2]-p_ys[1])))
# plt.gcf().text(0.9, 0.82, '%0.2f'%(np.abs(ys[2])-np.abs(p_ys[2]-p_ys[1])))
# plt.gcf().text(0.9, 0.79, '%0.2f'%(np.abs(ys[6])-np.abs(p_ys[6]-p_ys[1])))

# ax.set_xlabel('post --> ant')
# ax.set_ylabel('mediolateral')
# set_axes_equal(ax)
# plt.title('roll down')

### Pitch ant up/down around body center

In [ ]:
# MODEL FOOT PLACEMENT FOR DIFF BODY ORIENTATIONS - PLOT 3D - rotate around body center
from mpl_toolkits.mplot3d import Axes3D


flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0]
allY = np.hstack([-1*flatY,flatY])
body_height = 2/3
angle_OI = 1/3
view_ang = -75
view_ele = 20


def plot_tripods(xs,ys,zs,col):
#     ax.scatter(xs[1]-xs[1],ys[1]-ys[1],zs[1],s=20, c=col)
#     ax.scatter(xs[::4]-xs[1],ys[::4]-ys[1],zs[::4],s=4, c=col)
    ax.scatter(xs[2::4]-xs[1],ys[2::4]-ys[1],zs[2::4],s=20, c=col, alpha = 1)
    ax.plot3D(xs-xs[1],ys-ys[1],zs, ':', color = col, alpha = 1)

# flat ground option
xs = np.vstack([allX, np.zeros(6)]).flatten(order='F')
ys = np.vstack([allY, np.zeros(6)]).flatten(order='F')
zs = np.vstack([np.zeros(6), body_height*np.ones(6)]).flatten(order='F')


# PITCH UP
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1,2,1, projection = '3d')
ax.view_init(elev = view_ele, azim = view_ang)
col = '#8B5E3C'
# plot_tripods(xs,ys,zs,'k')
p_angle = np.tan(angle_OI)
p_xs, p_zs = rotate(xs,zs, 0, body_height, p_angle)
p_zs = p_zs + (0.5-(p_zs[-2]+p_zs[6])/2)
p_ys = ys
plot_tripods(p_xs,p_ys,p_zs,col)
ax.plot_surface(np.array([[-3,-3],[0.75,0.75]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[0.75,0.75],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[1,1],[1,1]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[0.75,0.75],[0.75,0.75]]), np.array([[-2,-2],[2,2]]), np.array([[0,1],[0,1]]), alpha = 0.2, color = 'k')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(p_xs[1]-1.5, p_xs[1]+1.5, p_ys[1], p_zs[1]-1.5*np.sin(p_angle), p_zs[1]+1.5*np.sin(p_angle), cyl_rad, col)
plt.title('pitching up')
ax._axis3don=False


# PITCH DOWN
ax = fig.add_subplot(1,2,2, projection = '3d')
ax.view_init(elev = view_ele, azim = view_ang)
col = 'k'
# plot_tripods(xs,ys,zs,'k')
p_angle = -1*np.tan(angle_OI)
p_xs, p_zs = rotate(xs,zs, 0, body_height, p_angle)
p_zs = p_zs + (-0.5-(p_zs[-2]+p_zs[6])/2)
p_ys = ys
plot_tripods(p_xs,p_ys,p_zs,col)
ax.plot_surface(np.array([[-3,-3],[0.75,0.75]]), np.array([[-2,2],[-2,2]]), np.array([[0,0],[0,0]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[0.75,0.75],[2,2]]), np.array([[-2,2],[-2,2]]), np.array([[-1,-1],[-1,-1]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[0.75,0.75],[0.75,0.75]]), np.array([[-2,-2],[2,2]]), np.array([[-1,0],[-1,0]]), alpha = 0.2, color = 'k')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(p_xs[1]-1.5, p_xs[1]+1.5, p_ys[1], p_zs[1]-1.5*np.sin(p_angle), p_zs[1]+1.5*np.sin(p_angle), cyl_rad, col)
plt.title('pitching down')
ax._axis3don=False

In [ ]:
# PITCH UP AND DOWN - 2D
# plt.close('all')

flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0] # stance feet move back 1 mm wrt body center
allY = np.hstack([-1*flatY,flatY])
allZ = np.zeros(6)
body_height = 2/3
angle_OI = 1/3
leg_lengths = np.linalg.norm(np.array([allX[1::2],allY[1::2],body_height*np.ones(3)]),axis=0)
leg_lengths2 = np.linalg.norm(np.array([allX[0::2],allY[0::2],body_height*np.ones(3)]),axis=0)


# PITCH UP
fig = plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    
p_angle = np.tan(angle_OI)
## rotate around body center and adjust
p_xs, p_zs = rotate(allX,allZ, 0, body_height, p_angle)
c_x, c_y, c_z = (0, 0, body_height+(0.5-(p_zs[-1]+p_zs[3])/2))
p_zs = p_zs + (0.5-(p_zs[-1]+p_zs[3])/2)
p_ys = allY
## rotate around back foot
# p_xs, p_zs = rotate(allX,allZ, allX[0], allZ[0], p_angle)
# p_ys = allY
# c_x, c_z = rotate(0,body_height, allX[0], allZ[0], p_angle)
# c_y = 0
plt.plot(c_x-c_x,0,'+r')
for start_idc in [1]:
    plt.plot(p_xs[start_idc::2]-c_x, p_ys[start_idc::2],'.r')
    tripod = plt.Polygon(np.array([p_xs[start_idc::2]-c_x,p_ys[start_idc::2]]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)

# midlimb on valley and peak
r = (leg_lengths[0]**2-(1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)
r = (leg_lengths[0]**2-(0-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)

# hindlimb on valley
r = (leg_lengths[1]**2-(0-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))+10 , color = 'r' , linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)

# forelimb on peak
r = (leg_lengths[2]**2-(1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))+10 , color = 'r', alpha = 0.4)
plt.gca().add_patch(h_arc)

plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('pitching up')


fig2 = plt.figure()
d_x = np.array([-1,0,1]) #np.mean(footX[1,0,:].reshape((2,3)),axis=0)/pix2mm
d_y = np.zeros(3) #np.mean(np.abs(footY[1,0,:]).reshape((2,3)),axis=0)/pix2mm
plt.plot(d_x,d_y,'.k')


# PITCH UP
pup = np.full((2,3,2),np.nan)
joint_nums = [1,0,2]
for jj in [1,3,5]:
    angOI = np.arctan2(p_ys[jj],p_xs[jj])
    for zz,z in enumerate([0,1]): # valley and peak vertical level
        r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
        pup[0,joint_nums[int((jj-1)/2)],zz] = np.cos(angOI)*r-allX[jj] # anterior posterior
        pup[1,joint_nums[int((jj-1)/2)],zz] = (np.sin(angOI)*r-allY[jj])*-1*np.sign(p_ys[jj]) # mediolateral
        pup[:,0,1]=np.nan
        pup[:,2,0]=np.nan
        plt.plot([d_x[joint_nums[int((jj-1)/2)]], d_x[joint_nums[int((jj-1)/2)]]+pup[0,joint_nums[int((jj-1)/2)],zz]],
                [d_y[joint_nums[int((jj-1)/2)]], d_y[joint_nums[int((jj-1)/2)]]+pup[1,joint_nums[int((jj-1)/2)],zz]],
                alpha = 0.8, color = 'g', linestyle=linestyles[zz])




plt.figure(fig.number)
plt.subplot(1,2,2)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    
p_angle = -1*np.tan(angle_OI)
## rotate around body center and adjust
p_xs, p_zs = rotate(allX,allZ, 0, body_height, p_angle)
c_x, c_y, c_z = (0, 0, body_height+ (-0.5-(p_zs[-1]+p_zs[3])/2))
p_zs = p_zs + (-0.5-(p_zs[-1]+p_zs[3])/2)
p_ys = allY

plt.plot(c_x-c_x,0,'+r')
for start_idc in [1]:
    plt.plot(p_xs[start_idc::2]-c_x, p_ys[start_idc::2],'.r')
    tripod = plt.Polygon(np.array([p_xs[start_idc::2]-c_x,p_ys[start_idc::2]]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    

# midlimb on valley and peak
r = (leg_lengths[0]**2-(-1-c_z)**2)**(1/2) # valley
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)
r = (leg_lengths[0]**2-(0-c_z)**2)**(1/2) # peak
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)

# hindlimb on peak
r = (leg_lengths[1]**2-(0-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))+10 , color = 'r' , linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)

# forelimb on valley
r = (leg_lengths[2]**2-(-1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))+10 , color = 'r', linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)

plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('pitching down')



# calc x,y of peak and valley stances for each limb
plt.figure(fig2.number)
pdown = np.full((2,3,2),np.nan)
joint_nums = [1,0,2]
for jj in [1,3,5]:
    angOI = np.arctan2(p_ys[jj],p_xs[jj])
    for zz,z in enumerate([-1,0]): # valley and peak vertical level
        r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
        pdown[0,joint_nums[int((jj-1)/2)],zz] = np.cos(angOI)*r-allX[jj] # anterior posterior
        pdown[1,joint_nums[int((jj-1)/2)],zz] = (np.sin(angOI)*r-allY[jj])*-1*np.sign(p_ys[jj]) # mediolateral
        pdown[:,0,0]=np.nan
        pdown[:,2,1]=np.nan
        plt.plot([d_x[joint_nums[int((jj-1)/2)]], d_x[joint_nums[int((jj-1)/2)]]+pdown[0,joint_nums[int((jj-1)/2)],zz]],
                [d_y[joint_nums[int((jj-1)/2)]], d_y[joint_nums[int((jj-1)/2)]]+pdown[1,joint_nums[int((jj-1)/2)],zz]],
                alpha = 0.8, color = 'b', linestyle=linestyles[zz])
plt.axis('equal')
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)

### Roll ant right/left around body center

In [ ]:
# MODEL - ROLL - 3D - rotate around body center
plt.close('all')

flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0]
allY = np.hstack([-1*flatY,flatY])
body_height = 2/3
angle_OI = 1/2.5
view_ang = -30
view_ele = 20


# flat ground option
xs = np.vstack([allX, np.zeros(6)]).flatten(order='F')
ys = np.vstack([allY, np.zeros(6)]).flatten(order='F')
zs = np.vstack([np.zeros(6), body_height*np.ones(6)]).flatten(order='F')


# ROLL RIGHT
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1,2,1, projection = '3d')
ax.view_init(elev = view_ele, azim = view_ang)
col = '#8B5E3C'
# plot_tripods(xs,ys,zs,pltcolors[0])
p_angle = np.tan(angle_OI)
p_ys, p_zs = rotate(ys,zs, 0, body_height, p_angle)
p_zs = p_zs + (0.5-(p_zs[2]+p_zs[6])/2)
p_xs = xs
plot_tripods(p_xs,p_ys,p_zs,col)
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,0.1],[-2,0.1]]), np.array([[0,0],[0,0]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[0.1,2],[0.1,2]]), np.array([[1,1],[1,1]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[0.1,0.1],[0.1,0.1]]), np.array([[0,1],[0,1]]), alpha = 0.2, color = 'k')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(p_xs[1]-1.5, p_xs[1]+1.5, p_ys[1], p_zs[1], p_zs[1], cyl_rad, col)
plt.title('roll right')
ax._axis3don=False

# ROLL LEFT
ax = fig.add_subplot(1,2,2, projection = '3d')
ax.view_init(elev = view_ele, azim = -1*view_ang)
col = 'k'
# plot_tripods(xs,ys,zs,pltcolors[0])
p_angle = -1*np.tan(angle_OI)
p_ys, p_zs = rotate(ys,zs, 0, body_height, p_angle)
p_zs = p_zs + (-0.5-(p_zs[2]+p_zs[6])/2)
p_xs = xs
plot_tripods(p_xs,p_ys,p_zs, col)
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-2,-0.2],[-2,-0.2]]), np.array([[0,0],[0,0]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-0.2,2],[-0.2,2]]), np.array([[-1,-1],[-1,-1]]), alpha = 0.1, color = 'k')
ax.plot_surface(np.array([[-3,-3],[2,2]]), np.array([[-0.2,-0.2],[-0.2,-0.2]]), np.array([[0,-1],[0,-1]]), alpha = 0.2, color = 'k')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(p_xs[1]-1.5, p_xs[1]+1.5, p_ys[1], p_zs[1], p_zs[1], cyl_rad, col)
plt.title('roll left')
ax._axis3don=False

In [ ]:
# ROLL - 2D
# plt.close('all')

flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0] # stance feet move back 1 mm wrt body center
allY = np.hstack([-1*flatY,flatY])
allZ = np.zeros(6)
body_height = 2/3
angle_OI = 1/3
leg_lengths = np.linalg.norm(np.array([allX[1::2],allY[1::2],body_height*np.ones(3)]),axis=0)
leg_lengths2 = np.linalg.norm(np.array([allX[0::2],allY[0::2],body_height*np.ones(3)]),axis=0)


# ROLL RIGHT
fig = plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    
p_angle = np.tan(angle_OI)
## rotate around body center and adjust
p_ys, p_zs = rotate(allY,allZ, 0, body_height, p_angle)
c_x, c_y, c_z = (0, 0, body_height+(0.5-(p_zs[1]+p_zs[3])/2))
p_zs = p_zs + (0.5-(p_zs[1]+p_zs[3])/2)
p_xs = allX


plt.plot(c_x-c_x,0,'+r')
for start_idc in [1]:
    plt.plot(p_xs[start_idc::2]-c_x, p_ys[start_idc::2],'.r')
    tripod = plt.Polygon(np.array([p_xs[start_idc::2]-c_x,p_ys[start_idc::2]]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)

# midlimb on valley 
r = (leg_lengths[0]**2-(0-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)
# hindlimb on peak
r = (leg_lengths[1]**2-(1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))+10 , color = 'r' , linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)
# forelimb on peak
r = (leg_lengths[2]**2-(1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))+10 , color = 'r', linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)

plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('roll right')


fig2 = plt.figure()
d_x = np.array([-1,0,1]) #np.mean(footX[1,0,:].reshape((2,3)),axis=0)/pix2mm
d_y = np.zeros(3) #np.mean(np.abs(footY[1,0,:]).reshape((2,3)),axis=0)/pix2mm
plt.plot(d_x,d_y,'.k')
pup = np.full((2,3,2),np.nan)
joint_nums = [1,0,2]
for jj in [1,3,5]:
    angOI = np.arctan2(p_ys[jj],p_xs[jj])
    for zz,z in enumerate([0,1]): # valley and peak vertical level
        r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
        pup[0,joint_nums[int((jj-1)/2)],zz] = np.cos(angOI)*r-allX[jj] # anterior posterior
        pup[1,joint_nums[int((jj-1)/2)],zz] = (np.sin(angOI)*r-allY[jj])*-1*np.sign(p_ys[jj]) # mediolateral
        pup[:,0,0]=np.nan
        pup[:,2,0]=np.nan
        pup[:,1,1]=np.nan
        plt.plot([d_x[joint_nums[int((jj-1)/2)]], d_x[joint_nums[int((jj-1)/2)]]+pup[0,joint_nums[int((jj-1)/2)],zz]],
                [d_y[joint_nums[int((jj-1)/2)]], d_y[joint_nums[int((jj-1)/2)]]+pup[1,joint_nums[int((jj-1)/2)],zz]],
                alpha = 0.8, color = 'g', linestyle=linestyles[zz])
plt.ylabel('lateral <--> medial')
plt.xlabel('posterior <--> anterior')


# ROLL LEFT
plt.figure(fig.number)
plt.subplot(1,2,2)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)

## rotate around body center and adjust
p_angle = -1*np.tan(angle_OI)
p_ys, p_zs = rotate(allY,allZ, 0, body_height, p_angle)
c_x, c_y, c_z = (0, 0, body_height+(-0.5-(p_zs[1]+p_zs[3])/2))
p_zs = p_zs + (-0.5-(p_zs[1]+p_zs[3])/2)
p_xs = allX

plt.plot(c_x-c_x,0,'+r')
for start_idc in [1]:
    plt.plot(p_xs[start_idc::2]-c_x, p_ys[start_idc::2],'.r')
    tripod = plt.Polygon(np.array([p_xs[start_idc::2]-c_x,p_ys[start_idc::2]]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    
# midlimb on peak
r = (leg_lengths[0]**2-(0-c_z)**2)**(1/2) # peak
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[1],p_xs[1]-c_x))+10 , color = 'r', linestyle = '-', alpha = 0.4)
plt.gca().add_patch(h_arc)

# hindlimb on valley
r = (leg_lengths[1]**2-(-1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[3],p_xs[3]-c_x))+10 , color = 'r' , linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)

# forelimb on valley
r = (leg_lengths[2]**2-(-1-c_z)**2)**(1/2)
h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))-10, theta2=np.rad2deg(np.arctan2(p_ys[5],p_xs[5]-c_x))+10 , color = 'r', linestyle = '--', alpha = 0.4)
plt.gca().add_patch(h_arc)

plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('roll left')

# calc x,y of peak and valley stances for each limb
plt.figure(fig2.number)
pdown = np.full((2,3,2),np.nan)
joint_nums = [1,0,2]
for jj in [1,3,5]:
    angOI = np.arctan2(p_ys[jj],p_xs[jj])
    for zz,z in enumerate([-1,0]): # valley and peak vertical level
        r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
        pdown[0,joint_nums[int((jj-1)/2)],zz] = np.cos(angOI)*r-allX[jj] # anterior posterior
        pdown[1,joint_nums[int((jj-1)/2)],zz] = (np.sin(angOI)*r-allY[jj])*-1*np.sign(p_ys[jj]) # mediolateral
        pdown[:,0,1]=np.nan
        pdown[:,1,0]=np.nan
        pdown[:,2,1]=np.nan
        plt.plot([d_x[joint_nums[int((jj-1)/2)]], d_x[joint_nums[int((jj-1)/2)]]+pdown[0,joint_nums[int((jj-1)/2)],zz]],
                [d_y[joint_nums[int((jj-1)/2)]], d_y[joint_nums[int((jj-1)/2)]]+pdown[1,joint_nums[int((jj-1)/2)],zz]],
                alpha = 0.8, color = 'b', linestyle=linestyles[zz])
plt.axis('equal')
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)

### Ant steps in ditches

In [ ]:
# DITCH - 2D
# plt.close('all')
from matplotlib import patches


flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0] # stance feet move back 1 mm wrt body center
allY = np.hstack([-1*flatY,flatY])
allZ = np.zeros(6)
body_height = 2/3
angle_OI = 1/3
leg_lengths = np.linalg.norm(np.array([allX[1::2],allY[1::2],body_height*np.ones(3)]),axis=0) # leg lengths from stance
leg_lengths2 = np.linalg.norm(np.array([allX[0::2],allY[0::2],body_height*np.ones(3)]),axis=0) # long leg lengths from stance location


# FORE UP
fig = plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)
    

# calc foot position on ground
c_x, c_y, c_z = (0, 0, body_height*1/3)
p_xs = np.full(3,np.nan)
p_ys = np.full(3,np.nan)
p_zs = np.array([-1,-1,0]) # hind, mid, fore
line_styles = ['--','-']
for j_idc, jj in enumerate([3, 1, 5]): # hind, mid, fore
    
    angOI = np.arctan2(allY[jj],allX[jj])
    z=p_zs[j_idc]
    print('joint %i, angle %0.1f, height %i'%(jj, angOI, z))
    r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
    p_xs[j_idc]= np.cos(angOI)*r
    p_ys[j_idc] = (np.sin(angOI)*r)
    h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(angOI)-10, theta2=np.rad2deg(angOI)+10 , color = 'r', 
                        linestyle = line_styles[z+1], alpha = 0.4)
    plt.gca().add_patch(h_arc)
plt.plot(p_xs, p_ys,'.r')
tripod = plt.Polygon(np.array([p_xs,p_ys]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
plt.gca().add_patch(tripod)
plt.plot(c_x-c_x,0,'+r')
plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('forelimb on peak')
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)
plt.axhline(y=0,color ='k',alpha = 0.2)
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axvline(x=0,color ='k',alpha = 0.2)


fig2 = plt.figure()
d_x = np.array([-1,0,1]) 
d_y = np.zeros(3) 
plt.plot(d_x,d_y,'.k')
pup = np.full((2,3,2),np.nan)
joint_nums = [3,1,5]
for jj in range(3):
    pup[0,jj,p_zs[jj]+1] = p_xs[jj]-allX[joint_nums[jj]] # anterior-posterior
    pup[1,jj,p_zs[jj]+1] = (p_ys[jj]-allY[joint_nums[jj]])*-1*np.sign(p_ys[jj]) # mediolateral
    plt.plot([d_x[jj], d_x[jj]+pup[0,jj,p_zs[jj]+1]],
            [d_y[jj], d_y[jj]+pup[1,jj,p_zs[jj]+1]],
            alpha = 0.8, color = 'g', linestyle=linestyles[p_zs[jj]+1])
plt.ylabel('lateral <--> medial')
plt.xlabel('posterior <--> anterior')


# FORELIMB DOWN
plt.figure(fig.number)
plt.subplot(1,2,2)
plt.plot(0,0,'+k')
plt.plot(allX,allY,'.k')
plt.plot([allX[::2], allX[::2]+1],[allY[::2],allY[::2]],':k')
for start_idc in [1]:
    tripod = plt.Polygon(np.array([allX[start_idc::2],allY[start_idc::2]]).T.reshape([3,2]),color = 'k', alpha = 0.2, edgecolor = None, linestyle = None)
    plt.gca().add_patch(tripod)

# calc foot position on ground
p_xs = np.full(3,np.nan)
p_ys = np.full(3,np.nan)
p_zs = np.array([0,0,-1]) # hind, mid, fore
line_styles = ['--','-']
for j_idc, jj in enumerate([3, 1, 5]): # hind, mid, fore
    angOI = np.arctan2(allY[jj],allX[jj])
    z=p_zs[j_idc]
#     print('joint %i, angle %0.1f, height %i'%(jj, angOI, z))
    r= (leg_lengths[int((jj-1)/2)]**2-(z-c_z)**2)**(1/2) 
    p_xs[j_idc]= np.cos(angOI)*r
    p_ys[j_idc] = (np.sin(angOI)*r)
    h_arc = patches.Arc((0,0), 2*r, 2*r, angle = 0, theta1=np.rad2deg(angOI)-10, theta2=np.rad2deg(angOI)+10 , color = 'r', 
                        linestyle = line_styles[z+1], alpha = 0.4)
    plt.gca().add_patch(h_arc)
plt.plot(p_xs, p_ys,'.r')
tripod = plt.Polygon(np.array([p_xs,p_ys]).T.reshape([3,2]), color = 'r', alpha = 0.2, edgecolor = None, linestyle = None)
plt.gca().add_patch(tripod)
plt.plot(c_x-c_x,0,'+r')
plt.gca().set_aspect('equal')
plt.xlabel('posterior --> anterior (mm)')
plt.ylabel('mediolateral')
plt.title('forelimb on valley')
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)
plt.axhline(y=0,color ='k',alpha = 0.2)
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axvline(x=0,color ='k',alpha = 0.2)


# calc x,y of peak and valley stances for each limb
plt.figure(fig2.number)
pdown = np.full((2,3,2),np.nan)
joint_nums = [3,1,5]
for jj in range(3):
    pdown[0,jj,p_zs[jj]+1] = p_xs[jj]-allX[joint_nums[jj]] # anterior-posterior
    pdown[1,jj,p_zs[jj]+1] = (p_ys[jj]-allY[joint_nums[jj]])*-1*np.sign(p_ys[jj]) # mediolateral
    plt.plot([d_x[jj], d_x[jj]+pdown[0,jj,p_zs[jj]+1]],
            [d_y[jj], d_y[jj]+pdown[1,jj,p_zs[jj]+1]],
            alpha = 0.8, color = 'b', linestyle=linestyles[p_zs[jj]+1])
plt.axis('equal')
plt.axvline(x=-1,color ='k',alpha = 0.2)
plt.axvline(x=1,color ='k',alpha = 0.2)
plt.axhline(y=-1,color ='k',alpha = 0.2)
plt.axhline(y=1,color ='k',alpha = 0.2)

In [ ]:
# MODEL DITCH - 3D 
from mpl_toolkits.mplot3d import Axes3D
plt.close('all')

flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical
flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm
allX = np.tile(flatX,2)-[1,0,1,0,1,0]
allY = np.hstack([-1*flatY,flatY])
body_height = 2/3
angle_OI = 1/2.5
view_ang = -70
view_ele = 80


# flat ground option
xs = np.vstack([allX, np.zeros(6)]).flatten(order='F')
ys = np.vstack([allY, np.zeros(6)]).flatten(order='F')
zs = np.vstack([np.zeros(6), body_height*np.ones(6)]).flatten(order='F')


# FORELIMB UP
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1,2,1, projection = '3d')
ax.view_init(elev = view_ele, azim = view_ang)
col = '#8B5E3C'
p_xs = xs.copy()
p_xs[2::4] = np.nanmean(pup[0,:,:],axis =1)+p_xs[2::4]
p_ys = ys.copy()
p_ys[2::4] = np.nanmean(pup[1,:,:],axis =1)+p_ys[2::4]
p_zs = zs.copy()-(body_height-c_z)
p_zs[2::4] = [-1,-1,0]

ax.scatter(p_xs[2::4],p_ys[2::4],p_zs[2::4],s=20, c=col, alpha = 1)
ax.plot3D(p_xs,p_ys,p_zs, ':', color = col, alpha = 1)
heights = np.array([0, -1])
y_offset = 0.5
x_offset = -0.1
for xb in range(-3,2):
    for yb in range(-3,2):
        if yb%2 == 1:
            zz= heights[xb%2]
        else:
            zz=heights[np.where(np.array(heights) !=xb%2)[0][0]]
        ax.plot_surface(np.array([[xb,xb],[xb+1,xb+1]])+x_offset, np.array([[yb,yb+1],[yb,yb+1]])+y_offset, np.ones([2,2])*zz, alpha = 1, color = ['beige','#D3D3D3'][-1*zz])#np.ones(3)*254/255)
        ax.plot_surface(np.array([[xb,xb],[xb+1,xb+1]])+x_offset, np.array([[yb+1,yb+1],[yb+1,yb+1]])+y_offset, np.array([[-1,0],[-1,0]]), alpha = 1, color = 'grey')
        ax.plot_surface(np.array([[xb+1,xb+1],[xb+1,xb+1]])+x_offset, np.array([[yb,yb],[yb+1,yb+1]])+y_offset, np.array([[-1,0],[-1,0]]), color = 'r')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(c_x-1.5, c_x+1.5, c_y, c_z, c_z, cyl_rad, col)
plt.title('forelimb on peak')
ax._axis3don=False

# FORELIMB DOWN
ax = fig.add_subplot(1,2,2, projection = '3d')
ax.view_init(elev = view_ele, azim = view_ang)
col = 'k'
p_xs = xs.copy()
p_xs[2::4] = np.nanmean(pdown[0,:,:],axis =1)+p_xs[2::4]
p_ys = ys.copy()
p_ys[2::4] = np.nanmean(pdown[1,:,:],axis =1)+p_ys[2::4]
p_zs = zs.copy()-(body_height-c_z)
p_zs[2::4] = [0,0,-1]

ax.scatter(p_xs[2::4],p_ys[2::4],p_zs[2::4],s=20, c=col, alpha = 1)
ax.plot3D(p_xs,p_ys,p_zs, ':', color = col, alpha = 1)
heights = np.array([0, -1])
y_offset = 0.5
x_offset = -0.1
for xb in range(-3,2):
    for yb in range(-3,2):
        if yb%2 == 0:
            zz= heights[xb%2]
        else:
            zz=heights[np.where(np.array(heights) !=xb%2)[0][0]]a
        ax.plot_surface(np.array([[xb,xb],[xb+1,xb+1]])+x_offset, np.array([[yb,yb+1],[yb,yb+1]])+y_offset, np.ones([2,2])*zz, alpha = 1, color = ['beige','#D3D3D3'][-1*zz])#np.ones(3)*254/255)
        ax.plot_surface(np.array([[xb,xb],[xb+1,xb+1]])+x_offset, np.array([[yb+1,yb+1],[yb+1,yb+1]])+y_offset, np.array([[-1,0],[-1,0]]), alpha = 1, color = 'grey')
        ax.plot_surface(np.array([[xb+1,xb+1],[xb+1,xb+1]])+x_offset, np.array([[yb,yb],[yb+1,yb+1]])+y_offset, np.array([[-1,0],[-1,0]]), color = 'r')
ax.set_xlabel('post --> ant')
ax.set_ylabel('mediolateral')
set_axes_equal(ax)
cyl_rad = 0.1
plot_cylinder(c_x-1.5, c_x+1.5, c_y, c_z, c_z, cyl_rad, col)
plt.title('forelimb on valley')
ax._axis3don=False

## KINEMATIC LANDSCAPE: travel direction vs. heading rotation

In [ ]:
# 2d dist plot of body travel dir. vs. facing rotation for all trials
from scipy.stats import kurtosis, skew
plt.close('all')
pix2mm = 1000/32
fps = 240
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))

lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values) })

print(len(all_strides))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
subtypes = sorted(list(set(df['substrate'].values)))
coltypes = sorted(list(set(df['substrate'].values)))

# ROTATION
# plt.figure(figsize = (15,5))
# for ss, subtype in enumerate(subtypes[0:4]):
#     plt.subplot(1,4, ss+1)
#     ax = sns.distplot(all_strides.loc[(all_strides['substrate']==subtype)]['St_rotation'].dropna().values, label = '%s'%subtype, color = pltcolors[ss])#,
#     plt.ylim((0,0.04))
#     plt.xlim((-200,200))
#     plt.title('%s'%subtype, loc = 'left')
#     if ss == 0:
#         plt.ylabel('probability')
#         plt.xlabel('stride rotation (deg)')
#     else:
#         ax.get_yaxis().set_visible(False)
        
        
# straight stride speed
# plt.figure(figsize = (15,5))
# for ss, subtype in enumerate(subtypes[0:4]):
#     plt.subplot(1,4, ss+1)
#     vals_OI = (all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_straight'].values/
#                all_strides.loc[(all_strides['substrate']==subtype)]['StDur_all'].values)
#     ax = sns.distplot(vals_OI[np.isfinite(vals_OI)], label = '%s'%subtype, color = pltcolors[ss])#,
#     plt.ylim((0,0.14))
#     plt.xlim((-50,50))
#     plt.title('%s'%subtype, loc = 'left')
#     if ss == 0:
#         plt.ylabel('probability')
#     else:
#         ax.get_yaxis().set_visible(False)
        
    
# straight/total dist
# plt.figure(figsize = (15,5))
# for ss, subtype in enumerate(subtypes[0:4]):
#     plt.subplot(1,4, ss+1)
#     vals_OI = (np.abs(all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_straight'].values)/
#                all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_total'].values)
#     ax = sns.distplot(vals_OI[np.isfinite(vals_OI)], label = '%s'%subtype, color = pltcolors[ss])#,
#     plt.ylim((0,40))
#     plt.xlim((-.1,1.1))
#     plt.title('%s'%subtype, loc = 'left')
#     if ss == 0:
#         plt.ylabel('probability')
#         plt.xlabel('straight/total dist')
#     else:
#         ax.get_yaxis().set_visible(False)
        
    

# 2D hist: straight/total dist vs. rotation
fig = plt.figure(figsize = (15,4))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
angle_buffer = 15
line_factor = angle_buffer #(angle_buffer/np.sin(np.deg2rad(45)))
bar_ax = plt.axes((0.8,0.1, 0.1, 0.1), facecolor = 'w')
ns = np.full((4,4),np.nan)

for ss, subtype in enumerate(subtypes[0:4]):
    ax =plt.subplot(1,4, ss+1)
#     xvals_OI = np.rad2deg(np.arccos(
#         all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_straight'].values/
#                all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_total'].values))
    xvals_OI = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!='Tunnel_20180329-30')]['St_travel_dir'].values
    yvals_OI = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!='Tunnel_20180329-30')]['St_rotation'].values
    
    # PLOT THINGS
    c_array = colors.ListedColormap(pltcolors[ss])(range(0,100))
    c_array[:, -1]=np.arange(0,1,0.01)
    new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[ss], colors = c_array)
    plt.hexbin(xvals_OI, yvals_OI, bins = 'log', gridsize = 40, vmin =0, vmax = 3.5, cmap = new_cmap, edgecolors = 'none')#, color = pltcolors[ss])#,
#     ax = sns.jointplot(x = xvals_OI, y = yvals_OI, kind = 'hex', color = pltcolors[ss], xlim = (0,1), ylim = (-150,150))
    if ss == 3:
        cax = fig.add_axes([.93, .1, .02, .8])
        plt.colorbar(cax = cax, label = 'log(probability)')
        plt.sca(ax)
    plt.axis('equal')
    plt.ylim((-180,180))
    plt.xlim((-180,180))
    
    
    # quadrants
    for vline in [90, -90, 0,]:
        plt.axhline(y=vline, color = [0.7, 0.7, 0.7], linestyle = ':', alpha = 0.4)
    for vline in [90, -90, 0,]:
        plt.axvline(x=vline, color = [0.7, 0.7, 0.7], linestyle = ':', alpha = 0.4)
    
    # unicycle and co-aligned lines
# #     plt.plot([angle_buffer,180-angle_buffer],[angle_buffer,180-angle_buffer],'--k', alpha = 0.4)
#     plt.plot([angle_buffer,180],[angle_buffer-line_factor,180-line_factor],':k', alpha = 0.4)
#     plt.plot([angle_buffer,180],[angle_buffer+line_factor,180+line_factor],':k', alpha = 0.4)
# #     plt.plot([-angle_buffer,-180+angle_buffer],[-angle_buffer,-180+angle_buffer],'--k', alpha = 0.4)
#     plt.plot([-angle_buffer,-180],[-angle_buffer+line_factor,-180+line_factor],':k', alpha = 0.4)
#     plt.plot([-angle_buffer,-180],[-angle_buffer-line_factor,-180-line_factor],':k', alpha = 0.4)
    plt.plot([-180,180],[-180+line_factor,180+line_factor],':k', alpha = 0.4)
    plt.plot([-180,180],[-180-line_factor,180-line_factor],':k', alpha = 0.4)
    plt.plot([-90,90],[-180,180],'-k', alpha = 0.4)
    
    
    # chasse-ing
#     plt.plot([angle_buffer+line_factor,135],[+angle_buffer,+angle_buffer],':k', alpha = 0.4)
    plt.plot([0,180],[-angle_buffer,-angle_buffer],':k', alpha = 0.4)
#     plt.plot([135,180],[angle_buffer,-angle_buffer],':k', alpha = 0.4)
#     plt.plot([-angle_buffer-line_factor,-135],[-angle_buffer,-angle_buffer],':k', alpha = 0.4)
    plt.plot([0,-180],[angle_buffer,angle_buffer],':k', alpha = 0.4)
#     plt.plot([-135,-180],[-angle_buffer,angle_buffer],':k', alpha = 0.4)
    
    # going backwards
#     for vline in [angle_buffer, -angle_buffer, 180-angle_buffer, -180+angle_buffer]:
#         plt.axvline(x=vline, color = 'k', linestyle = ':', alpha = 0.4)

    # going backwards
    for vline in [angle_buffer, -angle_buffer]:
        plt.axvline(x=vline, color = 'k', linestyle = ':', alpha = 0.4)

    plt.title('%s'%subtype, loc = 'left')
    if ss == 0:
        plt.ylabel('facing rotation over stride (deg)')
#         plt.xlabel('straight/total dist')
        plt.xlabel('body travel dir wrt original facing (deg)')
    else:
        plt.gca().get_yaxis().set_visible(False)
    plt.axis('equal')
    plt.text(50, 220, 'ang. buffer: %i'%angle_buffer, FontSize = 6)
    plt.text(50, 205, 'n strides: %i'%len(xvals_OI), FontSize = 6)
    n_straight = np.sum( np.abs(xvals_OI)<15 )
    n_unicycle = np.sum( (np.abs(yvals_OI-xvals_OI)<15) & (np.abs(xvals_OI)>15) )
    n_sideslip = np.sum((np.abs(yvals_OI) < np.abs(xvals_OI)-15) & (np.sign(yvals_OI-15) == np.sign(xvals_OI)))
    n_backup = np.sum( (np.sign(yvals_OI-15) != np.sign(xvals_OI)) & (np.abs(xvals_OI)>15)  & (np.abs(yvals_OI)>15))
    plt.text(50, 190, 'n unicycle: %i'%n_unicycle, FontSize = 6)
    plt.text(50, 175, 'n sideslip: %i'%n_sideslip, FontSize = 6)
    plt.text(50, 160, 'n straight: %i'%n_straight, FontSize = 6)
#     plt.text(50, 160, 'S vs. U: %0.2f%%'%(n_sideslip/n_unicycle*100), FontSize = 6)
    ns[ss,:]=np.array([n_straight, n_unicycle, n_sideslip, n_backup])


plt.figure
idcs = ([0,1,2,3]*np.ones((4,4))).T
heights = ns/np.repeat(np.sum(ns,axis=1)[:,np.newaxis], 4, axis =1)
bottoms = np.concatenate([np.zeros((1,4)),heights[:,:-1].T]).T
for ii in range(4):
    plt.bar(idcs[ii,:],heights[ii,:],1,bottom=bottoms[ii,:])

#     heights = np.array([n_straight, n_unicycle, n_sideslip, n_backup])/len(xvals_OI)*5
#     print(heights)
#     bottoms = np.array([0, n_straight, n_unicycle, n_sideslip])
#     idcs = ss*np.ones(4)
#     plt.sca(bar_ax)
    
#     plt.bar(idcs, heights, 1, bottom = bottoms)
#     aglkngew
        
    


        

# STRIDE LENGTH
# dist plot of all lengths
# max_val = ceil(all_strides['StLen_all'].max()*10)/10
# n_bins = 100
# plt.close('all')
# plt.figure(figsize = (15,5))
# plt.xlabel('stride len (mm)')
# allsubs = [tr['substrate'] for tr in trial_info]
# subtypes = sorted(list(set(allsubs)))
# allcols = [tr['colony'] for tr in trial_info]
# coltypes = sorted(list(set(allcols)))
# for ss,subtype in enumerate(subtypes[0:4]):
#     plt.subplot(1,4, ss+1)
#     ax = sns.distplot(all_strides.loc[(all_strides['substrate']==subtype)]['StLen_all'].dropna().values, label = '%s'%subtype)#, 
# #                                  bins = np.linspace(0,max_val, n_bins+1), label = '%s'%subtype)
#     str_k = kurtosis(all_strides.loc[(all_strides['substrate']==subtype)]['StLen_all'].dropna().values)
#     str_s = skew(all_strides.loc[(all_strides['substrate']==subtype)]['StLen_all'].dropna().values)
#     B = (str_s**2 +1 )/str_k
#     print('K : %0.2f, scew: %0.2f, B: %0.2f'%(str_k, str_s, B))
#     plt.title('%s'%subtype, loc = 'left')
    
#     plt.xlim((-1, 8))
#     plt.ylim((0,1.5))
#     plt.text(7.5, 1.4, 'n: %i'%len(all_strides.loc[(all_strides['substrate']==subtype)]['StLen_all'].dropna().values), ha = 'right')
#     plt.text(7.5, 1.3, 'kurtosis: %0.2f'%str_k, ha = 'right')
#     plt.text(7.5, 1.2, 'skew: %0.2f'%str_s, ha = 'right')
#     plt.text(7.5, 1.1, 'bimodality coeff: %0.2f'%B, ha = 'right')
#     plt.xlabel('stride length (mm)')
#     if ss == 0:
#         plt.ylabel('probability')

In [ ]:
plt.figure
idcs = ([0,1,2,3]*np.ones((4,4))).T
heights = ns/np.repeat(np.sum(ns,axis=1)[:,np.newaxis], 4, axis =1)
bottoms = np.cumsum(np.concatenate([np.zeros((1,4)),heights[:,:-1].T]).T,axis=1)
pltcolors = ['b','r','g','k']
for ii in range(4):
    print(idcs[ii,:], heights[ii,:]*100)
    plt.bar(idcs[ii,:], heights[ii,:],1, bottom=bottoms[ii,:], color = pltcolors)
plt.ylabel('fraction of strides')
plt.gca().get_xaxis().set_visible(False)

In [ ]:
# model what it would look like for a car
c_len = 1
# steering=-75
# c_thrust = 1
plt.close('all')
plt.figure()

steerings = -1*np.arange(0,92,5)
thrusts = np.arange(0.1,2,0.1)
headVtravel = np.full((len(steerings),len(thrusts)), np.nan)


for ss, steering in enumerate(steerings):

    for tt, c_thrust in enumerate(thrusts[0:1]):
        c_heading = np.deg2rad(steering)
        
        # initial conditions
        c_angle = np.deg2rad(90)
        c_loc = np.zeros(2)
        c_head = c_loc+c_len/2*np.array([np.cos(c_angle), np.sin(c_angle)])
        c_tail = c_loc-c_len/2*np.array([np.cos(c_angle), np.sin(c_angle)])
        
        # consitions at next time step
        c_tail = c_tail + c_thrust * np.array([np.cos(c_angle), np.sin(c_angle)])
        delta_angle =  c_thrust / c_len * np.tan(c_heading)
        c_angle = (c_angle + c_thrust / c_len * np.tan(c_heading))
       # print('steering: ', steering,' -- delta heading: ', delta_angle)
        
        if np.abs(delta_angle) < np.pi/2:

            c_head = c_tail + c_len * np.array([np.cos(c_angle), np.sin(c_angle)])
            c_loc = c_tail + c_len / 2 * np.array([np.cos(c_angle), np.sin(c_angle)])

            travel_dir = 90-np.rad2deg(np.arctan2(c_loc[1],c_loc[0]))
            heading_rot = 90-np.rad2deg(c_angle)
            headVtravel[ss,tt] = heading_rot/travel_dir
        



            ### plot things
            plt.plot([ 0,0], [-1*c_len/2, c_len/2], '-b')
            plt.plot(0,0,'.b')
            plt.plot([c_head[0],c_tail[0]], [c_head[1],c_tail[1]], '-r', alpha = 0.5)
            plt.plot(c_loc[0], c_loc[1],'.r', alpha = 0.5)
            plt.axis('equal')
        
        # print things
    #     print(np.linalg.norm(np.diff(np.vstack([c_head, c_tail]),axis=0))) # sanity check that new length is 1
            print('steering: ', steering,' -- travel dir: ', travel_dir , '-- heading: ', heading_rot)

plt.figure()
sns.heatmap(headVtravel, center = 1 , cmap = 'seismic', xticklabels = np.round(thrusts, decimals =1),
            yticklabels = steerings,  cbar_kws={'label': 'heading/travel dir.'})  
plt.ylabel('steering angle')
plt.xlabel('forward speed (body lengths/stride)')

## Violin plots of straight stride speed

In [ ]:
plt.close('all')
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))

lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "date" : np.repeat(df['date'].values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values),
                        "time" : np.repeat(df['time'].values, lens)})

all_strides['St_speed'] = all_strides['St_tdist_total']/all_strides['St_Dur_all'] # in mm/s

plt.close('all')
plt.figure()
coltypes = sorted(list(set(df['colony'].values)))
angle_buffer = 15
pltcolors = ['#464F56', '#BA4246', '#087E8B', '#701C6F']
my_pal = {"0mm": '#464F56', "1mm": '#BA4246', "3mm": '#087E8B', "5mm": '#701C6F'}
ax = sns.violinplot(x = 'substrate', y = 'St_speed',  
                    data = all_strides[(all_strides['colony'] != 'Tunnel_20180508-09') &
                                     (np.abs(all_strides['St_travel_dir'])<angle_buffer)], cut = 0 , palette=my_pal) #hue = 'substrate',
ax.set_ylabel('stride speed [mm/s]')
plt.gcf()



print('Saving median and bootstrap files as feathers')
import feather
temp = all_strides[(all_strides['colony'] != 'Tunnel_20180508-09') &
                                     (np.abs(all_strides['St_travel_dir'])<angle_buffer)]
colony_R = [col.split('20180')[-1][1:] for col in temp['colony'].values.tolist()]
date_days = [col[-2:] for col in temp['date'].values.tolist()]
day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
subs_string = temp['substrate'].values.tolist()
substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
st_speed_R = np.array(temp['St_speed'])
df_stspeed_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
                       "stride_speed" : st_speed_R } )
feather.write_dataframe(df_stspeed_R, vid_locations + 'St_Speed.feather')
for ss in list(set(df['substrate'].values)):
    print(ss, len(temp[temp['substrate']==ss]))
del colony_R, date_days, day_R, subs_string, substrate_R, st_speed_R, df_stspeed_R
del temp, all_strides

## Stride frequency vs. straight stride speed -- identify disrupted strides

In [421]:
# stride length or frequency vs. straight stride speed - hexbin plot
from scipy.spatial import ConvexHull

plt.close('all')
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='_St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_start'] = df.filter(regex='_St_start$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1) 
df['St_stop'] = df.filter(regex='_St_stop$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1) 
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))

lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_start" : np.concatenate(df['St_start'].values), "St_stop" : np.concatenate(df['St_stop'].values),    
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values),
                        "time" : np.repeat(df['time'].values, lens)})


subtypes = sorted(list(set(df['substrate'])))
coltypes = sorted(list(set(df['colony'].values)))
fig = plt.figure(1,figsize = (15,4))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
angle_buffer = 15

# Stride length vs. speed
slope, intercept, res_width = np.full(4,np.nan),np.full(4,np.nan),np.full(4,np.nan)
if 'density' not in locals():
    density = {}
for ss, subtype in enumerate(subtypes[0:4]):
    plt.figure(1)
    sp_ax=plt.subplot(1,4, ss+1)
    
    # FOR STRIDE FREQUENCY! - SPECIFY IF FOR ALL STRIDES OR JUST STRAIGHT ONES
    SL_OI = 1/all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Dur_all'].values
    vals_OI = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_tdist_total'].values/
               all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Dur_all'].values)
    str_idcs = np.isfinite(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])
#     str_idcs = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])<angle_buffer # only straight strides
    d_cutoff = 0.005 # stride duration

    
    # FOR STRIDE LENGTH
#     SL_OI = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Len_all'].values
#     vals_OI = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_tdist_total'].values/
#                all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Dur_all'].values)
#     str_idcs = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])<angle_buffer
#     d_cutoff = 0.03 # stride length
    

    plt.title('%s'%subtype, loc = 'left')
    if ss == 0:
        plt.ylabel('stride len (mm)')
        plt.ylabel('stride freq (Hz)')
#         plt.xlabel('total stride speed (mm/s)')
    else:
        plt.gca().get_yaxis().set_visible(False)
        

    # FIT LINE ONLY WHERE DENSE POINTS
    from sklearn.neighbors import KernelDensity
    x = vals_OI[str_idcs]
    y = SL_OI[str_idcs]
    no_nan_pts = np.logical_and(np.isfinite(x), np.isfinite(y))
    x = x[no_nan_pts]
    y = y[no_nan_pts]
    
    xy_train = np.vstack([y,x]).T
    d = np.min(xy_train.shape) # number of dimensions
    n = np.max(xy_train.shape) # number of points
    bw_sil = (n*(d+2) / 4.) ** (-1./ (d +4)) # silverman - "Density estimation for statistics and data analysis" 1986
    bw_sco = n**(-1./(d+4)) # scott's - "Multivariate density estimateion..." 1992
    print('bandwidths (scott vs. silverman): %0.9f, %0.9f'%(bw_sco, bw_sil))
    kde = KernelDensity(bandwidth = bw_sco, kernel = 'gaussian')
    kde.fit(xy_train)
    density[ss] = np.exp(kde.score_samples(np.vstack([y,x]).T))

    inlier_mask = density[ss] > d_cutoff
    outlier_mask = np.logical_not(inlier_mask)
    
    # if want to plot whole distribution
#     xx, yy = np.meshgrid(np.arange(np.floor(x.min()),np.ceil(x.max()),1), np.arange(np.floor(y.min()),np.ceil(y.max()),0.1))
#     xy_sample = np.vstack([yy.ravel(), xx.ravel()]).T
#     zz = np.reshape(np.exp(kde.score_samples(xy_sample)), xx.shape)
#     print(np.sum(np.sum(zz)))
#     # e.g. plt.pcolormesh(xx,yy,zz)
    
    
    # calc linear regression
    slope[ss], intercept[ss], r_val, p_val, std_err = stats.linregress(vals_OI[str_idcs][no_nan_pts][inlier_mask], SL_OI[str_idcs][no_nan_pts][inlier_mask])
    res_width[ss] = 4*np.nanstd(y[inlier_mask]-(slope[ss]*x[inlier_mask]+intercept[ss]))
    disrupted_strides = y>(slope[ss]*x+intercept[ss]+res_width[ss])
    
#     A = np.vstack([vals_OI[str_idcs][inlier_mask], np.ones(np.sum(inlier_mask))]).T 
    slope_origin = np.linalg.lstsq(vals_OI[str_idcs][no_nan_pts][inlier_mask][:,np.newaxis], SL_OI[str_idcs][no_nan_pts][inlier_mask])[0]
    print(slope_origin)
    
    
    # USE PCA TO GET RID OF OUTLIERS?
#     points = np.array([vals_OI[str_idcs], SL_OI[str_idcs]]).T
#     cent =np.mean(points, 0)
#     new_points = StandardScaler().fit_transform(points-cent)
#     pca = PCA(n_components = 2)
#     pca.fit(new_points)
#     el_angle = np.arctan2(pca.components_[0][1], pca.components_[1][1])
#     el_var = pca.explained_variance_
#     R = np.array([[np.cos(el_angle), np.sin(el_angle)],[np.sin(el_angle), -1*np.cos(el_angle)]])
#     print(points.shape, R.shape)
#     el_values = np.sum(np.square(np.einsum('ij, jk->ik', points-cent, R))/(1.96**2 * el_var)[np.newaxis,:], axis =1)
#     print('in: %i, out: %i'%(np.sum(el_values<=1), np.sum(el_values>1)))
#     inlier_mask = el_values <= 1
#     projected = pca.transform(new_points)
#     inlier_mask = np.logical_and(np.abs(projected[:,1]) < .5, np.abs(projected[:,0]) < 3)
#     inlier_mask = new_points[:,1] <= np.median(np.abs(new_points[:,1]))



    # plot using hex bin
#     c_array = colors.ListedColormap(pltcolors[ss])(range(0,1000))
#     c_array[:, -1]=np.arange(0,1,0.001)
#     new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[ss], colors = c_array)
#     plt.hexbin(vals_OI[str_idcs], SL_OI[str_idcs],
#                 gridsize = 40, vmin =0, vmax = 200, cmap = new_cmap, edgecolors = 'none')
    
    # plot using 2d hist
    c_array = colors.ListedColormap(pltcolors[ss])(range(0,1000))
    c_array[:, -1]=np.arange(0,1,0.001)
    new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[ss], colors = c_array)
    hb=sp_ax.hist2d(x, y, 50, weights =np.ones_like(x)/float(len(x)), vmin = 0, vmax =0.008, cmap = new_cmap)
    
    # convex hull of all inlier points
    hull = ConvexHull(np.vstack([y[inlier_mask],x[inlier_mask]]).T)
    for simplex in hull.simplices:
        plt.plot(x[inlier_mask][simplex], y[inlier_mask][simplex], 'k-')
    
    # plot cinnamon trials
#     cin_df = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']=='Tunnel_20180329-30') &
#                             ((all_strides['time'].map(lambda x: (int(x)>=90000) & (int(x[-3])%5-1<1))).values)]
#     SL_OI = 1/cin_df['St_Dur_all'].values
#     vals_OI = np.abs(cin_df['St_tdist_total'].values/cin_df['St_Dur_all'].values)
#     str_idcs = np.abs(cin_df['St_travel_dir'])<angle_buffer
#     plt.plot(vals_OI[str_idcs], SL_OI[str_idcs], '.k', alpha = 0.1, MarkerSize = 2)
    
    ### PLOT AESTHETICS
    # plot aesthetics - stride frequency
    plt.plot(np.arange(0,40), slope[ss]*np.arange(0,40)+intercept[ss], '--k')
    plt.plot(np.arange(0,40), slope_origin*np.arange(0,40), ':k')
    plt.plot(np.arange(0,40), slope[ss]*np.arange(0,40)+intercept[ss]+res_width[ss], '--k', alpha = 0.5)
    plt.text(20, 30, 'n: %i'%len(x))
    plt.text(20, 28, '%% disrupted:%0.1f'%(np.sum(disrupted_strides)/len(x)*100))
    plt.text(20, 26, 'slope: %0.3f'%slope[ss])
    plt.text(20, 24, 'intercept: %0.3f'%intercept[ss])
    plt.text(20, 22, 'slope_orig: %0.3f'%slope_origin)
    plt.xlim((0,40))
    plt.ylim((0,30))

    # plot aesthetics - stride length
#     plt.plot(np.arange(0,40), slope[ss]*np.arange(0,40)+intercept[ss], '--k')
#     plt.text(20, 4.5, 'n: %i'%len(x))
#     plt.text(20, 4.2, 'slope: %0.2f'%slope[ss])
#     plt.text(20, 3.9, 'intercept: %0.2f'%intercept[ss])
#     plt.xlim((0,40))
#     plt.ylim((0,5))
    
    
    if ss==3:
        ax=fig.add_axes([.93,.1,.02,.75])
        fig.colorbar(hb[3], cax=ax)
    fig.add_subplot(sp_ax)
    
#     if ss==2:
#         cax = fig.add_axes([.93, .1, .02, .8])
#         plt.colorbar(cax = cax, label = 'number of strides')
        
#         plt.figure()
#         plt.subplot(1,2,1)
#         inlier_mask = density[ss] > 0.005
#         plt.plot(x[inlier_mask],y[inlier_mask],'.k', alpha = 0.02)
#         plt.xlim((0,40))
#         plt.ylim((0,5))
#         plt.subplot(1,2,2)
#         inlier_mask = density[ss] > 0.03
#         plt.plot(x[inlier_mask],y[inlier_mask],'.k', alpha = 0.02)
#         plt.xlim((0,40))
#         plt.ylim((0,5))
    
    # FIT RANSAC REGRESSION - gets rid of outliers, not working great
#     ransac = linear_model.RANSACRegressor(stop_probability = 0.995)
#     ransac.fit(vals_OI[str_idcs][inlier_mask][:,np.newaxis], SL_OI[str_idcs][inlier_mask][:,np.newaxis])
#     inlier_mask2 = ransac.inlier_mask_
#     outlier_mask2 = np.logical_not(inlier_mask2)
#     line_X = np.arange(0,40)[:, np.newaxis]
#     line_y = ransac.predict(line_X)
#     plt.plot(line_X, line_y, '--k')
#     plt.plot(line_X, line_y+3, ':k', alpha = 0.5)
#     plt.plot(line_X, line_y-3, ':k', alpha = 0.5)
#     print(ransac.estimator_.coef_)
#     plt.plot(vals_OI[str_idcs][inlier_mask], SL_OI[str_idcs][inlier_mask], '.', color = pltcolors[ss], alpha = 0.02)
#     plt.plot(vals_OI[str_idcs][outlier_mask], SL_OI[str_idcs][outlier_mask], '.', color = 'k', alpha = 0.02)
    

    
    
# calculate if strides are disrupted based on stride length
def calc_SF_disrupted_df(x, slope, intercept, res_width, r_cutoff = 4):
    xs = x['St_tdist_total']/ x['St_Dur_all']
    ys = 1/x['St_Dur_all']
    
    ss = np.where(np.array(['0mm','1mm','3mm','5mm'])==x['substrate'])[0][0]
    
    disrupted_strides = ys>(slope[ss]*xs+intercept[ss]+res_width[ss])
    return disrupted_strides

df['St_disrupted_SF']= df.apply(calc_SF_disrupted_df, args = (slope, intercept, res_width, 4), axis=1)
    
del x, y, xy_train, kde, inlier_mask, outlier_mask, SL_OI, vals_OI, str_idcs, disrupted_strides
# del all_strides

bandwidths (scott vs. silverman): 0.175491039, 0.175491039


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[0.45316253]
bandwidths (scott vs. silverman): 0.205089878, 0.205089878
[0.5297038]
bandwidths (scott vs. silverman): 0.182314024, 0.182314024
[0.54010848]
bandwidths (scott vs. silverman): 0.166293335, 0.166293335
[0.51285222]


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:218: RuntimeWarning: invalid value encountered in greater


In [15]:
# save as feather for lme models in R
import feather
temp = df.copy()


# look only at trials longer than 50fr
idcs = [index for index, row in temp.iterrows() if len(row.St_disrupted_SF)>10]
longtracks = df.loc[idcs,]
# longtracks = longtracks.loc[(longtracks['colony']!=coltypes[-1])] # don't include cinnamon trial data
longtracks = longtracks.loc[(longtracks['colony']==coltypes[-1])] # just cinnamon trial data

# MEDIAN V
colony_R = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
subs_string = longtracks['substrate'].values.tolist()
substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
v_med_R = np.array(longtracks['median_v'])/pix2mm
n_disrupted_R = longtracks['St_disrupted_SF'].apply(lambda x: np.sum(x)).values
n_strides_R = longtracks['St_disrupted_SF'].apply(lambda x: len(x)).values
p_disrupted_R = longtracks['St_disrupted_SF'].apply(lambda x: np.sum(x)/len(x)).values
df_med_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
                       "v_med" : v_med_R, "n_disrupted": n_disrupted_R,
                      "n_strides": n_strides_R, "percent_disrupted": p_disrupted_R  } )

# SAVE AS FEATHER FOR USE WITH R
print('Saving disrupted stride data as feather for use in R')
# feather.write_dataframe(df_med_R, vid_locations + 'Disrupted_SF.feather')
feather.write_dataframe(df_med_R, vid_locations + 'Disrupted_SF_cinnamon.feather')
# testdf = feather.read_dataframe(vid_locations + 'Disrupted_SF.feather')
print('Done saving')

del idcs, colony_R, day_R, substrate_R, v_med_R, date_days, subs_string, temp, longtracks, n_disrupted_R
del df_med_R

Saving disrupted stride data as feather for use in R
Done saving


In [ ]:
# TEST OUT KERNEL DENSITY
plt.close('all')
ss=0
lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values),
                        "time" : np.repeat(df['time'].values, lens)})

allsubs = [tr['substrate'] for tr in trial_info]
subtypes = sorted(list(set(allsubs)))
coltypes = sorted(list(set(df['colony'].values)))
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
angle_buffer = 15

subtype = subtypes[0]
density = {}
    
SL_OI = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Len_all'].values
vals_OI = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_tdist_total'].values/
           all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Dur_all'].values)
str_idcs = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])<angle_buffer
del all_strides



from sklearn.neighbors import KernelDensity
x = vals_OI[str_idcs]
y = SL_OI[str_idcs]


xy_train = np.vstack([y,x]).T
d = np.min(xy_train.shape) # number of dimensions
n = np.max(xy_train.shape) # number of points
bw_sil = (n*(d+2) / 4.) ** (-1./ (d +4)) # silverman - "Density estimation for statistics and data analysis" 1986
bw_sco = n**(-1./(d+4)) # scott's - "Multivariate density estimateion..." 1992
print('bandwidths (scott vs. silverman): %0.9f, %0.9f'%(bw_sco, bw_sil))
kde = KernelDensity(bandwidth = bw_sco, kernel = 'gaussian')
kde.fit(xy_train)
density[ss] = np.exp(kde.score_samples(np.vstack([y,x]).T))


xx, yy = np.meshgrid(np.arange(np.floor(x.min()),np.ceil(x.max()),1), np.arange(np.floor(y.min()),np.ceil(y.max()),0.1))
xy_sample = np.vstack([yy.ravel(), xx.ravel()]).T
zz = np.reshape(np.exp(kde.score_samples(xy_sample)), xx.shape)


inlier_mask = density[ss] > 0.005
# outlier_mask = np.logical_not(inlier_mask)
# calc linear regression
# slope, intercept, r_val, p_val, std_err = stats.linregress(vals_OI[str_idcs][inlier_mask], SL_OI[str_idcs][inlier_mask])

plt.pcolormesh(xx,yy,zz)

# plt.figure()
# plt.subplot(1,2,1)
# inlier_mask = density[ss] > 0.005
# plt.plot(x[inlier_mask],y[inlier_mask],'.k', alpha = 0.02)
# plt.xlim((0,40))
# plt.ylim((0,5))
# plt.subplot(1,2,2)
# inlier_mask = density[ss] > 0.1
# plt.plot(x[inlier_mask],y[inlier_mask],'.k', alpha = 0.02)
# plt.xlim((0,40))
# plt.ylim((0,5))

plt.figure()
plt.scatter(x, y, c=density[0], s = 2)#, alpha = 0.01)

In [ ]:
# plot speed vs. % disrupted for trials with many strides:
from scipy.stats import poisson

plt.close('all')
# plt.figure()
for ss, subtype in enumerate(subtypes[0:4]):
    plt.figure()
    vals_OI = df.loc[(df['substrate']==subtype) & (df['colony']!=coltypes[-1])]['St_disrupted_SF'].apply(lambda x: np.sum(x)/len(x)).values
    med_v = df.loc[(df['substrate']==subtype) & (df['colony']!=coltypes[-1])]['median_v'].values
    n_strides = df.loc[(df['substrate']==subtype) & (df['colony']!=coltypes[-1])]['St_disrupted_SF'].apply(lambda x: len(x)).values
    
    glenna = vals_OI[n_strides>10]*100
    print(np.mean(glenna))

    plt.plot(vals_OI[n_strides>10], med_v[n_strides>10]/pix2mm, '.', color = pltcolors[ss], alpha = 0.05)
    plt.xlabel('perecent of strides disrupted based on stride frequency')
    plt.ylabel('median trackway speed (mm/s)')
    plt.xlim([0, 0.7])
    plt.ylim([0,35])
    
    print(stats.linregress(vals_OI[n_strides>10], med_v[n_strides>10]/pix2mm))
#     plt.hist(vals_OI, bins=np.arange(0,0.52,0.02), range = (0,0.5), alpha = 0.2, color = pltcolors[ss], density = True)
    
#     for power_OI in [1,0.5,1/3]:
#         dist_hist = np.histogram(np.power(vals_OI, power_OI), bins = np.arange(0,0.52,0.02), range = (0,0.5), density = True)[0]
#         plt.plot(np.arange(0,0.5,0.02), dist_hist, '.', label = 'power %0.1f'%power_OI)

#     resolution = 2

#     dist_hist = np.histogram(glenna, bins = np.arange(0,50+resolution,resolution), range = (0,50), density = True)[0]
#     plt.plot(np.arange(0,50,resolution), dist_hist, '.', label = 'raw hist')
    
    # get poisson from scipy to get expected probability density function
#     pois_comp = poisson.pmf(np.arange(0,50,resolution), np.mean(glenna))
#     plt.plot(np.arange(0,50,resolution), pois_comp, ':k', label = 'poisson L = %0.1f'%np.mean(glenna))
    
    
    # log
#     dist_hist = np.histogram(np.log(glenna*100), bins = np.arange(0,52,2), range = (0,0.5), density = True)[0]
#     plt.plot(np.arange(0,50,2), dist_hist, '.', label = 'log')
    plt.legend()
    
# del vals_OI, n_strides, med_v


# # print info for some trials:
# for tr in range(0,10):
#     print('\nTrial %i:')
#     print('median v: %0.2f, %i/%i disrupted strides'%(df['median_v'][tr], np.sum(df['St_disrupted_SF'][tr]) , len(df['St_disrupted_SF'][tr])))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.special import factorial
from scipy.stats import nbinom


def poisson(k, lamb):
    """poisson pdf, parameter lamb is the fit parameter"""
    return (lamb**k/factorial(k)) * np.exp(-lamb)


def negLogLikelihood(params, data):
    """ the negative log-Likelohood-Function"""
    lnl = - np.sum(np.log(poisson(data, params[0])))
    return lnl



def negLogLikelihood_NB(params, data):
    """ the negative log-Likelohood-Function"""
    lnl = - np.sum(np.log(nbinom(params[0], params[1]).pmf(data)  ))
    return lnl

# def CMPoisson(k, lamb, decay):
#     """poisson pdf, parameter lamb is the fit parameter"""
#     return (lamb**k/factorial(k)) * np.exp(-lamb)





# get poisson deviated random numbers
data = glenna

# # minimize the negative log-Likelihood
# result = minimize(negLogLikelihood,  # function to minimize
#                   x0=np.ones(1),     # start value
#                   args=(data,),      # additional arguments for function
#                   method='Powell',   # minimization method, see docs
#                   )

# minimize the negative log-Likelihood
result = minimize(negLogLikelihood_NB,  # function to minimize
                  x0=np.ones(1),     # start value
                  args=(data,),      # additional arguments for function
                  method='Powell',   # minimization method, see docs
                  )

# result is a scipy optimize result object, the fit parameters 
# are stored in result.x
print(result)

# plot poisson-deviation with fitted parameter
x_plot = np.linspace(0, 50, 1000)

plt.close('all')
plt.hist(data, bins=np.arange(0,50,2) - 0.5, normed=True)
plt.plot(x_plot, poisson(x_plot, result.x), 'r-', lw=2)
plt.show()





def convert_params(mu, theta):
    """
    Convert mean/dispersion parameterization of a negative binomial to the ones scipy supports

    See https://en.wikipedia.org/wiki/Negative_binomial_distribution#Alternative_formulations
    """
    var = mu + theta * mu ** 2
    p = (var - mu) / var
    return theta, 1 - p

In [243]:
# which limb most likely to have disrupted strides?
all_strides["disrupted_strides"] = np.concatenate(df['St_disrupted_SF'].values)


temp=all_strides['Joints_all'][all_strides['disrupted_strides']]
all_js = np.array([int(x[-1]) for x in temp])
temp = all_strides['substrate'][all_strides['disrupted_strides']]
all_ss = np.array([int(x[0]) for x in temp])

disrupted_st_by_limb = np.full([6,4], np.nan)
for s,ss in enumerate([0,1,3,5]):
    for jj in range(0,6):
        disrupted_st_by_limb[jj,s] = np.sum(np.logical_and(all_ss ==ss, all_js == jj))



plt.close('all')
plt.hist(clifton, bins= 6)
plt.xlabel('joint num')
plt.ylabel('# disrupted strides')


plt.figure()
for s,ss in enumerate([0,1,3,5]):
    for jj in range(0,3):
        plt.plot([jj,jj], disrupted_st_by_limb[[jj,jj+3], s], '-', color = pltcolors[s])
        plt.plot(jj, np.mean(disrupted_st_by_limb[[jj,jj+3], s]), '+', color = pltcolors[s])
        plt.plot(jj, np.mean(disrupted_st_by_limb[[jj,jj+3], s]), 'o', color = pltcolors[s])
plt.ylim([0,1500])
plt.yticks([0,500,1000,1500])
plt.text(0,1300, 'total n = %i'%np.sum(all_strides['disrupted_strides']) )
    

del temp

## Make video of sample disrupted strides

In [184]:
# functions for making video

def load_video(raw_video_path, frame_range, verbose):
    """
    Independent of the frame range loaded, background has to be computed over total video or else can run into
    tracking problems
    """
    vid = cv2.VideoCapture(raw_video_path)
    Height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    NumFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    if not (NumFrames > 0):
        raise IOError('Codec issue: cannot read number of frames.')

    # restrict to desired range of frames
    if frame_range is None:
        frame_range = (0, int(NumFrames))
    else:
        # check doesn't exceed number of frames
        if frame_range[0] + frame_range[1] > NumFrames:
            frame_range = (int(frame_range[0]), int(NumFrames - frame_range[0]))

    # initialize blank frames
    frames = np.zeros((frame_range[1], Height, Width), np.uint8)

    # set the first frame to read in
    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for kk in range(frame_range[0]):
        tru, ret = vid.read(1)
    # vid.set(cv.CAP_PROP_POS_FRAMES, frame) # this way of setting the frame doesn't work on all cv versions

    # read in all frames
    for kk in range(frame_range[1]):
        tru, ret = vid.read(1)

        # check if video frames are being loaded
        if not tru:
            raise IOError('Codec issue: cannot load frames.')
        frames[kk, :, :] = ret[:, :, 0]  # assumes loading color
        if ((kk % 100) == 0) and verbose:
            print(kk)
    return frames, NumFrames, frame_range, vid



def WRTant_to_WRTframe(val_x, val_y, frame_center_x, frame_center_y, ant_ang_deg):
    ant_ang = ant_ang_deg *np.pi/180
    R = np.array([[np.cos(ant_ang), -1*np.sin(ant_ang)],
                  [np.sin(ant_ang),    np.cos(ant_ang)]])
    rotated_vals = np.dot(R,np.array([val_x-100,val_y-100]))
    translated_vals = rotated_vals*np.array([1,1]) + np.array([frame_center_x, frame_center_y])  
    return translated_vals[0], translated_vals[1];

def plot_ant_pt(ant_part, ant_part_num, filt, df, tr_num, idx, ant_x, ant_y, ant_ang_deg, buffer): #filt = '' if want raw data
    x = df['%s%s_x%s'%(ant_part,str(ant_part_num),filt)][tr_num][idx]
    y = df['%s%s_y%s'%(ant_part,str(ant_part_num),filt)][tr_num][idx]
    conf = df['%s%s_conf'%(ant_part,str(ant_part_num))][tr_num][idx]
#     (newx, newy)= (x,y)
    (newx, newy) = WRTant_to_WRTframe(x, y, ant_x, ant_y, ant_ang_deg)
#     print('old vals: %i, %i  TO %0.1f, %0.1f'%(x,y,newx, newy))

    if ('joint' in ant_part) or ('antenna' in ant_part):
        if '_filt' in filt:
            
            # plot strides and touchdowns
            if 'joint%i_TD_idcs'%ant_part_num in df:
                if idx in df['joint%i_TD_idcs'%ant_part_num][tr_num][df['joint%i_good_TDs'%ant_part_num][tr_num]]:
#                     print('Joint %i -- Fr %i'%(ant_part_num, idx))
                    sca = plt.scatter(newx+buffer, newy+buffer, s = 10, facecolor = 'none', edgecolor = 'w')
                dur_starts = df['joint%i_TD_idcs'%ant_part_num][tr_num][:-1][df['joint%i_good_strides'%ant_part_num][tr_num]]
                dur_stops = df['joint%i_TD_idcs'%ant_part_num][tr_num][1:][df['joint%i_good_strides'%ant_part_num][tr_num]]
                if np.any(np.logical_and( ff>dur_starts, ff<dur_stops)):
                    str_OI = dur_starts[ np.logical_and( ff>dur_starts, ff<dur_stops)]
                    plt.plot([df['joint%i_x_filt_fullfr'%ant_part_num][tr_num][str_OI]+buffer, newx+buffer],
                             [df['joint%i_y_filt_fullfr'%ant_part_num][tr_num][str_OI]+buffer, newy+buffer], '-w', alpha = 0.2)
                
            # plot actual feet points
            if ant_part_num < 3:
                sca = plt.scatter(newx+buffer, newy+buffer, c = 'c', s = 10, edgecolor = 'none')# '.g')
            else:
                sca = plt.scatter(newx+buffer, newy+buffer, c = 'm', s = 10, edgecolor = 'none')
#             if ('joint' in ant_part):
#                 if df['frames_final'][tr_num][fr_num] in df['%s%i_TD_frs'%(ant_part, ant_part_num)][tr_num]:
#                     sca.set_edgecolor('w')
        else:
            # define colormap to show confidence
            norm2 = colors.Normalize(vmin=0, vmax=1)
            plt.scatter(newx+buffer, newy+buffer, c = conf, s = 10, cmap = cm.bwr,
                       edgecolor = 'none', norm=norm2)# '.g')
    else:
        plt.scatter(newx+buffer, newy+buffer, c = 'w', s = 10, edgecolor = 'none')
        
    return;


def crop_to_view(variable_to_use, tr_num, fr, x_dim, y_dim, buffer, axisOI):
    x = df[variable_to_use%'x'][tr_num][fr]
    y = df[variable_to_use%'y'][tr_num][fr]
    xrange = range(int(round(x)), int(round(x+2*buffer)))
    yrange = range(int(round(y)), int(round(y+2*buffer)))
    # account for if range goes outside of video frame
    xrange_actual = np.array(sorted(list( set(xrange) & set(range(0, x_dim+2*buffer) ) )))[[0,-1]]
    yrange_actual = np.array(sorted(list( set(yrange) & set(range(0, y_dim+2*buffer) ) )))[[0,-1]]
    plt.sca(axisOI)
    plt.xlim(xrange_actual)
    plt.ylim(yrange_actual)
    
    return xrange_actual, yrange_actual


def save_image(vlocation, nfig, name_base):
    pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
    plt.savefig(pname)
    nfig = nfig + 1
    plt.pause(0.2)
#     plt.close('all')
    return nfig


def save_video(vlocation, name_base):
    # save images as movie
    if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
        os.remove(vlocation + "/%s.mp4"%name_base)
        print('** Deleted %s.mp4 file'%name_base)
    print('saving %s.mp4 file'%name_base)
    command_p1 = "ffmpeg -r 10 -i '%s/%s"%(vlocation, name_base)
    command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
    command = command_p1 + "%01d.png'" + command_p2
#     print(command)
    os.system(command)
    plt.pause(10)

    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
    for pic in pics2delete:
        os.remove(pic)
    return

In [440]:
# plot images with tracked data and lowpass filtered data

buffer = 150
limbs = ['LH','LM','LF', 'RH', 'RM', 'RF']
vlocation = '/media/gravishlab/SeagateExpansionDrive/AntTrack'

#where are disrupted stides
plt.close('all')
fig = plt.figure(figsize=(8,8))
im_n = 0

# FOR DISRUPTED STRIDES
# for tt in np.arange(0,20):
#     tr_num = random.randint(0,len(df))

# FOR WEIRDLY LONG STRIDE DISTANCE
for tr_num in all_strides.loc[all_strides['St_tdist_total']>4]['trackway'].values:


    videofile = df.video[tr_num]
    print(videofile)
    
#     for st in np.where(df['St_disrupted_SF'][tr_num])[0]: # look at disrupted strides
    for st in np.where(df['St_tdist_total'][tr_num]>4)[0]: # look at long strides
        
        frame_range =[df['St_start'][tr_num][st]-20 , 
                      df['St_stop'][tr_num][st]+20 ]
        if frame_range[0]<np.min(df['frames'][tr_num]):
            frame_range[0] = int(np.min(df['frames'][tr_num]))
        if frame_range[1]>np.max(df['frames'][tr_num]):
            frame_range[1] = int(np.max(df['frames'][tr_num]))
        joint_OI = int(df['St_jointID'][tr_num][st][-1])
        print('    %i -- stride OI: %i, joint %i, starting fr %i'%(tr_num, st, joint_OI, frame_range[0]))

        frames, NumFrames, _, vid = load_video(videofile, frame_range, verbose = False)
        

        for ff, fr_OI in  enumerate(np.arange(frame_range[0], frame_range[1])):
            fr_id = np.where(df['frames'][tr_num] == fr_OI)[0][0]


            plt.clf()
            # load frame
            x_dim = frames.shape[2]
            y_dim = frames.shape[1]
            frame = np.stack((frames[ff,:,:],)*3,-1)

            # RAW IMAGE WITH FILTERED DATA
            ax3=fig.add_axes([0.1,0.1, 0.8, 0.8]) #plt.axes()
            black_frame = np.ones((y_dim+ 2*buffer, x_dim+ 2*buffer,3),dtype=np.uint8)* 1# 1.001# np.max(temp) # gray background  1.0001#
            bframe = black_frame.copy()
            bframe[buffer:-buffer, buffer:-buffer,:] = frame
            plt.imshow(bframe)
            xrange_actual, yrange_actual = crop_to_view('%s_raw', tr_num, fr_id, x_dim, y_dim, buffer, ax3)
            plt.text(xrange_actual[0]+10, yrange_actual[1]-20, 'Fr: %i'%fr_OI, color= 'w')
            plt.text(xrange_actual[0]+10, yrange_actual[1]-30, 'Joint OI: %i'%joint_OI, color= 'w')
            ss= np.where(np.array([0,1,3,5])==int(df['substrate'][tr_num][0]))[0]
            st_resid = (1/df['St_Dur_all'][tr_num][st] - (slope[ss]*df['St_tdist_total'][tr_num][st]/ df['St_Dur_all'][tr_num][st]+intercept[ss]))/(res_width[ss]/4)
            plt.text(xrange_actual[0]+10, yrange_actual[0]+40, 'Residual = %0.2f stdevs'%st_resid, color= 'w')
                
            # plot text and foot data if during disrupted stride
            if np.logical_and(fr_OI >= df['St_start'][tr_num][st], fr_OI <= df['St_stop'][tr_num][st] ):
                plt.text(xrange_actual[0]+10, yrange_actual[0]+30, 'Disrupted Stride', color= 'r')
                plt.plot(df['joint%i_x_filt_fullfr'%joint_OI][tr_num][fr_id]+buffer, df['joint%i_y_filt_fullfr'%joint_OI][tr_num][fr_id]+buffer, '.r')
            else:
                plt.scatter(df['joint%i_x_filt_fullfr'%joint_OI][tr_num][fr_id]+buffer, df['joint%i_y_filt_fullfr'%joint_OI][tr_num][fr_id]+buffer, 
                            s = 40, facecolors='none', edgecolors = 'w')

            titleparts = videofile.split('/')
            plt.suptitle( 'Tr: %i -- %s -- %s -- %s'
                  %(tr_num, titleparts[-2], titleparts[-1].split('_')[0], titleparts[-1].split('_')[1]),x=0.02, y=.95, horizontalalignment = 'left')
            plt.gca().invert_yaxis()

            plt.pause(0.1)
            save_image(vlocation, im_n, 'Disrupted_Strides_vid')
            im_n = im_n+1
save_video(vlocation, 'Disrupted_Strides_vid')

/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/3mm/20180313_093328_16276718-0000.mp4
    806 -- stride OI: 14, joint 3, starting fr 203
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/3mm/20180313_101847_16276718-0000.mp4
    835 -- stride OI: 33, joint 4, starting fr 134
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/5mm/20180313_093139_16276735-0000.mp4
    1213 -- stride OI: 4, joint 1, starting fr 123
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/5mm/20180313_094113_16276735-0000.mp4
    1220 -- stride OI: 3, joint 0, starting fr 438
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/5mm/20180313_101212_16276735-0000.mp4
    1244 -- stride OI: 1, joint 0, starting fr 292
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/5mm/20180313_105741_16276735-0000.mp4
    1298 -- stride OI: 18, joint 2, starting fr 407
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_2

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in greater


/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180320-21/3mm/20180321_100640_16276718-0000.mp4
    3606 -- stride OI: 17, joint 3, starting fr 120
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180320-21/5mm/20180321_085929_16276736-0000.mp4
    3912 -- stride OI: 11, joint 5, starting fr 6
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180322-23/1mm/20180322_093612_16276712-0000.mp4
    4416 -- stride OI: 11, joint 3, starting fr 152
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180322-23/3mm/20180322_091855_16276735-0000.mp4
    5025 -- stride OI: 2, joint 2, starting fr 0
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180322-23/3mm/20180322_093913_16276735-0000.mp4
    5089 -- stride OI: 14, joint 5, starting fr 57
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180322-23/3mm/20180323_115050_16276735-0000.mp4
    5768 -- stride OI: 36, joint 3, starting fr 301
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20

In [ ]:
ss=0
n_st = len(df['St_jointID'][tr_num])
np.reshape(np.vstack([df['St_jointID'][tr_num], df['St_start'][tr_num], df['St_Dur_all'][tr_num], 
                     1/df['St_Dur_all'][tr_num], slope[ss]*df['St_tdist_total'][tr_num]/ df['St_Dur_all'][tr_num]+intercept[ss]+res_width[ss]]).T, (n_st,5))

## Plot travel dist vs. stride length

In [422]:
plt.close('all')
lens = [len(item) for item in df['St_Len_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_start" : np.concatenate(df['St_start'].values), "St_stop" : np.concatenate(df['St_stop'].values),    
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values),
                        "time" : np.repeat(df['time'].values, lens), "date" : np.repeat(df['date'].values, lens)})


for ss, subtype in enumerate(subtypes[0:4]):
    plt.figure(1)
    sp_ax=plt.subplot(1,4, ss+1)
    
    # FOR STRIDE FREQUENCY! - SPECIFY IF FOR ALL STRIDES OR JUST STRAIGHT ONES
    SL_OI = all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_Len_all'].values
    vals_OI = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_tdist_total'].values)
#     str_idcs = np.isfinite(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])
    str_idcs = np.abs(all_strides.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!=coltypes[-1])]['St_travel_dir'])<angle_buffer # only straight strides
    d_cutoff = 0.005 # stride duration

    

    
    if ss == 0:
        plt.ylabel('foot stride len (mm)')
        plt.xlabel('body stride length (mm)')
    else:
        plt.gca().get_yaxis().set_visible(False)
                     
                     
                     
    x = vals_OI[str_idcs]
    y = SL_OI[str_idcs]
    no_nan_pts = np.logical_and(np.isfinite(x), np.isfinite(y))
    x = x[no_nan_pts]
    y = y[no_nan_pts]            
    # plot using 2d hist
    c_array = colors.ListedColormap(pltcolors[ss])(range(0,1000))
    c_array[:, -1]=np.arange(0,1,0.001)
    new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[ss], colors = c_array)
    hb=sp_ax.hist2d(x, y, 50, weights =np.ones_like(x)/float(len(x)), vmin = 0, vmax =0.008, cmap = new_cmap)
    
    
    plt.title('%s'%subtype, loc = 'left')
    plt.ylim([0,5])
    plt.xlim([0,5])

In [423]:
# save body and foot stride distance data as feather for stats modeling in R

temp = all_strides.copy()
longtracks = temp.loc[(temp['colony']!=coltypes[-1])] # don't include cinnamon trial data
# longtracks = longtracks.loc[(longtracks['colony']==coltypes[-1])] # just cinnamon trial data

# Get variables ready for R
colony_R = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
subs_string = longtracks['substrate'].values.tolist()
substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
b_SLen_R = np.array(longtracks['St_tdist_total'])
f_SLen_R = np.array(longtracks['St_Len_all'])
f_SDur_R = np.array(longtracks['St_Dur_all'])

df_med_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
                       "body_len" : b_SLen_R, "foot_len": f_SLen_R,
                      "foot_dur": f_SDur_R  } )

# SAVE AS FEATHER FOR USE WITH R
print('Saving disrupted stride data as feather for use in R')
feather.write_dataframe(df_med_R, vid_locations + 'Body_distance.feather')
print('Done saving')

del colony_R, day_R, substrate_R, date_days, subs_string, temp, longtracks
del df_med_R, b_SLen_R, f_SLen_R, f_SDur_R

Saving disrupted stride data as feather for use in R
Done saving


## Save stride speed vs. foot placement data for analysis in R

In [157]:
# calculate average foot displacement for TDs bookending each stride
def find_foot_disp_df(x, part, flat_XY):
    j = int(part[-1])
    all_x = x['%s_x_filt_WRTneck'%part]
    all_y = x['%s_y_filt_WRTneck'%part]
    d_x_1 = all_x[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]]-flat_XY[0][j] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    d_y_1 = all_y[x['%s_TD_idcs'%part][:-1][x['%s_good_strides'%part]]]-flat_XY[1][j]
#     d_x_2 = all_x[x['%s_TD_idcs'%part][1:][x['%s_good_strides'%part]]]-flat_XY[0][j] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
#     d_y_2 = all_y[x['%s_TD_idcs'%part][1:][x['%s_good_strides'%part]]]-flat_XY[1][j]
#     disp_pix = np.mean([np.linalg.norm([d_x_1,d_y_1],axis=0), np.linalg.norm([d_x_2,d_y_2],axis=0)],axis=0) # can take average or sum
#     return disp_pix
    return np.linalg.norm([d_x_1,d_y_1],axis=0)

for joint_num in range(0,6):
    df['joint%i_St_foot_disp'%joint_num]= df.apply(find_foot_disp_df, args = ('joint%i'%joint_num, flat_XY), axis=1)
print('done calculating average foot displacment for each limb')

df['St_foot_disp'] = df.filter(regex='_St_foot_disp$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
print('compiled foot displacements for all limbs')

done calculating average foot displacment for each limb
compiled foot displacements for all limbs


In [ ]:
# make dataframe for R and save as a feather

# compile dataframe of all strides
lens = [sum(item<=15) for item in df['St_travel_dir']]
all_St_disp = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "date" : np.repeat(df['date'].values, lens), 
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values)[np.concatenate(df['St_travel_dir'].values)<=15],
                        "St_Dur_all": np.concatenate(df['St_Dur_all'].values)[np.concatenate(df['St_travel_dir'].values)<=15],
                        "St_Len_all": np.concatenate(df['St_Len_all'].values)[np.concatenate(df['St_travel_dir'].values)<=15],     
                        "St_foot_disp" : np.concatenate(df['St_foot_disp'].values)[np.concatenate(df['St_travel_dir'].values)<=15]/pix2mm, # in mm
                        "St_tdist_total_v" : (np.concatenate(df['St_tdist_total'].values)/np.concatenate(df['St_Dur_all'].values))[np.concatenate(df['St_travel_dir'].values)<=15], # in mm/s
                        })
print('done compiling all straight strides into dataframe')


# save as feather for lme models in R
import feather

# MEDIAN V
colony_R = [col.split('20180')[-1][1:] for col in all_St_disp['colony'].values.tolist()]
date_days = [col[-2:] for col in all_St_disp['date'].values.tolist()]
day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
subs_string = all_St_disp['substrate'].values.tolist()
substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
v_R = np.array(all_St_disp['St_tdist_total_v'])
foot_disp_R = all_St_disp['St_foot_disp']
dur_R = all_St_disp['St_Dur_all']
len_R = all_St_disp['St_Len_all']
dist_R = all_St_disp['St_tdist_total']
df_med_R = pd.DataFrame( { "colony" : colony_R, "day" : day_R, "substrate" : substrate_R, "trackway": all_St_disp['trackway'],
                       "v" : v_R, "foot_displacement": foot_disp_R, "stride_dist": dist_R, "stride_dur": dur_R, "stride_len": len_R } )
print('%i strides in dataframe for analysis in R'%len(df_med_R))

# SAVE AS FEATHER FOR USE WITH R
print('\nSaving disrupted stride data as feather for use in R')
feather.write_dataframe(df_med_R, vid_locations + 'Foot_Displacement.feather')
print('Done saving')

# del colony_R, day_R, substrate_R, v_R, date_days, subs_string, all_St_disp, foot_disp_R, lens
del df_med_R, dist_R, dur_R, len_R

## 3D scatter of duration, length and distance traveled

In [ ]:
# 3D scatter of St duration vs. St length vs. dist traveled
from mpl_toolkits.mplot3d import Axes3D
plt.close('all')
pix2mm = 1000/32
fps = 240
df['St_Len_all'] = df.filter(regex='_St_Len$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1) # in mm
df['St_Dur_all'] = df.filter(regex='St_Dur$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/fps, axis = 1) # in sec
df['St_tdist_total'] = df.filter(regex='_St_tdist_total$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_tdist_straight'] = df.filter(regex='_St_tdist_straight$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0))/pix2mm, axis = 1)
df['St_rotation'] = df.filter(regex='_St_rotation$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_travel_dir'] = df.filter(regex='_St_travel_dir$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['St_jointID'] = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: np.sum(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))

lens = [len(item) for item in df['StLen_all']]
all_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "Joints_all" : np.concatenate(df['St_jointID'].values),
                        "St_Len_all" : np.concatenate(df['St_Len_all'].values), "St_Dur_all" : np.concatenate(df['St_Dur_all'].values),
                        "St_tdist_total" : np.concatenate(df['St_tdist_total'].values), "St_tdist_straight" : np.concatenate(df['St_tdist_straight'].values), 
                        "St_rotation" : np.concatenate(df['St_rotation'].values), "St_travel_dir" : np.concatenate(df['St_travel_dir'].values) })

def save_image(vlocation, nfig, name_base):
    pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
    plt.savefig(pname)
    nfig = nfig + 1
    plt.pause(0.2)
#     plt.close('all')
    return nfig

def save_video(vlocation, name_base):
    # save images as movie
    if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
        os.remove(vlocation + "/%s.mp4"%name_base)
        print('** Deleted %s.mp4 file'%name_base)
    print('saving %s.mp4 file'%name_base)
    command_p1 = "ffmpeg -r 4 -i '%s/%s"%(vlocation, name_base)
    command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
    command = command_p1 + "%01d.png'" + command_p2
#     print(command)
#     os.system(command)
    call(command, shell = True)
    plt.pause(15)

    # delete all trackway vids
    pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
    for pic in pics2delete:
        os.remove(pic)
    return

def save_gif(vlocation, name_base):
    if os.path.isfile((vlocation+'/%s.gif'%name_base)):
        os.remove(vlocation + "/%s.gif"%name_base)
        print('** Deleted %s.gif file'%name_base)
    command_p1 = "ffmpeg -i '%s/%s.mp4'"%(vlocation, name_base)
    command_p2 = " -filter_complex '[0:v] fps=5,split[a][b];[a] palettegen [p];[b][p] paletteuse' '%s/%s.gif'"%(vlocation, name_base) # - gif
    command = command_p1 + command_p2
    print('saving gif')
#     os.system(command)
    call(command, shell = True)
    plt.pause(2)
    return


########## WHAT VARIABLES TO USE #############
y_var = 'frequency'
z_var = 'speed'
##############################################


fig = plt.figure(figsize = (15,4))
for ss, subtype in enumerate(subtypes[0:4]):
    ax=fig.add_subplot(140+ss+1, projection='3d')
    
    if z_var == 'speed':
        z =np.abs(all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_total'].values/
               all_strides.loc[(all_strides['substrate']==subtype)]['St_Dur_all'].values)
        norm = colors.Normalize(vmin=0, vmax=40)
        ax.set_zlim((0,50))
        ax.set_zticklabels([0,'',20,'',40, ''])
        ax.set_zlabel('body speed (mm/s)')
    elif z_var == 'distance':
        z =np.abs(all_strides.loc[(all_strides['substrate']==subtype)]['St_tdist_total'].values)
        norm = colors.Normalize(vmin=0, vmax=5)
        ax.set_zlim((0,5))
        ax.set_zticklabels([0,'',2,'',4, ''])
        ax.set_zlabel('dist. traveled (mm)')
    else:
        print('z variable specified is not recognized')
        break
        
        
    if y_var == 'duration':
        y =all_strides.loc[(all_strides['substrate']==subtype)]['St_Dur_all'].values
        ax.set_ylim((0,1))
        ax.set_yticks(np.linspace(0,1,5))
        ax.set_yticklabels([0,'',0.5,'',1])
        ax.set_ylabel('stride dur (s)')
    elif y_var == 'frequency':
        y =1/all_strides.loc[(all_strides['substrate']==subtype)]['St_Dur_all'].values
        ax.set_ylim((0,40))
        ax.set_yticks(np.linspace(0,40,5))
        ax.set_yticklabels([0,'',20,'',40])
        ax.set_ylabel('stride fq (Hz)')
    else:
        print('y variable specified is not recognized')
        break


    x =all_strides.loc[(all_strides['substrate']==subtype)]['St_Len_all'].values
    str_idcs = np.abs(all_strides.loc[(all_strides['substrate']==subtype)]['St_travel_dir'])<angle_buffer
    ax.scatter(x[str_idcs], y[str_idcs], z[str_idcs], c=z[str_idcs], marker='.', alpha = 0.02, norm = norm )
    ax.set_xlim((0,4))
    ax.set_xticklabels([0,'',2,'',4])
    ax.set_xlabel('stride len (mm)')
    ax.plot(x[str_idcs],z[str_idcs], '.', c = [0.6,0.6,0.6], zdir = 'y', zs = ax.get_ylim()[1], alpha = 0.002)
    ax.plot(y[str_idcs],z[str_idcs], '.', c = [0.6,0.6,0.6], zdir = 'x', zs = ax.get_xlim()[1], alpha = 0.002)
    plt.show()

del all_strides, x, y, z
    
save_location = '/media/gravishlab/SeagateExpansionDrive/AntTrack'
name_base = '3D_%s_%s'%(y_var, z_var)
print(name_base)
for nfig, ang in enumerate(range(-175, -90, 2)):
#     print(ang)
    for ax in fig.get_axes():
#         print(ss)
        ax.view_init(20,ang)
    plt.show()
    plt.pause(0.2)
    
    save_image(save_location, nfig, name_base)
    
save_video(save_location,name_base)
save_gif(save_location,name_base)

del ax

## Sequential Strides

In [42]:
# FIND SEQUENTIAL STRIDES AND PLOT:
# -- ALL STRIDE PARAMETERS ON ONE PLOT
# -- ONLY TOTAL STRIDE DISTANCE
# -- QUIVER PLOT FOR FACING ROTATION VS. TRAVEL DIRECTION
from matplotlib.colors import LogNorm
plt.close('all')

def find_sequential_strides_df(x, joint_num, parameter): #'rotation', 'Len', 'Dur', 'travel_dir'
    arr = x['joint%s_St_%s'%(joint_num, parameter)]
    good_st = x['joint%s_good_strides'%joint_num]
    idcs = range(len(good_st))
    st_idcs = np.cumsum(good_st)-1
    st_len_groups  = [arr[st_idcs[np.array(list(g))]] for k,g in groupby(iter(idcs), lambda x: good_st[x]) if k == True]
    arr_0 = [g[:-1] for g in st_len_groups if len(g)>1]
    arr_1 = [g[1:] for g in st_len_groups if len(g)>1]
    if len(arr_0)>0:
        arr_0 = np.concatenate(arr_0)
        arr_1 = np.concatenate(arr_1)
    return arr_0, arr_1


# for parameter  in ['Len', 'Dur', 'tdist_total', 'travel_dir', 'rotation']:
#     print(parameter)
#     # for each joint, find the 1st (0) and 2nd (1) parameter values for sequential strides
#     for joint_num in range(0,6):
#         df['joint%s_SeqSt0_%s'%(joint_num, parameter)], df['joint%s_SeqSt1_%s'%(joint_num, parameter)]= \
#             zip(*df.apply(find_sequential_strides_df, args = (joint_num, parameter), axis=1))
#     # combine all joints
#     df['SeqSt0_%s'%parameter] = df.filter(regex='_SeqSt0_%s$'%parameter, axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
#     df['SeqSt1_%s'%parameter] = df.filter(regex='_SeqSt1_%s$'%parameter, axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
#     # get joint ID info
#     if parameter == 'tdist_total':
#         df['SeqSt_jointID'] = df.filter(regex='_SeqSt0_tdist_total$', axis=1).applymap(lambda x: len(x)).apply(
#             lambda x: np.concatenate([x]), axis = 1).map(
#             lambda x: np.repeat(['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5'], x))
#     # remove colulmns for individual joint sequential stride info
#     for joint_num in range(0,6):
#         df = df.drop(['joint%s_SeqSt0_%s'%(joint_num, parameter), 'joint%s_SeqSt1_%s'%(joint_num, parameter)], axis =1)
    
    
# new df with just sequential stride info of interest
lens = [len(item) for item in df['SeqSt0_travel_dir']]
seq_strides = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "joint_ID" : np.repeat(df['SeqSt_jointID'].values, lens), 
                        "Len_0" : np.concatenate(df['SeqSt0_Len'].values/pix2mm), "Len_1" : np.concatenate(df['SeqSt1_Len'].values/pix2mm),
                        "Dur_0" : np.concatenate(df['SeqSt0_Dur'].values/fps), "Dur_1" : np.concatenate(df['SeqSt1_Dur'].values/fps),
                        "travel_dist_0" : np.concatenate(df['SeqSt0_tdist_total'].values/pix2mm), 
                        "travel_dist_1" : np.concatenate(df['SeqSt1_tdist_total'].values/pix2mm),
                        "speed_0" : np.concatenate(df['SeqSt0_tdist_total'].values/pix2mm)/np.concatenate(df['SeqSt0_Dur'].values/fps), 
                        "speed_1" : np.concatenate(df['SeqSt1_tdist_total'].values/pix2mm)/np.concatenate(df['SeqSt1_Dur'].values/fps),
                        "travel_dir_0" : np.concatenate(df['SeqSt0_travel_dir'].values), "travel_dir_1" : np.concatenate(df['SeqSt1_travel_dir'].values),
                        "rotation_0" : np.concatenate(df['SeqSt0_rotation'].values), "rotation_1" : np.concatenate(df['SeqSt1_rotation'].values)    })

print('\ndone analyzing')


#####################
# ONLY DIST TRAVELED -- PLOT CURRENT VS. PAST STRIDE
pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
subtypes = sorted(list(set(df['substrate'])))
coltypes = sorted(list(set(df['colony'].values)))

plt.close('all')
fig =plt.figure(figsize = (17,5))
p_toplot = 'travel_dist'
density = {}
coltypes = sorted(list(set(df['colony'].values)))
xlimits = [np.floor(np.nanmin(seq_strides['%s_0'%p_toplot].values)), np.ceil(np.nanmax(seq_strides['%s_0'%p_toplot].values))]
cutoff_info = np.full((4,5),np.nan)
for ss, subtype in enumerate(subtypes[0:4]):
    sp_ax = plt.subplot(1,4,ss+1)
    xvals = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']!=coltypes[-2])]['%s_0'%p_toplot].values
    yvals = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']!=coltypes[-2])]['%s_1'%p_toplot].values
#     sp_ax.plot(xvals, yvals, '.', alpha = 0.008, c = pltcolors[ss])
    
#     # plot cinnamon colony
#     xvals = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']=='Tunnel_20180329-30')]['%s_0'%p_toplot].values
#     yvals = seq_strides.loc[(seq_strides['substrate']==subtype) & (seq_strides['colony']=='Tunnel_20180329-30')]['%s_1'%p_toplot].values
#     sp_ax.plot(xvals, yvals, '.', alpha = 0.008, c = 'k')
#     plt.text(4,5,'n: %i'%len(xvals), color = 'k')

    # plot unity lines
    plt.plot([0,6],[0,6], '--k', alpha = 0.4)
    # 10% of previous stride length is considered the same
    plt.plot([0,6],[0,6.6], ':k', alpha = 0.4)
    plt.plot([0,6],[0,5.4], ':k', alpha = 0.4)
#     plt.plot([0,6],[0.5,6.5], ':k')
#     plt.plot([0,6],[-.5,5.5], ':k')

    # use density and pca to identify side clouds
    x = xvals
    y = yvals
    H, xbins, ybins =np.histogram2d(x, y, 50, weights =np.ones_like(x)/float(len(x)))
    xloc = np.array([np.where(xbins[:-1]<= xi)[0][-1]  for xi in x])
    yloc = np.array([np.where(ybins[:-1]<= yi)[0][-1]  for yi in y])
    density[ss] = H[xloc,yloc]
    cutoff = 0.002# 0.0025
    inlier_mask = density[ss] > cutoff
    outlier_mask = np.logical_not(inlier_mask)
    inlier_mask_org = inlier_mask.copy()
#     sp_ax.plot(xvals[inlier_mask], yvals[inlier_mask], '.', alpha = 0.008, c = pltcolors[ss])
#     sp_ax.plot(xvals[outlier_mask], yvals[outlier_mask], '.', alpha = 0.008, c = 'k')
    
    points = np.array([x, y]).T
    cent =np.mean(points[inlier_mask], axis = 0)
    pca = PCA(n_components = 2)
    pca.fit_transform(points[inlier_mask])
    projected_in= pca.transform(points[inlier_mask])
    a = 4*np.linalg.norm(pca.inverse_transform([np.std(projected_in, axis =0)[0],0])-cent)
    b = 4*np.linalg.norm(pca.inverse_transform([np.std(projected_in, axis =0)[1],0])-cent)
    new_unit =pca.inverse_transform([1,0])-cent
    el_angle = np.rad2deg(np.arctan2(new_unit[1], new_unit[0]))
    
    projected= pca.transform(points)
    inlier_mask = np.logical_or(np.sum(np.square(projected)/[a**2,b**2],axis=1)<1, np.sign(el_angle)*projected[:,0]>0)
    inlier_mask = np.logical_or(inlier_mask, (y-x)/x < 0.1)
    outlier_mask = np.logical_not(inlier_mask)
    cutoff_info[ss,:] = [cent[0], cent[1], a, b, el_angle]
    
#     # plot using scatter
# #     sp_ax.plot(xvals[inlier_mask], yvals[inlier_mask], '.', alpha = 0.01, c = pltcolors[ss], ms = 2)
# #     sp_ax.plot(xvals[outlier_mask], yvals[outlier_mask], '.', alpha = 0.01, c = 'k', ms =2)
    
    # plot using hist
    plt.plot(cent[0],cent[1],'.k')
    c_array = colors.ListedColormap(pltcolors[ss])(range(0,1000))
    c_array[:, -1]=np.arange(0,1,0.001)
    new_cmap = LinearSegmentedColormap.from_list(name = '%s_alpha'%pltcolors[ss], colors = c_array)
#     hb=sp_ax.hist2d(x, y, 50,  cmap = new_cmap, norm = LogNorm(), vmin = 1, vmax = 800)
    hb=sp_ax.hist2d(x, y, 50, weights =np.ones_like(x)/float(len(x)), vmin = 0, vmax = 0.008, cmap = new_cmap)

    # plot pca ellipse cut-off
#     el_axes = np.sqrt(el_var)*cutoff/2
#     el_dim = np.linalg.norm(pca.inverse_transform([[a,0],[0,b]]),axis =1)*2
#     el = Ellipse(cent, el_dim[0], el_dim[1], angle = np.rad2deg(el_angle), ec = 'k', fc = 'None', LineStyle = '--')
    el = Ellipse(cent, 2*a,2*b, angle = el_angle, ec = 'k', fc = 'None', LineStyle = '--')
    plt.gca().add_patch(el)
    
    # convex hull around dense data
    hull = ConvexHull(np.vstack([y[inlier_mask_org],x[inlier_mask_org]]).T)
    for simplex in hull.simplices:
        plt.plot(x[inlier_mask_org][simplex], y[inlier_mask_org][simplex], 'k-')
#     hull = ConvexHull(np.vstack([y[inlier_mask],x[inlier_mask]]).T)
#     for simplex in hull.simplices:
#         plt.plot(x[inlier_mask][simplex], y[inlier_mask][simplex], 'b:')
    
    
#     # how many strides in each category
#     n_total = len(inlier_mask)
#     n_normal=np.sum(inlier_mask)
#     n_disrupted=np.sum(outlier_mask)
#     n_normal_constant = np.sum(np.abs(y[inlier_mask]-x[inlier_mask])<0.1*x[inlier_mask])
#     n_normal_notconstant = n_normal - n_normal_constant
#     plt.text(4, 5.5,'n: %i'%n_normal, color = pltcolors[ss])
#     plt.text(4, 5.1, 'n: %i'%n_disrupted, color = 'k')
#     plt.text(4, 3, '%% disrupted: \n%0.2f'%(n_disrupted/n_total*100))
#     ax=fig.add_axes([.26+ss*.2,.25,.02,.2])
#     plt.bar(np.zeros(3), np.array([n_normal_constant, n_normal_notconstant, n_disrupted])/n_total, 
#             bottom = np.array([0, n_normal_constant, n_normal])/n_total, color=[pltcolors[ss],'m','k'], alpha = 0.4)
#     ax.set_ylim([0,1])
#     ax.axis('off')
    
    # plot aesthetics
    sp_ax.set_aspect('equal')
    sp_ax.set_xlim(xlimits)
    sp_ax.set_ylim(xlimits)
    sp_ax.set_ylabel('current stride %s (mm)'%p_toplot)
    sp_ax.set_xlabel('previous stride %s (mm)'%p_toplot)
    if ss==3:
        ax=fig.add_axes([.93,.2,.02,.75])
        fig.colorbar(hb[3], cax=ax)
    fig.add_subplot(sp_ax)
    
fig.show()


#####################
# # FOR EACH KINEMATIC PARAMETER, PLOT CURRENT VS. PAST STRIDE
# plt.close('all')
# fig =plt.figure(figsize = (14,9))
# outer = gridspec.GridSpec(2,3, wspace = 0.2, hspace = 0.2)
# for sp, p_toplot in enumerate(['travel_dist', 'Dur', 'speed', 'travel_dir', 'rotation', 'Len']):
#     print(sp)
#     inner = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec = outer[sp], wspace = 0, hspace = 0)
#     xlimits = [np.floor(np.nanmin(seq_strides['%s_0'%p_toplot].values)), np.ceil(np.nanmax(seq_strides['%s_0'%p_toplot].values))]
#     for ss, subtype in enumerate(subtypes[0:4]):
#         sp_ax = plt.Subplot(fig, inner[ss])
#         xvals = seq_strides.loc[(seq_strides['substrate']==subtype)]['%s_0'%p_toplot].values
#         yvals = seq_strides.loc[(seq_strides['substrate']==subtype)]['%s_1'%p_toplot].values
#         sp_ax.plot(xvals, yvals, '.', alpha = 0.002, c = pltcolors[ss])
#         sp_ax.set_aspect('equal')
#         sp_ax.set_xlim(xlimits)
#         sp_ax.set_ylim(xlimits)
#         if ss != 2:
#             sp_ax.set_xticks([])
#             sp_ax.set_yticks([])
#         else:
#             sp_ax.set_ylabel('%s'%p_toplot)
#             sp_ax.set_xlabel('%s'%p_toplot)
#         fig.add_subplot(sp_ax)
# fig.show()


#########################
# # PLOT QUIVER PLOT ON BODY TRAVEL DIRECTION VS. FACING ROTATION
# plt.figure(figsize = (17,4))
# for ss, subtype in enumerate(subtypes[0:4]):
#     x = seq_strides.loc[(seq_strides['substrate']==subtype)]['travel_dir_0'].values
#     y = seq_strides.loc[(seq_strides['substrate']==subtype)]['rotation_0'].values
#     dx = seq_strides.loc[(seq_strides['substrate']==subtype)]['travel_dir_1'].values - seq_strides.loc[(seq_strides['substrate']==subtype)]['travel_dir_0'].values
#     dy = seq_strides.loc[(seq_strides['substrate']==subtype)]['rotation_1'].values - seq_strides.loc[(seq_strides['substrate']==subtype)]['rotation_0'].values
#     idcs = np.linalg.norm([dx,dy], axis = 0) > 80
#     ax = plt.subplot(1,4,ss+1)
#     plt.quiver(x, y, dx, dy, color = 'k', alpha = 0.03, angles = 'xy', scale_units = 'xy', scale = 1)
#     plt.quiver(x[idcs], y[idcs], dx[idcs], dy[idcs], color = pltcolors[ss], alpha = 0.1,  angles = 'xy', scale_units = 'xy', scale = 1)
#     plt.xlim((-180,180))
#     plt.ylim((-180,180))
#     plt.xlabel('travel dir')
#     plt.ylabel('facing rotation')



del xvals, yvals, sp_ax, ss, xlimits, p_toplot, seq_strides



# calculate if strides are disrupted based on stride length
def calc_SS_disrupted_df(x, cutoff_info):
    ss0 = x['SeqSt0_tdist_total']/pix2mm
    ss1 = x['SeqSt1_tdist_total']/pix2mm
    ss = np.where(np.array(['0mm','1mm','3mm','5mm'])==x['substrate'])[0][0]
    (c_x, c_y, a, b, ang) = cutoff_info[ss,:]
    ang = np.deg2rad(ang)

    rad = ((ss0-c_x)*np.cos(ang)+(ss1-c_y)*np.sin(ang))**2/a**2 + ((ss0-c_x)*np.sin(ang)-(ss1-c_y)*np.cos(ang))**2/b**2
    p_diff = (ss1-ss0)/ss0
    disrupted_strides = np.logical_and(rad>1, np.abs(p_diff)>0.1)
    return disrupted_strides

print('calculating disrupted sequential strides for each trial')
df['St_disrupted_SS']= df.apply(calc_SS_disrupted_df, args = (cutoff_info,), axis=1)
print('all done')


done analyzing
calculating disrupted sequential strides for each trial
all done


In [ ]:
# # test out seq stride df function on one trial
ss0 = df['SeqSt0_tdist_total'][0]/pix2mm
ss1 = df['SeqSt1_tdist_total'][0]/pix2mm
ss = np.where(np.array(['0mm','1mm','3mm','5mm'])==df['substrate'][0])[0][0]
(c_x, c_y, a, b, ang) = cutoff_info[ss,:]
ang = np.deg2rad(ang)
rad = ((ss0-c_x)*np.cos(ang)+(ss1-c_y)*np.sin(ang))**2/a**2 + ((ss0-c_x)*np.sin(ang)-(ss1-c_y)*np.cos(ang))**2/b**2
disrupted_strides = rad>1
print(rad)

# look at seq stride disrtupted results for one trial
tr= 689
np.vstack([df['SeqSt_jointID'][tr], df['St_disrupted_SS'][tr], df['SeqSt0_tdist_total'][tr]/pix2mm, df['SeqSt1_tdist_total'][tr]/pix2mm  ]).T

In [44]:
# save as feather for lme models in R
import feather
temp = df.copy()

# look only at trials longer than 50fr
idcs = [index for index, row in temp.iterrows() if len(row.St_disrupted_SS)>10]
longtracks = df.loc[idcs,]
longtracks = longtracks.loc[(longtracks['colony']!=coltypes[-1])] # don't include cinnamon trial data

# MEDIAN V
colony_R = [col.split('20180')[-1][1:] for col in longtracks['colony'].values.tolist()]
date_days = [col[-2:] for col in longtracks['date'].values.tolist()]
day_R = [col.split('-').index(day) for day, col in zip(date_days, colony_R)]
subs_string = longtracks['substrate'].values.tolist()
substrate_R = np.array([int(s.split('mm')[0]) for s in subs_string])
v_med_R = np.array(longtracks['median_v'])/pix2mm
n_disrupted_R = longtracks['St_disrupted_SS'].apply(lambda x: np.sum(x)).values
n_strides_R = longtracks['St_disrupted_SS'].apply(lambda x: len(x)).values
p_disrupted_R = longtracks['St_disrupted_SS'].apply(lambda x: np.sum(x)/len(x)).values
df_med_R = pd.DataFrame( {"colony" : colony_R, "day" : day_R, "substrate" : substrate_R,
                       "v_med" : v_med_R, "n_disrupted": n_disrupted_R,
                      "n_strides": n_strides_R, "percent_disrupted": p_disrupted_R  } )

# SAVE AS FEATHER FOR USE WITH R
print('Saving disrupted stride data as feather for use in R')
feather.write_dataframe(df_med_R, vid_locations + 'Disrupted_SS.feather')
# testdf = feather.read_dataframe(vid_locations + 'Disrupted_SF.feather')
print('Done saving')

del idcs, colony_R, day_R, substrate_R, v_med_R, date_days, subs_string, temp, longtracks, p_disrupted_R
del df_med_R, n_strides_R, n_disrupted_R

Saving disrupted stride data as feather for use in R
Done saving


In [19]:
points = np.array([x, y]).T
cent =np.mean(points[inlier_mask], 0)
new_points = points[inlier_mask]-cent




pca = PCA(n_components = 2)
t_points = pca.fit_transform(new_points)
el_angle = np.arctan2(pca.components_[0][1], pca.components_[0][0])
print(np.rad2deg(el_angle))
a = np.max(np.abs(t_points[:,0]))*1.2
b = np.max(np.abs(t_points[:,1]))*1.2


el_var = pca.explained_variance_


# el_values = np.sum(np.square(np.einsum('ij, jk->ik', points-cent, R))/(1.96**2 * el_var)[np.newaxis,:], axis =1)
#     print('in: %i, out: %i'%(np.sum(el_values<=1), np.sum(el_values>1)))
#     inlier_mask = el_values <= 1
print(el_var)
projected = pca.transform(points-cent)
cutoff = 12
pca_mask = np.logical_or(np.sum(np.square(projected)/[a**2,b**2],axis=1)<1, np.sign(el_angle)*projected[:,0]>0)

# pca_mask = np.logical_and(np.any(np.abs(projected)>[a,b],axis =1), projected[:,0]<0)
# pca_mask =np.any(np.abs(projected)>[a,b],axis =1)
# pca_mask = np.sum(np.square(projected)/[a**2,b**2],axis=1)<1

# inlier_mask = np.logical_or(np.linalg.norm(projected, axis =1)<1, np.sign(el_angle)*projected[:,0]>0)
# outlier_mask = np.logical_not(inlier_mask)

plt.figure()
plt.plot(projected[:,0], projected[:,1], '.k', alpha = 0.02, ms = 2)
plt.plot(t_points[:,0], t_points[:,1], '.b', alpha = 0.2, ms = 2)
plt.plot(projected[pca_mask,0], projected[pca_mask, 1], '.g', alpha = 0.2, ms =2)
glenna= pca.inverse_transform(projected)
# plt.plot(glenna[:,0], glenna[:,1],'.g', alpha = 0.2, ms = 2)
# plt.plot(glenna[pca_mask,0], glenna[pca_mask,1], '.m', alpha = 0.2, ms = 2)
# plt.plot(1,0,'.r')


# el_axes = np.abs(pca.inverse_transform([1,0]))
# el = Ellipse(cent, el_axes[0], el_axes[1], angle = np.rad2deg(el_angle), ec = 'k', fc = 'None', LineStyle = '--')
# plt.gca().add_patch(el)
plt.axis('equal')

55.64351655537718
[0.12914604 0.10330766]


(-2.858317009384496, 3.450483020032533, -2.2619134098066223, 4.381794585606526)

### 

In [ ]:
trs = range(474,475)


def load_video(raw_video_path, frame_range, verbose):
    """
    Independent of the frame range loaded, background has to be computed over total video or else can run into
    tracking problems
    """
    vid = cv2.VideoCapture(raw_video_path)
    Height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    NumFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    if not (NumFrames > 0):
        raise IOError('Codec issue: cannot read number of frames.')
    # restrict to desired range of frames
    if frame_range is None:
        frame_range = (0, int(NumFrames))
    else:
        # check doesn't exceed number of frames
        if frame_range[0] + frame_range[1] > NumFrames:
            frame_range = (int(frame_range[0]), int(NumFrames - frame_range[0]))
    # initialize blank frames
    frames = np.zeros((frame_range[1], Height, Width), np.uint8)
    # set the first frame to read in
    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_range[0]) # this way of setting the frame doesn't work on all cv versions
    # read in all frames
    for kk in range(frame_range[1]):
        tru, ret = vid.read(1)
        # check if video frames are being loaded
        if not tru:
            raise IOError('Codec issue: cannot load frames.')
        frames[kk, :, :] = ret[:, :, 0]  # assumes loading color
        if ((kk % 100) == 0) and verbose:
            print(kk)
    return frames, NumFrames, frame_range, vid


for tr in trs:
    for n_tri in [0]:
#         n_tri = 0
        subset_df = df.filter(regex='_TD_idcs$', axis=1).iloc[tr,range(0,6)[n_tri::2]]

        stops = []
        starts = []
        all_TDs = []
        for ii in range(0,3):
            all_TDs = np.append(all_TDs, df['joint%i_TD_idcs'%(n_tri+2*ii)][tr][df['joint%i_good_TDs'%(n_tri+2*ii)][tr]])
            starts = np.append(starts, df['joint%i_TD_idcs'%(n_tri+2*ii)][tr][:-1][df['joint%i_good_strides'%(n_tri+2*ii)][tr]])
            stops = np.append(stops, df['joint%i_TD_idcs'%(n_tri+2*ii)][tr][1:][df['joint%i_good_strides'%(n_tri+2*ii)][tr]])


        all_TDs = np.sort(all_TDs)
        if len(all_TDs)==0:
            continue
        all_stride_diff = np.insert(np.diff(np.sort(all_TDs)),0,0)

        arr = all_TDs
        arr_diff = np.cumsum(all_stride_diff>10)
        idcs = range(len(arr_diff))
        TD_groups  = [arr[(list(g))] for k,g in groupby(iter(idcs), lambda x: arr_diff[x])]

        # lone strides
        group_is_one  = np.array([len(g) for g in TD_groups])==1
        if not np.any(group_is_one):
            continue
        lone_TDs = np.concatenate(np.array(TD_groups)[group_is_one])

        for lTD in lone_TDs:
            fr = int(df['frames'][tr][int(lTD)])
            is_disrupted = np.sum(np.logical_and(lTD>=starts, lTD < stops))>2


            if is_disrupted ==True:
                
                which_joint = np.where(subset_df.apply(lambda x: np.isin(lTD, x)).values)[0]*2+n_tri
                other_tjoints = np.where(np.array(range(n_tri,6,2)) != which_joint)[0]*2+n_tri
                frames, _, _, _ = load_video(df['video'][tr], [fr-60,120], verbose= False)
                
                print('\n%i **** '%tr, df['video'][tr])
                print('tripod: %i -- fr: %i -- joint: %i'%(n_tri, fr, which_joint))
                for loop in range(2):
                    plt.close('all')
                    fig = plt.figure()
                    for f_num in range(len(frames)):
                        plt.cla()
                        plt.imshow(frames[f_num,:,:], cmap = 'gray')
                        plt.plot(df['joint%i_x_filt_fullfr'%which_joint][tr][int(lTD-60)+f_num], df['joint%i_y_filt_fullfr'%which_joint][tr][int(lTD-60)+f_num] ,
                                     '.r', MarkerSize = 3)
                        for jj in other_tjoints:
                            plt.plot(df['joint%i_x_filt_fullfr'%jj][tr][int(lTD-60)+f_num], df['joint%i_y_filt_fullfr'%jj][tr][int(lTD-60)+f_num] ,
                                         '.w', MarkerSize = 3)
                        plt.xlim([-100, 100] + df['thorax_x_filt_fullfr'][tr][int(lTD)])
                        plt.ylim([-100, 100] + df['thorax_y_filt_fullfr'][tr][int(lTD)])
                        plt.gca().invert_yaxis()
                        plt.text(df['thorax_x_filt_fullfr'][tr][int(lTD)]+70, df['thorax_y_filt_fullfr'][tr][int(lTD)]-90 ,'joint %i'%which_joint, color = 'r')
                        plt.scatter(df['joint%i_x_filt_fullfr'%which_joint][tr][int(lTD)], df['joint%i_y_filt_fullfr'%which_joint][tr][int(lTD)] ,
                                    s=40, edgecolor= 'r', facecolor = 'None', linestyle = ':')
                        plt.show()
                        if f_num == 60:
                            plt.pause(0.003)
                        else:
                            plt.pause(0.01)
                    plt.close(fig)

In [ ]:
def other_strides_in_group(group, group_idx, n_tri, starts, stops, good_strides, stride_ID):
    joints_not_in_group = np.array(range(n_tri,6,2))[np.logical_not( np.isin(range(n_tri,6,2), group_idx) )]
    n_other_strides = 0
    if (len(joints_not_in_group) >0) and (np.sum(good_strides)>0):
        where_stride_edges = np.where(np.diff(stride_ID[good_strides]).astype(bool))[0]+1
        starts_joints = np.split(starts[good_strides], where_stride_edges)
        stops_joints = np.split(stops[good_strides], where_stride_edges)
        joint_joints = np.split(stride_ID[good_strides], where_stride_edges)
#         print([jj[0] for jj in joint_joints])
        joints_not_in_group_id = [[jj[0] for jj in joint_joints].index(ii) for ii in joints_not_in_group if ii in [jj[0] for jj in joint_joints]]
        if len(joints_not_in_group_id)>0:
            starts_OI = np.concatenate(np.array(starts_joints)[joints_not_in_group_id])
            stops_OI = np.concatenate(np.array(stops_joints)[joints_not_in_group_id])
            n_other_strides = np.sum(np.logical_and(np.mean(group)>starts_OI, np.mean(group)<stops_OI))
    return n_other_strides


for tr in [476, 479,489]:#range(475,490):
    frames = df['frames'][tr]
    print('%i **** '%tr, df['video'][tr])

    all_TDs = df.filter(regex='_TD_idcs$', axis=1).applymap(lambda x: x).apply(lambda x: np.concatenate(np.concatenate([x])), axis = 1)[tr]
    starts = df.filter(regex='_TD_idcs$', axis=1).applymap(lambda x: x[:-1]).apply(lambda x: np.concatenate(np.concatenate([x])), axis = 1)[tr]
    stops = df.filter(regex='_TD_idcs$', axis=1).applymap(lambda x: x[1:]).apply(lambda x: np.concatenate(np.concatenate([x])), axis = 1)[tr]
    good_strides = df.filter(regex='_good_strides$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x])), axis = 1)[tr]
    good_TDs = df.filter(regex='_good_TDs$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x])), axis = 1)[tr]
    TDs_ID = df.filter(regex='_TD_idcs$', axis=1).applymap(lambda x: len(x)).apply(
        lambda x: np.concatenate([x]), axis = 1).map(lambda x: np.repeat(np.array(range(0,6)), x))[tr]
    stride_ID = df.filter(regex='_good_strides$', axis=1).applymap(lambda x: len(x)).apply(
        lambda x: np.concatenate([x]), axis = 1).map(
        lambda x: np.repeat(np.array(range(0,6)), x))[tr]
    good_stride_ID = stride_ID.copy().astype(float)
    good_stride_ID[np.logical_not(good_strides)] = np.nan
    stride_order = np.argsort(starts)

    # keep starts and stops of strides as trusted TDs
    where_strides = np.diff(np.insert(TDs_ID, len(TDs_ID),-1)) == 0
    temp = np.full(where_strides.shape, False)
    temp[where_strides] = good_strides
    temp = np.insert(temp,0,False)
    trusted_TDs = np.logical_or(temp[:-1],temp[1:])
    del temp


#     plt.figure(figsize = (14,2))
#     pltcolors = np.array(['r', 'm', 'y', 'g', 'c', 'b'])
#     tricolors = np.array(['r','g'])
#     plt.scatter(frames[np.concatenate([starts,stops])], np.tile(stride_ID,2), s=4, c=pltcolors[stride_ID], alpha = 0.3)#, color = list(pltcolors[stride_ID]));
#     plt.plot(frames[np.array([starts[good_strides],stops[good_strides]])], np.tile(stride_ID[good_strides],(2,1)), '--k', alpha = 0.5);
#     plt.scatter(frames[all_TDs[trusted_TDs]], TDs_ID[trusted_TDs], s=8, c=pltcolors[TDs_ID[trusted_TDs]])
#     for tt,td in enumerate(TDs_ID[trusted_TDs]):
#         plt.text(frames[all_TDs[trusted_TDs][tt]], 6, '%i'%td, color = pltcolors[td], horizontalalignment = 'center')
#     plt.ylim([-.5,6])
#     plt.gca().invert_yaxis()
#     plt.title('Tr %i -- %s'%(tr,(' -- ').join(df['video'][tr].split('/')[-2:]) ), loc = 'left')

    for n_tri in [0,1]:

        # define only tripod of interest
        tripod_TDs = all_TDs[TDs_ID%2==n_tri]
        tripod_good_TDs = good_TDs[TDs_ID%2==n_tri]
        tripod_TDs_ID = TDs_ID[TDs_ID%2 == n_tri]

        # sort good TDs into clusters
        arr = tripod_TDs[tripod_good_TDs]
        arr_sort = np.argsort(arr)
        arr_diff = np.insert(np.diff(arr[arr_sort]),0,0)>15
        TD_groups  = np.split(arr[arr_sort], np.where(arr_diff)[0])
        TD_groups_ID  = np.split(tripod_TDs_ID[tripod_good_TDs][arr_sort], np.where(arr_diff)[0])
        
        # split any groups that have more than one TD from same foot
        temp = [gg for gg,[gr,idx] in enumerate(zip(TD_groups,TD_groups_ID)) if len(np.unique(idx))<len(idx)]

        for gr in temp[::-1]:
            temp2 = TD_groups[gr]
#             print(np.split(temp2,[np.argmax(np.diff(temp2))+1]))
            TD_groups[gr:gr+1]=np.split(temp2,[np.argmax(np.diff(temp2))+1])
            TD_groups_ID[gr:gr+1]=np.split(TD_groups_ID[gr],[np.argmax(np.diff(temp2))+1])
            del temp2

        # how many TDS in each group
        TD_groups_n_TDs = np.array([len(x) for x in TD_groups])

        # how many good strides from tripod are trusted around each cluster
        TD_groups_n_other_strides = np.array([other_strides_in_group(group, group_idx, n_tri, starts, stops, good_strides, stride_ID) 
                                     for group, group_idx in zip(TD_groups, TD_groups_ID)])

#         print(TD_groups_n_TDs)
#         print(TD_groups_n_other_strides)

        tripods = np.array(TD_groups)[TD_groups_n_TDs == 3]
        pairs = np.array(TD_groups)[TD_groups_n_TDs == 2]
        disrupted = np.array(TD_groups)[np.logical_and(TD_groups_n_TDs == 1, TD_groups_n_other_strides >1)]
        disrupted_idx = np.array(TD_groups_ID)[np.logical_and(TD_groups_n_TDs == 1, TD_groups_n_other_strides >1)]

#         for t in tripods:
#             rect = Rectangle([frames[np.min(t)],-.5],np.max(t)-np.min(t),6.5, color = tricolors[n_tri], alpha = 0.2)
#             plt.gca().add_patch(rect)
#         for t in pairs:
#             rect = Rectangle([frames[np.min(t)],-.5],np.max(t)-np.min(t),6.5, color = tricolors[n_tri], alpha = 0.05)
#             plt.gca().add_patch(rect)
        for t, ii in zip(disrupted, disrupted_idx):
#             plt.axvline(x=frames[t], color = pltcolors[ii][0])
            fr = frames[t]
            which_joint = ii
            other_tjoints = np.where(np.array(range(n_tri,6,2)) != which_joint)[0]*2+n_tri
            vframes, _, _, _ = load_video(df['video'][tr], [fr-60,120], verbose= False)

#             print('\n%i **** '%tr, df['video'][tr])
            print('     tripod: %i -- fr: %i -- joint: %i'%(n_tri, fr, which_joint))
#             for loop in range(1):
            plt.close('all')
            fig = plt.figure()
            for f_num in range(len(vframes)):
                plt.cla()
                plt.imshow(vframes[f_num,:,:], cmap = 'gray')
                plt.plot(df['joint%i_x_filt_fullfr'%which_joint][tr][int(t-60)+f_num], df['joint%i_y_filt_fullfr'%which_joint][tr][int(t-60)+f_num] ,
                             '.r', MarkerSize = 3)
                for jj in other_tjoints:
                    plt.plot(df['joint%i_x_filt_fullfr'%jj][tr][int(t-60)+f_num], df['joint%i_y_filt_fullfr'%jj][tr][int(t-60)+f_num] ,
                                 '.w', MarkerSize = 3)
                plt.xlim([-100, 100] + df['thorax_x_filt_fullfr'][tr][t])
                plt.ylim([-100, 100] + df['thorax_y_filt_fullfr'][tr][t])
                plt.gca().invert_yaxis()
                plt.text(df['thorax_x_filt_fullfr'][tr][t]+70, df['thorax_y_filt_fullfr'][tr][t]-90 ,'joint %i'%which_joint, color = 'r')
                plt.scatter(df['joint%i_x_filt_fullfr'%which_joint][tr][t], df['joint%i_y_filt_fullfr'%which_joint][tr][t] ,
                            s=40, edgecolor= 'r', facecolor = 'None', linestyle = ':')
                plt.show()
                if f_num == 60:
                    plt.pause(0.003)
                else:
                    plt.pause(0.01)
            plt.close(fig)
            

        del TD_groups, TD_groups_ID, temp

# Look at ant placement wrt block edge for each substrate

In [ ]:
# Calculate where in checkerboard box for user defined trials

def split_by_bool(arr, boo):
    split_idcs = np.nonzero(boo[1:] != boo[:-1])[0]+1
    split_arr = np.split(arr, split_idcs)
    split_arr = [split_arr[0::2] if boo[0] else split_arr[1::2]]
    return split_arr[0]

plt.close('all')
for tr in range(875,896):

    # collect data into easy to use variables
    all_x = df['thorax_x_filt_fullfr'][tr]
    all_y = df['thorax_y_filt_fullfr'][tr]
    col_OI = df['colony'][tr]
    sub_OI = df['substrate'][tr]

    # find horizontal and vertical lines
    step_height_OI = step_height_df[
        step_height_df['substrate'].str.contains(sub_OI) & step_height_df['colony'].str.contains(col_OI)]
    height_profile = step_height_OI['height_profile'].values[0]
    vlines = step_height_OI['vlines'].values[0]
    n_vlines = len(vlines)
    hlines = step_height_OI['hlines'].values[0]
    hlines_sep = np.diff(hlines[:,1]*500+hlines[:,0]) # if close-by hline it's a ledge and remove
    where_ledge = np.where((np.median(hlines_sep)-hlines_sep)>30)[0]
    if len(where_ledge)>0:
        if where_ledge[0]==0:
            hlines=np.delete(hlines,0,axis=0)
            height_profile=np.delete(height_profile,0,axis=0)
        else:
            hlines=np.delete(hlines,where_ledge[0]+1,axis=0)
            height_profile=np.delete(height_profile,where_ledge[0],axis=0)
    n_hlines = len(hlines)

    # find where close to middle vertical line
    vert_box_cutoff = 0.5 # in mm
    where_in_vert_box = np.full(all_x.shape, False)
    if sub_OI == '0mm':
        vline_OI = 0
        for vs in [1,3]:
            a = np.mean(1/vlines[vline_OI:vline_OI+2,1])/2*vs
            b = np.mean(vlines[vline_OI:vline_OI+2,0]/vlines[vline_OI:vline_OI+2,1])/2*vs
            x_pred =all_y*a-b
            temp = np.abs(all_x-x_pred)<(vert_box_cutoff*pix2mm)
            where_in_vert_box = np.logical_or(temp, where_in_vert_box)
    else:
        for vs in [1,2]:
            vline_OI = int(np.round(n_vlines/3*vs))-1
            a = np.mean(1/vlines[vline_OI:vline_OI+2,1])
            b = np.mean(vlines[vline_OI:vline_OI+2,0]/vlines[vline_OI:vline_OI+2,1])
            x_pred =all_y*a-b
            temp = np.abs(all_x-x_pred)<(vert_box_cutoff*pix2mm)
            where_in_vert_box = np.logical_or(temp, where_in_vert_box)
    
    

    # find if far from edges
    edge_cutoff = [1.5, 1.5, 3, 2.5][subtypes.index(sub_OI)]# in mm
    is_far_from_top = all_y>(all_x*hlines[0,1]+hlines[0,0]+edge_cutoff*pix2mm)
    is_far_from_bottom = all_y<(all_x*hlines[-1,1]+hlines[-1,0]-edge_cutoff*pix2mm)
    good_points = is_far_from_top & where_in_vert_box & is_far_from_bottom

    # isolate trails through vertical box
    x_chunks = split_by_bool(all_x, good_points)
    y_chunks = split_by_bool(all_y, good_points)

    if len(y_chunks)>0:
        y_box_fraction = np.full(len(y_chunks), np.nan)
        checkerboard_height = np.full(len(y_chunks), np.nan)
        # find average y_location
        for cc in range(len(y_chunks)):

            if len(y_chunks[cc])>10:
                
                plt.plot(all_x,all_y,'-k')
                
                y_mean = np.nanmedian(y_chunks[cc])
                x_mean = np.nanmedian(x_chunks[cc])
                
                # check if going straight through window
                slope = (y_chunks[cc][-1]-y_chunks[cc][0])/(x_chunks[cc][-1]-x_chunks[cc][0])
                max_divergence = np.max( np.abs( y_chunks[cc]-y_mean ))
                if np.abs(slope) > 0.75:
                    print('too slope-y: %0.1f'%slope)
                    continue
                elif max_divergence/pix2mm > 1/2:
                    print('too divergent: %0.1f mm'%(max_divergence/pix2mm))
                    continue

                plt.plot(x_chunks[cc],y_chunks[cc],'.r', Markersize = 2)
                
                # find where in checkerboard box
                y_pred = hlines[:,1]*x_mean+hlines[:,0]
                y_top = y_pred[y_mean>hlines[:,1]*x_mean+hlines[:,0]][-1]
                y_bot = y_pred[y_mean<hlines[:,1]*x_mean+hlines[:,0]][0]
                y_box_fraction[cc] =np.abs(y_mean-np.mean([y_top,y_bot]))/np.diff([y_top,y_bot])*2
                
                # find checkerboard box height
                row = np.where(y_mean<hlines[:,1]*x_mean+hlines[:,0])[0][0]-1
                col = np.where(x_mean<(1/vlines[:,1])*y_mean-vlines[:,0]/vlines[:,1])[0][0]-1
                checkerboard_height[cc] = height_profile[row,col]

                
                
            else:
                plt.plot(all_x,all_y,'-k',alpha = 0.5)
                
            print('%s - fraction %0.3f - slope %0.1f - divergence %0.1f - height: %0.0f'%(tr,y_box_fraction[cc],slope,max_divergence/pix2mm,checkerboard_height[cc]))
    else:
        print('%s - no trail through'%tr)
        plt.plot(all_x,all_y,'-k', alpha = 0.5)
        
for vv in range(n_vlines):
    plt.plot(np.linspace(0,550,551)*(1/vlines[vv,1])- vlines[vv,0]/vlines[vv,1], np.linspace(0,550,551),'k', alpha = 0.1)
for vv in range(n_hlines):
    plt.plot(np.linspace(0,1000,1001), np.linspace(0,1000,1001)*(hlines[vv,1])+hlines[vv,0],'k', alpha = 0.1)
plt.plot(np.linspace(0,1000,1001), hlines[0,1]*np.linspace(0,1000,1001)+hlines[0,0],'k', alpha = 0.3)
plt.plot(np.linspace(0,1000,1001), hlines[-1,1]*np.linspace(0,1000,1001)+hlines[-1,0],'k', alpha = 0.3)
plt.plot(np.linspace(0,1000,1001), hlines[0,1]*np.linspace(0,1000,1001)+hlines[0,0]+edge_cutoff*pix2mm,'--k', alpha = 0.3)
plt.plot(np.linspace(0,1000,1001), hlines[-1,1]*np.linspace(0,1000,1001)+hlines[-1,0]-edge_cutoff*pix2mm,'--k', alpha = 0.3)
# plt.plot(np.linspace(0,550,551)*(1/vlines[vline_OI,1])- vlines[vline_OI,0]/vlines[vline_OI,1], np.linspace(0,550,551),'k', alpha = 0.3)
# plt.plot(np.linspace(0,550,551)*(1/vlines[vline_OI,1])- vlines[vline_OI,0]/vlines[vline_OI,1]+pix2mm*vert_box_cutoff, np.linspace(0,550,551),':k', alpha = 0.3)
# plt.plot(np.linspace(0,550,551)*(1/vlines[vline_OI,1])- vlines[vline_OI,0]/vlines[vline_OI,1]-pix2mm*vert_box_cutoff, np.linspace(0,550,551),':k', alpha = 0.3)
plt.ylim([0,550])
plt.xlim([0,1000])
plt.axis('equal')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.gca().invert_yaxis()

In [ ]:
# find where in checkerboard for all dataframe

def split_by_bool(arr, boo):
    split_idcs = np.nonzero(boo[1:] != boo[:-1])[0]+1
    split_arr = np.split(arr, split_idcs)
    split_arr = [split_arr[0::2] if boo[0] else split_arr[1::2]]
    return split_arr[0]


def checkerboard_fraction_df(x, step_height_df):
    subtypes = ['0mm', '1mm', '3mm', '5mm']

    # collect data into easy to use variables
    all_x = x['thorax_x_filt_fullfr']
    all_y = x['thorax_y_filt_fullfr']
    col_OI = x['colony']
    sub_OI = x['substrate']

    # find horizontal and vertical lines
    step_height_OI = step_height_df[
        step_height_df['substrate'].str.contains(sub_OI) & step_height_df['colony'].str.contains(col_OI)]
    height_profile = step_height_OI['height_profile'].values[0]
    vlines = step_height_OI['vlines'].values[0]
    n_vlines = len(vlines)
    hlines = step_height_OI['hlines'].values[0]
    hlines_sep = np.diff(hlines[:,1]*500+hlines[:,0]) # if close-by hline it's a ledge and remove
    where_ledge = np.where((np.median(hlines_sep)-hlines_sep)>30)[0]
    if len(where_ledge)>0:
        if where_ledge[0]==0:
            hlines=np.delete(hlines,0,axis=0)
        else:
            hlines=np.delete(hlines,where_ledge[0]+1,axis=0)
    n_hlines = len(hlines)

    # find where close to middle vertical lines
    vert_box_cutoff = 0.5 # in mm
    where_in_vert_box = np.full(all_x.shape, False)
    if sub_OI == '0mm':
        vline_OI = 0
        for vs in [1,3]:
            a = np.mean(1/vlines[vline_OI:vline_OI+2,1])/2*vs
            b = np.mean(vlines[vline_OI:vline_OI+2,0]/vlines[vline_OI:vline_OI+2,1])/2*vs
            x_pred =all_y*a-b
            temp = np.abs(all_x-x_pred)<(vert_box_cutoff*pix2mm)
            where_in_vert_box = np.logical_or(temp, where_in_vert_box)
    else:
        for vs in [1,2]:
            vline_OI = int(np.round(n_vlines/3*vs))-1
            a = np.mean(1/vlines[vline_OI:vline_OI+2,1])
            b = np.mean(vlines[vline_OI:vline_OI+2,0]/vlines[vline_OI:vline_OI+2,1])
            x_pred =all_y*a-b
            temp = np.abs(all_x-x_pred)<(vert_box_cutoff*pix2mm)
            where_in_vert_box = np.logical_or(temp, where_in_vert_box)

    # find if far from edges
    edge_cutoff = [1.5, 1.5, 3, 2.4][subtypes.index(sub_OI)] # in mm
    is_far_from_top = all_y>(all_x*hlines[0,1]+hlines[0,0]+edge_cutoff*pix2mm)
    is_far_from_bottom = all_y<(all_x*hlines[-1,1]+hlines[-1,0]-edge_cutoff*pix2mm)
    good_points = is_far_from_top & where_in_vert_box & is_far_from_bottom

    # isolate trails through vertical box
    x_chunks = split_by_bool(all_x, good_points)
    y_chunks = split_by_bool(all_y, good_points)

    y_box_fraction = []
    checkerboard_height = []
    if len(y_chunks)>0:
        
        # find average y_location
        for cc in range(len(y_chunks)):
            
            if len(y_chunks[cc])>10:
                
                y_mean = np.median(y_chunks[cc])
                x_mean = np.median(x_chunks[cc])
                
                # check if going straight through window
                slope = (y_chunks[cc][-1]-y_chunks[cc][0])/(x_chunks[cc][-1]-x_chunks[cc][0])
                max_divergence = np.max( np.abs( y_chunks[cc]-y_mean ))
                if np.abs(slope) > 0.75:
                    continue
                elif max_divergence/pix2mm > 1/2:
                    continue

                # find where in checkerboard box
                y_pred = hlines[:,1]*x_mean+hlines[:,0]
                y_top = y_pred[y_mean>hlines[:,1]*x_mean+hlines[:,0]][-1]
                y_bot = y_pred[y_mean<hlines[:,1]*x_mean+hlines[:,0]][0]
                y_box_fraction =np.append(y_box_fraction, np.abs(y_mean-np.mean([y_top,y_bot]))/np.diff([y_top,y_bot])*2 )
                
                # find checkerboard height
                if sub_OI == '0mm':
                    checkerboard_height = np.append(checkerboard_height, 1)
                else:
                    row = np.where(y_mean<hlines[:,1]*x_mean+hlines[:,0])[0][0]-1
                    col = np.where(x_mean<(1/vlines[:,1])*y_mean-vlines[:,0]/vlines[:,1])[0][0]-1
                    checkerboard_height = np.append(checkerboard_height, height_profile[row,col])
                
    if len(y_box_fraction)>0:
        return y_box_fraction, checkerboard_height
    else:
        return np.full(1,np.nan), np.full(1,np.nan)

            

df['vertical_checkerboard_fraction'], df['checkerboard_height'] =  zip(*df.apply(checkerboard_fraction_df, args = ( step_height_df,), axis=1))
print('done calculated checkerboard fractions')

In [ ]:
# plot histogram plot
# plt.close('all')
plt.figure()
line_styles = ['--','-']

precision = 50 # how many bins per 1 unit
bins = np.linspace(0, 1, precision+1)

for ss,subtype in enumerate(subtypes):
    hist_OI_raw = np.hstack(df.loc[(df['substrate']==subtype) & (df['colony']!= coltypes[-1])]['vertical_checkerboard_fraction'].values)
    heights = np.hstack(df.loc[(df['substrate']==subtype) & (df['colony']!= coltypes[-1])]['checkerboard_height'].values)
    print('%i data points out of %i trailway chunks'%(np.sum(np.isfinite(hist_OI_raw)), len(hist_OI_raw)))
    hist_OI= hist_OI_raw[np.isfinite(hist_OI_raw)]
    heights= heights[np.isfinite(hist_OI_raw)]
    
    
    # plot all together
#     # find histogram counts
#     h_counts = np.histogram(hist_OI, bins = bins)[0]
#     plt.hist(hist_OI, bins = bins[:-1], color = pltcolors[ss], alpha = 0.2, weights = np.ones(hist_OI.shape)*(1/np.sum(h_counts)))
#     h_counts = h_counts/np.sum(h_counts)
#     plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts[h_counts>0], ':', color = pltcolors[ss])

#     # lowpass filter histogram data
#     b,a = signal.butter(2, 1/(precision/4))
#     temp = signal.filtfilt(b,a, np.hstack([np.flipud(h_counts[h_counts>0]),h_counts[h_counts>0], np.flipud(h_counts[h_counts>0])]) , padlen = 10)
#     h_counts_filt= temp[ len(h_counts[h_counts>0]):(2*len(h_counts[h_counts>0]))]
#     plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts_filt, linestyle = line_styles[height], color = pltcolors[ss])
#     plt.text(0.82, .06-ss*0.003, 'n: %i'%len(hist_OI), color  = pltcolors[ss])
    
    
    
    # separate by height
    for height in [0,1]:
        
        if len(hist_OI[heights==height])==0:
            continue
        
        h_counts = np.histogram(hist_OI[heights==height], bins = bins)[0]
        h_counts = h_counts/np.sum(h_counts)
#         plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts[h_counts>0], ':', color = pltcolors[ss])
    
        # lowpass filter histogram data
        b,a = signal.butter(2, 1/(precision/4))
        temp = signal.filtfilt(b,a, np.hstack([np.flipud(h_counts[h_counts>0]),h_counts[h_counts>0], np.flipud(h_counts[h_counts>0])]) , padlen = 10)
        h_counts_filt= temp[ len(h_counts[h_counts>0]):(2*len(h_counts[h_counts>0]))]
        plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts_filt, linestyle = line_styles[height], color = pltcolors[ss])
        plt.text(0.72+height/10, .06-ss*0.003, 'n: %i'%len(hist_OI[heights==height]), color  = pltcolors[ss])
        plt.ylim([0,0.07])

plt.xlabel('center  <---->  edge')

## Plot TDs on video image to see foot placement strategies 

In [ ]:
tr_num = 1332
fr_range = [14, 300]

# 0 mm - 79 - 25, 200
# 1 mm - 412 - 548,680
# 3 mm - 824 - 193,430
# 5 mm - 1332 - 14, 300

file_idc = np.where(np.array(['%s_%s'%(df['date'][tr_num],df['time'][tr_num]) in f for f in file_list]))[0][0]
file = file_list[file_idc]
print(file)
lcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
jcolors = ['xkcd:goldenrod',  'xkcd:crimson', 'xkcd:fuchsia',  'xkcd:green', 'xkcd:azure', 'xkcd:aqua']
linetypes = ['-.',':','--']
ss= np.where(['1mm'== s for s in subtypes])[0][0]
# ['xkcd:aqua', 'xkcd:azure', 'xkcd:crimson', 'xkcd:fuchsia', 'xkcd:goldenrod', 'xkcd:green']


plt.close('all')
cap = cv2.VideoCapture(file)
width = int(cap.get(3))
height = int(cap.get(4))
vid_length = np.min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),500])
rgb_convert = np.array([[[0.2989]],[[0.5870]],[[0.1140]]]).T
frames_to_read = fr_range #np.arange(0,vid_length,100)]
frames = np.zeros((len(frames_to_read), height, width), np.uint8)
for kk in range(0,len(frames_to_read)):
    fr_OI=frames_to_read[kk]
    cap.set(1,fr_OI)
    # Capture frame-by-frame
    ret, frame2 = cap.read()
    if ret:
        frames[kk,:,:] = np.sum(frame2*rgb_convert, axis = 2)
    else:
        print('couldnt get frame')
cap.release()
img = np.mean(frames, axis = 0)


plt.figure(figsize = (13,7))
plt.imshow(img, cmap = 'gray')


if df['substrate'][tr_num] != '0mm':
    lname = ('/').join(file_list[0].split('/')[:-2]) + '/%s/%s_%s_Step_Height.pkl'%(
        df['substrate'][tr_num], df['colony'][tr_num], df['substrate'][tr_num])
    with open(lname, 'rb') as f:
        hlines, vlines, height_profile,_ = pickle.load(f)
        plt.plot()
    f.close()
    plt.plot( np.repeat(np.array([0,1000])[:,np.newaxis],len(hlines), axis =1) , 
             np.array([hlines[:,0], hlines[:,0]+hlines[:,1]*1000]), '-', color = lcolors[ss], alpha = 0.2)
    plt.plot(np.array([-1*vlines[:,0]/vlines[:,1], -1*vlines[:,0]/vlines[:,1]+1/vlines[:,1]*1000])  , 
             np.repeat(np.array([0,550])[:,np.newaxis],len(vlines), axis =1) ,  '-', color = lcolors[ss], alpha = 0.2)

    
for jj in range(0,6):
    idcs = df['joint%i_TD_idcs'%jj][tr_num]
    idcs_OI = idcs[np.logical_and(df['frames'][tr_num][idcs]>=fr_range[0]-10, df['frames'][tr_num][idcs]<=fr_range[1]+5)]
    xs = df['joint%i_x_filt_fullfr'%jj][tr_num][idcs_OI]
    ys = df['joint%i_y_filt_fullfr'%jj][tr_num][idcs_OI]
#     plt.xlim([0,1000])
#     plt.ylim([550,0])
#     print(xs)
    plt.plot(xs, ys, '.', color = jcolors[jj])


    
plt.figure(figsize = (15,4))
for jj in range(0,6):
    plt.subplot(2,1,(jj%2)+1)
    idcs = df['joint%i_TD_idcs'%jj][tr_num]
    idcs_OI = idcs[np.logical_and(df['frames'][tr_num][idcs]>=fr_range[0]-10, df['frames'][tr_num][idcs]<=fr_range[1]+10)]
    plt.plot(df['frames'][tr_num][:-1], df['joint%i_vel'%jj][tr_num],'-',color = jcolors[jj])
    plt.plot(df['frames'][tr_num][np.array([idcs_OI,idcs_OI])], np.repeat(np.array([0,10])[:,np.newaxis],len(idcs_OI), axis =1), 
             color = jcolors[jj], linestyle = linetypes[jj%3], alpha = 0.4)

## Check follow the leader foot placement

In [ ]:
# FOR ONE TRIAL (TEST OUT ANALYSIS)
tr=300
x=df

for joint_num in [0,1,3,4]:
    
    if not np.any(np.isfinite(x['joint%i_x_filt_fullfr'%joint_num][tr])): # not just a single nan
        print('no data for filt_fullfr data')
        continue
    if not np.any(np.isfinite(x['joint%i_x_filt_fullfr'%(joint_num+1)][tr])): # not just a single nan
        print('no data for filt_fullfr data')
        continue
    
    # posterior limb info
    post_x = x['joint%i_x_filt_fullfr'%joint_num][tr]
    post_y = x['joint%i_y_filt_fullfr'%joint_num][tr]
    post_TD_starts = x['joint%i_TD_idcs'%joint_num][tr][:-1][x['joint%i_good_strides'%joint_num][tr]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    post_TD_stops = x['joint%i_TD_idcs'%joint_num][tr][1:][x['joint%i_good_strides'%joint_num][tr]]
    
    # anterior limb info
    ant_x = x['joint%i_x_filt_fullfr'%(joint_num+1)][tr]
    ant_y = x['joint%i_y_filt_fullfr'%(joint_num+1)][tr]
    ant_TD_starts = x['joint%i_TD_idcs'%(joint_num+1)][tr][:-1][x['joint%i_good_strides'%(joint_num+1)][tr]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    ant_TD_stops = x['joint%i_TD_idcs'%(joint_num+1)][tr][1:][x['joint%i_good_strides'%(joint_num+1)][tr]]

    all_dists = []
    St_idcs = []
    d_xy = np.full((1,2),np.nan)
    print('post TD stops: ', x['frames'][tr][post_TD_stops])
    final_post_TD_stops=post_TD_stops.copy().astype(float )
    for post_idc in range(0,len(post_TD_stops)):
#         print(post_TD_stops[post_idc], np.logical_and(ant_TD_starts<post_TD_stops[post_idc],ant_TD_stops>post_TD_stops[post_idc]))
        ant_idc = np.where(np.logical_and(post_TD_stops[post_idc]-ant_TD_starts > 7, ant_TD_stops-post_TD_stops[post_idc]>7))[0]
        if len(ant_idc)<2:
            if np.isfinite(ant_idc):
                ant_loc = np.hstack([ant_x[ant_TD_starts[ant_idc[0]]],  ant_y[ant_TD_starts[ant_idc[0]]]])
                post_loc = np.hstack( [post_x[post_TD_stops[post_idc]],  post_y[post_TD_stops[post_idc]]])
                d_xy = np.append(d_xy, np.array([post_loc-ant_loc])/pix2mm, axis = 0)
                dist = np.linalg.norm([post_loc-ant_loc])/pix2mm
                all_dists.append(dist)
                Stride_angle = np.abs(x['joint%i_St_travel_dir'%(joint_num+1)][tr][ant_idc])
                St_idcs.append( Stride_angle <=15)
                frame_OI = x['frames'][tr][post_TD_stops[post_idc]]
                print('joint %i, frame %i, dist = %0.2f mm, turning = %0.1f'%(joint_num, frame_OI, dist, Stride_angle))
            else:
                final_post_TD_stops[post_idc]=np.nan
        else:
            final_post_TD_stops[post_idc]=np.nan
            
    final_post_TD_stops = final_post_TD_stops[~np.isnan(final_post_TD_stops)].astype(np.uint8)
    dx = d_xy[1:,0]
    dy = d_xy[1:,1]
    
    
    # rotate dxy wrt ant facing
    if np.any(final_post_TD_stops):
#         print(x['frames'][tr][final_post_TD_stops])
        thorax_x = x['thorax_x_filt_fullfr'][tr][final_post_TD_stops]
        thorax_y = x['thorax_y_filt_fullfr'][tr][final_post_TD_stops]
        neck_x = x['neck_x_filt_fullfr'][tr][final_post_TD_stops]
        neck_y = x['neck_y_filt_fullfr'][tr][final_post_TD_stops]
    
        val_coord = np.array([dx,dy])#-np.array([thorax_x,thorax_y])
        neck_coord = np.array([neck_x-thorax_x,neck_y-thorax_y])
        ang = np.arctan2( (neck_y-thorax_y),(neck_x-thorax_x))
#         print(np.rad2deg(ang))
        c, s = np.cos(ang), np.sin(ang)
        Rx = np.array([c,s])
        Ry = np.array([-s,c])
        newx = np.einsum('mn,mn->n', val_coord, Rx)
        newy = np.einsum('mn,mn->n', val_coord, Ry)
    else:
        newx = dx
        newy = dy
#     print(dx,dy, ' to ', newx, newy)
                
# del x, post_x, post_y, post_TD_starts, post_TD_stops
# del ant_x, ant_y, ant_TD_starts, ant_TD_stops
# del all_dists, St_idcs, d_xy
# del Stride_angle, frame_OI

In [ ]:
def get_TD_dists_df(x, joint_num):
    
    if not np.any(np.isfinite(x['joint%i_x_filt_fullfr'%joint_num])): # not just a single nan
        return [],[],[],[]
    if not np.any(np.isfinite(x['joint%i_x_filt_fullfr'%(joint_num+1)])): # not just a single nan
        return [],[],[],[]
    
    # posterior limb info
    post_x = x['joint%i_x_filt_fullfr'%joint_num]
    post_y = x['joint%i_y_filt_fullfr'%joint_num]
    post_TD_starts = x['joint%i_TD_idcs'%joint_num][:-1][x['joint%i_good_strides'%joint_num]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    post_TD_stops = x['joint%i_TD_idcs'%joint_num][1:][x['joint%i_good_strides'%joint_num]]
    
    # anterior limb info
    ant_x = x['joint%i_x_filt_fullfr'%(joint_num+1)]
    ant_y = x['joint%i_y_filt_fullfr'%(joint_num+1)]
    ant_TD_starts = x['joint%i_TD_idcs'%(joint_num+1)][:-1][x['joint%i_good_strides'%(joint_num+1)]] # USING ONLY TRUSTED STRIDES, NOT ALL TDs
    ant_TD_stops = x['joint%i_TD_idcs'%(joint_num+1)][1:][x['joint%i_good_strides'%(joint_num+1)]]

    all_dists = []
    St_angles = []
    d_xy = np.full((1,2),np.nan)
    final_post_TD_stops = post_TD_stops.copy().astype(float)
    for post_idc in range(0,len(post_TD_stops)):
        ant_idc = np.where(np.logical_and(post_TD_stops[post_idc]-ant_TD_starts > 7, ant_TD_stops-post_TD_stops[post_idc]>7))[0]
        if len(ant_idc)<2:
            if np.isfinite(ant_idc):
                ant_loc = np.hstack([ant_x[ant_TD_starts[ant_idc][0]],  ant_y[ant_TD_starts[ant_idc][0]]])
                post_loc = np.hstack( [post_x[post_TD_stops[post_idc]],  post_y[post_TD_stops[post_idc]]])
                d_xy = np.append(d_xy, np.array([post_loc-ant_loc])/pix2mm, axis = 0)
                dist = np.linalg.norm([post_loc-ant_loc])/pix2mm
                all_dists.append(dist)
                Stride_angle = np.abs(x['joint%i_St_travel_dir'%(joint_num+1)][ant_idc])
                St_angles.append( (Stride_angle <=15)[0])
    #             print(ant_loc, post_loc, 'dist = %0.2f mm'%dist)
            else:
                final_post_TD_stops[post_idc]=np.nan
        else:
            final_post_TD_stops[post_idc]=np.nan
    
    final_post_TD_stops = final_post_TD_stops[~np.isnan(final_post_TD_stops)].astype(np.uint8)
    dx = d_xy[1:,0]
    dy = d_xy[1:,1]

    # rotate dxy wrt ant facing
    if np.any(final_post_TD_stops):
        thorax_x = x['thorax_x_filt_fullfr'][final_post_TD_stops]
        thorax_y = x['thorax_y_filt_fullfr'][final_post_TD_stops]
        neck_x = x['neck_x_filt_fullfr'][final_post_TD_stops]
        neck_y = x['neck_y_filt_fullfr'][final_post_TD_stops]
        val_coord = np.array([dx,dy])
        ang = np.arctan2( (neck_y-thorax_y),(neck_x-thorax_x))
        c, s = np.cos(ang), np.sin(ang)
        Rx = np.array([c,s])
        Ry = np.array([-s,c])
        newx = np.einsum('mn,mn->n', val_coord, Rx)
        newy = np.einsum('mn,mn->n', val_coord, Ry)
    else:
        newx = dx
        newy = dy

    return np.array(all_dists), newx, newy, np.array(St_angles)


# compile data of TD locations and heights
print('\ncalculating step distances')
for joint_num in [0,1,3,4]:
    print(joint_num)
    df['joint%i_TD_dists'%joint_num], df['joint%i_TD_dist_x'%joint_num], df['joint%i_TD_dist_y'%joint_num] , df['joint%i_TD_dist_idcs'%joint_num]= zip(*
                                df.apply(get_TD_dists_df, args = (joint_num, ), axis=1))

print('\nDone analyzing step distances!')

In [ ]:
#  distribution plot of anterior-posterior step distances
plt.close('all')
subtypes = sorted(df['substrate'].unique())
coltypes = sorted(df['colony'].unique())


df['TD_dists_all'] = df.filter(regex='_TD_dists$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1) 
df['TD_dist_x_all'] = df.filter(regex='_TD_dist_x$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1) 
df['TD_dist_y_all'] = df.filter(regex='_TD_dist_y$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1) 
df['TD_dist_straight'] = df.filter(regex='_TD_dist_idcs$', axis=1).apply(lambda x: np.concatenate(np.concatenate([x], axis = 0)), axis = 1)
df['TD_dist_jointID'] = df.filter(regex='_TD_dists$', axis=1).applymap(lambda x: len(x)).apply(
    lambda x: np.concatenate([x]), axis = 1).map(
    lambda x: np.repeat([0,1,3,4], x))

lens = [len(item) for item in df['TD_dists_all']]
all_TD_dists = pd.DataFrame( {"substrate" : np.repeat(df['substrate'].values, lens), "trackway" : np.repeat(df.index.values, lens),
                        "colony" : np.repeat(df['colony'].values, lens), "TD_dists_all" : np.concatenate(df['TD_dists_all'].values),
                        "TD_dist_x_all" : np.concatenate(df['TD_dist_x_all'].values), "TD_dist_y_all" : np.concatenate(df['TD_dist_y_all'].values),
                        "TD_dist_straight" : np.concatenate(df['TD_dist_straight'].values).astype(bool), "TD_dist_jointID" : np.concatenate(df['TD_dist_jointID'].values)})


pltcolors = ['#B1740F', '#BA4246', '#087E8B', '#701C6F']
sp_max = 3
precision = 20 # how many bins per 1 unit
bins = np.linspace(0,sp_max, sp_max*precision+1)

for ss,subtype in enumerate(subtypes[0:4]):
    print(subtype)
#     sub_df = all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & (all_TD_dists['colony']!=coltypes[-1])]
    vals = all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & 
                                   (all_TD_dists['colony']!=coltypes[-1]) &
                                   (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all']
    sns.distplot(  vals , bins = bins, color = pltcolors[ss], kde_kws={'clip': (np.min(vals), np.max(vals))})
    
#     # combine all joints together
#     vals_OI = sub_df['TD_dists_all'].values
#     straight_vals = sub_df['TD_dist_straight'].values.astype(bool)
#     hist_OI = np.histogram(vals_OI[straight_vals], bins, density = True)[0]
# #     hist_OI = vals_OI.mean()
#     plt.bar(bins[:-1], hist_OI , width =1/precision, color = pltcolors[ss], alpha = 0.3, align = 'edge')
# #     kde_data = np.repeat(bins[:-1]+1/(2*precision),np.round((hist_OI*10000)).astype(int))
# #     kde = stats.gaussian_kde(kde_data)
# #     kde_fit = kde.evaluate(bins[:-1])/precision
# # #     if ss == 0:
# # #         ref_speed = bins[np.argmax(kde_fit)]+1/(2*precision)
# # #     plt.axvline(x=ref_speed, ymin = 0, ymax = 1, color = 'k', linestyle = ':', alpha = 0.4)
# #     plt.plot(bins[:-1][hist_OI != 0]+1/(2*precision), kde_fit[hist_OI != 0], '-', color = pltcolors[ss], alpha = 0.4)
    n_pts = len(all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & 
                                   (all_TD_dists['colony']!=coltypes[-1]) &
                                   (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all'] )
    plt.text(2, 2-ss*0.2, 'n: %i'%n_pts, color = pltcolors[ss], alpha = 0.4)

In [ ]:
# calc if distributions are sig diff
for ss,subtype in enumerate(subtypes[0:4]):
    vals_0 = all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & 
                                   (all_TD_dists['colony']!=coltypes[-1]) &
                                   (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all']
    for comp in range(ss+1, 4):
        vals_1 = all_TD_dists.loc[(all_TD_dists['substrate']==subtypes[comp]) & 
                                   (all_TD_dists['colony']!=coltypes[-1]) &
                                   (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all']
        pval = stats.ttest_ind(vals_0, vals_1, equal_var = False)[1]
        print('\ncompare %s and %s: p = '%(subtype, subtypes[comp]), pval, ' -- means: %0.2f vs. %0.2f'%(np.mean(vals_0), np.mean(vals_1)))
        
        # perform whitney mann comparison (doesn't assume normally-distributed)
        _, pval = stats.ranksums(vals_0, vals_1)
        print('Mann-Whitney-Wilcox: p = ', pval)

print('\nanova comparing all groups')
_, p_val = stats.f_oneway( all_TD_dists.loc[(all_TD_dists['substrate']==subtypes[0]) & (all_TD_dists['colony']!=coltypes[-1]) & (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all'], 
                         all_TD_dists.loc[(all_TD_dists['substrate']==subtypes[1]) & (all_TD_dists['colony']!=coltypes[-1]) & (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all'], 
                         all_TD_dists.loc[(all_TD_dists['substrate']==subtypes[2]) & (all_TD_dists['colony']!=coltypes[-1]) & (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all'], 
                         all_TD_dists.loc[(all_TD_dists['substrate']==subtypes[3]) & (all_TD_dists['colony']!=coltypes[-1]) & (all_TD_dists['TD_dist_straight']==True)]['TD_dists_all'], 
                         )
print('anova p-val: ', p_val)

In [ ]:
# plot location of TD relative to anterior step
plt.close('all')

def WRTant_to_WRTneck_df(df, ant_part):
    x = df['%s_x_filt'%ant_part]
    y = df['%s_y_filt'%ant_part]
    thorax_x = df['thorax_x_filt']
    thorax_y = df['thorax_y_filt']
    neck_x = df['neck_x_filt']
    neck_y = df['neck_y_filt']
    
    val_coord = np.array([x,y])-np.array([thorax_x,thorax_y])
    neck_coord = np.array([neck_x-thorax_x,neck_y-thorax_y])
    ang = np.arctan( (neck_y-thorax_y)/(neck_x-thorax_x))
    c, s = np.cos(ang), np.sin(ang)
    Rx = np.array([c,s])
    Ry = np.array([-s,c])
    newx = np.einsum('mn,mn->n', val_coord, Rx)
    newy = np.einsum('mn,mn->n', val_coord, Ry)
    return newx, newy

def plot_convex_hull(points, pltcolors, ss_num):
    hull = ConvexHull(points)
    cent =np.mean(points, 0)
    pts = []
    for pt in points[hull.simplices]:
        pts.append(pt[0].tolist())
        pts.append(pt[1].tolist())
    pts.sort(key=lambda p: np.arctan2(p[1]-cent[1], p[0] - cent[0]))
    pts = pts[0::2]
    pts.insert(len(pts), pts[0])
    k= 1.0
    poly = Polygon(k*(np.array(pts)-cent) + cent, closed = True, facecolor = pltcolors[ss_num], alpha = 0.05)
    poly.set_capstyle('round')
    plt.gca().add_patch(poly)
    plt.plot(cent[0],cent[1],'.', color = pltcolors[ss_num])
    return 

def plt_pca_ellipse(points, pltcolors, ss_num):
    
    cent =np.mean(points, axis = 0)
    pca = PCA(n_components = 2)
    pca.fit_transform(points)
    projected = pca.transform(points)
    a = np.linalg.norm(pca.inverse_transform([np.std(projected, axis =0)[0],0])-cent)
    b = np.linalg.norm(pca.inverse_transform([np.std(projected, axis =0)[1],0])-cent)
    new_unit =pca.inverse_transform([1,0])-cent
    el_angle = np.rad2deg(np.arctan2(new_unit[1], new_unit[0]))
    lstyle = '-'
    xoffset = 0

    el = Ellipse(cent, 2*a, 2*b, angle = el_angle, ec = pltcolors[ss_num], fc = 'None', LineStyle = lstyle)
    plt.gca().add_patch(el)
    el = Ellipse(cent, 2*1.96*a, 2*1.96*b, angle = el_angle, ec = pltcolors[ss_num], fc = 'None', LineStyle = lstyle)
    plt.gca().add_patch(el)
#     plt.text(-100 + xoffset, -115 + 10*joint_num, '%i'%len(points), color = pltcolors[joint_num])
#     SD = np.sqrt(a*b) # radius of circle with same area as ellipse
#     plt.text(-60 + xoffset, -115 + 10*joint_num, '%0.1f'%SD, color = pltcolors[joint_num])
    return 

plt.figure(figsize = (5,5))
sp_order = [3,1,4,2]
for ff,foot in enumerate([0,1,3,4]):
    plt.subplot(2,2,sp_order[ff])
    for ss,subtype in enumerate(subtypes[0:4]):
        
        vals_0 = all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & (all_TD_dists['colony']!=coltypes[-1]) &
                                    (all_TD_dists['TD_dist_straight']==True) & (all_TD_dists['TD_dist_jointID']==foot)]['TD_dist_x_all'].values
        vals_1 = all_TD_dists.loc[(all_TD_dists['substrate']==subtype) & (all_TD_dists['colony']!=coltypes[-1]) &
                                    (all_TD_dists['TD_dist_straight']==True) & (all_TD_dists['TD_dist_jointID']==foot)]['TD_dist_y_all'].values
        tmp = vals_0
        vals_0=vals_0[~np.logical_or(np.isnan(tmp),np.isnan(vals_1))]
        vals_1=vals_1[~np.logical_or(np.isnan(tmp),np.isnan(vals_1))]
        del tmp
        plot_convex_hull(np.array([vals_1,vals_0]).T, pltcolors, ss)
        plt_pca_ellipse(np.array([vals_1,vals_0]).T, pltcolors, ss)
        plt.text((ss-2),-1.8,'%i'%len(vals_0), color = pltcolors[ss])
    plt.text(0.5,1.5,'%i wrt %i'%(foot, foot+1))
    
    plt.xlim(-2,2)
    plt.ylim(-2,2)
#         plt.plot(vals_1,vals_0, '.', color = pltcolors[ss], alpha = 0.02)

In [ ]:
# how to delete a bunch of columns from a dataframe

# for joint_num in range(0,6):
#     columns_to_drop = ['TD_dists_all', 'TD_dist_x', 'TD_dist_y', 'TD_dist_idcs',
#                       'joint%i_TD_dists'%joint_num, 'joint%i_TD_dist_x'%joint_num, 'joint%i_TD_dist_y'%joint_num, 'joint%i_TD_dist_idcs'%joint_num]
#     for colmn in columns_to_drop:
#         if colmn in df: # remove columns if already exist
#             df = df.drop(colmn, axis = 1)

## MOVED TO OWN NOTEBOOK: MODEL where ant limbs can touchdown based on body location

In [ ]:
# plt.close('all')
import itertools
from scipy.linalg import toeplitz

# USER INPUT
for box_size in [1,3,5]:
    
    # box_size = 1
    precision = box_size/50
    plot_things = False
    print('box_size is %i, precision %i'%(box_size, precision))


    data = []
    plt.close('all')

    floor_x_range = [-5,10]
    floor_y_range = [-5,10]
    h_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
    v_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
    heights = toeplitz(np.arange(0,len(h_lines))%2,r=np.arange(0,len(v_lines))%2)-1
    colors = ['grey','silver']

    # neutral conditions
    neutral_body_height = 2/3
    flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical --> hind, mid, fore
    flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm * [1,-1,1]
    leg_lengths = np.linalg.norm(np.array([flatX,flatY,neutral_body_height*np.ones(3)]),axis=0) # leg lengths from stance
    leg_lengths = leg_lengths*1.1

    # for a given body location
    for body_height in neutral_body_height+np.linspace(0,-1,10):
        print(body_height)
        dict_to_df = {}

        # initialize figure
        if plot_things:
            plt.figure()
            plt.title('body height = %0.2f mm'%body_height)
            for hh,h in enumerate(h_lines[:-1]):
                for vv,v in enumerate(v_lines[:-1]):
                    rect = patches.Rectangle((h,v), box_size, box_size, color = colors[heights[hh,vv]+1])
                    plt.gca().add_patch(rect)
                    plt.show()
            plt.xlim(floor_x_range)
            plt.xlim(floor_y_range)
            plt.axis('equal')

        for body_y in np.arange(0,box_size+precision, precision):

            for body_x in np.arange(0,2*box_size+precision, precision):
                foot_xs, foot_ys, foot_thetas, foot_phis, foot_dists, foot_heights = np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan)

                body_loc = [body_x,body_y]

                if plot_things:
                    plt.plot(body_x,body_y,'+k')
                    foot_colors = ['r','g','b']
                    line_styles = ['--','-']

                # for each foot, 
                for ff in [0,1,2]:
                    foot_x = body_loc[0]+flatX[ff]
                    foot_y = body_loc[1]+flatY[ff]
                    angOI = np.arctan2(flatY[ff],flatX[ff])

                    if plot_things:
                        plt.plot(foot_x,foot_y, '+', color = foot_colors[ff], alpha = 0.5)

                    for zz in [-1,0]: 

                        if body_height-zz>leg_lengths[ff]: # if leg can't touch surface
                            continue

                        # find best touchdown location on that surface
                        r= (leg_lengths[ff]**2-(zz-body_height)**2)**(1/2) 
                        thetas = np.arange(-180,180,0.1)
                        arc_xs = r*np.cos(np.deg2rad(thetas))+body_x
                        arc_ys = r*np.sin(np.deg2rad(thetas))+body_y
                        arc_rows = np.argmax(np.repeat(arc_ys[np.newaxis,:],len(v_lines),axis=0)<np.repeat(v_lines[:,np.newaxis],len(thetas),axis = 1), axis =0)-1
                        arc_cols = np.argmax(np.repeat(arc_xs[np.newaxis,:],len(h_lines),axis=0)<np.repeat(h_lines[:,np.newaxis],len(thetas),axis = 1), axis =0)-1
                        arc_heights = heights[arc_rows,arc_cols]
                        good_thetas = (arc_heights ==zz) & (np.abs(thetas-np.rad2deg(angOI))<30) & (np.sign(thetas)==np.sign(angOI))
                        arc_dist_to_neutral = np.linalg.norm(np.vstack([arc_xs-foot_x,arc_ys-foot_y]),axis =0)
                        if np.sum(good_thetas)==0:
                            continue
                        theta_min = np.min(thetas[good_thetas])
                        theta_max = np.max(thetas[good_thetas])
                        tmp = arc_dist_to_neutral.copy().astype(float)
                        tmp[np.logical_not(good_thetas)]=np.nan
                        best_theta_idc = np.nanargmin(tmp)
                        del tmp

                        if plot_things:
                            h_arc = patches.Arc(body_loc, 2*r, 2*r, angle = 0, theta1=theta_min, theta2=theta_max , color = foot_colors[ff], 
                                            linestyle = line_styles[zz+1], alpha = 0.8)
                            plt.gca().add_patch(h_arc)
                            plt.plot(arc_xs[best_theta_idc], arc_ys[best_theta_idc], '.', color = foot_colors[ff], alpha = 0.5)
                            plt.plot()
                            plt.show()

                        # save to variables
                        foot_xs[zz+1,ff]=arc_xs[best_theta_idc]
                        foot_ys[zz+1,ff]=arc_ys[best_theta_idc]
                        foot_thetas[zz+1,ff]=thetas[best_theta_idc]
                        foot_phis[zz+1,ff]=np.rad2deg(np.arccos(r/leg_lengths[ff]))
                        foot_dists[zz+1,ff]=arc_dist_to_neutral[best_theta_idc]
                        foot_heights[zz+1,ff]=arc_heights[best_theta_idc]

                # append data for each valid stance
                tmp =foot_xs.copy()
                tmp[np.isfinite(tmp)]=np.sum(np.argwhere(np.isfinite(foot_xs))*np.array([3,1]),axis=1)
                list_of_TDs = [i[np.isfinite(i)].astype(int).tolist() for i in np.split(tmp.T,3, axis =0)]
                if len(list_of_TDs)<3:
                    continue
                stance_idcs = list(itertools.product(*list_of_TDs))
                for ss in range(len(stance_idcs)):
                    dict_to_df['box_size']=box_size
                    dict_to_df['body_x']=body_x
                    dict_to_df['body_y']=body_y
                    dict_to_df['body_z']=body_height
                    dict_to_df['foot_xs']=foot_xs.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_ys']=foot_ys.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_d_xs']=foot_xs.flatten()[np.array(stance_idcs[ss])] - (body_loc[0]+flatX)
                    dict_to_df['foot_d_ys']=foot_ys.flatten()[np.array(stance_idcs[ss])] - (body_loc[1]+flatY)
                    dict_to_df['foot_thetas']=foot_thetas.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_phis']=foot_phis.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_dists']=foot_dists.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_heights']=foot_heights.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['stance_type']=['flat','pitch','roll'][int(np.sum(np.abs(np.diff( foot_heights.flatten()[np.array(stance_idcs[ss])]))))]
                    dict_to_df['stance_displacement']=np.sum(foot_dists.flatten()[np.array(stance_idcs[ss])])
                    dict_to_df['stance_height']=['valley','peak'][(np.sum(foot_heights.flatten()[np.array(stance_idcs[ss])])>=-1).astype(int)]

                    # append to dict
                    data.append(dict_to_df.copy())


foot_placement_df = pd.DataFrame(data)
print('done making foot placement dataframe')

In [ ]:
# analyze dataframe of foot placement info
plt.close('all')


floor_x_range = [-5,10]
floor_y_range = [-5,10]
h_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
v_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
heights = toeplitz(np.arange(0,len(h_lines))%2,r=np.arange(0,len(v_lines))%2)-1
colors = ['grey','silver']
text_locations = np.meshgrid(h_lines[2:4]-box_size/2,v_lines[1::2]-box_size/2)

# where are box cut-offs for 2d histogram
xedges = np.arange(floor_x_range[0],floor_x_range[1],precision*2)-precision
yedges = np.arange(floor_y_range[0],floor_y_range[1],precision*2)-precision

# 2d histogram
for box_size in [1,3,5]:
    fig=plt.figure(figsize = (9,5))
    n_stances = np.full((2,3), np.nan)
    n_stances_adjusted = np.full((2,3), np.nan)
    stance_types = ['flat','pitch','roll']
    for ss,s_type in enumerate(stance_types):
        for hh,h_type in enumerate(['valley', 'peak']):


            df_OI = foot_placement_df.loc[(foot_placement_df['box_size']==box_size) & (foot_placement_df['stance_type']==s_type) & (foot_placement_df['stance_height']==h_type) ]
            n_stances[hh,ss] = len(df_OI)
            n_stances_adjusted[hh,ss] = np.sum(df_OI['foot_dists'].apply(lambda x: 1/np.sum(x)).values)
            print('stance type: %s, height: %s, n: %i, adjusted: %0.1f'%(s_type, h_type, n_stances[hh,ss], n_stances_adjusted[hh,ss]))

            H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges)) # count number of stances
        #     H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges) # weight each stance by "cost" (how much cumulative distance from neutral position)
        #                             , weights = 1/foot_placement_df.loc[(foot_placement_df['stance_type']==s_type)]['stance_displacement'].values)
            H = H.T

            plt.subplot(2,3,ss+1+hh*3)
            for ii in range(0,3):
                plt.axvline(x=box_size*ii, color ='w', linestyle='-')
                plt.axhline(y=box_size*ii, color ='w', linestyle='-')
            X,Y = np.meshgrid(xedges, yedges)
            pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 100) # number stanced
        #     pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 80) # weighted
            plt.axis('equal')
            plt.xlim([-1*box_size,3*box_size])
            plt.ylim([-1*box_size,box_size*2])
            if hh == 0:
                plt.title('%s stances'%s_type)
                plt.gca().get_xaxis().set_visible(False)
            for ttx, tx in enumerate(range(0,box_size*6,box_size)):
                for tty, ty in enumerate(range(0,box_size*3,box_size)):
                    if (tty==1) & ((ttx==1) or (ttx==2)):
                        continue
                    plt.text(tx-box_size/2,ty-box_size/2, ['v','p'][(ttx+tty)%2], color ='w' , ha='center')

    cax = fig.add_axes([0.92,0.1,0.02,0.8])
    #         plt.colorbar(pc, cax = cax, label = 'more <---> less displacement')
    plt.colorbar(pc, cax = cax, label = '# stances')





# plot stacked bar graph of number of stances
plt.figure()
total_n = np.sum(np.sum(n_stances))
total_n_adjusted = np.sum(np.sum(n_stances_adjusted))
bcolors = ['r','g','b']
for box_size in [1,3,5]:
    for ss in range(0,3):
        for hh in range(0,2):
            plt.bar(box_size-1, n_stances[hh,ss]/total_n, width = 1/2, 
                    bottom = np.insert(np.cumsum(n_stances.T.flatten()),0,0)[ss*2+hh]/total_n,  color = bcolors[ss], alpha = 0.2+0.3*hh)
            plt.bar(box_size, n_stances_adjusted[hh,ss]/total_n_adjusted, width = 1/2, 
                    bottom = np.insert(np.cumsum(n_stances_adjusted.T.flatten()),0,0)[ss*2+hh]/total_n_adjusted,  color = bcolors[ss], alpha = 0.2+0.3*hh)
plt.xlim([-10,6])
plt.gca().get_xaxis().set_ticks([])
plt.text(-9.5,0.95,'total n stances: %i'%total_n)
plt.title('%s mm number of possible stances'%box_size)
plt.xlabel('1, 3, 5 mm substrates (regular/adjusted)')
plt.ylabel('proportion of stances')




# # plt point clouds of where feet place down for model
# plt.figure(figsize = (12,10))
# for ss,s_type in enumerate(stance_types):
    
#         df_OI = foot_placement_df.loc[(foot_placement_df['stance_type']==s_type)]
#         d_x = np.vstack(df_OI['foot_d_xs'].values)
#         d_y = np.vstack(df_OI['foot_d_ys'].values)
#         heights = np.vstack(df_OI['foot_heights'].values)
        
#         for ll in range(0,3):
#             plt.subplot(3,3, ss*3+ll+1)
# #             plt.xlim([-1.25,1.25])
            
#             for hh in [-1,0]:
#                 where_height = heights[:,ll]==hh
#                 plt.plot(d_x[where_height,ll], d_y[where_height,ll]*(ll%2*2-1), '.', color = ['b','g'][hh+1], alpha = 0.02, MarkerSize = 2)
#                 plt.plot(np.mean(d_x[where_height,ll]), np.mean(d_y[where_height,ll]*(ll%2*2-1)), '+', color = ['b','g'][hh+1])
#                 plt.plot([0,np.mean(d_x[where_height,ll])], [0,np.mean(d_y[where_height,ll])*(ll%2*2-1)], '-', color = 'k')
                
# #                 weights_to_use = np.vstack(1/df_OI['foot_dists'].values)[where_height, ll]
#                 weights_to_use = 1/df_OI['stance_displacement'].values[where_height]
#                 x_mean = np.average(d_x[where_height,ll], weights = weights_to_use)
#                 y_mean = np.average(d_y[where_height,ll]*(ll%2*2-1),  weights = weights_to_use)
#                 plt.plot([0,x_mean], [0,y_mean], '--', color = 'r')
#                 plt.plot(0,0,'.k')

#             if ll==0:
#                 plt.ylabel('%s'%s_type)
#             else:
#                 plt.gca().get_yaxis().set_visible(False)
#             if ss ==0:
#                 plt.title('%s'%['fore','mid','hind'][ll])
#             plt.ylim([-1.25,1.25])
#             plt.xlim([-1.25,1.25])
            
            
# plot distribution of stances along y-axis for 1mm range in center of block
plt.figure()
vals_OI = np.abs(foot_placement_df['body_x'].values%box_size-box_size/2)<=0.5
stance_ys_OI = np.abs( (foot_placement_df['body_y'].values[vals_OI])/box_size - 0.5)/0.5


hprecision = box_size/precision/2 # how many bins per 1 unit
hbins = np.linspace(0, 1, hprecision+1)-0.01

h_counts = np.histogram(stance_ys_OI, bins = hbins)[0]
h_counts[0] = 2*h_counts[0]
n_data = np.sum(stance_ys_OI)
h_counts = h_counts/n_data
plt.plot(hbins[:-1][h_counts>0]+(1/(2*hprecision)), h_counts[h_counts>0], ':', color = pltcolors[np.where(np.array([0,1,3,5])==box_size)[0][0]])
plt.bar(hbins[:-1][h_counts>0], h_counts[h_counts>0], color = pltcolors[np.where(np.array([0,1,3,5])==box_size)[0][0]], alpha = 0.2, width =1/hprecision, align='edge')
# plt.hist(stance_ys_OI, bins = hbins, color = pltcolors[2], alpha = 0.2, weights = np.ones(stance_ys_OI.shape)*(1/n_data))
plt.xlabel('center   <------>   edges')
plt.ylabel('proportion of stances')
    
#         # lowpass filter histogram data
#         b,a = signal.butter(2, 1/(precision/4))
#         temp = signal.filtfilt(b,a, np.hstack([np.flipud(h_counts[h_counts>0]),h_counts[h_counts>0], np.flipud(h_counts[h_counts>0])]) , padlen = 10)
#         h_counts_filt= temp[ len(h_counts[h_counts>0]):(2*len(h_counts[h_counts>0]))]
#         plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts_filt, linestyle = line_styles[height], color = pltcolors[ss])
#         plt.text(0.72+height/10, .06-ss*0.003, 'n: %i'%len(hist_OI[heights==height]), color  = pltcolors[ss])
#         plt.ylim([0,0.07])



# del df_OI

## Compile video of random times during trials to see how many feet in contact - UNFINISHED

In [ ]:
def load_and_save_frames(raw_video_path, frame_range, verbose, video_to_save):
    """
    Independent of the frame range loaded, background has to be computed over total video or else can run into
    tracking problems
    """
    vid = cv2.VideoCapture(raw_video_path)
    Height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    Width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    NumFrames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    if not (NumFrames > 0):
        raise IOError('Codec issue: cannot read number of frames.')

    # restrict to desired range of frames
    if frame_range is None:
        frame_range = (0, int(NumFrames))
    else:
        # check doesn't exceed number of frames
        if frame_range[0] + frame_range[1] > NumFrames:
            frame_range = (int(frame_range[0]), int(NumFrames - frame_range[0]))

    # initialize blank frames
    frames = np.zeros((frame_range[1], Height, Width), np.uint8)

    # set the first frame to read in
    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for kk in range(frame_range[0]):
        tru, ret = vid.read(1)
    # vid.set(cv.CAP_PROP_POS_FRAMES, frame) # this way of setting the frame doesn't work on all cv versions

    # read in all frames
    for kk in range(frame_range[1]):
        tru, ret = vid.read(1)

        # check if video frames are being loaded
        if not tru:
            raise IOError('Codec issue: cannot load frames.')
        frames[kk, :, :] = ret[:, :, 0]  # assumes loading color
        
        # crop to center around ant
        video_to_save.write(ret)
        
        if ((kk % 100) == 0) and verbose:
            print(kk)
            
    vid.release()
    return frames, NumFrames, frame_range


# def save_image(vlocation, nfig, name_base):
#     pname = os.path.join(vlocation, '%s%d.png'%(name_base,nfig))
#     plt.savefig(pname)
#     nfig = nfig + 1
#     plt.pause(0.2)
# #     plt.close('all')
#     return nfig


# def save_video(vlocation, name_base):
#     # save images as movie
#     if os.path.isfile((vlocation+'/%s.mp4'%name_base)):
#         os.remove(vlocation + "/%s.mp4"%name_base)
#         print('** Deleted %s.mp4 file'%name_base)
#     print('saving %s.mp4 file'%name_base)
#     command_p1 = "ffmpeg -r 4 -i '%s/%s"%(vlocation, name_base)
#     command_p2 = " -vcodec libx264 '%s/%s.mp4'"%(vlocation, name_base)
#     command = command_p1 + "%01d.png'" + command_p2
# #     print(command)
#     os.system(command)
#     plt.pause(10)

#     # delete all trackway vids
#     pics2delete = glob.glob(os.path.join(vlocation, '%s*.png'%name_base))
#     for pic in pics2delete:
#         os.remove(pic)
#     return

In [ ]:
subtypes = sorted(list(set(df['substrate'].values)))
coltypes = sorted(list(set(df['substrate'].values)))
vlocation = '/media/gravishlab/SeagateExpansionDrive/AntTrack'
# vid_to_save = vlocation+'/Feet_In_Contact_%s.mp4'%subtype

for ss, subtype in enumerate(subtypes[0:1]):
    vid_name = vlocation+'/Feet_In_Contact_%s.mp4'%subtype
    vid_to_save = cv2.VideoWriter(vid_name, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (1000,550) )

    t_idcs = df.loc[(df['substrate']==subtype) & (df['colony']!='Tunnel_20180329-30')].index.values
    idcs_OI = np.random.choice(t_idcs,100)
#     print(idcs_OI)
    
    for ii,idc in enumerate(idcs_OI[0:1]):
        non_nan_frs = df.iloc[idc]['frames'][np.logical_and(
            np.isfinite(df.iloc[idc]['thorax_x_filt_fullfr']), np.isfinite(df.iloc[idc]['thorax_y_filt_fullfr']) )]
        while len(non_nan_frs)==0:
            idc = idc+1
            non_nan_frs = df.iloc[idc]['frames'][np.logical_and(
                np.isfinite(df.iloc[idc]['thorax_x_filt_fullfr']), np.isfinite(df.iloc[idc]['thorax_y_filt_fullfr']) )]
            idcs_OI[ii]=idc
            
        t_xs = df.iloc[idc]['thorax_x_filt_fullfr']
        t_ys = df.iloc[idc]['thorax_y_filt_fullfr']
        frs = df.iloc[idc]['frames']
        
        fr_OI = np.random.choice(non_nan_frs,1)
        fr_idc_OI = np.where(frs==fr_OI)[0][0]
        t_x = t_xs[fr_idc_OI]
        fr_OI = frs[fr_idc_OI]
        t_y = t_ys[fr_idc_OI]
        
        print('trial %i - frame OI: %i - x,y: %0.1f,%0.1f'%(idc,fr_OI, t_x,t_y))
        
        frame_range = [int(fr_OI-5),11]
        frames, NumFrames, frame_range = load_and_save_frames(df.iloc[idc]['video'], frame_range, vid_to_save, t_x, t_y)
#         frames, NumFrames, frame_range = load_video(df.iloc[idc]['video'], [int(fr_OI-5),int(fr_OI+5)], True)

        del idcs_OI, t_idcs, ii, idc, t_xs, t_ys, non_nan_frs, fr_OI, fr_idc_OI, t_x, t_y



    
    vid_to_save.release()
#         for fr_idc in range(frames.shape[0]):
#             frame_to_save = frames[fr_idc,:,:]
#             gray_frame = cv2.normalize(frame_to_save, None, 255, 0, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8U)
#             save_image(vlocation, im_n, 'LEAPtracking_filter')
# save_video(vlocation, 'LEAPtracking_filter')
        
        
    
#     yvals_OI = df.loc[(all_strides['substrate']==subtype) & (all_strides['colony']!='Tunnel_20180329-30')].index

## Compare manual and automatic tracked TDs

In [ ]:
# find trials to manually track

# which trials?
# for tt in range(2000,2030):#[1662]:
#     print(tt, ' -- ', np.sum(df['joint3_good_strides'][tt]), np.sum(df['joint4_good_strides'][tt]), np.sum(df['joint5_good_strides'][tt]), ' -- %s'%df['time'][tt])

# which ROI?
# for tt in [2417,2437,2448,2464]:
for tt in [1960, 1990, 2011]: 
    print('\n', tt, ' -- ', np.sum(df['joint3_good_strides'][tt]), np.sum(df['joint4_good_strides'][tt]), np.sum(df['joint5_good_strides'][tt]), ' -- %s'%df['time'][tt])
    for jj in range(3,6):
        print(df['frames'][tt][df['joint%i_TD_idcs'%jj][tt][:-1][df['joint%i_good_strides'%jj][tt]]])

In [ ]:
# [1616,1617,1626,1662]
# [1924,1943,1961,2016,1960,1990],
# [2106,2114,2129,2177]
# [2417,2437,2448,2464]

from numpy import genfromtxt
cutoff = 10

all_trials = ([
    [1616,1617,1626,1662],
    [1924,1943,1961,2016,1960,1990],
    [2106,2114,2129,2177],
    [2417,2437,2448,2464]])


for trials in all_trials[0:4]:


    all_TD_offsets = [[],[],[]]
    total_TDs_OI = 0
    total_TDs_correct = 0

    for tt in trials:
        print('\n', tt, ' -- ', np.sum(df['joint3_good_strides'][tt]), np.sum(df['joint4_good_strides'][tt]), np.sum(df['joint5_good_strides'][tt]))
        mfile = ('/').join(df['video'][tt].split('/')[:-1]) + '/' + df['video'][tt].split('/')[-1].split('1627')[0] + 'manualTDs.csv'
        print(mfile)
        manual_data = np.fliplr(genfromtxt(mfile, delimiter=','))

        for jj in range(0,3):

            mTDs = manual_data[:,jj]
            mTDs = mTDs[np.isfinite(mTDs)]
            aTDs =df['frames'][tt][df['joint%i_TD_idcs'%(jj+3)][tt][:-1][df['joint%i_good_strides'%(jj+3)][tt]]]

            temp = np.tile(aTDs[:,np.newaxis],len(mTDs)).T-np.tile(mTDs[:,np.newaxis],len(aTDs))
            aTDs_OI = np.sum(np.abs(temp)<15,axis=0)>0
            n_aTDs_OI = np.sum(aTDs_OI)

            aTDs_OI_checked_idcs = []
            aTDs_OI_checked =[]
            TDs_offset = []
            temp2 = mTDs.copy()
            for aTD in aTDs[aTDs_OI]:
                idx = np.argmin(np.abs(temp2-aTD))
                aTDs_OI_checked_idcs.append(0)
                if np.abs(temp2-aTD)[idx]<cutoff:
                    aTDs_OI_checked.append(temp2[idx])
                    TDs_offset.append(aTD-temp2[idx])
                    temp2 = np.delete(temp2, idx)
                del idx


            n_aTDs_OI_checked = len(aTDs_OI_checked)



            all_TD_offsets[jj]=all_TD_offsets[jj]+TDs_offset
            total_TDs_OI = total_TDs_OI + n_aTDs_OI
            total_TDs_correct = total_TDs_correct + n_aTDs_OI_checked

            print(aTDs, aTDs[aTDs_OI], mTDs)
#             print(aTDs[aTDs_OI], mTDs)
            print('%i/%i TDs correctly identfied, offsets:'%( n_aTDs_OI_checked ,n_aTDs_OI), TDs_offset)


    ave_offset = np.mean(np.array([item for sublist in all_TD_offsets for item in sublist]))
    print('\n\nTOTAL %i/%i TDs correctly identified, ave offset = %0.2f\n\n'%(total_TDs_correct, total_TDs_OI, ave_offset))

### list and inspect trials with weird values

In [ ]:
# find df and file_list locations of weird trials
trackways_OI = np.unique(all_strides['trackway'][np.abs(all_strides['St_rotation'].values)>100].values)
print('Trackways: ', trackways_OI[0:10])
files_OI = np.where(np.isin(np.array(file_list), df['video'][trackways_OI].values))[0]
print('Files: ', files_OI[0:10])



# is it an angle issue?
tway = 416
print('\nVIDEO: ', df['video'][tway])
plt.close('all')
plt.figure(figsize=(10,5))
plt.subplot2grid((2,2),(0,0))
plt.plot(df['angle_improved'][tway], '.k')
plt.ylabel('angle improved')
plt.subplot2grid((2,2),(1,0))
plt.plot(df['thorax_x_filt'][tway], '.k')
plt.plot(df['neck_x_filt'][tway], '.r')
plt.ylabel('x filt WRT ant')

#     print(str_starts, str_ends)
#     plt.plot()
plt.subplot2grid((2,2),(0,1), rowspan =2)
plt.plot(df['thorax_x_filt_fullfr'][tway],df['thorax_y_filt_fullfr'][tway], '.k')
n_str = 0
for jj in range(0,1):#6):
    str_starts = df['joint%i_TD_idcs'%jj][tway][:-1][df['joint%i_good_strides'%jj][tway]]
    str_ends = df['joint%i_TD_idcs'%jj][tway][1:][df['joint%i_good_strides'%jj][tway]]-1
    pltcolors = np.random.rand(len(str_starts),3)#plt.cm.get_cmap('hsv', len(str_starts))
    plt.gca().set_prop_cycle(color = pltcolors)
    plt.plot(np.array([df['thorax_x_filt_fullfr'][tway],df['neck_x_filt_fullfr'][tway]])[:,str_starts],
             np.array([df['thorax_y_filt_fullfr'][tway],df['neck_y_filt_fullfr'][tway]])[:,str_starts], '-')
    plt.gca().set_prop_cycle(color = pltcolors)
    plt.plot(np.array([df['thorax_x_filt_fullfr'][tway],df['neck_x_filt_fullfr'][tway]])[:,str_ends],
             np.array([df['thorax_y_filt_fullfr'][tway],df['neck_y_filt_fullfr'][tway]])[:,str_ends], ':')
    plt.gca().set_prop_cycle(color = pltcolors)
    plt.plot(np.array([df['neck_x_filt_fullfr'][tway][str_starts],df['neck_x_filt_fullfr'][tway][str_ends]]),
             np.array([df['neck_y_filt_fullfr'][tway][str_starts],df['neck_y_filt_fullfr'][tway][str_ends]]), ':')
    for ii,kk in enumerate(str_starts):
        textsize = 6
        if np.abs(df['joint%i_St_rotation'%jj][tway][ii]) > 80:
            textsize = 12
        plt.text(df['thorax_x_filt_fullfr'][tway][kk]+10, df['thorax_y_filt_fullfr'][tway][kk]+5, 
                 '%0.1f, %0.1f'%(df['joint%i_travel_dir'%jj][tway][ii], df['joint%i_St_rotation'%jj][tway][ii]), color = pltcolors[ii], 
                fontsize = textsize)
    n_str = n_str + len(str_starts)
plt.ylabel('y filt WRT full frame')
plt.ylabel('x filt WRT full frame')
plt.gca().axis('equal')
plt.gca().invert_yaxis()


print('\nFACING ROTATIONS:')
for jj in range(0,6):
    print('Joint %i: '%jj, df['joint%i_St_rotation'%jj][tway].astype(int))
    
print('\nTRAVEL DIR WRT STARTING FACING:')
for jj in range(0,6):
    print('Joint %i: '%jj, df['joint%i_travel_dir'%jj][tway].astype(int))

## Fourier Analysis of stride frequency 

In [ ]:
# fourier of one trackway
plt.close('all')

subOI = '0mm'
colOI = '20180313-14'
trOI = '13_093129'

win_wid = 130
n_dp_cutoff = 120
inc = 10 # how much bump window with each iteration
rate = 240

# for comparison to manually tracked stepping
# st_timing = [1.000,1.112,1.233,1.337,1.441,1.533,1.687,1.799,2.062,2.200,2.320,2.429,2.549,2.783,2.887]
# st_frs = [round(st*239.2) for st in st_timing]


# tr_nums = longtracks[(longtracks['colony']=='Tunnel_' + colOI) & 
#                      (longtracks['substrate']==subOI) &
#                      (longtracks['datetime']==
#                       datetime.datetime.strptime('201803'+trOI, "%Y%m%d_%H%M%S"))].index

tr_nums = [91]

fig = plt.figure(figsize = (7,13))
gs = gridspec.GridSpec(3,1)
gs.tight_layout(fig, rect = [0.5, 0, 1,1], h_pad = 0.5)
ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[1,0])
ax3 = plt.subplot(gs[2,0])

for tr_num in tr_nums:

    # tr_num = 8978

    gs = gridspec.GridSpec(3,1)
    gs.update( hspace = 0.3, wspace = 0.01)
    trC = np.random.rand(3,)


    x = longtracks.loc[tr_num].copy()
    print(x.colony, x.substrate)
    blankvaltrace = np.full(x.frames.shape,np.nan)
    blankvtrace = np.full(x.frames.shape,np.nan)
    frs = [ frame in x.frames_final for frame in x.frames] #[int(fr) for fr in x.frames_final]
    blankvaltrace[frs] = x.dist_90fr
    blankvtrace[frs] = x.v_final

    # PLOT THINGS
#     ax1.plot([st_frs, st_frs],[np.zeros(len(st_frs)), 1500*np.ones(len(st_frs))],
#                              ':', c = 'k', alpha = 0.5, linewidth = 1)
    ax1.plot(x.frames[1:], x.vfilt, '-k', alpha = 0.5);
    ax1.plot(x.frames+1, blankvtrace, '-', c = trC);
    ax1.plot(x.frames, blankvaltrace*(240/90), '.r', alpha = 0.5);
    ax1.set_xlim((0,717))
    ax1.set_xlabel('frame');
    ax1.set_ylabel('v (pix/s)')
    ax1.set_title('%s -- %s -- %s' % (x.colony, x.substrate, x.datetime.strftime('%H_%M_%S')));

    # fft on whole track
    fft_input = blankvtrace[np.isfinite(blankvtrace)]
    p = 20* np.log10(np.abs(np.fft.rfft(fft_input)))
    xf = np.linspace(0,rate/2,len(p))
    ax2.plot(xf,p, c = trC)
    ax2.set_ylabel('log(power)');
    ax2.set_xlabel('freq (Hz)');
    ax2.set_title('FFT for all non-nan data');
    

    # fft each chunk of non nan
    starts = np.where(np.diff(np.isfinite(blankvtrace)))[0][0::2]
    stops = np.where(np.diff(np.isfinite(blankvtrace)))[0][1::2]
    start_frs = x.frames[1:][(np.diff(np.isfinite(blankvtrace)))][0::2]+1
    for sta, sto, stafr in zip(starts,stops, start_frs):
        n_dp = sto - sta
        if n_dp > n_dp_cutoff:
    #                     print('---- # data points in chunk: %s' %n_dp)
            chunk = blankvtrace[sta+1:sto]
            cushion = np.full(win_wid-10,np.nan)
            chunk = np.hstack((cushion,chunk,cushion))
            

            powers = np.full(int(120*10+1),np.nan)
            power_vs = []

            for kk,win_edge in enumerate(range(0, n_dp+win_wid-10, inc)):
    #             print('---%s' % win_edge)
                win_data = chunk[win_edge:win_edge+win_wid]
                fft_input = win_data[np.isfinite(win_data)]
    #             print('-- %i: %i' %(win_edge, len(fft_input)))
                if (len(fft_input)> n_dp_cutoff) and (np.nanstd(fft_input) < 350):
                    p = 20* np.log10(np.abs(np.fft.rfft(fft_input)))
                    xf = np.linspace(0,rate/2,len(p))
#                     print('-- %0.3f -- %0.3f' %(np.nanstd(fft_input), np.nanstd(fft_input)/np.nanmean(fft_input)))
                    

                    randomC = np.random.rand(3,)
                    ax3.plot(xf,p, c = randomC)
                    ax1.plot([win_edge-(win_wid-10)+stafr, win_edge+10+stafr],[np.nanmean(fft_input), np.nanmean(fft_input)],
                             '-', c = randomC, alpha = 0.5)
                    ax1.add_patch(Rectangle(
                        (win_edge-(win_wid-10)+stafr, np.nanmean(fft_input)-np.nanstd(fft_input)), win_wid, 2*np.nanstd(fft_input),
                                 alpha = 0.1, color = randomC ))

                    temp = np.interp(np.linspace(0,120,120*10+1),xf,p)
                    print('-- %0.3f -- Peak FRs: %0.2f, %0.2f' %(np.nanmean(fft_input), 
                                                                 argrelextrema(temp, np.greater)[0][0]/10,
                                                                argrelextrema(temp, np.greater)[0][1]/10))
                    powers = np.vstack((powers,temp))
                    power_vs = np.append(power_vs, np.nanmean(fft_input))

    powers = powers[1::]
    avg_power = np.nanmean(powers,axis=0)

    ax3.plot(np.linspace(0,rate/2,len(temp)),avg_power, '--k', linewidth = 3, label = 'ave')
    ax3.set_ylabel('log(power)');
    ax3.set_xlabel('frequency (Hz)');
    ax3.set_title('FFT for moving window w/ 50-60 non-nan data points')
    ax3.text(90,80,'increment = 10 fr')
    ax3.legend()

In [ ]:
# HEATMAP - fourier vs travel velocity for each chunk
import matplotlib.gridspec as gridspec


subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())

win_wid = 120
n_dp_cutoff = 110
inc = 10 # how much bump window with each iteration
rate = 240
all_chunks = []

for coltype in coltypes[0:1]:
    plt.close('all')
    fig = plt.figure(figsize = (15,12))
    gs = gridspec.GridSpec(1,8, width_ratios = [2,0.5,2,0.5,2,0.5,2,0.5])
    gs.update(left = 0.05, right = 0.95, hspace = 0.3, wspace = 0.0)
    cbar_ax = fig.add_axes([.95,.1,.03,.8])
    
    for ss,subtype in enumerate(subtypes[0:4]):
        print('%s -- %s' % (coltype, subtype))
        allpowers = np.full(int(120*10+1),np.nan)
        allpowers_vs = []
        allpowers_sorted = []
        allpowers_vs_sorted = []
        allpeaks = []
        
        for k,x in longtracks.loc[(longtracks['substrate']==subtype) & (longtracks['colony']==coltype)].iterrows():
#             print('-- %s' % x.datetime.strftime('%Y%m%d_%H%M%S'))

            blankvaltrace = np.full(x.frames.shape,np.nan)
            blankvtrace = np.full(x.frames.shape,np.nan)
            frs = [ frame in x.frames_final for frame in x.frames] #[int(fr) for fr in x.frames_final]
            blankvaltrace[frs] = x.dist_90fr
            blankvtrace[frs] = x.v_final

            # fft each chunk of non nan
            starts = np.where(np.diff(np.isfinite(blankvtrace)))[0][0::2]
            stops = np.where(np.diff(np.isfinite(blankvtrace)))[0][1::2]
            for sta, sto in zip(starts,stops):
                n_dp = sto - sta
                if n_dp > n_dp_cutoff:
#                     print('---- # data points in chunk: %s' %n_dp)
                    chunk = blankvtrace[sta+1:sto]
                    cushion = np.full(win_wid-10,np.nan)
                    chunk = np.hstack((cushion,chunk,cushion))
            
                    powers = np.full(int(120*10+1),np.nan)
                    power_vs = []
                    peak_fqs = []
                    
                    
                    for kk,win_edge in enumerate(range(0, n_dp+win_wid-10, inc)):
                        tmp = {}
                        win_data = chunk[win_edge:win_edge+win_wid]
                        win_data_nonan = win_data[np.isfinite(win_data)]
            #             print('-- %i: %i' %(win_edge, len(fft_input)))
                        if (len(win_data_nonan)> n_dp_cutoff) and (np.nanstd(win_data_nonan) < 350):
                            fft_input = win_data_nonan - np.nanmean(win_data_nonan) # get rid of low freq by subtracting mean value
                            p = 20* np.log10(np.abs(np.fft.rfft(fft_input)))
                            xf = np.linspace(0,rate/2,len(p))

                            temp = np.interp(np.linspace(0,120,120*10+1),xf,p)
                            powers = np.vstack((powers,temp))
                            power_vs = np.append(power_vs, np.nanmean(win_data_nonan))
                            
                            # find rel max
                            rel_max_idcs = argrelextrema(temp,np.greater)[0]
                            rel_maxs = temp[rel_max_idcs]
                            peak_freqs = rel_max_idcs[np.argsort(rel_maxs)[::-1]]/10
                            
                            # save to dataframe
                            tmp['colony']= coltype
                            tmp['substrate'] = subtype
                            tmp['video'] = x.video
                            tmp['datetime'] = x.datetime
                            tmp['v'] = fft_input
                            tmp['v_ave'] = np.nanmean(win_data_nonan)
                            tmp['frames'] = np.linspace(win_edge-(win_wid-10)+sta,win_edge+10+sta, win_wid)
                            tmp['pow_spec'] = temp
                            tmp['pow_spec_raw'] = p
                            tmp['pow_freq_raw'] = xf
                            tmp['peak_freqs'] = peak_freqs
                            all_chunks.append(tmp)
                            
                            peak_fqs = np.append(peak_fqs, peak_freqs[0])

                    if not np.isnan(powers).all(): # only the holder vector of nan -- vel std too large for all windows
                        powers = powers[1::]
                        avg_power = np.nanmean(powers,axis=0)
                        allpowers = np.vstack((allpowers,powers))
                        allpowers_vs = np.append(allpowers_vs, power_vs)
                        allpeaks = np.append(allpeaks, peak_fqs)


        allpowers = allpowers[1::]
        allpowers_sorted = allpowers[np.argsort(allpowers_vs)]
        allpowers_vs_sorted = np.sort(allpowers_vs)
        allpeaks_sorted = allpeaks[np.argsort(allpowers_vs)]
        
        
        
        # PLOT THINGS
        plt.subplot(gs[0,ss*2])
        v_box = 5
        if ss == 0:
            max_v = int(v_box*round(np.max(allpowers_vs)/v_box))
        min_v = 0
        subsample = int((max_v-min_v)/v_box+1)
        target_vs = np.linspace(min_v,max_v, subsample)
        
        # plot chunk closest to target v
#         target_vs_idcs = [np.abs(allpowers_vs_sorted-val).argmin() if np.abs(allpowers_vs_sorted-val).min() < (max_v-min_v)/(subsample-1)*(1/2) 
#                           else np.nan for val in target_vs]
#         hmap_to_plot = np.vstack( [allpowers_sorted[idc] if not np.isnan(idc) else np.full(int(120*10+1),np.nan) 
#                                    for idc in target_vs_idcs ])
        
        # average all chunks within range
        def average_powerspec(val):
            chunkidcs = []
            chunkidcs = np.logical_and(allpowers_vs_sorted>val, allpowers_vs_sorted<val+5) #[((v > val) and (v < val+v_box)) for v in allpowers_vs_sorted]
#             chunkidcs = list(allpowers_vs_sorted>val) and list(allpowers_vs_sorted<(val+5))
            if chunkidcs.any(): # are there any chunk power spectrums within velocity range of interest?
                chunks_ave = np.mean(allpowers_sorted[chunkidcs], axis=0)
            else:
                chunks_ave = np.full(int(120*10+1),np.nan)
            return chunks_ave
        hmap_to_plot = np.vstack( [average_powerspec(val) for val in target_vs[0:-1] ])
        chunk_hist = [sum(np.logical_and(allpowers_vs_sorted>val, allpowers_vs_sorted<val+5)) for val in target_vs[0:-1]]

        # plot heatmap
        if ss == 1:
            hmap = sns.heatmap(hmap_to_plot, xticklabels = 100, yticklabels = 10, cbar = True, cbar_kws = {'label': 'log(power)'}, cbar_ax = cbar_ax, vmin = 0, vmax = 100)
        else:
            hmap = sns.heatmap(hmap_to_plot, xticklabels = 100, yticklabels = 10, cbar = False, vmin = 0, vmax = 100)
        
        if ss == 0:
            plt.ylabel('ave v (pix/s)');
            plt.yticks(rotation=0)
            hmap.set_yticklabels(['%i' % v for v in target_vs[::10]])
        else:
            hmap.set_yticklabels([])
            plt.gca().get_yaxis().set_visible(False)
        plt.xticks(rotation=-90)
        hmap.set_xticklabels(np.arange(0,121,10))
#         plt.axvline(x=201, color = 'k', linestyle = ':', alpha = 0.5, linewidth = 1)
        plt.xlabel('freq (Hz)');
        plt.xlim((0,600))
        plt.title('%s -- total n: %i' % (subtype, len(allpowers_sorted)), loc = 'left')
        
        
        # plot hist
        plt.subplot(gs[0,ss*2+1])
        plt.hist(allpowers_vs_sorted,subsample-1,(min_v,max_v), orientation ='horizontal',color = 'k', alpha = 0.5)
        plt.gca().get_yaxis().set_visible(False)
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().set_ylim((min_v,max_v))
        plt.gca().set_xlim((0,250))
        plt.gca().invert_yaxis()
        plt.gca().set_frame_on(False)
        
        # run linear regression model and plot on top
# #         x = fft_data[(fft_data['colony']==coltype) & (fft_data['substrate']==subtype)]['v_ave']
# #         y = np.array([fs[0] for fs in fft_data[(fft_data['colony']==coltype) & (fft_data['substrate']==subtype)]['peak_freqs']])
# #         y = y[np.argsort(x).values]
# #         x = np.array(sorted(x))
#         x = allpowers_vs_sorted
#         y = allpeaks_sorted
        
#         x_c = sm.add_constant(x) # if do not want model to go through zero
#         x_i = np.linspace(min_v, max_v, subsample)
#         model = sm.OLS(y, x_c).fit()
#         predictions = model.predict(x_c)
#         predictions_interp = np.interp(x_i, x, predictions)
#         hmap.plot(predictions_interp*10,x_i/v_box,':b')
#         hmap.text(400,10,'freq = %0.3f *x + %0.2f'% (model.params[1],model.params[0]), color = 'b')

        
        
#         plt.tight_layout()
    plt.suptitle('Colony: %s -- from %i to %i pix/s, averaged over box of %i pix/s' % (coltype, min_v, max_v, v_box), 
                 x=0.02, y = 0.95, fontsize = 16, horizontalalignment = 'left')
    plt.savefig(vid_locations + 'Figures/FFT/C%i_Heatmap.png' % (coltypes.index(coltype)+1))
#     plt.savefig(vid_locations + 'Figures/FFT/C%i_Heatmap.eps' % (coltypes.index(coltype)+1))

fft_data = pd.DataFrame(all_chunks)
print('ALL DONE!')

In [ ]:
# SAVE FFT_DATA AS PICKLE FOR FUTURE USE
fft_data.to_pickle(vid_locations + 'FFT_data')

In [ ]:
# FIND INFO FOR A SINGLE CHUNK SO THAT CAN COMPARE TO MANUALLY COUNTING STRIDE RATE
# fft_data = 

# find subset of all chunks for a given colony and substrate type
dataOI = fft_data[(fft_data['colony']==coltypes[0]) & (fft_data['substrate']==subtypes[3])].copy()
sorted_vs = dataOI['v_ave'].sort_values(axis = 0, ascending = True)

def print_chunk_info(g):
    peak_fq = find_peak_fq(g['pow_spec'])
    print('\t%s, %s, %s, %0.2f Hz, %i to %i, %0.3f-%0.3f s' %(g['colony'].split('201803')[1], 
                                                                                     g['substrate'],g['datetime'], peak_fq,
                                                                                     g['frames'][0], g['frames'][-1], g['frames'][0]/239.2, 
                                                                                     g['frames'][-1]/239.2) )

def find_peak_fq(pspec):
    if np.isnan(pspec).all():
        return np.nan
    idcs = argrelextrema(pspec, np.greater)[0]
    rel_maxs = pspec[idcs[idcs>30]]
    peak_freqs = idcs[idcs>30][np.argsort(rel_maxs)[::-1]]/10
    return peak_freqs[0]
    

# n_vals = 10
# for val in np.linspace(list(sorted_vs)[0]+5,list(sorted_vs)[-1]-5,10):

for val in [250,400,550]:
    print('\n%0.1f\n'%val)
    
    chunksOI_idcs = sorted_vs[(sorted_vs>(val-2.5)) & (sorted_vs<(val+2.5))].index
    chunksOI_idcs_sorted = (dataOI.loc[chunksOI_idcs]['datetime']).sort_values(axis=0).index
    [print_chunk_info(dataOI.loc[idx]) for idx in chunksOI_idcs_sorted]
#     print(dataOI.iloc[chunksOI_idcs][['substrate','colony']], dataOI.iloc[chunksOI_idcs]['v_ave'])

In [ ]:
# RUN LINEAR REGRESSION AVERAGING ALL POWER SPECTRA WITHIN VELOCITY RANGES 
from scipy import optimize
plt.close('all')

# print('read in data')
subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())

# define what data to use for specific colony and substrate
colOI = coltypes[3]
subOI = subtypes[1]
v_ave = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['v_ave']
peak_fqs = np.array([fs[0] for fs in fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['peak_freqs']])
ps = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['pow_spec']

# define features for averaging over velocity range
v_box = 5
max_v = int(round(np.max(v_ave)))
min_v = 0
subsample = int((max_v-min_v)/v_box+1)
target_vs = np.linspace(min_v,max_v, subsample)

# functions
def average_powerspec(val):
    chunkidcs = []
    chunkidcs = np.logical_and(v_ave>val, v_ave<val+5) 
    if chunkidcs.any(): # are there any chunk power spectrums within velocity range of interest?
        chunks_ave = np.mean(ps[chunkidcs], axis=0)
    else:
        chunks_ave = np.full(int(120*10+1),np.nan)
    return chunks_ave
def find_peak_fq(pspec):
    if np.isnan(pspec).all():
        return np.nan
    idcs = argrelextrema(pspec, np.greater)[0]
    rel_maxs = pspec[idcs[idcs>30]]
    peak_freqs = idcs[idcs>30][np.argsort(rel_maxs)[::-1]]/10
    return peak_freqs[0]
    
# average power spectra over velocity range
ps_averaged = np.vstack( [average_powerspec(val) for val in target_vs[0:-1] ])
x = target_vs[0:-1]+v_box/2
y = [find_peak_fq(temp) for temp in ps_averaged]
peak_idcs = [find_peak_fq(temp) if (not np.isnan(temp).any()) else np.nan for temp in ps_averaged]
# get rid of nans
x_nonan = x[np.logical_not(np.isnan(y))]
x_c = sm.add_constant(x_nonan)
y_nonan = np.array(y)[np.logical_not(np.isnan(y))]

# fit slope and intercept model
model = sm.OLS(y_nonan, x_c).fit()
predictions = model.predict(x_c)



# look at specific power spectra to find criterion for including point in linear regression
fig  = plt.figure(figsize = (8,12))
ax1 =plt.axes([0.1, 0.6, .8, 0.3])
plt.plot(x_nonan,y_nonan,'.k', alpha = 0.2)
plt.plot(x_nonan, predictions, '-k')
plt.title('Col: %s -- Sub: %s\npeak freq for power spectra averaged over every 5pix/s window' % (colOI, subOI));
plt.text(700,7,'freq = %0.3f * v + %0.2f'% (model.params[1],model.params[0]), color = 'k')

for sp in range(0,6):
    plt.figure(fig.number)
    ranC = np.random.rand(3,)
    ax =plt.axes([0.08+sp*.15, 0.45, .15, 0.1])
    pt =12+sp*10#int((len(x)-10)/5.5)
    while np.isnan(y[pt]):
        pt = pt + 1
    ps_averaged[pt][ps_averaged[pt]<0] = np.nan #0.001 # get rid of negative power spectra values
    xs = np.linspace(0,120,120*10+1)
    xs_nonan = xs[np.logical_not(np.isnan(ps_averaged[pt]))]
    ys = ps_averaged[pt]
    ys_nonan = ps_averaged[pt][np.logical_not(np.isnan(ps_averaged[pt]))]
    plt.plot(xs,ys, c = ranC)
#     norm2 = colors.Normalize(vmin=0, vmax = 100)
#     sc = plt.scatter(xs,ys, s= 5,
#             cmap=cm.magma, c = ys, alpha = 0.8, norm=norm2)
    plt.ylim((0,100))
    plt.xlim((0,120))
    plt.axvline(x = y[pt], alpha = 0.2)  
    if not sp == 0:
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
    else:
        plt.ylabel('log(power)')
    ax1.plot(x[pt],y[pt], '.', c= ranC, Markersize = 10)
    ax1.text(x[pt],0, '%0.1f'%x[pt], color= ranC)
    
    
#     # fit y = A*e^(B*x)
#     coeffs = np.polyfit(xs, np.log(ys), 1, w=np.sqrt(ys))
#     ax.plot(xs, np.exp(coeffs[1]) * np.exp(xs * coeffs[0]),':k', alpha = 0.5, label = 'y=A*e^(B*x)')
#     peak_res = ps_averaged[pt][int(y[pt]*10)] - np.exp(coeffs[1]) * np.exp(xs[int(y[pt]*10)] * coeffs[0])
#     ax.text(90, 15, '%0.1f'% peak_res, color = 'k', alpha = 0.5, fontsize = 8)
    
    
    # fit y = A*x^B
    coeffs = np.polyfit(np.log(xs_nonan[6::]), np.log(ys_nonan[6::]), 1)
#     print(coeffs)
    ax.plot(xs[1::], np.exp(coeffs[1]) * xs[1::]**coeffs[0],'--', alpha = 0.5, color= 'k', label = ' y=A*x^B')
    peak_res = ps_averaged[pt][int(y[pt]*10)] - (np.exp(coeffs[1]) * y[pt]**coeffs[0])
    ax.text(115, 5, 'peak res: %0.1f'% peak_res, color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    if sp == 5:
        ax.legend(fontsize = 6, frameon=False, bbox_to_anchor = (.95, 1.25));
    
    
    # fit average line
    cv_n = 200
    fitline = np.convolve(ys_nonan[10:], np.ones((cv_n,))/cv_n, mode = 'valid')
    fitline_i = np.interp(xs,xs_nonan[int(cv_n/2+10):int((-cv_n)/2+1)],fitline)
    ax.plot(xs,fitline_i, ':b', alpha = 0.5)
    peak_res = ps_averaged[pt][int(y[pt]*10)] - fitline_i[int(y[pt]*10)]
#     print(np.nanmean(ys-fitline_i))
    ax.text(115, 15, 'peak res: %0.1f'% peak_res, color = 'r', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')

    plt.figure()
    plt.plot(ps_averaged[pt]-fitline_i, '-', color= ranC)
    residuals = ps_averaged[pt][30:]-fitline_i[30:]
    plt.axhline(y=np.nanmean(residuals))
    plt.axhline(y=np.nanmean(residuals)+np.nanstd(residuals), LineStyle = '--')
    plt.text(200,-50, 'stdev: %0.1f'% np.nanstd(residuals), color= ranC)
    plt.axvline(x = y[pt]*10, alpha = 0.2)  
    plt.text(200,-40, 'stdev of 60+ Hz: %0.1f'% np.nanstd(residuals[600-30:]), color= ranC)
    
#     # fit gaussian to peak
#     def gauss(x, *p):
#         A, mu, sigma, B = p
#         return A* np.exp(-(x-mu)**2/(2.*sigma**2)) + B
#     # restrict fitting to around found peak
#     if int(y[pt]*10-100) < 0:
#         ROI = range(5, int(2*y[pt]*10+5))
#     else:
#         ROI = range(int(y[pt]*10-100), int(y[pt]*10+100)) # only look at points around peak
#     axI =plt.axes([0.18+sp*.15, 0.5, .04, 0.04])
#     axI.plot(xs[ROI], ys[ROI]-np.nanmin(ys[ROI]), '-', c= ranC, alpha = 0.5)
#     axI.set_frame_on(False)
#     axI.get_xaxis().set_visible(False)
#     axI.get_yaxis().set_visible(False)
#     p0 = [2, y[pt], 3, 10] # initial guess of coefficients (A = amplitude?, mu = center, sigma = spread)
#     try:
#         coeff, var_matrix = curve_fit(gauss, xs[ROI][np.logical_not(np.isnan(ys[ROI]))], 
#                                       ys[ROI][np.logical_not(np.isnan(ys[ROI]))]-np.nanmin(ys[ROI]), 
#                                       p0=p0, bounds = ((0,0,0,0), (100,100,20,100)))
# #         print( sp, ' -- ', coeff)
#         gauss_fit = gauss(xs[ROI], *coeff)
#         axI.plot(xs[ROI],gauss_fit, ':k', alpha = 0.5)
#         ax.text(115, 90, r'A: %0.1f, $\sigma$: %0.1f'% (coeff[0],coeff[2]), color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
#         plt.axis('equal')
#     except:
#         print(sp,'-- Could not find gauss fit parameters')
        
    
    
        

# # REDO LIN REGRESSION WHEN PEAK IS FAR FROM BEST FIT
# def find_peak_res(pspec,peak_freq):
#     if np.isnan(pspec).all():
#         return np.nan

#     pspec[pspec<0] = np.nan
#     xs = np.linspace(0,120,120*10+1)
#     xs_nonan = xs[np.logical_not(np.isnan(pspec))]
#     pspec_nonan = pspec[np.logical_not(np.isnan(pspec))]
# #     print(len(xs), len(pspec), len(pspec_nonan))
#     coeffs = np.polyfit(np.log(xs_nonan[6:]), np.log(pspec_nonan[6:]), 1, w=np.sqrt(pspec_nonan[6:])) # fit y = A*e^(B*x)
    
#     # find peak freq
# #     idcs = argrelextrema(pspec_nonan, np.greater)[0]
# #     rel_maxs = pspec_nonan[idcs]
# #     peak_freq = (idcs[np.argsort(rel_maxs)[::-1]]/10)[0]
#     peak_res = pspec[int(peak_freq*10)] - (np.exp(coeffs[1]) * peak_freq**coeffs[0])
#     return peak_res

# def find_peak_var(pspec,peak_fq):
#     if np.isnan(peak_fq):
#         return np.nan
#     if int(peak_fq*10-100) < 0:
#         ROI = range(5, int(peak_fq*10+105))
#     else:
#         ROI = range(int(peak_fq*10-100), int(peak_fq*10+100)) # only look at points around peak
#     xs = np.linspace(0,120,120*10+1)
#     p0 = [2, y[pt], 3, 10] # initial guess of coefficients (A = amplitude?, mu = center, sigma = spread)
# #     plt.figure()
# #     plt.plot(xs,pspec,'-k',alpha = 0.3)
# #     plt.plot(xs[ROI],pspec[ROI],'-k',alpha = 0.7)
#     try:
#         coeff, var_matrix = curve_fit(gauss, xs[ROI][np.logical_not(np.isnan(pspec[ROI]))], 
#                                       pspec[ROI][np.logical_not(np.isnan(pspec[ROI]))]-np.nanmin(pspec[ROI]), p0=p0, bounds = ((0,0,0,0), (100,100,20,100)))
# #         print(' -- ', coeff)
#         gauss_fit = gauss(xs[ROI], *coeff)
#         return coeff[2]
# #         plt.plot(xs[ROI],gauss_fit, ':r')
#     except:
#         print(' Could not find gauss fit parameters')


# peak_res = np.array([find_peak_res(temp, pf) for temp, pf in zip(ps_averaged, y)])
# peak_var = np.array([find_peak_var(temp, pf) for temp, pf in zip(ps_averaged, y)])
# res_cutoff = 1
# ax1 =plt.axes([0.1, 0.1, .8, 0.3])
# norm2 = colors.Normalize(vmin=0, vmax = 6)
# plt.plot(x_nonan,y_nonan,'.k', alpha = 0.2)
# sc = plt.scatter(np.array(x)[peak_res>res_cutoff],np.array(y)[peak_res>res_cutoff], s= 5,
#             cmap=cm.magma, c = peak_var[peak_res>res_cutoff], alpha = 0.8, norm=norm2)
# # plt.colorbar.make_axes(fig.add_axes([.95,.1,.03,.3]))

# x_nonan = np.array(x)[peak_res>res_cutoff]
# x_c = sm.add_constant(x_nonan)
# y_nonan = np.array(y)[peak_res>res_cutoff]
# model = sm.OLS(y_nonan, x_c).fit()
# predictions = model.predict(x_c)
# plt.plot(x_nonan, predictions, '-k')
# plt.title('linear regression for peak frequencies with residual > %i' % res_cutoff);
# plt.xlabel('v (pix/s)')
# plt.xlim((0,1200))
# ax1.text(700,7,'freq = %0.3f * v + %0.2f'% (model.params[1],model.params[0]), color = 'k')

# cbar = plt.colorbar(sc, cax = fig.add_axes([.92,.1,.01,.3]))#fig.add_axes([.95,.1,.03,.3]))
# cbar.set_label('gaussian fit sigma (Hz)')

In [ ]:
# FOR A GIVEN COLONY AND SUBSTRATE, FIT A LINEAR MODEL TO THE PEAK FREQUENCY VS. VELOCITY FOR ***ALL CHUNKS***"
plt.close('all')
from scipy.optimize import curve_fit

# fft_data = pd.read_pickle(vid_locations + 'FFT_data')
# print('read in data')
subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())

# define what data to use for specific colony and substrate
colOI = coltypes[0]
subOI = subtypes[0]
v_ave = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['v_ave']
peak_fqs = np.array([fs[0] for fs in fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['peak_freqs']])
ps = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['pow_spec']

# define inputs for models
y = peak_fqs[np.argsort(v_ave).values].astype(np.float32)
ps_sorted = list(ps[np.argsort(v_ave).values].values)
x = np.array(sorted(v_ave))

# interpolate data so can fit intercept model to it
v_box = 5
max_v = int(round(np.max(x)))
min_v = 0
subsample = int((max_v-min_v)/v_box+1)
target_vs = np.linspace(min_v,max_v, subsample)

x_c = sm.add_constant(x) # if do not want model to go through zero
x_i = np.linspace(min_v, max_v, subsample)

# plot raw data
fig  = plt.figure(figsize = (8,12))
ax1 =plt.axes([0.1, 0.6, .8, 0.3])
plt.plot(x,y,'.k', alpha = 0.2)
plt.title('Peak freq from all data chunks')
plt.xlabel('v (pix/s)')
plt.ylabel('freq with peak power')

# intercept and slope model
model = sm.OLS(y, x_c).fit()
predictions = model.predict(x_c)
predictions_interp = np.interp(x_i, x, predictions)
# model.summary()
plt.plot(x_i, predictions_interp,'-b')
# plt.plot(x, predictions, ':r')
plt.text(700,35,'freq = %0.3f * v + %0.2f'% (model.params[1],model.params[0]), color = 'b')

# slope only model
model = sm.OLS(y, x).fit()
predictions = model.predict(x)
# model.summary()
plt.plot(x, predictions,'--g')
plt.text(700,37,'freq = %0.3f * v'% (model.params[0]), color = 'g')

# intercept and slope model removing any values where peak fr > 30
# model = sm.OLS(y[y<30], x_c[y<30]).fit()
# predictions = model.predict(x_c[y<30])
# predictions_interp = np.interp(x_i, x[y<30], predictions)
# model.summary()
# plt.plot(x[y<30], predictions, ':g')
# plt.text(700,35,'freq = %0.3f *x + %0.2f'% (model.params[1],model.params[0]), color = 'b')


# look at specific power spectra to find criterion for including point in linear regression
def gauss(x, *p):
    A, mu, sigma, B = p
    return A* np.exp(-(x-mu)**2/(2.*sigma**2)) + B

for sp in range(0,6):
    plt.figure(fig.number)
    ranC = np.random.rand(3,)
    ax =plt.axes([0.08+sp*.15, 0.45, .15, 0.1])
#     pt = 10+sp*int(len(x)/5.1)
    pt = 10+sp*50
    while np.isnan(y[pt]):
        pt = pt + 1
    ps_sorted[pt][ps_sorted[pt]<0] = np.nan #0.001 # get rid of negative power spectra values
    xs = np.linspace(0,120,120*10+1).astype(np.float32)
    ys = ps_sorted[pt]
    xs_nonan = xs[np.logical_not(np.isnan(ps_sorted[pt]))]
    ys_nonan = ps_sorted[pt][np.logical_not(np.isnan(ps_sorted[pt]))]
    plt.plot(xs,ys, c = ranC)
    plt.ylim((0,100))
#     plt.xlim((0,40))
    plt.axvline(x = y[pt], alpha = 0.2)  
    if not sp == 0:
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
    else:
        plt.ylabel('log(power)')
        plt.xlabel('freq (Hz)')
    ax1.plot(x[pt],y[pt], '.', c= ranC, Markersize = 10)
    
    
    # fit y = A*x^B
    coeffs = np.polyfit(np.log(xs_nonan[1::]), np.log(ys_nonan[1::]), 1)
    ax.plot(xs, np.exp(coeffs[1]) * xs**coeffs[0],'--', alpha = 0.5, color= 'k', label = ' y=A*x^B')
    peak_res = ps_sorted[pt][int(y[pt]*10)] - np.exp(coeffs[1]) * xs[int(y[pt]*10)]**coeffs[0]
    ax.text(39, 5, 'peak res: %0.1f'% peak_res, color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    print('res: %f - ave res: %f ' % (peak_res, np.nanmean(ys[1:]-(np.exp(coeffs[1]) * xs[1:]**coeffs[0])) ))
    if sp == 5:
        ax.legend(fontsize = 6, frameon=False, bbox_to_anchor = (.4, 1.15));
        
    # fit y = A*e^(B*x)
    coeffs = np.polyfit(xs_nonan, np.log(ys_nonan), 1, w=np.sqrt(ys_nonan))
    ax.plot(xs, np.exp(coeffs[1]) * np.exp(xs * coeffs[0]),':k', alpha = 0.5, label = 'y=A*e^(B*x)')
    peak_res = ps_sorted[pt][int(y[pt]*10)] - np.exp(coeffs[1]) * np.exp(xs[int(y[pt]*10)] * coeffs[0])
    ax.text(39, 15, '%0.1f'% peak_res, color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
        
    # fit average line
    cv_n = 200
    fitline = np.convolve(ys_nonan[10:], np.ones((cv_n,))/cv_n, mode = 'valid')
    fitline_i = np.interp(xs,xs_nonan[int(cv_n/2+10):int((-cv_n)/2+1)],fitline)
    ax.plot(xs,fitline_i, ':r', alpha = 0.5)
    peak_res = ps_sorted[pt][int(y[pt]*10)] - fitline_i[(xs==y[pt])]
#     print('res: %f - ave res: %f - rel res: %f' % (peak_res, np.nanmean(ys-fitline_i), np.abs(peak_res/np.nanmean(ys-fitline_i)) ))
    ax.text(115, 15, 'peak res: %0.1f'% peak_res, color = 'r', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    
    # fit gaussian to peak
    # restrict fitting to around found peak
    if int(y[pt]*10-100) < 0:
        ROI = range(5, int(y[pt]*10+105))
    else:
        ROI = range(int(y[pt]*10-100), int(y[pt]*10+100)) # only look at points around peak
    axI =plt.axes([0.19+sp*.15, 0.51, .03, 0.03])
    axI.plot(xs[ROI], ys[ROI]-np.nanmin(ys[ROI]), '-', c= ranC, alpha = 0.5)
    axI.set_frame_on(False)
    axI.get_xaxis().set_visible(False)
    axI.get_yaxis().set_visible(False)
    p0 = [2, y[pt], 3, 10] # initial guess of coefficients (A = amplitude?, mu = center, sigma = spread)
    try:
        coeff, var_matrix = curve_fit(gauss, xs[ROI][np.logical_not(np.isnan(ys[ROI]))], 
                                      ys[ROI][np.logical_not(np.isnan(ys[ROI]))]-np.nanmin(ys[ROI]), 
                                      p0=p0, bounds = ((0,0,0,0), (100,100,20,100)))
        
#         print( sp, ' -- ', coeff)
        gauss_fit = gauss(xs[ROI], *coeff)
        axI.plot(xs[ROI],gauss_fit, ':k', alpha = 0.5)
        ax.text(115, 90, r'A: %0.1f, $\sigma$: %0.1f'% (coeff[0],coeff[2]), color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    except:
        print(sp,'-- Could not find gauss fit parameters')
        
        
        
    plt.figure()
    plt.plot(ps_sorted[pt]-fitline_i, '-', color= ranC)
    residuals = ps_sorted[pt][30:]-fitline_i[30:]
    plt.axhline(y=np.nanmean(residuals))
    plt.axhline(y=np.nanmean(residuals)+np.nanstd(residuals), LineStyle = '--')
    plt.text(200,5, 'stdev: %0.1f'% np.nanstd(residuals), color= ranC)
    plt.axvline(x = y[pt]*10, alpha = 0.2)  
    plt.text(200,2, 'stdev of 60+ Hz: %0.1f'% np.nanstd(residuals[600-30:]), color= ranC)
    
    
def find_residuals(pspec):
    cv_n = 200
    xs = np.range(0,len(pspec))
    xs_nonan = xs[np.logical_not(np.isnan(pspec))]
    ys_nonan = pspec[np.logical_not(np.isnan(pspec))]
    fitline = np.convolve(ys_nonan[10:], np.ones((cv_n,))/cv_n, mode = 'valid')
    fitline_i = np.interp(xs,xs_nonan[int(cv_n/2+10):int((-cv_n)/2+1)],fitline)
    residuals = pspec[30:]-fitline_i[30:]
    res_mean = np.nanmean(residuals)
    res_std = np.nanstd(residuals)


        
        

# # REDO LIN REGRESSION WHEN PEAK IS FAR FROM BEST FIT
# def find_peak_res(pspec,peak_freq):
#     if np.isnan(pspec).all():
#         return np.nan

#     pspec[pspec<0] = np.nan
#     xs = np.linspace(0,120,120*10+1)
#     xs_nonan = xs[np.logical_not(np.isnan(pspec))]
#     pspec_nonan = pspec[np.logical_not(np.isnan(pspec))]
# #     print(len(xs), len(pspec), len(pspec_nonan))
#     coeffs = np.polyfit(np.log(xs_nonan), np.log(pspec_nonan), 1, w=np.sqrt(pspec_nonan)) # fit y = A*e^(B*x)
    
#     # find peak freq
# #     idcs = argrelextrema(pspec_nonan, np.greater)[0]
# #     rel_maxs = pspec_nonan[idcs]
# #     peak_freq = (idcs[np.argsort(rel_maxs)[::-1]]/10)[0]
#     peak_res = pspec[int(peak_freq*10)] - np.exp(coeffs[1]) * xs[int(peak_freq*10)]**coeffs[0]
#     return peak_res

# def find_peak_var(pspec,peak_fq):
#     if np.isnan(peak_fq):
#         return np.nan
#     if int(peak_fq*10-100) < 0:
#         ROI = range(5, int(peak_fq*10+105))
#     else:
#         ROI = range(int(peak_fq*10-100), int(peak_fq*10+100)) # only look at points around peak
#     xs = np.linspace(0,120,120*10+1)
#     p0 = [2, y[pt], 3, 10] # initial guess of coefficients (A = amplitude?, mu = center, sigma = spread)
# #     plt.figure()
# #     plt.plot(xs,pspec,'-k',alpha = 0.3)
# #     plt.plot(xs[ROI],pspec[ROI],'-k',alpha = 0.7)
#     try:
#         coeff, var_matrix = curve_fit(gauss, xs[ROI][np.logical_not(np.isnan(pspec[ROI]))], 
#                                       pspec[ROI][np.logical_not(np.isnan(pspec[ROI]))]-np.nanmin(pspec[ROI]), p0=p0, bounds = ((0,0,0,0), (100,100,20,100)))
# #         print(' -- ', coeff)
#         gauss_fit = gauss(xs[ROI], *coeff)
#         return coeff[2]
# #         plt.plot(xs[ROI],gauss_fit, ':r')
#     except:
#         print(' Could not find gauss fit parameters')


# peak_res = np.array([find_peak_res(temp, pf) for temp, pf in zip(ps_sorted, y)])
# # peak_var = np.array([find_peak_var(temp, pf) for temp, pf in zip(ps_sorted, y)])
# res_cutoff = 1
# ax1 =plt.axes([0.1, 0.1, .8, 0.3])
# # norm2 = colors.Normalize(vmin=0, vmax = 15)
# # plt.plot(x,y,'.k', alpha = 0.05)
# sc = plt.scatter(np.array(x)[peak_res>res_cutoff],np.array(y)[peak_res>res_cutoff], s= 5,
#              c = 'b', alpha = 0.2, norm=norm2)
# # sc = plt.scatter(np.array(x)[peak_res>res_cutoff],np.array(y)[peak_res>res_cutoff], s= 5,
# #             cmap=cm.cool, c = peak_var[peak_res>res_cutoff], alpha = 0.8, norm=norm2)
# # # plt.colorbar.make_axes(fig.add_axes([.95,.1,.03,.3]))

# x_nonan = np.array(x)[peak_res>res_cutoff]
# x_c = sm.add_constant(x_nonan)
# y_nonan = np.array(y)[peak_res>res_cutoff]
# model = sm.OLS(y_nonan, x_c).fit()
# predictions = model.predict(x_c)
# plt.plot(x_nonan, predictions, '-k')
# plt.title('linear regression for peak frequencies with residual > %i' % res_cutoff);
# plt.xlabel('v (pix/s)')
# ax1.text(700,7,'freq = %0.3f * v + %0.2f'% (model.params[1],model.params[0]), color = 'k')

# # cbar = plt.colorbar(sc, cax = fig.add_axes([.92,.1,.01,.3]))#fig.add_axes([.95,.1,.03,.3]))
# # cbar.set_label('gaussian fit sigma (Hz)')

In [ ]:
# Run through all chunks and remove "bad" trials based on noise from moving avg line

plt.close('all')
from scipy.optimize import curve_fit

# fft_data = pd.read_pickle(vid_locations + 'FFT_data')
# print('read in data')
subtypes = sorted(longtracks['substrate'].unique())
coltypes = sorted(longtracks['colony'].unique())

# define what data to use for specific colony and substrate
colOI = coltypes[0]
subOI = subtypes[1]
v_ave = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['v_ave']
peak_fqs = np.array([fs[0] for fs in fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['peak_freqs']])
ps = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['pow_spec']

def find_peak_fq(pspec): # find rel max ignoring first 3 Hz
    if np.isnan(pspec).all():
        return np.nan
    idcs = argrelextrema(pspec, np.greater)[0]
    rel_maxs = pspec[idcs[idcs>30]]
    peak_freqs = idcs[idcs>30][np.argsort(rel_maxs)[::-1]]/10
    return peak_freqs[0]

ps_sorted = ps.loc[v_ave.sort_values().index].values #list(ps.values[np.argsort(v_ave).values])
peak_freqs2 = np.array([find_peak_fq(ps) for ps in ps_sorted])
y=peak_freqs2
x = v_ave.sort_values().values


def find_residuals(pspec):
    cv_n = 200
    xs = np.linspace(0, 120, len(pspec))
#     print(len(xs))
    xs_nonan = xs[np.logical_not(np.isnan(pspec))]
    ys_nonan = pspec[np.logical_not(np.isnan(pspec))]
    fitline = np.convolve(ys_nonan[10:], np.ones((cv_n,))/cv_n, mode = 'valid')
    fitline_i = np.interp(xs,xs_nonan[int(cv_n/2+10):int((-cv_n)/2+1)],fitline)
    residuals = pspec[30:]-fitline_i[30:]
    res_mean = np.nanmean(residuals[600-30:])
    res_std = np.nanstd(residuals[30:])
    res_std_half = np.nanstd(residuals[600-30:])
    return res_mean, res_std, res_std_half
    

res_info = np.array([find_residuals(ps) for ps in ps_sorted])
res_mean = res_info[:,0]
res_std = res_info[:,1]
res_std_half = res_info[:,2]
res_cutoff = 0.7




# PLOT THINGS

fig  = plt.figure(figsize = (8,12))
ax1 =plt.axes([0.1, 0.65, .8, 0.3])
plt.plot(x,y,'.k', alpha = 0.02)
plt.title('Peak freq from all data chunks')
plt.xlabel('v (pix/s)')
plt.ylabel('freq with peak power')



ax2 =plt.axes([0.1, 0.05, .8, 0.25])
# apply conditions
newy = peak_freqs2[np.logical_and(res_std_half<res_cutoff,peak_freqs2>5)]
newx = x[np.logical_and(res_std_half<res_cutoff,peak_freqs2>5)]
plt.plot(x,peak_freqs2,'.k', alpha = 0.02)
plt.plot(newx,newy,'.r', alpha = 0.1)
# plt.plot(x,peak_freqs2,'.k', alpha = 0.02)
# plt.plot(x[res_std_half<res_cutoff],peak_freqs2[res_std_half<res_cutoff],'.r', alpha = 0.2)
plt.xlabel('v (pix/s)')
plt.ylabel('freq with peak power')






for sp in range(0,5):
    plt.figure(fig.number)
    ranC = np.random.rand(3,)
    ax =plt.axes([0.08+sp*.15, 0.5, .15, 0.1])
#     pt = 10+sp*int(len(x)/5.1)
    pt = 10+sp*350
    while np.isnan(y[pt]) or res_std_half[pt]>res_cutoff:
        pt = pt + 1
    ps_sorted[pt][ps_sorted[pt]<0] = np.nan #0.001 # get rid of negative power spectra values
    xs = np.linspace(0,120,120*10+1).astype(np.float32)
    ys = ps_sorted[pt]
    xs_nonan = xs[np.logical_not(np.isnan(ps_sorted[pt]))]
    ys_nonan = ps_sorted[pt][np.logical_not(np.isnan(ps_sorted[pt]))]
    plt.plot(xs,ys, c = ranC)
    plt.ylim((0,100))
    plt.xlim((0,60))
    plt.axvline(x = y[pt], alpha = 0.2)  
    if not sp == 0:
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
#     else:
#         plt.ylabel('log(power)')
#         plt.xlabel('freq (Hz)')
    ax1.plot(x[pt],y[pt], '.', c= ranC, Markersize = 10)
    
    
    # fit y = A*x^B
    coeffs = np.polyfit(np.log(xs_nonan[1::]), np.log(ys_nonan[1::]), 1)
    ax.plot(xs, np.exp(coeffs[1]) * xs**coeffs[0],'--', alpha = 0.5, color= 'k', label = ' y=A*x^B')
    fitline_e = np.exp(coeffs[1]) * xs**coeffs[0]
    peak_res = ps_sorted[pt][int(y[pt]*10)] - np.exp(coeffs[1]) * xs[int(y[pt]*10)]**coeffs[0]
    peak_res2 = ps_sorted[pt][int(peak_freqs2[pt]*10)] - np.exp(coeffs[1]) * xs[int(peak_freqs2[pt]*10)]**coeffs[0]
    ax.text(58, 15, 'peak res: %0.1f'% peak_res2, color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
#     ax.text(58, 5, 'peak res: %0.1f'% peak_res, color = 'b', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    print('res: %f - ave res: %f ' % (peak_res2, np.nanmean(ys[1:]-(np.exp(coeffs[1]) * xs[1:]**coeffs[0])) ))
    if sp == 5:
        ax.legend(fontsize = 6, frameon=False, bbox_to_anchor = (.4, 1.15));

        
    # fit average line
    cv_n = 200
    fitline = np.convolve(ys_nonan[10:], np.ones((cv_n,))/cv_n, mode = 'valid')
    fitline_i = np.interp(xs,xs_nonan[int(cv_n/2+10):int((-cv_n)/2+1)],fitline)
    ax.plot(xs,fitline_i, ':r', alpha = 0.5)
    peak_res = ps_sorted[pt][int(y[pt]*10)] - fitline_i[(xs==y[pt])]
#     print('res: %f - ave res: %f - rel res: %f' % (peak_res, np.nanmean(ys-fitline_i), np.abs(peak_res/np.nanmean(ys-fitline_i)) ))
    ax.text(58, 5, 'peak res: %0.1f'% peak_res, color = 'r', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
    
    
    # plot residuals
    axr =plt.axes([0.08+sp*.15, 0.36, .15, 0.1])
#     plt.plot(xs,ps_sorted[pt]-fitline_i, '-', color= ranC)
    plt.plot(xs,ps_sorted[pt]-fitline_e, '-', color= ranC)
    
    
    def find_peak_residual(pspec, fitline): # find rel max ignoring first 3 Hz
        if np.isnan(pspec).all():
            return np.nan
        residuals = pspec - fitline
        residuals[np.isnan(residuals)]=0
        p_idcs = argrelextrema(residuals, np.greater)[0]
        peaks = residuals[p_idcs]
        t_idcs = argrelextrema(residuals, np.lesser)[0]
        troughs = residuals[t_idcs]
#         rel_maxs = pspec[idcs[idcs>30]]
#         peak_freqs = idcs[idcs>30][np.argsort(rel_maxs)[::-1]]/10
#         return peak_freqs[0]
    
    plt.axhline(y=res_mean[pt])
    plt.axhline(y=res_mean[pt]+res_std[pt], LineStyle = '--')
#     plt.axhline(y=res_mean[pt]+res_std_half[pt], LineStyle = ':')
    plt.axvline(x=y[pt], alpha = 0.2)
    plt.axvline(x=peak_freqs2[pt], Color ='r', alpha = 0.2)
    ax.axvline(x=peak_freqs2[pt], Color ='r', alpha = 0.2)
    plt.ylim((-10,15))
    plt.xlim((0,60))
    if not sp == 0:
        axr.get_yaxis().set_visible(False)
#         ax.get_xaxis().set_visible(False)
#     plt.text(37,8, '%0.1f'% res_std_half[pt], color= ranC)

In [ ]:
# test another way of finding peak frequencies
plt.close('all')
colOI = coltypes[0]
subOI = subtypes[1]
v_ave = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['v_ave']
ps = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['pow_spec_raw']
ps_fr = fft_data[(fft_data['colony']==colOI) & (fft_data['substrate']==subOI)]['pow_freq_raw']
ps_sorted = ps.loc[v_ave.sort_values().index].values 
ps_fr_sorted = ps_fr.loc[v_ave.sort_values().index].values 


def find_res_peak(pspec, fitline):
    residuals = pspec - fitline
    residuals[np.isnan(residuals)]=0
    p_idcs = argrelextrema(residuals, np.greater)[0]
    if not p_idcs.any() < 0:
        return np.nan
    t_idcs = argrelextrema(residuals, np.less)[0]
    t_idcs = t_idcs[t_idcs > p_idcs[0]]
    t_idcs = t_idcs[:np.min([len(p_idcs), len(t_idcs)])]
    p_idcs = p_idcs[:np.min([len(p_idcs), len(t_idcs)])]
    peaks = residuals[p_idcs]
    troughs = residuals[t_idcs]
    true_peaks = p_idcs[np.logical_and((peaks > 5),(troughs < 0))]
    print(len(xs[true_peaks]))
    if xs[true_peaks].any():
        return xs[true_peaks][0]
    else:
        return np.nan

def fit_exponential(pspec, pspec_xf):
    coeffs = np.polyfit(np.log(pspec_xf[1::]), np.log(pspec[1::]), 1)
    fitline_e = np.exp(coeffs[1]) * pspec_xf**coeffs[0]
    return fitline_e

test = np.array([fit_exponential(y,x) for y,x in zip(ps_sorted, ps_fr_sorted)])
peak_freqs = np.array([find_res_peak(y, fit_exponential(y,x)) for y,x in zip(ps_sorted, ps_fr_sorted)])
# v_ave_sorted = v_ave.sort_values().values


# plt.figure(figsize = (8,12))
# ax1 =plt.axes([0.1, 0.65, .8, 0.3])
# plt.plot(v_ave_sorted,peak_freqs,'.k', alpha = 0.02)

# for sp in range(0,6):
#     pt = 10+sp*250

#     xs = ps_fr_sorted[pt]
#     ys = ps_sorted[pt]
#     ax =plt.axes([0.08+sp*.15, 0.5, .15, 0.1])
#     ax.plot(xs,ys,'-k')
    

#     # fit y = A*x^B
#     coeffs = np.polyfit(np.log(xs[1::]), np.log(ys[1::]), 1)
#     fitline_e = np.exp(coeffs[1]) * xs**coeffs[0]
#     ax.plot(xs, fitline_e,'--', alpha = 0.5, color= 'k', label = ' y=A*x^B')
#     plt.ylim((0,100))
#     plt.xlim((0,60))
# #     peak_res = ps_sorted[pt][int(y[pt]*10)] - np.exp(coeffs[1]) * xs[int(y[pt]*10)]**coeffs[0]
# #     peak_res2 = ps_sorted[pt][int(peak_freqs2[pt]*10)] - np.exp(coeffs[1]) * xs[int(peak_freqs2[pt]*10)]**coeffs[0]
# #     ax.text(58, 15, 'peak res: %0.1f'% peak_res2, color = 'k', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
# #     #     ax.text(58, 5, 'peak res: %0.1f'% peak_res, color = 'b', alpha = 0.5, fontsize = 8, horizontalalignment = 'right')
# #     print('res: %f - ave res: %f ' % (peak_res2, np.nanmean(ys[1:]-(np.exp(coeffs[1]) * xs[1:]**coeffs[0])) ))


    

#     axr =plt.axes([0.08+sp*.15, 0.36, .15, 0.1])
#     axr.plot(xs, ys-fitline_e)
#     plt.ylim((-15,15))
#     plt.xlim((0,60))
#     plt.axhline(y=0, LineStyle = '--', Color = 'k', alpha = 0.3)
#     if not sp == 0:
#         axr.get_yaxis().set_visible(False)


    
#     ax.axvline(x=xs[true_peaks[0]], Color ='r', alpha = 0.2)
#     axr.axvline(x=xs[true_peaks[0]], Color ='r', alpha = 0.2)

# #         rel_maxs = pspec[idcs[idcs>30]]
# #         peak_freqs = idcs[idcs>30][np.argsort(rel_maxs)[::-1]]/10
# #         return peak_freqs[0]

In [ ]:
from scipy import signal
# t = np.linspace(-1, 1, 200, endpoint=False)
# sig  = np.cos(2 * np.pi * 7 * t) + signal.gausspulse(t - 0.4, fc=2)


plt.close('all')

tr_num = 8976
x = longtracks.loc[tr_num].copy()
t = x.frames_final
sig= x.v
widths = np.arange(1, 31)
fig = plt.figure()
plt.axes([.1,.55,.75,.4])
plt.plot(t,sig)
plt.xlim(t[0],t[-1])
plt.gca().get_xaxis().set_visible(False)


cwtmatr = signal.cwt(sig, signal.ricker, widths)
ax2 = plt.axes([.1,.1,.75,.4])
ax3 = fig.add_axes([.88,.1,.02,.4])
im = ax2.imshow(cwtmatr, extent=[t[0],t[-1], widths.max(), 1], cmap='PRGn', aspect='auto',vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())
ax2.set_ylabel('widths')
ax2.set_xlabel('time')
plt.colorbar(im, cax = ax3)
plt.show()

plt.figure()
fs = 10e3
N = 1e5
amp = 2 * np.sqrt(2)
noise_power = 0.01 * fs / 2
time = np.arange(N) / float(fs)
mod = 500*np.cos(2*np.pi*0.25*time)
carrier = amp * np.sin(2*np.pi*3e3*time + mod)
noise = np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
noise *= np.exp(-time/5)
x = carrier + noise
# f, t2, Sxx = signal.spectrogram(np.array(sig), 240)
# plt.pcolormesh(t2,f,Sxx)
f, t, Sxx = signal.spectrogram(x, fs)
plt.pcolormesh(t,f,Sxx)
plt.ylabel('Freq (Hz)')
plt.xlabel('Time (s)')
plt.show()




# plt.colorbar()

In [ ]:
# PLOT A HEATMAP FOR A SINGLE COLONY AND SUBSTRATE - SUBSAMPLE EVERY Nth SAMPLE

plt.figure(figsize = (5,12))

subsample = 20
hmap = sns.heatmap(allpowers_sorted[::subsample], xticklabels = 1000, yticklabels = 10, cbar_kws = {'label': 'log(power)'})
hmap.set_yticklabels(['%i' % v for v in allpowers_vs_sorted[::subsample][::10]])
hmap.set_xticklabels(np.arange(0,121,10))
plt.yticks(rotation=0)
plt.ylabel('ave v (pix/s)');
plt.xlabel('freq (Hz)');
plt.title('%s -- %s -- subsample: %i' % (subtype, coltype[-5:], subsample))
plt.tight_layout()

## Save trials for Shai/Brian

In [ ]:
plt.close('all')
for tr in range(1000,1020):
    
    plt.figure()
    plt.text(0, 70, 'Trial: %i'%tr)

    for joint_num in range(0,6):
        plt.plot(df['joint%i_x_filt_WRTneck'%joint_num][tr])

In [23]:
# calculate orientation of ant


def find_orientation_df(x): #'rotation', 'Len', 'Dur', 'travel_dir'
    orie = np.arctan2(x['neck_y_filt_fullfr']-x['thorax_y_filt_fullfr'], x['neck_x_filt_fullfr']-x['thorax_x_filt_fullfr'])
    return orie


df['orientation']= df.apply(find_orientation_df, args = (), axis=1)

In [25]:
for tr in [5,1007,24,433,22,439]:
    print('saving csv for trial %i'%tr)
    print('    ', df['video'][tr])
    
    s_location = vid_locations + 'GaitPhasing/' + 'Trial_%04.0f'%tr
    vars_OI = ['frames', 'frames_final', 'x_kal', 'y_kal', 'x_raw', 'y_raw', 'x', 'y', 'angle', 'angle_improved', 'orientation']
    for wrt in ['', '_filt_fullfr']:
        for jj in ['thorax','joint']:
            if jj == 'thorax':
                for coord in ['x','y']:
                    vars_OI.append('%s_%s%s'%(jj,coord,wrt))
            else:
                for jn in range(0,6):
                    for coord in ['x','y']:
                        vars_OI.append('%s%i_%s%s'%(jj,jn,coord,wrt))
    for wrt in ['_filt_WRTneck']:
        for jn in range(0,6):
            for coord in ['x','y']:
                vars_OI.append('joint%i_%s%s'%(jn,coord,wrt))

    n_frs = len(df['angle'][tr])-1
    tmp = np.full((n_frs, len(vars_OI)), np.nan)
    idcs = np.isin(df['frames'][tr][1:],df['frames_final'][tr])

    for vv,var in enumerate(vars_OI):
        tmp2 = df[var].iloc[tr]
        if (len(tmp2)-1)==n_frs:
            tmp[:,vv] = tmp2[1:]
        else:
            tmp[idcs,vv] = tmp2
        del tmp2


    glenna = pd.DataFrame(tmp, columns = vars_OI)
    glenna['frames'] = glenna['frames'].astype(int)
    glenna.to_csv(s_location, sep = '\t', index=False, float_format = '%11.4f', na_rep = 'nan')

saving csv for trial 5
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080521_16276736-0000.mp4
saving csv for trial 1007
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/3mm/20180314_094259_16276718-0000.mp4
saving csv for trial 24
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_083709_16276736-0000.mp4
saving csv for trial 433
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/1mm/20180313_095948_16276712-0000.mp4
saving csv for trial 22
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_083333_16276736-0000.mp4
saving csv for trial 439
     /media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/1mm/20180313_100402_16276712-0000.mp4


In [ ]:
np.arctan2(df['neck_y_filt_fullfr'][1]-df['thorax_y_filt_fullfr'][1], df['neck_x_filt_fullfr'][1]-df['thorax_x_filt_fullfr'][1])

## Find fastest trials

Use these df trackways to make sure that you aren't getting tracking artifacts

In [ ]:
sortbyvs = longtracks.copy()

allvs = longtracks.v
allidcs = list(longtracks.index)
maxvs = allvs.apply(np.max)
sortbyvs['maxv'] = maxvs
sortbyvs = sortbyvs.sort_values(['maxv'], ascending = False)

print(sortbyvs.iloc[0:10][['maxv','substrate','datetime']])

In [ ]:
tr_num = 124
norm2 = colors.Normalize(vmin=0, vmax = 1500)
# plt.scatter(glenna.loc[tr_num].x, glenna.loc[tr_num].y)

plt.figure()

blankvtrace = np.full(glenna.loc[tr_num].frames.shape,np.nan)

plt.plot(glenna.loc[tr_num].frames[1:], glenna.loc[tr_num].vfilt, '-k', alpha = 0.5);
plt.plot(glenna.loc[tr_num].frames_final+1, glenna.loc[tr_num].v_final, '-g');
plt.plot(glenna.loc[tr_num].frames[30-1:-30], glenna.loc[tr_num].v_runave, '.b', alpha = 0.5);
plt.plot(glenna.loc[tr_num].frames_final, glenna.loc[tr_num].dist_90fr*(240/90), '.r', alpha = 0.5);
plt.legend(('filtered v','v within ROI/moving','v_runavg','net pix over 90 fr'))
# plt.scatter(df.frames_final[tr_num], temp, #'-' ,
#          cmap=cm.cool, c=temp, edgecolor='none', norm=norm2)



# longtracks.apply(np.convolve, args = (np.ones((60,))/60, mode = 'valid'))

## Kalman filtering to smooth out tracked data
